<a href="https://colab.research.google.com/github/genaiconference/Agentic_RAG_Workshop/blob/main/01_Handling_Multi_Modal_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyzing Document and Processing using Azure Document Intelligence

This notebook demonstrates how to:
- process a document using Azure Document Intelligence,
- extract information
- process images within the document & Interpret using llm
- Insert the image interpretation in place of the image

In [1]:
!git clone https://github.com/genaiconference/Agentic_RAG_Workshop.git

Cloning into 'Agentic_RAG_Workshop'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 144 (delta 67), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (144/144), 7.36 MiB | 17.43 MiB/s, done.
Resolving deltas: 100% (67/67), done.


## Setup and Installations
Install necessary libraries for document processing, data handling, and interacting with Azure Document Intelligence and OpenAI.

In [3]:
!pip install -r /content/Agentic_RAG_Workshop/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.0/106.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.9/187.9 kB 15.2 MB/s eta

## Load Environment Variables
Load environment variables containing API keys and endpoint information

In [1]:
import os

os.chdir("/content/Agentic_RAG_Workshop/")

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

doc_intelligence_endpoint=os.getenv("doc_intelligence_endpoint")
doc_intelligence_key=os.getenv("doc_intelligence_key")


llm = ChatOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4.1",
    temperature=0,
)

## Data Source Details

We will be dealing with 2 types of Data Sources
1. HR Policies
    - Leave Policy Document
    - Insurance Policy Document
2. Annual Reports (10-K Documents)
    - Microsoft 2023 & 2024 Reports
    - Apple 2023 & 2024 Reports

## Define Paths and Load Documents

In [2]:
import os
import pickle
from IPython.display import Markdown

# --- Configuration ---
# Define paths to all input files in the data folder and its subfolders
DATA_DIR = os.path.join(os.getcwd(),"data")
INPUT_FILE_PATHS = []

for root, _, files in os.walk(DATA_DIR):
    for file in files:
        if file.endswith(".pdf"):
            INPUT_FILE_PATHS.append(os.path.join(root, file))

PICKLE_DIR = os.path.join(os.getcwd(), "DI_output")
IMG_SAVE_DIR = os.path.join(PICKLE_DIR, "images")

# --- Ensure output directories exist ---
os.makedirs(PICKLE_DIR, exist_ok=True)
os.makedirs(IMG_SAVE_DIR, exist_ok=True)

print("Input files found:")
for file_path in INPUT_FILE_PATHS:
    print(file_path)

Input files found:
/content/Agentic_RAG_Workshop/data/HR Policies/Insurance Policy.pdf
/content/Agentic_RAG_Workshop/data/HR Policies/Leave Policy.pdf
/content/Agentic_RAG_Workshop/data/Annual Reports/Microsoft_2024.pdf
/content/Agentic_RAG_Workshop/data/Annual Reports/Apple_2023.pdf
/content/Agentic_RAG_Workshop/data/Annual Reports/Apple_2024.pdf
/content/Agentic_RAG_Workshop/data/Annual Reports/Microsoft_2023.pdf


## Run Document Intelligence (DI)

In [3]:
import di_utils

def run_document_intelligence(INPUT_FILE_PATH):
  """
  Analyzes a document using Azure Document Intelligence, extracts information,
  processes images within the document, and integrates image descriptions
  into the document's text representation.

  Returns:
      tuple: A tuple containing:
          - md_result: The markdown result object from Azure Document Intelligence layout analysis.
          - result_with_image_descp: A string containing the document's markdown text with image descriptions inserted.
  """
  # Step 1: Analyze Document Layout
  md_result = di_utils.analyze_document_layout(INPUT_FILE_PATH, doc_intelligence_endpoint, doc_intelligence_key)

  # Step 2: Build Span Map
  span_map = di_utils.extract_span_map(md_result)

  # Step 3: Process Images and Integrate with Text
  image_chunks = di_utils.process_images(INPUT_FILE_PATH, IMG_SAVE_DIR, md_result, llm)
  result_with_image_descp = di_utils.insert_figures_into_full_text(span_map, md_result, image_chunks)

  return md_result, result_with_image_descp

## Save the DI results as pickle file

In [4]:
import os
import pickle
from IPython.display import Markdown

def process_document_and_save_results(input_file_path, output_dir):
    """
    Processes a document using Azure Document Intelligence, extracts information,
    processes images, integrates image descriptions, and saves the results to a pickle file.

    Args:
        input_file_path (str): The path to the input document file.
        output_dir (str): The directory to save the pickle file and images.

    Returns:
        tuple: A tuple containing:
            - md_result: The markdown result object from Azure Document Intelligence layout analysis.
            - result_with_image_descp: A string containing the document's markdown text with image descriptions inserted.
    """
    md_result, result_with_image_descp = run_document_intelligence(input_file_path)

    # Derive pickle file name from input file name
    file_name = os.path.basename(input_file_path)
    pickle_file_name = os.path.splitext(file_name)[0] + "_results.pkl"
    pickle_file_path = os.path.join(output_dir, pickle_file_name)

    # Save results to a single pickle file as a dictionary
    results_dict = {
        "md_result": md_result,
        "result_with_image_descp": result_with_image_descp
    }

    with open(pickle_file_path, "wb") as f:
        pickle.dump(results_dict, f)

    return result_with_image_descp

### Processing Insurance Policy Document

In [5]:
DI_result = process_document_and_save_results(INPUT_FILE_PATHS[0], PICKLE_DIR)

Markdown(DI_result)

starting up...
entered...
	Caption: 
page numer: 1
Opening PDF file: %s /content/Agentic_RAG_Workshop/data/HR Policies/Insurance Policy.pdf
image cropped
image saved as/content/Agentic_RAG_Workshop/DI_output/images/Insurance Policy_cropped_image_0.png
entered...
	Caption: 
page numer: 3
Opening PDF file: %s /content/Agentic_RAG_Workshop/data/HR Policies/Insurance Policy.pdf
image cropped
image saved as/content/Agentic_RAG_Workshop/DI_output/images/Insurance Policy_cropped_image_1.png
entered...
	Caption: 
page numer: 3
Opening PDF file: %s /content/Agentic_RAG_Workshop/data/HR Policies/Insurance Policy.pdf
image cropped
image saved as/content/Agentic_RAG_Workshop/DI_output/images/Insurance Policy_cropped_image_2.png
entered...
	Caption: 
page numer: 3
Opening PDF file: %s /content/Agentic_RAG_Workshop/data/HR Policies/Insurance Policy.pdf
image cropped
image saved as/content/Agentic_RAG_Workshop/DI_output/images/Insurance Policy_cropped_image_3.png
entered...
	Caption: 
page numer: 6
O

<!-- PageHeader="AON" -->


# Employee Benefits Manual 2023-24 Novartis Group


<figure>
</figure>


<!-- PageFooter="Aon India Insurance Brokers Private Limited (Formerly Anviti Insurance Brokers Private Limited) CIN U66000KA2016PTC095559, Composite Insurance Broker, IRDAI Licence No.624, Valid till 15/10/2023, aon.com Registered Office - Unit 102, 1st Floor, The Estate, 121 Dickenson Road, Bengaluru 560042" -->
<!-- PageBreak -->


## Group Hospitalization Policy

The Group Medical policy covers
expenses for the insured (employee &
eligible family members) on account of
hospitalization due to sickness or
accident. The policy covers expenses
incurred on room rent, medicines, surgery
etc. Expenses for hospitalization are
payable only if a 24-hour hospitalization
has been taken. Group Mediclaim is an
annual plan effective from 1st July 2023
to 30th June 2024; terms and conditions
are subject to renewal on annual basis.

8

<!-- PageFooter="Novartis Group-Employee Benefits Manual 2022-23" -->

2

<!-- PageFooter="AON" -->
<!-- PageBreak -->


## Make The Best Use Of Your Benefits


<figure>


The process begins with enrollment, which involves planning the enrollment and following the necessary steps to obtain an E-Card. It is essential to ensure that all dependent details are accurately listed, including their names, dates of birth, gender, age, and relationship information. This information must be correctly entered into the HR CORE system, which is the Novartis HR Database, to complete the process successfully. The flow indicates a sequential process, starting from enrollment planning to the verification and entry of dependent information in the HR system.
Getting Enrolled

· Plan Enrollment &
Process for getting
your E-Card

O

Please ensure that all your
dependent details are listed, and
their corresponding information
provided Name, DOB, Gender,
Age, and relationship details etc.
are captured correctly in HR
CORE (Novartis HR Database)

</figure>


Benefit Plan Details

· Plan Information,
Benefit Details &
General Exclusions

O

Please refer Employee
Insurance Benefit Manual for
information on Coverage,
exclusions & do save
emergency contact points, this
will be very handy in case of
emergency.


<figure>


The process of making a claim involves understanding the hospitalization procedure, reviewing the claims checklist, and referring to key frequently asked questions. It is crucial to submit all required documents in their entirety at the first opportunity, ensuring that original documents are provided. Failure to submit complete documentation may result in the claim being closed. It is also important to keep copies of all documents submitted to the insurer for personal records.
Making A Claim

· Hospitalization
Procedure, Claims
Check List and Key
FAQS

O

Please ensure that your documents are
submitted entirely at the first instance
itself and originals are submitted.
Incomplete submissions may lead to
the claim getting closed. Please also
retain a copy of all claim documents
submitted to the insurer.

</figure>


<figure>

<!-- PageFooter="AON" -->

</figure>


<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="3" -->
<!-- PageBreak -->


## Your Plan Details


<table>
<tr>
<td colspan="2">Plan Name : Group Mediclaim Policy</td>
</tr>
<tr>
<td>Policy Holder</td>
<td>Novartis Healthcare Pvt Ltd, Novartis India Limited and Sandoz Pvt Ltd.</td>
</tr>
<tr>
<td>Policy period</td>
<td>01-Jul-2023 To 30-Jun-2024</td>
</tr>
<tr>
<td>Insurer</td>
<td>Aditya Birla Health Insurance Co. Ltd</td>
</tr>
<tr>
<td>TPA</td>
<td>Medi Assist India TPA Pvt. Ltd.</td>
</tr>
<tr>
<td>Sum Insured Limits</td>
<td>Uniform INR 7 Lacs</td>
</tr>
<tr>
<td>Geographical Limits</td>
<td>India</td>
</tr>
<tr>
<td>Family Definition</td>
<td>· Single Associates - Self and 2 Dependent Parents or Partner · Married Associates - Self, Spouse or Partner and All Dependent Children upto 25 years ("Partner" includes married spouse, Domestic Partner (unmarried same sex or opposite sex) If Husband and Wife both are working with Novartis Group, family definition will differ specifically for those i.e. one employee can cover spouse and other can cover parents.</td>
</tr>
<tr>
<td>Mid-Term Enrollment</td>
<td>Allowed only for New Joiner, Newly Married Spouse &amp; Newborn Baby</td>
</tr>
</table>


AON

<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="4" -->
<!-- PageBreak -->


## Your Plan Details Family Definition


<table>
<tr>
<th>Particular</th>
<th>Included</th>
<th>Special Condition if any</th>
</tr>
<tr>
<td>Employee</td>
<td>Yes</td>
<td>-</td>
</tr>
<tr>
<td>Spouse</td>
<td>Yes</td>
<td>-</td>
</tr>
<tr>
<td>Child</td>
<td>Yes</td>
<td>· All dependent children till age of 25 years · No age limit for differently abled children · Legally adopted children of single employees/ same sex LGBT employees</td>
</tr>
<tr>
<td>Parents</td>
<td>Yes</td>
<td>Only for unmarried employees Parents over the age of 85 are covered only if they were covered in previous year policy</td>
</tr>
<tr>
<td>Live -in/same sex LGBT Partner</td>
<td>Yes</td>
<td>Only for unmarried employees after due declaration submission and if parents are not covered</td>
</tr>
<tr>
<td>Parent-in-Laws</td>
<td>No</td>
<td>-</td>
</tr>
<tr>
<td>Sibling</td>
<td>No</td>
<td>-</td>
</tr>
</table>


AON

<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="5" -->
<!-- PageBreak -->


## Your Plan Details


<table>
<tr>
<td>Standard Hospitalization</td>
<td>Covered</td>
</tr>
<tr>
<td>Pre-existing Diseases</td>
<td>Covered</td>
</tr>
<tr>
<td>Restriction on Room-Rent</td>
<td>For Normal-1% of Sum insured or single private AC room whichever is higher For ICU- As per actuals</td>
</tr>
<tr>
<td>Maternity Benefits</td>
<td>Covered</td>
</tr>
<tr>
<td>Maternity Limits</td>
<td>INR 75,000 for Normal &amp; INR 1 Lac (New) for Caesarean Section to only 2 deliveries</td>
</tr>
<tr>
<td>Pre &amp; Post Natal Expenses</td>
<td>Covered up to INR 5,000 , over and above the maternity limit, to only 2 deliveries</td>
</tr>
<tr>
<td>Newborn Baby Cover</td>
<td>Covered from day one subject to intimation to HR within 15 days from date of birth</td>
</tr>
<tr>
<td>Congenital Disease</td>
<td>Internal Congenital - Covered External Congenital - Covered in Life Threatening Situations only</td>
</tr>
<tr>
<td>Dental Benefit</td>
<td>Covered only in Accident cases (Hospitalization)</td>
</tr>
<tr>
<td>Critical Illness</td>
<td>Covered on voluntary basis</td>
</tr>
<tr>
<td>Diagnostics Expenses</td>
<td>Standalone diagnostic not covered</td>
</tr>
<tr>
<td>Ambulance Services</td>
<td>Local emergency Ambulance covered up to INR 3,000/- per person per policy. Ambulance cover to hometown for carriage of mortal remains of employee, covered up to INR 100,000</td>
</tr>
<tr>
<td>Non-Medical Expense</td>
<td>In the event of an employee's death during treatment or procedure, the plan will cover the entire cost of the including non-medical charges, otherwise, the associate is typically responsible for non- medical expenses</td>
</tr>
</table>


<figure>

AON

</figure>


<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="6" -->
<!-- PageBreak -->


## Your Plan Details Policy Benefits


<table>
<tr>
<td>Pre &amp; Post Hospitalization Expenses</td>
<td>30 days prehospitalization and 60 days post hospitalization.</td>
</tr>
<tr>
<td>Claims Submission Timeframes</td>
<td>Submission within 30 days from Date of Discharge</td>
</tr>
<tr>
<td>Claims Intimation Timeframes</td>
<td>Waived off</td>
</tr>
<tr>
<td>Therapy area brief - Plaque Psoriasis, Psoriatic Arthritis &amp; Ankylosing Spondylitis</td>
<td>covered up to INR 2,00,000 per family on OPD &amp; IPD basis</td>
</tr>
<tr>
<td>Refractive error or sight correction $\left( + / - \right) 7$</td>
<td>For Myopia, Astigmatism and Hypermetropia) covered with a cap of INR 50,000/- family limit</td>
</tr>
<tr>
<td>Macular Degeneration of Retina</td>
<td>Macular Degeneration of Retina (covering specific macular Indications - wAMD, DME, Diabetic Retinopathy , Retinal Vein Occlusion (cRVO and bRVO) and Retinopathy of prematurity) covered up to 50% of the sum insured either as in patient or as part of day care treatment</td>
</tr>
<tr>
<td>Gender Dysphoria (Gender change)</td>
<td>covered within family sum insured limits</td>
</tr>
<tr>
<td>Migraine management</td>
<td>Treatment using Monoclonal Antibodies for Episodic and Chronic Migraine patients with a capped at payout of INR 125,000 per family</td>
</tr>
</table>


<figure>

<!-- PageFooter="AON" -->

</figure>


<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="7" -->
<!-- PageBreak -->


## Your Plan Details New Age Treatment covered under policy

$$N e w \quad A g e \quad T r e a t m e n t \quad o r \quad P r o c e d u r e$$


<table>
<tr>
<th></th>
<th>Limit as per (Per Policy Period)</th>
</tr>
<tr>
<td>Uterine Artery Embolization and HIFU (High intensity focused ultrasound)</td>
<td>Upto 50% of Sum Insured</td>
</tr>
<tr>
<td>Balloon Sinuplasty</td>
<td>Upto 50% of Sum Insured</td>
</tr>
<tr>
<td>Deep Brain stimulation</td>
<td>Upto 50% of Sum Insured</td>
</tr>
<tr>
<td>Oral chemotherapy</td>
<td>Upto 50% of Sum Insured</td>
</tr>
<tr>
<td>Immunotherapy- Monoclonal Antibody to be given as injection</td>
<td>Upto 50% of Sum Insured</td>
</tr>
<tr>
<td>Robotic surgery</td>
<td>Upto 50% of Sum Insured</td>
</tr>
<tr>
<td>Stereotactic radio surgery</td>
<td>Upto 50% of Sum Insured</td>
</tr>
<tr>
<td>Bronchial Thermoplasty</td>
<td>Upto 50% of Sum Insured</td>
</tr>
<tr>
<td>Vaporisation of the prostrate (Green laser treatment or holmium laser treatment)</td>
<td>Upto 50% of Sum Insured</td>
</tr>
<tr>
<td>IONM - (Intra Operative Neuro Monitoring)</td>
<td>Upto 50% of Sum Insured</td>
</tr>
<tr>
<td>Stem cell therapy: Hematopoietic stem cells for bone marrow transplant for haematological conditions to be covered</td>
<td>Upto 50% of Sum Insured</td>
</tr>
</table>


<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="8" -->
<!-- PageBreak -->


## Your Plan Details What Is Covered?

If any Insured Person suffers an Illness or Accident during the Policy Period that requires Insured Person's hospitalization as an inpatient,
then the insurer will reimburse reasonable and customary expenses towards the below mentioned hospitalization under your group
medical plan.

· Inpatient Treatment

· Room rent and boarding expenses

· Doctor's fees ( who needs to be a medical practitioner)

· Intensive Care Unit

· Nursing expenses, Anesthesia, blood, oxygen, operation theatre charges, surgical appliances,

· Medicines, drugs and consumables (Dressing, ordinary splints and plaster casts)

· Diagnostic procedures (such as laboratory, x-ray, diagnostic tests)

· Costs of prosthetic devices if implanted internally during a surgical procedure

· Organ transplantation including the treatment costs of the donor but excluding the costs of the organ

The expenses shall be reimbursed provided they are incurred in India and are within the policy period. Expenses will be reimbursed
to the covered member depending on the level of cover that he/she is entitled to. Expenses that are of a diagnostic nature only or
are incurred from a preventive perspective with no active line of treatment and do not warrant a hospitalization admission are not
covered under the plan.

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="9" -->
<!-- PageBreak -->


## Your Plan Details Pre & Post Hospitalization Expenses


<table>
<tr>
<td colspan="2">Pre-hospitalization expenses</td>
</tr>
<tr>
<td>Pre-hospitalization Expenses</td>
<td>If the Insured Person is diagnosed with an Illness which results in his or her Hospitalization and for which the Insurer accepts a claim, the Insurer will reimburse the Insured Person's Pre-hospitalization Expenses for up to 30 days prior to his Hospitalization as long as the 30-day period commences and ends within the Policy Period.</td>
</tr>
<tr>
<td>Duration</td>
<td>Within 30 days before hospitalization</td>
</tr>
</table>


<table>
<tr>
<td colspan="2">Post hospitalization expenses</td>
</tr>
<tr>
<td>Post-hospitalization Expenses</td>
<td>If the Insurer accepts a claim above and, immediately following the Insured Person's discharge, he requires further medical treatment directly related to the same condition for which the Insured Person was Hospitalized, the Insurer will reimburse the Insured Person's Post-hospitalization Expenses</td>
</tr>
<tr>
<td>Duration</td>
<td>Within 60 days post discharge</td>
</tr>
</table>


<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="10" -->
<!-- PageBreak -->


## Your Plan Details Maternity Benefits

· Maternity benefits are admissible only if the expenses are incurred in Hospital / Nursing Home as in-patients in India.

· Those Insured Persons who already have two or more living children will not be eligible for this benefit.

· Expenses incurred in connection with voluntary medical termination of pregnancy during the first 12 weeks from the date
of conception are not covered. Infertility Treatment and sterilization are excluded from the policy.


### The maternity benefit under your Group Medical Plan


<table>
<tr>
<td>Maximum Benefit</td>
<td>INR 75,000 for Normal &amp; INR 1 Lac (New) for C - Section within Sum Insured Limit</td>
</tr>
<tr>
<td>9-months waiting period</td>
<td>Waived off</td>
</tr>
<tr>
<td>Pre-Post Natal expenses</td>
<td>Covered up to INR 5,000 over and above the maternity limit</td>
</tr>
<tr>
<td>Newborn baby covered from day 1</td>
<td>Covered from day 1, subject to intimation to HR within 15 days from date of birth</td>
</tr>
</table>


IMPORTANT :

For maternity reimbursements and employees on subsequent maternity leave, please do not wait till you have returned
back to office to submit a claim as it will cross the claim submission within 30 days to avoid denial of claim. Please also
immediately inform your HR about the new baby coverage as your dependent as A subsequent complication may be A
possibility and intimation is mandatory prior to coverage


<figure>

<!-- PageFooter="AON" -->

</figure>


<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="11" -->
<!-- PageBreak -->


## Your Plan Details


<table>
<tr>
<th>Policy Benefit</th>
<th>Definition</th>
<th>Covered/Not Covered</th>
</tr>
<tr>
<td>Pre-existing Diseases</td>
<td>Any Pre-Existing Condition or related condition for which care, treatment or advice was recommended by or received from a Doctor or which was first manifested prior to the commencement date of the Insured Person's first Health Insurance policy with the Insurer</td>
<td>Covered</td>
</tr>
<tr>
<td>First 30 days' waiting period</td>
<td>Any Illness diagnosed or diagnosable within 30 days of the effective date of the Policy Period if this is the first Health Policy taken by the Policyholder with the Insurer.</td>
<td>Covered</td>
</tr>
<tr>
<td>First Year Waiting Period</td>
<td>During the first year of the operation of the policy the expenses on treatment of diseases such as Cataract, Benign Prostatic Hypertrophy, Hysterectomy for Menorrhagia or Fibromyoma, Hernia, Hydroceie, Congenital Internal Diseases, Fistula in anus, Piles, Sinusitis and related disorders are not payable. If these diseases are pre- existing at the time of proposal, they will not be covered even during subsequent period or renewal too</td>
<td>Waived off</td>
</tr>
<tr>
<td>Day Care</td>
<td>Day Care Procedure means the course of medical treatment, or a surgical procedure listed in the Schedule which is undertaken under general or local anesthesia in a Hospital by a Doctor in not less than 2 hours and not more than 24 hours.</td>
<td>Covered</td>
</tr>
<tr>
<td>Diagnostic Expenses</td>
<td>All diagnostic tests and lab tests as part of hospitalization and pre-post hospitalization including OPD. Diagnostic tests without treatment or not related to treatment are not covered</td>
<td>Covered and Only incase of 24-hr hospitalization related to treatment</td>
</tr>
</table>


AON

<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="12" -->
<!-- PageBreak -->


## Your Plan Details


### How to Enroll?


<table>
<tr>
<th>Particular</th>
<th>Description</th>
<th>Special Condition if any</th>
</tr>
<tr>
<td>Enrollment of existing employees' and their dependents</td>
<td>Enrollment/Validation</td>
<td>Employee to either declare new dependents or validate current dependent details at the time of renewal every year</td>
</tr>
<tr>
<td>Enrollment of New Joinees (New Employees + their Dependents) during the policy year</td>
<td>Declaration</td>
<td>Dependent information needs to be updated with 30 days from date of joining on HR system</td>
</tr>
</table>


### Mid Term Enrollment Eligibility?


<table>
<tr>
<th>Particular</th>
<th>Description</th>
<th>Special Condition if any</th>
</tr>
<tr>
<td>Mid-Term Enrollment of Existing employees' Dependents(as on plan start date)</td>
<td>Not Allowed</td>
<td></td>
</tr>
<tr>
<td>Mid-Term Enrollment of New Joinees (New Employees +Their Dependents)</td>
<td>Allowed *</td>
<td></td>
</tr>
<tr>
<td>Mid-Term Enrollment of New Dependents (Spouse/Children)</td>
<td>Allowed *</td>
<td>Midterm enrollment of new dependents (Spouse / Children) is allowed for employees within 30 days from Date of Marriage/ Date of Birth.</td>
</tr>
<tr>
<td>Mid term inclusion of Parents</td>
<td>Allowed</td>
<td>Only in case of legal divorce, death of spouse and if no children are enrolled in the policy, required documents have to be submitted to respective HR spocs</td>
</tr>
</table>


<figure>

AON

</figure>


<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="13" -->
<!-- PageBreak -->


## Getting Enrolled Process for Domestic Partner Enrolment

· DOMESTIC PARTNERSHIP* means a committed relationship between two adults (same or opposite sex),
that meets all of the requirements as per the attached declaration form

· Dependent Children: To qualify as a dependent child, the child must be legally adopted child of an
employee or his/her domestic partner should be a legal parent of child(ren), legal document to be
submitted as a part of the declaration process

· Please email the following below mentioned details to your respective HR SPOCs with duly filled and
signed Declaration Form

A. Your Employee Code

B. Full Name of the Partner

C. DOB of the Partner

D. Gender of the Partner


<figure>

PDF

Domestic
Declaration Form

</figure>


Please Note:

If you opt for the above partner enrolment, then in line with
the eligibility guidelines your parents shall no longer be
eligible for cover under the medical insurance plan and will
be removed from coverage .

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="14" -->
<!-- PageBreak -->


## Health E- Cards Process of Download E-Health Id Cards


<figure>

Medi Assist

</figure>


Test000

«Home

EMenu

[+ Sign out

· Enter URL in address bar of
your browser :
https://portal.medibuddy.in/lo
gin.aspx

. Sign in with Username and
Password as received via mail
from

. Click on the menu, located at
top right corner

. Click on "E Card" to download
pdf of your E 
A comprehensive health benefits dashboard is organized into several functional sections, each representing a key aspect of employee health coverage and support services. The main categories include claims management, wellness resources, pharmacy access, health information updates, and hospital search functionality. There is a dedicated area for understanding health cover details, including access to the employee policy, enrollment options, and the ability to download electronic cards. Profile management is available, along with options for cashless services, reimbursement initiation, and claim submission. Contact information is clearly provided, including a helpline number and an email contact option. Each section is distinctly labeled, ensuring users can easily navigate to the desired service or information, supporting efficient management of health benefits and related actions.
cards.


<figure>

Wellness

Your claims

Pharmacy

Health bites

Search hospitals

Know
your
health
cover

Enrolment

Employee policy

Download E-cards

Contact us

eCashless

1800-425-9449

Write to us

Your profile

Intimate
Reimbursement

Submit claim

</figure>


<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="15" -->
<!-- PageBreak -->


## Voluntary Critical Illness - Plan Details Policy Benefits


<table>
<tr>
<td>Cover Definition</td>
<td>A policy that pays a lump sum amount equal to the sum insured upon first diagnosis of $a$ critical illness covered under the policy. In this case, 36 Critical Illnesses are covered</td>
</tr>
<tr>
<td>Sum Insured</td>
<td>INR 10 Lakhs</td>
</tr>
<tr>
<td>Cover Type</td>
<td>Individual cover of INR 10 Lakhs</td>
</tr>
<tr>
<td>Family Definition</td>
<td>Employee, Spouse &amp; Dependent Children (min 5 years age)</td>
</tr>
<tr>
<td>Waiting Period</td>
<td>90 Days</td>
</tr>
<tr>
<td>Survival Period</td>
<td>30 days</td>
</tr>
<tr>
<td>Pre-Existing Diseases</td>
<td>Not covered</td>
</tr>
<tr>
<td>Number of Critical Illness cover</td>
<td>36 Critical Illness covered (separate slide of covers in detail)</td>
</tr>
</table>


<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="16" -->
<!-- PageBreak -->


## Critical Illness (1/2) List of New 36 Critical Illness


<table>
<tr>
<th>Cancer of specific severity</th>
<th>Multiple Sclerosis with Persisting Symptoms</th>
</tr>
<tr>
<td>Myocardial Infarction</td>
<td>Coma of Specified Severity</td>
</tr>
<tr>
<td>Open Chest CABG</td>
<td>Motor Neurone Disease with Permanent Symptoms</td>
</tr>
<tr>
<td>Open Heart Replacement or Repair of Heart Valves</td>
<td>Loss of Vision (Blindness)</td>
</tr>
<tr>
<td>Kidney Failure Requiring Regular Dialysis</td>
<td>Major burns</td>
</tr>
<tr>
<td>Stroke Resulting in Permanent Symptoms</td>
<td>Parkinson's Disease</td>
</tr>
<tr>
<td>Major Organ / Bone Marrow Transplant</td>
<td>Benign Brain Tumor</td>
</tr>
<tr>
<td>Permanent Paralysis of Limbs</td>
<td>Alzheimer's Disease</td>
</tr>
<tr>
<td>Bacterial Meningitis</td>
<td>Apallic Syndrome or Persistent Vegetative State (PVS)</td>
</tr>
</table>


<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="17" -->
<!-- PageBreak -->


## Critical Illness (2/2) List of New 36 Critical Illness


<table>
<tr>
<th>Aorta Graft Surgery</th>
<th>Encephalitis</th>
</tr>
<tr>
<td>Loss of Hearing (Deafness)</td>
<td>Fulminant Hepatitis</td>
</tr>
<tr>
<td>Loss of Limbs</td>
<td>Chronic Relapsing Pancreatitis</td>
</tr>
<tr>
<td>Loss of Speech</td>
<td>Major Head Trauma</td>
</tr>
<tr>
<td>Aplastic Anaemia</td>
<td>Medullary Cystic Disease</td>
</tr>
<tr>
<td>End Stage Liver Failure</td>
<td>Muscular Dystrophy</td>
</tr>
<tr>
<td>End Stage Lung Failure</td>
<td>Poliomyelitis</td>
</tr>
<tr>
<td>Primary (Idiopathic) Pulmonary Hypertension</td>
<td>Systemic Lupus Erythematous</td>
</tr>
<tr>
<td>Coronary Angioplasty (PTCA)[1]</td>
<td>Brain Surgery</td>
</tr>
</table>


AON

<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="18" -->
<!-- PageBreak -->


## Critical Illness General Exclusions


<table>
<tr>
<td>1</td>
<td>We shall not be liable to make any payment in respect of any Critical Illness whose signs or symptoms first occur within the first 90 days from the Inception Date</td>
</tr>
<tr>
<td>2</td>
<td>Any claim with respect to any Critical Illness diagnosed or which manifested prior to the Inception Date</td>
</tr>
<tr>
<td>3</td>
<td>Any Pre-Existing Disease or any complication arising therefrom</td>
</tr>
<tr>
<td>4</td>
<td>Any Critical Illness arising out of use, abuse or consequence or influence of any substance, intoxicant, drug, alcohol or hallucinogen</td>
</tr>
<tr>
<td>5</td>
<td>Any Critical Illness directly or indirectly caused due to intentional self-injury, suicide or attempted suicide; whether the person is medically sane or insane</td>
</tr>
<tr>
<td>6</td>
<td>Working in underground mines, tunneling or involving electrical installations with high tension supply, or as jockeys or circus personnel</td>
</tr>
<tr>
<td>7</td>
<td>Any loss resulting directly or indirectly contributed or aggravated or prolonged by childbirth or from pregnancy</td>
</tr>
<tr>
<td>8</td>
<td>Any Critical Illness arising or resulting from the Insured Person committing any breach of law or participating in an actual or attempted felony, riot, crime, misdemeanor or civil commotion with criminal intent</td>
</tr>
</table>


AON

<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="19" -->
<!-- PageBreak -->


## Voluntary Parent's Policy- Plan Details


<table>
<tr>
<td colspan="2">Plan Name : Voluntary Parent's Mediclaim Policy</td>
</tr>
<tr>
<td>Policy Holder</td>
<td>Novartis Healthcare Pvt Ltd, Novartis India Limited and Sandoz Pvt Ltd.</td>
</tr>
<tr>
<td>Policy period</td>
<td>01-Jul-2023 To 30-Jun-2024</td>
</tr>
<tr>
<td>Insurer</td>
<td>Aditya Birla Health Insurance Co. Ltd</td>
</tr>
<tr>
<td>TPA</td>
<td>Medi Assist India TPA Pvt. Ltd.</td>
</tr>
<tr>
<td>Sum Insured Limits</td>
<td>INR 3 Lacs &amp; INR 5 Lacs</td>
</tr>
<tr>
<td>Geographical Limits</td>
<td>India</td>
</tr>
<tr>
<td>Family Definition</td>
<td>. Parents or Parent In Laws . Cross combination not allowed Parents here refers the Primary Insured's natural parents or parents that have legally adopted him/her</td>
</tr>
<tr>
<td>Entry Age for Parents</td>
<td>For Parents/ in laws- Entry age restricted upto 85 years</td>
</tr>
</table>


<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="20" -->
<!-- PageBreak -->


## Voluntary Parent's Policy - Plan Details (12)


<table>
<tr>
<td>Room Rent Restriction</td>
<td>2% of the in-patient Sum Insured, if admitted to regular room. All benefits as an inpatient in a hospital attached to room will be restricted to the room, which falls within the room rent limits allowed. The enhanced difference in expenses due to opting rooms with higher room rent than what has been allowed will be borne by the insured only. Wherever the room rent based tariff for the other expenses is not available, the payment would be done in the same proportion as per the entitlement of room rent under the policy excluding medicines, consumables and implants medically prescribed by the treating doctor under the policy. In case of package treatment where individual bifurcation of room rent, medicines, operation theatre expenses, doctor's consultation charges etc. are not available, then the package charges shall be proportionately linked to the entitled room rent of the insured person under the policy.</td>
</tr>
<tr>
<td>Pre &amp; Post Hospitalization expenses</td>
<td>30 Days Pre-Hospitalization &amp; 60 days Post-Hospitalization (to be claimed as reimbursements)</td>
</tr>
<tr>
<td>Road Ambulance Charges</td>
<td>Covered upto INR 2500 per case in case of emergency only. Ambulance charges will be applicable for transferring patient to Hospital or between Hospitals in the Hospitals ambulance or in an ambulance provided by any ambulance service provider only</td>
</tr>
<tr>
<td>Internal Congenital</td>
<td>Covered</td>
</tr>
<tr>
<td>Cataract Surgery</td>
<td>Mono and/or Multifocal Lens covered</td>
</tr>
<tr>
<td>Survival Period</td>
<td>30 days</td>
</tr>
<tr>
<td>Co-Pay</td>
<td>10% Co-Pay on each and every claim</td>
</tr>
</table>


<figure>

AON

</figure>


<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="21" -->
<!-- PageBreak -->


## Voluntary Parent's Policy - Plan Details (2/2)


<table>
<tr>
<td>Special Conditions</td>
<td>Pandemic and Dental and Vision in cases of accidents covered, only in case of accidents. Refractive error or sight correction $\left( + / - \right) 7$ and macular degeneration of retina- on IPD and OPD basis, Final decision will be taken on receipt of complete set of documents. 50% Co-Pay for cyber-knife treatment, Gamma Knife treatment and Stem Cell Transplantation, Robotic Surgery, Femto laser treatment for eye. It will be applicable for each eye each event. Cochlear Implant treatment shall be restricted to 50% of the SI. Coverage under this benefit is subject to the procedure being authority-approved indication for the medical condition, and should not be experimental and unproven treatment as on date</td>
</tr>
<tr>
<td>Pre-Existing exclusions/1st year exclusions/30 days waiting period</td>
<td>Waived off</td>
</tr>
<tr>
<td>Claims Submission Timeframes</td>
<td>Submission within 30 days from Date of Discharge</td>
</tr>
<tr>
<td>Claims Intimation Timeframes</td>
<td>Waived off</td>
</tr>
<tr>
<td>Terrorism</td>
<td>Covered</td>
</tr>
</table>


<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="22" -->
<!-- PageBreak -->


## Know Your Room Rent Limit Proportionate Deduction Clause

· Insured employees are requested to use prudence and proper negotiation with Hospital/ Nursing home in availing
the eligible room category.

. Please remember, higher the room category higher is the cost of treatment. This may result in faster exhaustion
of your total available eligibility

· Employee opting for a higher room category will have to bear the proportionate increase in cost on all categories /
heads.

. For example: if an employee opts for room of INR 10,000 per day , 50% deduction shall be applicable for overall
bill except for the cost of medicines.

· Any expenses like nursing / electricity / monitor charges / etc. charged separately will be considered under room
rent cost and will not be payable if charged separately.

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="23" -->
<!-- PageBreak -->


## Your Plan Details(1/3) General Exclusions

· Injury or disease directly or indirectly caused by or arising from or attributable to War, Invasion, Act of Foreign Enemy, War like
operations (whether war be declared or not) or by nuclear weapons / materials.

· Circumcision (unless necessary for treatment of a disease not excluded hereunder or as may be necessitated due to any accident),
vaccination, inoculation or change of life or cosmetic or of aesthetic treatment of any description, plastic surgery other than as may
be necessitated due to an accident or as a part of any illness.

· Surgery for correction of eye- sight, cost of spectacles, contact lenses, hearing aids etc.

· Cosmetic or of aesthetic procedure, filling of cavity, root canal including wear and tear etc unless arising from disease or injury and
which requires hospitalization for treatment.

· Congenital external diseases or defects/anomalies

· Convalescence, general debility, "run down" condition or rest cure, congenital external diseases or defects or anomalies, sterility,
any fertility, sub-fertility or assisted conception procedure, venereal diseases, intentional self-injury/suicide, all psychiatric and
psychosomatic disorders and diseases / accident due to and or use, misuse or abuse of drugs / alcohol or use of intoxicating
substances or such abuse or addiction etc.

. Any cosmetic or plastic surgery except for correction of injury

· Expenses incurred at Hospital or Nursing Home primarily for evaluation / diagnostic purposes which is not followed by active
treatment for the ailment during the hospitalized period.

· Expenses on vitamins and tonics etc unless forming part of treatment for injury or disease as certified by the attending physician.

· Any Treatment arising from or traceable to pregnancy, miscarriage, abortion or complications of any of these including changes in
chronic condition as a result of pregnancy except where covered under the maternity section of benefits.

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="24" -->
<!-- PageBreak -->


## Your Plan Details(2/3) General Exclusions

· Doctor's home visit charges, Attendant / Nursing charges during pre and post Hospitalization period.

· Treatment which is continued before hospitalization and continued even after discharge for an ailment / disease / injury different
from the one for which hospitalization was necessary.

· Naturopathy treatment, unproven procedure or treatment, experimental or alternative medicine and related treatment including
acupressure, acupuncture, magnetic and such other therapies etc.

· External and or durable Medical / Non-medical equipment of any kind used for diagnosis and or treatment including CPAP, CAPD,
Infusion pump etc., Ambulatory devices i.e. walker , Crutches, Belts , Collars , Caps , splints, slings, braces , Stockings etc of any
kind, Diabetic footwear, Glucometer / Thermometer and similar related items etc and also any medical equipment which is
subsequently used at home etc ..

. All non-medical expenses including Personal comfort and convenience items or services such as telephone, television, Aya /
barber or beauty services, diet charges, baby food, cosmetics, napkins, toiletry items etc, guest services and similar incidental
expenses or services etc ..

· Change of treatment from one pathy to other pathy unless being agreed / allowed and recommended by the consultant under whom
the treatment is taken.

· Treatment of obesity or condition arising therefrom (including morbid obesity) and any other weight control programme, services or
supplies etc ..

· Any treatment required arising from Insured's participation in any hazardous activity including but not limited to scuba diving, motor
racing, parachuting, hang gliding, rock or mountain climbing etc unless specifically agreed by the Insurance Company.

. Any treatment received in convalescent home, convalescent hospital, health hydro, nature care clinic or similar establishments.

· Any stay in the hospital for any domestic reason or where no active regular treatment is given by the specialist.

· Massages, Steam bathing, Shirodhara not covered.

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="25" -->
<!-- PageBreak -->


## Your Plan Details(3/3) General Exclusions

. Any kind of Service charges, Surcharges, Admission fees / Registration charges etc levied by the hospital.

. Non-prescribed drugs and medical supplies, Hormone replacement therapy

· Expenses incurred for investigation or treatment irrelevant to the diseases diagnosed during Hospitalization or primary reasons for
admission. Private nursing charges, Referral fee to family doctors, Out station consultants / Surgeon fees etc,.

· Vitamins and tonics unless used for treatment of injury or disease

· Infertility treatment, Intentional self Injury, Outpatient treatment.

· Family planning Operations (Vasectomy or tubectomy) etc

. All expenses arising out of any condition directly or indirectly caused by or associated with Human T-cell Lymphotropic Virus Type III
(HTLD - III) or Lymohadinopathy Associated Virus (LAV) or the Mutants Derivative or Variations Deficiency Syndrome or any
Syndrome or condition of similar kind commonly referred to as AIDS, HIV and its complications including sexually transmitted
diseases.

. External and or durable Medical / Non-medical equipment of any kind used for diagnosis and or treatment like Prosthetics etc.

. Any device/instrument/machine that does not become part of the human anatomy/body but would contribute/replace the function
of an organ is not covered.

· Warranted that treatments on trial/experimental basis are not covered under scope of the policy.

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="26" -->
<!-- PageBreak -->


## Getting Enrolled The Procedure : What Must You Remember ?

. Employees have to provide all the details of dependents in the prescribed format provided in the joining docket for Mediclaim
coverage. Dependents once declared cannot be changed during the policy period.

· Existing Employees are covered as on date of policy commencement (or date of joining for new employees joining after 1st July 2022)
along with their eligible dependents as per data provided by HR to Insurance Company.

. No midterm inclusion of dependents would be allowed except in case of spouse due to marriage of an employee and birth of child.

· Midterm enrollment of new dependents (Spouse / Children) is allowed for employees within 15 days from Date of Marriage/ Date of
Birth. The details need to be provided to HR within 15 days from date of event.

. Eligible Dependent covered under the policy for existing employees can be viewed on the TPA website.

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="27" -->
<!-- PageBreak -->


## Making A Claim Hospitalization Procedure

You can avail either cashless facility or submit the claim for reimbursement.


### Definition of Cashless

· Cashless hospitalization means the TPA may authorize (upon an Insured person's request) for direct settlement of eligible services
and the corresponding charges between a Standard Network / PPN Network Hospital and the TPA. In such case, the TPA will
directly settle all eligible amounts with the Network Hospital and the Insured Person may not have to pay any deposits at the
commencement of the treatment or bills after the end of treatment to the extent these services are covered under the Policy. Denial
of cashless does not mean that the treatment is not covered by the policy.


### Definition of Reimbursement

. In case you choose a non-network hospital, you will have to liaise directly with the hospital for admission. However, you are advised
to follow the preauthorization procedure and intimate the TPA about the claim to ensure eligibility for reimbursement of
hospitalization expenses from the insurer.

· To know about cashless or reimbursement, please visit the desired section mentioned below:

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="28" -->
<!-- PageBreak -->


## Making A Claim Process for Cashless

Cashless hospitalization means the Administrator may authorizes upon a Policyholder's request for direct settlement of
eligible services and its according charges between a Network Hospital and the Administrator. In such case the
Administrator will directly settle all eligible amounts with the Network Hospital and the Insured Person may not have to
pay any deposits at the commencement of the treatment or bills after the end of treatment to the extent as these
services are covered under the Policy.


<table>
<tr>
<td colspan="2">List of hospitals in the TPA's network eligible for cashless hospitalization</td>
</tr>
<tr>
<td>Hospital Network List</td>
<td></td>
</tr>
<tr>
<td>For Network Hospital List Click</td>
<td>Essential email ids and contact numbers:</td>
</tr>
<tr>
<td>https://www.medibuddy.in/networkHospitals</td>
<td></td>
</tr>
<tr>
<td>Note : Employee can always download the updated network list from TPA website on real time basis.</td>
<td rowspan="2">24x7 Toll Free Number for Claim Intimation: 040 6817 2735 Claim Intimation E-mail: Novartis@mediassist.in</td>
</tr>
<tr>
<td>For TPA Website Click:</td>
</tr>
<tr>
<td>https://portal.medibuddy.in/login.aspx</td>
<td></td>
</tr>
</table>


<figure>
</figure>



A list of resources and contact information is provided for employees seeking cashless hospitalization through the TPA’s network. Employees can access the network hospital list by visiting a specified online link, and are informed that the list is updated in real time and can be downloaded from the TPA website. Another link is provided for direct access to the TPA website. For claim intimation, a 24x7 toll-free phone number is available, along with a dedicated email address for claim notifications. All essential contact details, including web links, phone number, and email, are clearly specified to facilitate easy access to information and support.
Please reach out to MediAssist TPA to check the admissibility of any claim in advance to avoid any ambiguity

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="29" -->
<!-- PageBreak -->


## Medi Assist TPA point of contacts

For any support in Cashless claims, Reimbursement claims, E-cards, network hospital support, cover details,
please contact Medi Assist TPA


<table>
<tr>
<th>Hyderabad</th>
<th>Mumbai</th>
</tr>
<tr>
<td>Helpline no - 040-68172735</td>
<td>Helpline no - 040-68213616</td>
</tr>
<tr>
<td>E-Mail Id - novartis@mediassist.in</td>
<td>E-Mail Id - novartis@mediassist.in</td>
</tr>
<tr>
<td>1st Level - K Raj Reddy - kancharla.reddy@mediassist.in</td>
<td>1st Level - Ramesh Baswal - ramesh.baswal@mediassist.in</td>
</tr>
<tr>
<td>9731847099</td>
<td>9515121283</td>
</tr>
<tr>
<td>2nd Level - Achanta Hari Tulasi - achanta.tulasi@mediassist.in</td>
<td>2nd Level - Achanta Hari Tulasi - achanta.tulasi@mediassist.in</td>
</tr>
<tr>
<td>9620856349</td>
<td>9620856349</td>
</tr>
<tr>
<td>3rd Level - Devendar Rao CH - devendar.rao@mediassist.in 6366420429</td>
<td>3rd Level - Devendar Rao CH - devendar.rao@mediassist.in 6366420429</td>
</tr>
</table>


Please reach out to MediAssist TPA to check the admissibility of any claim in advance to avoid any ambiguity

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="30" -->
<!-- PageBreak -->


## Making A Claim Group Medical- Cashless Hospitalization


<figure>


Planned hospitalization requires the patient to approach the hospital at least 48 hours before admission, present a TPA card along with a government ID, and complete pre-authorization formalities. The hospital then sends a pre-authorization letter to the TPA for approval. If all documents are in order, the TPA issues an authorization letter to the hospital within three hours. If the case is declined, a denial letter is issued to the hospital, with a note clarifying that denial of cashless service does not equate to denial of treatment or claim. If additional information is needed, the TPA will notify the hospital or employee. This process ensures a structured approach to planned hospital admissions, clarifies the steps for both approval and denial, and highlights the communication protocol for additional requirements.
Planned
Hospitalization

Approach hospital minimum 48 hrs. prior
to admission , produces TPA card with
Govt ID and completes pre-authorization
formalities

Faxes Pre-Authorization letter to TPA for
Approval

If all the documents are in order, TPA will
issue authorization letter to hospital
within 3 hours

If the case is Declined, Denial Letter will
be issued to hospital

(denial of cashless does not mean denial
of treatment or claim)

Incase additional information is required,
TPA will inform the Hospital / Employee

</figure>


<figure>


Emergency hospitalization involves a sequence of steps starting with admission to the hospital. Within 24 hours, pre-authorization formalities must be completed and sent to the Third Party Administrator (TPA) for approval. If all required documents are in order, the TPA issues an authorization letter to the hospital within three hours. If the case is declined, a denial letter is issued, clarifying that denial of cashless service does not equate to denial of treatment or claim. If additional information is needed, the TPA will notify the hospital or employee accordingly. This process ensures timely communication and clarity regarding the approval or denial of cashless hospitalization services.
Emergency
Hospitalization

Admission in Hospital

Pre-Authorization formalities to be
completed within 24 hours and sent to
TPA for Approval

If all the documents are in order, TPA
will issue authorization letter to hospital
within 3 hours

If the case is Declined, Denial Letter will
be issued

(denial of cashless does not mean
denial of treatment or claim)

Incase additional information is
required, TPA will inform the Hospital /
Employee

</figure>


For Cashless Network Hospital List Click

<!-- PageFooter="AON" -->
<!-- PageFooter="https://www.medibuddy.in/networkHospitals" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="31" -->
<!-- PageBreak -->


## Making A Claim Group Medical- Reimbursement 
The process for claiming insurance after treatment at a non-network hospital begins with the insured seeking treatment at such a hospital. After treatment, the insured is discharged and pays for the treatment out of pocket. The insured then collects all original documents, receipts, and investigation reports from the hospital. All these original hospital documents, along with a filled claim form, must be submitted to the Third Party Administrator (TPA) within 21 days from the date of discharge.

Once the TPA receives the claim documents, they acknowledge receipt via email and start the claim process. If additional information is required, the TPA will inform the employee via email and send reminders. If the required documents are not submitted within 30 days, the claim may be closed.

If the claim is found to be payable, payment will be made to the employee via NEFT. If the claim is declined, a denial email will be sent to the employee.

This process ensures that all necessary documentation and timelines are followed for claim assessment, with clear communication at each step and defined outcomes based on the claim's status.
Claim


<figure>

Insured visits non network
hospital for treatment

Takes discharge, pays for
treatment

Collects all original
documents, receipts and
investigation reports from
Hospital

Submits all original
Hospital documents along
with filled claim form
within 21 days from date
of discharge to TPA

TPA acknowledges
receipt of claim
documents via email and
commences claim process

information is required, TPA
will inform the employee via
email with reminders, If
documents are not
submitted within 30 days,
claim may be closed/

If claim is payable,
payment will be made to
employee via NEFT

If claim is declined, denial
mail will be sent.

</figure>


<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="32" -->
<!-- PageBreak -->


## Making A Claim Claims Document Check List & Attachments


<table>
<tr>
<th>Sr. No.</th>
<th>Document Required (All in ORIGINAL)</th>
</tr>
<tr>
<td>1</td>
<td>Claim intimation is mandatory within 7 days from DOA and before DOD</td>
</tr>
<tr>
<td>2</td>
<td>Main Hospital bills in original (Original Hospital Payment Receipt with serial number, with bill no; signed and stamped by the hospital) &amp; itemized bills.</td>
</tr>
<tr>
<td>3</td>
<td>Discharge Card/Summary (original).</td>
</tr>
<tr>
<td>4</td>
<td>Attending doctors' bills and receipts and certificate regarding diagnosis (if separate from hospital bill) Indoor case papers</td>
</tr>
<tr>
<td>5</td>
<td>Original investigation reports or attested copies of Bills &amp; Receipts for Medicines, Investigations along with Doctors prescription in Original &amp; Laboratory</td>
</tr>
<tr>
<td>6</td>
<td>Follow-up advice or letter for line of treatment after discharge from hospital, from Doctor.</td>
</tr>
<tr>
<td>7</td>
<td>Break up with details of Pharmacy items, Materials, Investigations even though it is there in the main bill</td>
</tr>
<tr>
<td>8</td>
<td>In non- network hospitalization, please get the hospital and doctor's registration number in Hospital letterhead and get the same signed and stamped by the hospital.</td>
</tr>
<tr>
<td>9</td>
<td>In case of accidents, please note FIR or MLC (medico legal certificate) is mandatory. Original Death Summary: In case of Death Claims</td>
</tr>
<tr>
<td>10</td>
<td>One Cancelled Cheque with Employee Name printed to settle the claim to Employee Bank account.</td>
</tr>
</table>


<figure>

<!-- PageFooter="AON" -->

</figure>


<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="33" -->
<!-- PageBreak -->


## Making A Claim Group Medical- Important Video Links- Medi Assist


<table>
<tr>
<th>Topic</th>
<th>Link</th>
</tr>
<tr>
<td rowspan="2">Hassle-free healthcare</td>
<td>https://www.youtube.com/watch?v=Fth</td>
</tr>
<tr>
<td>U5JU1QTU</td>
</tr>
<tr>
<td rowspan="2">One app for all your healthcare needs</td>
<td>https://www.youtube.com/watch?v=lhi</td>
</tr>
<tr>
<td>WAeAdY54</td>
</tr>
<tr>
<td>E-cashless to beat the que</td>
<td>https://youtu.be/urNNfFva864</td>
</tr>
<tr>
<td>Self-help anytime anywhere</td>
<td>https://youtu.be/QUowKC8jb3M</td>
</tr>
<tr>
<td>Paperless mobile on the go</td>
<td>https://youtu.be/l78q44WlDJY</td>
</tr>
<tr>
<td>Health benefits info at your fingertips</td>
<td>https://youtu.be/Cb0gpM3alDU</td>
</tr>
<tr>
<td rowspan="2">E-Cashless</td>
<td>https://youtu.be/3dgifO8QpWE</td>
</tr>
<tr>
<td></td>
</tr>
<tr>
<td>Network hospital search</td>
<td>https://youtu.be/U XPR7LYNvo</td>
</tr>
</table>


<table>
<tr>
<th>Topic</th>
<th>Link</th>
</tr>
<tr>
<td>MediBuddy Chatbot</td>
<td>https://youtu.be/0wfV5KfiAoQ</td>
</tr>
<tr>
<td>Real time Claims Tracking</td>
<td>https://youtu.be/U4ocgz0QnLk</td>
</tr>
<tr>
<td>Network hospital search with #MediBuddy #SelfHelp #Healthcare #Wellness #HealthTech</td>
<td>https://youtu.be/GbbZA964_zg</td>
</tr>
<tr>
<td>Online reimbursement claims submission</td>
<td>https://youtu.be/nycXXW2m56I</td>
</tr>
<tr>
<td>Pharmacy</td>
<td>https://youtu.be/kReYoxxJJH0</td>
</tr>
<tr>
<td>Ecard</td>
<td>https://youtu.be/OX2N7HCkr00</td>
</tr>
<tr>
<td>Maternity with #MediBuddy #MaternityProgram</td>
<td>https://youtu.be/651LEr1ItM0</td>
</tr>
</table>


AON

<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="34" -->
<!-- PageBreak -->


## Making A Claim Group Medical- Important FAQ

What are network hospitals? What should I do when I reach the hospital (NETWORK)?

. These are hospitals where TPA has a tie up for the cashless hospitalization. There are two kinds of network hospitals; PPN Network
hospitals where cashless services can be obtained for emergency and planned treatments and Standard (Non PPN) network
hospitals where cashless services can be obtained for planned Hospitalization.

. Once you have reached there, please show your ID card for identification. TPA will also send $a$ letter of credit (on pre-authorization)
to the hospital to make sure that they extend credit facility. Please complete the pre-authorization procedure listed earlier. If the
pre-authorization is not done, you must collect all reports and discharge card when you get discharged. Please make sure that you
sign the hospital bill before leaving the hospital. You can then submit the claim along with all the necessary supporting documents
to TPA as a reimbursement . If, however, you go to a non network hospital , it is still advisable to fill the preauthorization form ( use
the copy attached with the Benefits Manual). Please fill the claim form, attach the relevant documents and send it to TPA office for
reimbursement.


### How can I claim my pre & post hospitalization expenses?

. The policy covers pre-hospitalization expenses made prior to 60 days of hospitalization and incurred towards the same illness/
disease due to which hospitalization happens. It also covers all medical expenses for up to 60 days post discharge as advised by the
Medical Practitioner. All bills with summary have to be sent to TPA for reimbursement within 7 days of incurring the expense.

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="35" -->
<!-- PageBreak -->


## Making A Claim(1/2) Things To Remember

Always aim to pre-authorize your benefits with the TPA
This will help you in the following ways:

. You will be informed in advance regarding your coverage for the treatment and whether it is covered under your medical plan or
not . This will help you know in advance if your claim may get rejected at a later stage and you do not end up paying out of pocket.

. It will help you ensure that the treatment cost is appropriate and not inflated. as the TPA will be able to cross check costs with the
hospital in question. This will also help TPA in planning your hospitalization expenditure such that you do not run out of the cover
that you are entitled to.

. It will help TPA in registering the impending claim with the insurer

Ensure your dependent list is always updated and claims submitted as per protocols

· Please ensure that all your dependents are covered and have a valid card at the outset itself as it will not be possible to add
dependents at a later stage. Submit your reimbursement claims within timelines from the hospital. Please do not postpone this till
later as it may mean that your claim gets rejected due to late submission .

. Please check that your documents are submitted completely at the first instance itself and originals are submitted wherever
requested for . Do note that incomplete submissions will not be considered as exceptions by the insurers and will only delay the
process further for you and a delay may lead to the claim getting closed. Please also retain a copy of all claim documents submitted
to the insurer

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="36" -->
<!-- PageBreak -->


## Making A Claim (2/2) Things To Remember

Know that it is possible that benefits under your plan could be reduced v/s your eligible sum insured

The following are some common reasons for rejection although these are NOT the only reasons why a claim could be reduced

· Limits for the specific ailment exceed the reasonable cap on ailments listed in the manual,

· Claim amount exceeds the permissible limit under the policy for you ( denied to the extent of the excess),

. Some expense items are non payable for e.g. toiletries , food charges for visitors etc.

Know that it is possible that your claim could also be completely rejected under the plan?

The following are some common reasons for rejection although these are NOT the only reasons why a claim could be rejected

· Treatment taken after leaving the organization. (If you have been transferred from one group business to another, please confirm
with your HR that you have been included for coverage under your new entity)

· Treatment that should have been taken on an outpatient basis (unnecessary inpatient admission and / or no active line of
treatment.) or where hospitalization has been done primarily from a preventive perspective. Please remember that on occasion your
personal doctor may recommend hospital admission for observation purposes however such admissions are not covered under
your medical plan

· Treatment taken is not covered as per policy conditions or excluded, under the policy. Please go through the list of standard
exclusions listed earlier. (for e.g. : Ailment is a because of alcohol abuse is a standard exclusion, similarly cosmetic treatments or
treatments for external conditions like squint correction etc are not covered) . Hospitalization taken in a hospital which is not
covered as per policy conditions (Ex. less than 10 bed hospitals), Admission is before/after the policy period or details of the
member are not updated on the insurer's list of covered members . Additionally in case original documents are not submitted as
per the claim submission protocol.


<figure>

<!-- PageFooter="AON" -->

</figure>


<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="37" -->
<!-- PageBreak -->


## Group Personal Accident Policy

The Group Personal Accident policy covers
expenses for the insured persons (only employee
covered) on account of permanent / temporary, total
or partial disability or death due to an accident.

Accidental Permanent Disablement means
disablement caused due to an accident which
entirely prevents an insured person from attending
to any business or occupation of any and every kind
and which lasts 12 months and at the expiry of that
period is beyond hope of improvement.

Accidental Temporary Total Disablement means
disablement caused due to an accident which
temporarily and totally prevents the Insured Person
from attending to the duties of his usual business or
occupation and shall be payable during such
disablement from the date on which the Insured
person first became disabled.

Accidental Permanent Partial Disablement is a
doctor certified total and continuous loss or
impairment of a body part or sensory organ caused
due to an accident, to the extent specified in the
chart provided by the insurer


<figure>

<!-- PageFooter="AON" -->

</figure>


<figure>

Nadada

Jakuna Man

</figure>


<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageBreak -->


## Your Plan Details


<table>
<tr>
<td colspan="2">Plan Name : Group Accident Plan</td>
</tr>
<tr>
<td>Policy Holder</td>
<td>Novartis Healthcare Pvt Ltd, Novartis India Limited and Sandoz Pvt Ltd.</td>
</tr>
<tr>
<td>Period of the Cover</td>
<td>12 months</td>
</tr>
<tr>
<td>Policy Effective Date</td>
<td>1st Jul 2023</td>
</tr>
<tr>
<td>Expiry Date</td>
<td>30th Jun 2024</td>
</tr>
<tr>
<td>Insurer</td>
<td>Aditya Birla Health Insurance Co. Ltd</td>
</tr>
<tr>
<td>Sum Insured Limit</td>
<td>3 times of Annual Base Salary</td>
</tr>
<tr>
<td>Members covered under the plan</td>
<td>Employees of Novartis Healthcare Pvt Ltd, Novartis India Limited and Sandoz Pvt Ltd.</td>
</tr>
</table>


<table>
<tr>
<td>Key Benefit Features</td>
<td></td>
</tr>
<tr>
<td>Accidental Death</td>
<td>Covered up to the full sum insured limit</td>
</tr>
<tr>
<td>Permanent Total Disablement</td>
<td>Covered up to the % of sum insured limit</td>
</tr>
<tr>
<td>Permanent Partial Disablement</td>
<td>Covered up to a specified percentage of the full sum insured limit</td>
</tr>
<tr>
<td>Temporary Total Disablement (Weekly Benefit)</td>
<td>Covered for 1% of Sum Insured or INR 5,000 or Actual subject to a maximum limit per week, up to 104 weeks</td>
</tr>
</table>


<figure>

AON

</figure>


<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="39" -->
<!-- PageBreak -->


## Your Plan Details


<table>
<tr>
<td>Key Benefit Features</td>
<td></td>
</tr>
<tr>
<td>Medical Expenses</td>
<td>Limit of 10% of maximum eligible limit or 40% of admissible claims amount or actual whichever is lower subject to a maximum of INR 500,000.</td>
</tr>
<tr>
<td>Education Fund</td>
<td>In case of death of the employee, an Educational Fund for dependent children up to INR 100,000/- for 2 children up to the age of 23 years or 4 years whichever is earlier.</td>
</tr>
<tr>
<td>Ambulance Charges</td>
<td>Covered up to INR 5,000</td>
</tr>
<tr>
<td>Repatriation of Mortal Remains</td>
<td>Covered up to INR 5,000</td>
</tr>
<tr>
<td>Carriage of Dead Body</td>
<td>In the event of death of the insured person due to accident, company shall pay for transportation of insured person's death body to the place resident of a lump sum of 2% of capital sum insured or INR 2,500/- whichever is less.</td>
</tr>
<tr>
<td>Other Coverages</td>
<td>a) Terrorism is covered b) Snake or Animal bite is coved as an accident</td>
</tr>
</table>


AON

<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="40" -->
<!-- PageBreak -->


## Your Plan Details


### Key Exclusions

· Suicide, attempted suicide (whether sane or insane) or intentionally self-inflicted Injury or illness, or sexually transmitted
conditions, mental or nervous disorder, anxiety, stress or depression, Acquired Immune Deficiency Syndrome (AIDS), Human
Immune-deficiency Virus (HIV) infection; or

· Being under the influence of drugs, alcohol, or other intoxicants or hallucinogens unless properly prescribed by a Physician and
taken as prescribed; or

· Participation in an actual or attempted felony, riot, crime, misdemeanor, (excluding traffic violations) or civil commotion; or

· Operating or learning to operate any aircraft or performing duties as a member of the crew on any aircraft; or Scheduled
Aircraft .; or

· Self exposure to needless peril (except in an attempt to save human life);

· Loss due to childbirth or pregnancy

· Self exposure to adventurous sports like sky diving / scuba diving / rock climbing, car racing, etc.

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="41" -->
<!-- PageBreak -->


## Making A Claim Group Personal Accident - Claim Procedure


<figure>


The process begins when an employee or beneficiary notifies the HR department, which then informs the insurer and submits the necessary claims documents within 14 days of the event. Once all relevant documents are obtained, the insurer starts processing the claims. The next step involves claim investigation and review after all required documents have been submitted.

A decision point follows, where it is determined whether the claim is approved. If the claim is approved, a cheque is sent to the HR department or NEFT details are shared with HR, who then communicates this information to the employee or beneficiary. If the claim is rejected, the insurer provides a valid reason for the rejection to the HR, employee, or beneficiary.

This process ensures a structured workflow for claim submission, investigation, approval, or rejection, with clear responsibilities and communication channels at each stage.
Employee / Beneficiary
notifies HR, who in turn would
intimate Insurer and submit required
claims documents within 14 days of
the event

On obtaining all relevant documents,
Insurer begins processing the claims

Claim Investigation and Review
post submission of all the
required documents

Yes

Is claim
approved
?

No

On approval, the cheque is sent to the HR
or NEFT details shared with HR, from
where the information is shared to the
Employee / Beneficiary

On rejection of the claim, Insurer would
provide a valid reason for the rejection to
HR / Employee / Beneficiary

</figure>


<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="42" -->
<!-- PageBreak -->


## Making A Claim Document Check List


<table>
<tr>
<td>Sr No.</td>
<td>Document Details for Accidental Death</td>
</tr>
<tr>
<td>1</td>
<td>Completed Claim form with photocopy of ID card</td>
</tr>
<tr>
<td>2</td>
<td>Copy of postmortem examination report</td>
</tr>
<tr>
<td>3</td>
<td>Copy of the first information report from police department / copy of the medico-legal certificate</td>
</tr>
<tr>
<td>4</td>
<td>Original death summary from the hospital</td>
</tr>
<tr>
<td>5</td>
<td>Copy of the legal heir certificate, if the claim is for the death of the principle insured</td>
</tr>
<tr>
<td>6</td>
<td>Viscera Report for death due to poisoning OR snake bite</td>
</tr>
<tr>
<td>7</td>
<td>Death certificate from Municipal Authority OR Gram Panchayat (Applicable for claim reported from rural areas)</td>
</tr>
<tr>
<td>8</td>
<td>Salary slip prior to Accident</td>
</tr>
</table>


<table>
<tr>
<td>Sr No.</td>
<td>Document Details for Disablement Claims</td>
</tr>
<tr>
<td>1</td>
<td>Completed Claim form with photocopy of ID card</td>
</tr>
<tr>
<td>2</td>
<td>Original detailed discharge summary / day care summary from the hospital</td>
</tr>
<tr>
<td>3</td>
<td>Treating doctor's certificate giving details of injuries (How, when and where injury sustained)</td>
</tr>
<tr>
<td>4</td>
<td>Copy of the first information report from police department / copy of the medico-legal certificate</td>
</tr>
<tr>
<td>5</td>
<td>First consultation letter and subsequent treatment papers</td>
</tr>
<tr>
<td>6</td>
<td>Disability certificate from a concerned specialist affiliated with government hospital confirming the extent and nature of disability</td>
</tr>
</table>


This is an indicative list of documents and there may be additional documents required by the insurer.
It is mandatory to provide the details for nomination of beneficiary.


<figure>

<!-- PageFooter="AON" -->

</figure>


<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="43" -->
<!-- PageBreak -->


## Making A Claim Document Check List


<table>
<tr>
<td>Sr No.</td>
<td>Document Details for Temporary Total Disablement</td>
</tr>
<tr>
<td>1</td>
<td>Completed Claim form with photocopy of ID card</td>
</tr>
<tr>
<td>2</td>
<td>Salary Slip prior to Accident</td>
</tr>
<tr>
<td>3</td>
<td>Fitness Certificate by the treating doctor</td>
</tr>
<tr>
<td>4</td>
<td>Medical/Hospital Documents if any</td>
</tr>
</table>


This is an indicative list of documents and there may be
additional documents required by the insurer.
It is mandatory to provide the details for nomination of
beneficiary.


<table>
<tr>
<td>Sr No.</td>
<td>Document Details for Emergency Medical Expenses/Emergency Expenses (Accident Only)</td>
</tr>
<tr>
<td>1</td>
<td>Completed Claim form with photocopy of ID card</td>
</tr>
<tr>
<td>2</td>
<td>Original consolidated hospital bill with breakup of each Item, duly signed by the insured</td>
</tr>
<tr>
<td>3</td>
<td>Original payment receipt of the hospital bill</td>
</tr>
<tr>
<td>4</td>
<td>Original bills, original payment receipts and reports for investigation</td>
</tr>
<tr>
<td>5</td>
<td>Original medicine bills and receipts with corresponding prescriptions</td>
</tr>
<tr>
<td>6</td>
<td>Original invoice/bills for implants (viz. Stent /PHS Mesh / IOL etc.) with original payment Receipts</td>
</tr>
<tr>
<td>7</td>
<td>Treating doctor's certificate giving details of injuries (How, when and where injury sustained) including whether claimant was under the influence of any intoxicating material</td>
</tr>
<tr>
<td>8</td>
<td>Copy of the medico-legal certificate</td>
</tr>
</table>


<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="44" -->
<!-- PageBreak -->


## Group Term Life Benefits

Group Term Life Insurance Scheme is
meant to provide life insurance protection
to the employees. The Policy provides for
payment of a lump sum to the nominated
beneficiary in the unfortunate event of the
employee's death due to any cause. Plans
are subject to a Free Cover Limit and
requirement for medical tests. Employees
whose sum insured is above the Free
Cover Limit, will have to undergo medical
test as informed/prescribed by the
insurer. In case any employee fails to
complete the criteria of the insurer, sum
insured benefit will be restricted up to
Free Cover Limit only.

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee \ Benefits | Manual 2023-24" -->
<!-- PageBreak -->


## Your Plan Details


### Plan Name : Group Term Life


<table>
<tr>
<td>Policy Holder</td>
<td>Novartis Healthcare Pvt Ltd, Novartis India Limited and Sandoz Pvt Ltd.</td>
</tr>
<tr>
<td>Period of the Cover</td>
<td>12 months</td>
</tr>
<tr>
<td>Policy Effective Date</td>
<td>1st July 2023</td>
</tr>
<tr>
<td>Expiry Date</td>
<td>30th June 2024</td>
</tr>
<tr>
<td>Insurer</td>
<td>HDFC Life Insurance Co. Ltd</td>
</tr>
<tr>
<td>Basis of Sum Assured</td>
<td>3 times of Annual Base Salary (Minimum Sum assured of INR 30 lacs) upto FCL limit of INR 4 Crores - Maximum Age 59 years</td>
</tr>
<tr>
<td>Terminal Illness Rider</td>
<td>100% of Base Life Cover subject to maximum of INR 2 Cr.</td>
</tr>
<tr>
<td>Members covered under the plan</td>
<td>Employees of Novartis Healthcare Pvt Ltd, Novartis India Limited and Sandoz Pvt Ltd.</td>
</tr>
<tr>
<td>Geographical Limits</td>
<td>24*7 World-Wide</td>
</tr>
</table>


<table>
<tr>
<th>Benefit</th>
<th>Coverage</th>
</tr>
<tr>
<td>Death</td>
<td>Due to any cause</td>
</tr>
</table>


<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="46" -->
<!-- PageBreak -->


## Document Checklist


<table>
<tr>
<th>Type of Claim</th>
<th>Requirement</th>
</tr>
<tr>
<td rowspan="9">Death (all causes of death #)</td>
<td>· Claim Forms Part I: Application Form for Death Claim (Claimant's Statement) # Part II: Physician's Statement, relevant Hospital records and report from the concerned medical specialist giving nature of disability and illness (for Critical Illness claims).</td>
</tr>
<tr>
<td>· Death Certificate issued by a local government body like Municipal Corporation/Village Panchayat</td>
</tr>
<tr>
<td>· Medical Cause of Death Certificate issued by attending physician/hospital</td>
</tr>
<tr>
<td>· Attested True Copies of Indoor case Papers of the hospital(s)</td>
</tr>
<tr>
<td>· Post-mortem Report (Autopsy Report) &amp; Chemical Viscera Report - if performed</td>
</tr>
<tr>
<td>· The Beneficiary : o Photo ID with DOB with relationship to the insured</td>
</tr>
<tr>
<td>o Proof of legal title to the claim proceeds (e.g., legal succession papers, assignment deed etc.)</td>
</tr>
<tr>
<td>· Employer's Certificate</td>
</tr>
<tr>
<td>· Leave Records for the past 3 years</td>
</tr>
<tr>
<td rowspan="3">If Death due to Accident (submit in addition to the above)</td>
<td>All Police Reports / First Information &amp; Final Investigation Report</td>
</tr>
<tr>
<td>Proof of Accident - Panchnama / Inquest Report</td>
</tr>
<tr>
<td>Newspaper cutting / Photographs of the accident - if available</td>
</tr>
</table>


The above is an indicative list of documents, and the insurer reserves the right to ask for additional proofs & documents in support of the
claim. Policyholder shall inform the insurance company of any claim within 30 days of the claim event.


<figure>

<!-- PageFooter="AON" -->

</figure>


<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="47" -->
<!-- PageBreak -->


## Thank You

About Aon: Aon plc (NYSE:AON) is a leading global
professional services firm providing a broad range of risk,
retirement and health solutions. Our 50,000 colleagues
in 120 countries empower results for clients by using
proprietary data and analytics to deliver insights that
reduce volatility and improve performance. The
information contained herein, and the statements
expressed are of a general nature and are not intended to
address the circumstances of any particular individual or
entity. Although we endeavour to provide accurate and
timely information and use sources, we consider reliable,
there can be no guarantee that such information is
accurate as of the date it is received or that it will
continue to be accurate in the future. No one should act
on such information without appropriate professional
advice after a thorough examination of the particular
situation.

www.aon.com

AON

<!-- PageNumber="48" -->


### Processing Leave Policy Document

In [6]:
DI_result = process_document_and_save_results(INPUT_FILE_PATHS[1], PICKLE_DIR)

Markdown(DI_result)

# LEAVE POLICY


## Objective

To provide guidelines to avail leave and to specify the leave procedures at Novartis
Hyderabad.


## Applicability

The policy is applicable to all associates on a permanent employment contract with the
Company.


## Types of Leave


<table>
<tr>
<th>Leave</th>
<th>Entitlement</th>
<th>Encashment</th>
<th>Accumulation</th>
<th>Approved By</th>
</tr>
<tr>
<td>Annual</td>
<td>29 days</td>
<td>On separation or retirement</td>
<td>10 per annum up to 65 days</td>
<td>Immediate Manager</td>
</tr>
<tr>
<td>Sick</td>
<td>Open</td>
<td>Nil</td>
<td>Nil</td>
<td>Immediate Manager</td>
</tr>
<tr>
<td>Parental Leave (Birth / Surrogacy / Adoption)</td>
<td>26 weeks</td>
<td>Nil</td>
<td>Nil</td>
<td>Immediate Manager</td>
</tr>
<tr>
<td>Transition/Relo cation Leave</td>
<td>5 days</td>
<td>Nil</td>
<td>Nil</td>
<td>Immediate Manager</td>
</tr>
<tr>
<td>Marriage Leave</td>
<td>2 days</td>
<td>Nil</td>
<td>Nil</td>
<td>Immediate Manager</td>
</tr>
<tr>
<td>Bereavement Leave</td>
<td>2 days</td>
<td>Nil</td>
<td>Nil</td>
<td>Immediate Manager</td>
</tr>
<tr>
<td>Leave Without Pay</td>
<td>Maximum of 30 days</td>
<td>NA</td>
<td>NA</td>
<td>LF Head &amp; HR Business Partner</td>
</tr>
<tr>
<td>Compensatory Off</td>
<td>2 days</td>
<td>Nil</td>
<td>Nil</td>
<td>Immediate Manager</td>
</tr>
</table>


<!-- PageFooter="Novartis Hyderabad Business use only" -->
<!-- PageFooter="Back to Contents" -->
<!-- PageNumber="Page 1 of 7" -->
<!-- PageBreak -->


## Leave Guidelines

☐
For the purpose of calculating leave accounts, 'year' shall mean the year commencing
on the first day of April and ending on the last day of March next year ('Financial
year').

☐
] Discretion is reserved with the manager empowered to sanction leave, to refuse or
revoke leave at any time according to the exigencies of the organization's work.

☐
Leave must be applied for through HR Core.
☐
The associate is responsible for his/her attendance and leave records and should
always keep it updated.


## 1.1 Annual Leave (AL)

Entitlement
☐
All associates are entitled to 29 working days of AL for the financial year, credited on
1 st April every year. Associates joining during the year will be credited with leave on
a pro-rate basis for the remaining part of the financial year.

New Joiners joining on or before the 15th of the month are entitled to (2) two days
☐
AL for the first month and those joining after the 15th of the month are entitled to 1
(one) AL for the month.

Associates can avail a minimum of half day Annual Leave at a time.
☐

Accumulation

☐
In a given year, an associate can carry forward a maximum of 10 days of AL. The
balance AL of that year would automatically lapse on 31 st March.
☐
For hires during the leave calendar year, the leave accumulation will be pro-rated up
to a maximum of 10 days.

☐
AL can be accumulated up to a maximum of 65 days in total. Any leave balance in
excess of 65 days will lapse without any encashment.
☐
] For associates joining during the year, the accumulation of leaves will be pro-rated
up to a maximum limit of 10 days.

<!-- PageFooter="Novartis Hyderabad Business use only" -->
<!-- PageFooter="Back to Contents" -->
<!-- PageNumber="Page 2 of 7" -->
<!-- PageBreak -->


# Encashment

☐
Encashment of leaves will be calculated on Base Salary' (not basic salary) as per the
formula "Base Salary / 26 * no. of leaves"

Guidelines
☐
Intervening Holidays do not form part of the Annual Leave.
☐
AL can be combined with Sick Leave.
☐
If associate could not make use of the leave after it was sanctioned, he/she is required
to cancel the leave giving reasons to their manager in the system.
☐
[ On resignation/superannuation of services from the company, excess AL in the
current year, against entitlement, will be adjusted in the full & final settlement on a
pro-rata basis subject to a cap of 10 days AL.


## 1.2 Sick Leave

Objective & Eligibility

To give associates time off to recover and recuperate in the event of illness or accidents.
All internal associates are eligible for this policy.


### Procedures and Conditions

☐
The company may, at its discretion, grant sick leave to an associate, based on the
nature of sickness.
☐

[ In case of absence beyond 3 days, the associate is required to submit a medical
certificate.

[ In case where absence is likely to be prolonged beyond 3 days, the likely duration of
☐

absence along with the doctor's certificate certifying the same should be submitted.

[ Any sick leave in excess of 30 continuous days will require the approval of the division
☐

management team. The management team will review the case and may at its
discretion approve additional sick leave with pay or without pay (Loss of Pay - LOP).

[ The company may request the associate to undergo a medical examination by a
☐

nominated medical practitioner if felt necessary.
☐
Sick leaves can neither be accumulated nor encashed nor carried forward to the next
year.

<!-- PageFooter="Novartis Hyderabad Business use only" -->
<!-- PageFooter="Back to Contents" -->
<!-- PageNumber="Page 3 of 7" -->
<!-- PageBreak -->


### 1.3 Parental Leave

Both Birthing mother and Non-birthing parent are eligible for Parental Leave of 26 weeks
following the birth, surrogacy or adoption of a child.

Associates must avail the Parental Leave within one year of child's birth/surrogacy/adoption.
Any Parental Leave that is not used within one year of a covered life event will be forfeited
and will not be financially compensated, or carried forward. For details, refer to the policy
document - Parental Leave Policy - India


# Leave Policy for Trainees

Interns are eligible for a maximum of one day per month during their internship period in
case of medical and personal exigencies, subject to Manager's approval.


## Guidelines

☐
A minimum of one and maximum leave is at the discretion of the Manager.
☐
Intervening Holidays do not form part of the leave

☐
Any leave taken in excess of these 15 days in a given financial year would be treated
as leave without pay.


# Leave Without Pay (LWP)


## Eligibility

Leave without Pay (LWP) will be given to associates only in exceptional circumstances purely
at the discretion of the management. LWP can be availed only AFTER exhausting the Annual
Leave


## Process

Any associate planning to go on LWP will need to apply for the same to the immediate
Manager which would be approved by the SLT member and HR.


## Entitlement

Maximum 30 days Leave Without Pay in a year, at the discretion of the SLT member.

Unauthorised Absence

<!-- PageFooter="Novartis Hyderabad Business use only" -->
<!-- PageFooter="Back to Contents" -->
<!-- PageNumber="Page 4 of 7" -->
<!-- PageBreak -->

☐
Unauthorized absence refers to absence from work without requisite approval.
☐
The associate will need to offer an explanation to $\mathrm { h i s / h e r }$ immediate manager in the
event of any unauthorized absence.
☐
In the event of an associate failing to give an explanation to the satisfaction of the

immediate manager, the employment is liable to termination as per organization
rules.

The associate will not be eligible for payment of salary during this absence.
☐


### General Compliance on Leave Rules

☐
Each associate is responsible for maintaining his/her updated leave record with all
leaves availed by him/her during the year on HR Core.

An associate on leave may be recalled to duty if an exceptional situation arises, and
the Company deems fit to do so.
☐
An associate going out of station during his/her leave must furnish to the manager
the contact numbers at which he/she can be contacted while out of station.
☐


### 1.4 Transition/Relocation Leave

Objective & Eligibility

Applicable only for those Associates who relocate (at the time of joining / internal
transfer) from other cities, it aims to provide them some time off to comfortably relocate
and settle in the new location.

Procedures and Conditions

☐
Eligible Associates will be entitled to Five (5) working days of Relocation Leave.
☐
This leave must be availed within Six (6) months from the date of joining / transfer
to the new location.
☐
The leave can be availed continuously or as and when required.

<!-- PageFooter="Novartis Hyderabad Business use only" -->
<!-- PageFooter="Back to Contents" -->
<!-- PageNumber="Page 5 of 7" -->
<!-- PageBreak -->


### 1.5 Marriage Leave

Entitlement
Associates can avail 2 days leave at the time of own marriage


#### Conditions

2 days need to be taken consecutively at the time of marriage. If the marriage falls over the
weekend, marriage leave can be clubbed with the weekend.


# 1.6 Bereavement Leave

Entitlement

In the unfortunate event of the death of a parent, sibling, spouse or child, associates may
avail 2 days leave within a period of 3 months of the event.


## 1.7 Compensatory Offs

Objective & Eligibility

In order to meet business exigency associates have to come to work on weekly off or
public holidays. Associates who work on public holidays / weekly off day in case of
exigencies of work will be provided with a compensatory off. Compensatory off is allowed
subject to approval from Operational Manager

Eligibility: All associates at GJFA 5 to 8

Procedures and Conditions

1\. An associate should have worked at least for 8 hours on any off day to avail
compensatory off for the same.

2\. The compensatory off must be availed within two months.

3\. The compensatory off cannot be taken in advance.

4\. The compensatory off will not be carried forward or accumulated or en-cashed.

5\. Maximum 2 compensatory offs can be availed in a month

<!-- PageFooter="Novartis Hyderabad Business use only" -->
<!-- PageFooter="Back to Contents" -->
<!-- PageNumber="Page 6 of 7" -->
<!-- PageBreak -->


### 1.8 Periodical Review of Leave Records

It is the responsibility of the manager to ensure that the leave records of their respective
teams are periodically reviewed and any discrepancies identified should be immediately
brought to the notice of the HRBP. HRBPs will then initiate corrections, if required, by
sending it to the HR Services team (XDHR) or through the employee/manager on HR
Core directly. In case any leaves are not recorded, managers should ensure that the
leaves are recorded on HR Core. Sufficient evidences of periodical review of leave records
should be maintained by the managers for their teams.

<!-- PageFooter="Novartis Hyderabad Business use only" -->
<!-- PageFooter="Back to Contents" -->
<!-- PageNumber="Page 7 of 7" -->


### Processing Annual Reports Policy Document

In [7]:
DI_result = process_document_and_save_results(INPUT_FILE_PATHS[2], PICKLE_DIR)

Markdown(DI_result)

# UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549 FORM 10-K

☒
☒
ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF
1934
For the Fiscal Year Ended June 30, 2024
OR

☐
☐
TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF
1934
For the Transition Period From
to

Commission File Number 001-37845


# MICROSOFT CORPORATION

WASHINGTON
(STATE OF INCORPORATION)
ONE MICROSOFT WAY, REDMOND, WASHINGTON 98052-6399
(425) 882-8080
www.microsoft.com/investor

91-1144442

(I.R.S. ID)

Securities registered pursuant to Section 12(b) of the Act:


<table>
<tr>
<th>Title of each class</th>
<th>Trading Symbol</th>
<th>Name of exchange on which registered</th>
</tr>
<tr>
<td>Common stock, $0.00000625 par value per share</td>
<td>MSFT</td>
<td>NASDAQ</td>
</tr>
<tr>
<td>3.125% Notes due 2028</td>
<td>MSFT</td>
<td>NASDAQ</td>
</tr>
<tr>
<td>2.625% Notes due 2033</td>
<td>MSFT</td>
<td>NASDAQ</td>
</tr>
</table>


Securities registered pursuant to Section 12(g) of the Act:

None

Indicate by check mark if the registrant is a well-known seasoned issuer, as defined in Rule 405 of the Securities Act.
☒
Yes
No
☐
No
Indicate by check mark if the registrant is not required to file reports pursuant to Section 13 or Section 15(d) of the Act. Yes
☐

☒
Indicate by check mark whether the registrant (1) has filed all reports required to be filed by Section 13 or 15(d) of the Securities Exchange
Act of 1934 during the preceding 12 months (or for such shorter period that the registrant was required to file such reports), and (2) has
been subject to such filing requirements for the past 90 days.
☒
Yes
☐
No

Indicate by check mark whether the registrant has submitted electronically every Interactive Data File required to be submitted pursuant to Rule
405 of Regulation S-T (§232.405 of this chapter) during the preceding 12 months (or for such shorter period that the registrant was required to
submit such files).
☒
Yes
☐
No

Indicate by check mark whether the registrant is a large accelerated filer, an accelerated filer, a non-accelerated filer, a smaller reporting
company, or an emerging growth company. See the definitions of "large accelerated filer," "accelerated filer," "smaller reporting company,"
and "emerging growth company" in Rule 12b-2 of the Exchange Act.

Large Accelerated Filer
☒
Non-accelerated Filer
☐

☐
Accelerated Filer

☐
Smaller Reporting Company
Emerging Growth Company
☐

If an emerging growth company, indicate by check mark if the registrant has elected not to use the extended transition period for complying
with any new or revised financial accounting standards provided pursuant to Section 13(a) of the Exchange Act.
☐
☐

Indicate by check mark whether the registrant has filed a report on and attestation to its management's assessment of the effectiveness of its
internal control over financial reporting under Section 404(b) of the Sarbanes-Oxley Act (15 U.S.C. 7262(b)) by the registered public accounting
firm that prepared or issued its audit report.
☒
☒

If securities are registered pursuant to Section 12(b) of the Act, indicate by check mark whether the financial statements of the registrant
included in the filing reflect the correction of an error to previously issued financial statements.
☐
☐

Indicate by check mark whether any of those error corrections are restatements that required a recovery analysis of incentive-based
compensation received by any of the registrant's executive officers during the relevant recovery period pursuant to §240.10D-1(b).
☐
☐

<!-- PageBreak -->

Indicate by check mark whether the registrant is a shell company (as defined in Rule 12b-2 of the Act). Yes
☐
No
☒

As of December 31, 2023, the aggregate market value of the registrant's common stock held by non-affiliates of the registrant was $2.8
trillion based on the closing sale price as reported on the NASDAQ National Market System. As of July 25, 2024, there were 7,433,038,381
shares of common stock outstanding.


## DOCUMENTS INCORPORATED BY REFERENCE

Portions of the definitive Proxy Statement to be delivered to shareholders in connection with the Annual Meeting of Shareholders to be held
on December 10, 2024 are incorporated by reference into Part III.

<!-- PageBreak -->


# MICROSOFT CORPORATION FORM 10-K For the Fiscal Year Ended June 30, 2024 INDEX


<table>
<tr>
<th></th>
<th></th>
<th></th>
<th>Page</th>
</tr>
<tr>
<td>PART I</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td></td>
<td>Item 1.</td>
<td>Business</td>
<td>3</td>
</tr>
<tr>
<td></td>
<td></td>
<td>Information about our Executive Officers</td>
<td>18</td>
</tr>
<tr>
<td></td>
<td>Item 1A.</td>
<td>Risk Factors</td>
<td>20</td>
</tr>
<tr>
<td></td>
<td>Item 1B.</td>
<td>Unresolved Staff Comments</td>
<td>34</td>
</tr>
<tr>
<td></td>
<td>Item 1C.</td>
<td>Cybersecurity</td>
<td>34</td>
</tr>
<tr>
<td></td>
<td>Item 2.</td>
<td>Properties</td>
<td>36</td>
</tr>
<tr>
<td></td>
<td>Item 3.</td>
<td>Legal Proceedings</td>
<td>36</td>
</tr>
<tr>
<td></td>
<td>Item 4.</td>
<td>Mine Safety Disclosures</td>
<td>36</td>
</tr>
<tr>
<td>PART II</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td></td>
<td>Item 5.</td>
<td>Market for Registrant's Common Equity, Related Stockholder Matters, and Issuer Purchases of Equity Securities</td>
<td>37</td>
</tr>
<tr>
<td></td>
<td>Item 6.</td>
<td>[Reserved]</td>
<td>37</td>
</tr>
<tr>
<td></td>
<td>Item 7.</td>
<td>Management's Discussion and Analysis of Financial Condition and Results of Operations</td>
<td>38</td>
</tr>
<tr>
<td></td>
<td>Item 7A.</td>
<td>Quantitative and Qualitative Disclosures About Market Risk</td>
<td>54</td>
</tr>
<tr>
<td></td>
<td>Item 8.</td>
<td>Financial Statements and Supplementary Data</td>
<td>55</td>
</tr>
<tr>
<td></td>
<td>Item 9.</td>
<td>Changes in and Disagreements with Accountants on Accounting and Financial Disclosure</td>
<td>98</td>
</tr>
<tr>
<td></td>
<td>Item 9A.</td>
<td>Controls and Procedures</td>
<td>98</td>
</tr>
<tr>
<td></td>
<td></td>
<td>Report of Management on Internal Control over Financial Reporting</td>
<td>98</td>
</tr>
<tr>
<td></td>
<td></td>
<td>Report of Independent Registered Public Accounting Firm</td>
<td>99</td>
</tr>
<tr>
<td></td>
<td>Item 9B.</td>
<td>Other Information</td>
<td>100</td>
</tr>
<tr>
<td></td>
<td>Item 9C.</td>
<td>Disclosure Regarding Foreign Jurisdictions that Prevent Inspections</td>
<td>100</td>
</tr>
<tr>
<td>PART III</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td></td>
<td>Item 10.</td>
<td>Directors, Executive Officers, and Corporate Governance</td>
<td>100</td>
</tr>
<tr>
<td></td>
<td>Item 11.</td>
<td>Executive Compensation</td>
<td>101</td>
</tr>
<tr>
<td></td>
<td>Item 12.</td>
<td>Security Ownership of Certain Beneficial Owners and Management and Related Stockholder Matters</td>
<td>101</td>
</tr>
<tr>
<td></td>
<td>Item 13.</td>
<td>Certain Relationships and Related Transactions, and Director Independence</td>
<td>101</td>
</tr>
<tr>
<td></td>
<td>Item 14.</td>
<td>Principal Accountant Fees and Services</td>
<td>101</td>
</tr>
<tr>
<td colspan="2">PART IV</td>
<td></td>
<td></td>
</tr>
<tr>
<td></td>
<td>Item 15.</td>
<td>Exhibit and Financial Statement Schedules</td>
<td>102</td>
</tr>
<tr>
<td></td>
<td>Item 16.</td>
<td>Form 10-K Summary</td>
<td>115</td>
</tr>
<tr>
<td></td>
<td></td>
<td>Signatures</td>
<td>116</td>
</tr>
</table>


<!-- PageNumber="2" -->
<!-- PageBreak -->

<!-- PageHeader="PARTI Item 1" -->


# Note About Forward-Looking Statements

This report includes estimates, projections, statements relating to our business plans, objectives, and expected
operating results that are "forward-looking statements" within the meaning of the Private Securities Litigation Reform
Act of 1995, Section 27A of the Securities Act of 1933, and Section 21E of the Securities Exchange Act of 1934.
Forward-looking statements may appear throughout this report, including the following sections: "Business" (Part I,
Item 1 of this Form 10-K), "Risk Factors" (Part I, Item 1A of this Form 10-K), and "Management's Discussion and
Analysis of Financial Condition and Results of Operations" (Part II, Item 7 of this Form 10-K). These forward-looking
statements generally are identified by the words "believe," "project," "expect," "anticipate," "estimate," "intend,"
"strategy," "future," "opportunity," "plan," "may," "should," "will," "would," "will be," "will continue," "will likely result," and
similar expressions. Forward-looking statements are based on current expectations and assumptions that are subject
to risks and uncertainties that may cause actual results to differ materially. We describe risks and uncertainties that
could cause actual results and events to differ materially in "Risk Factors," "Management's Discussion and Analysis of
Financial Condition and Results of Operations," and "Quantitative and Qualitative Disclosures About Market Risk" (Part
II, Item 7A of this Form 10-K). Readers are cautioned not to place undue reliance on forward-looking statements, which
speak only as of the date they are made. We undertake no obligation to update or revise publicly any forward-looking
statements, whether because of new information, future events, or otherwise.


# PART I ITEM 1. BUSINESS


## GENERAL


### Embracing Our Future

Microsoft is a technology company committed to making digital technology and artificial intelligence ("Al") available
broadly and doing so responsibly, with a mission to empower every person and every organization on the planet to
achieve more. We create platforms and tools, powered by AI, that deliver innovative solutions that meet the evolving
needs of our customers. From infrastructure and data, to business applications and collaboration, we provide unique,
differentiated value to customers. We strive to create local opportunity, growth, and impact in every country around the
world.

We have entered a new age of AI that will fundamentally transform productivity for every individual, organization, and
industry on earth, while helping us address some of our most pressing challenges. Microsoft's Al offerings, including
Copilot and our Copilot stack, are already orchestrating a new era of Al transformation, driving better business
outcomes across every role and industry. As a company, we believe we can be the democratizing force for this new
generation of technology and the opportunity it will help unlock for every country, community, and individual.

We believe Al should be as empowering across communities as it is powerful, and we're committed to ensuring it is
responsibly designed and built with safety and security from the outset.


### What We Offer

Founded in 1975, we develop and support software, services, devices, and solutions that deliver new value for
customers and help people and businesses realize their full potential.

We offer an array of services, including cloud-based solutions that provide customers with software, services,
platforms, and content, and we provide solution support and consulting services. We also deliver relevant online
advertising to a global audience.

Our products include operating systems, cross-device productivity and collaboration applications, server applications,
business solution applications, desktop and server management tools, software development tools, and video games.
We also design and sell devices, including PCs, tablets, gaming and entertainment consoles, other intelligent devices,
and related accessories.

<!-- PageNumber="3" -->
<!-- PageBreak -->

<!-- PageHeader="PARTI Item 1" -->


### The Ambitions That Drive Us

To achieve our vision, our research and development efforts focus on three interconnected ambitions:

· Reinvent productivity and business processes.

· Build the intelligent cloud and intelligent edge platform.

· Create more personal computing.


### Reinvent Productivity and Business Processes

At Microsoft, we provide technology and resources to help our customers create a secure, productive work
environment. Our family of products plays a key role in the ways the world works, learns, and connects.

Our growth depends on securely delivering continuous innovation and advancing our leading productivity and
collaboration tools and services, including Microsoft 365, LinkedIn, and Dynamics 365. Microsoft 365 is an Al first
platform that brings together Office, Windows, Copilot, and Enterprise Mobility + Security to help organizations
empower their employees. Copilot for Microsoft 365 combines Al with business data in the Microsoft Graph and
Microsoft 365 applications. Microsoft Teams is a comprehensive platform for communication and collaboration, with
meetings, calling, chat, file collaboration, and the ability to bring all of the applications teams use into a single place.
Microsoft Viva is an employee experience platform that brings together communications, knowledge, learning,
resources, and insights.

Together, the Microsoft Cloud, Dynamics 365, Microsoft Teams, and our AI offerings bring a new era of collaborative
applications for every role and business function to get insights and business impact faster. Dynamics 365 is a portfolio
of intelligent business applications that delivers operational efficiency and breakthrough customer experiences. Our
role-based extensions of Microsoft Copilot - Copilot for Sales, Copilot for Service, and Copilot for Finance - bring
together the power of Copilot for Microsoft 365 with role-specific insights and workflow assistance to streamline
business processes. Copilot Studio allows customers to customize Copilot for Microsoft 365 or build their own Copilot.
Microsoft Power Platform helps domain experts drive productivity gains with low-code/no-code tools, robotic process
automation, virtual agents, and business intelligence. Copilot Pro is a consumer subscription service that offers faster
and more powerful Al assistance in Microsoft 365 apps and on the web. Linkedln combines our unique data with this
new generation of Al to transform the way professionals learn, sell, market, and get hired.


### Build the Intelligent Cloud and Intelligent Edge Platform

Digital transformation and adoption of AI continues to revolutionize more business workstreams for organizations in
every sector across the globe. For enterprises, digital technology empowers employees, optimizes operations,
engages customers, and in some cases, changes the very core of products and services. We continue to invest in high
performance and sustainable computing to meet the growing demand for fast access to Microsoft services provided by
our network of cloud computing and Al infrastructure and datacenters.

Our cloud business benefits from three economies of scale: datacenters that deploy computational resources at
significantly lower cost per unit than smaller ones; datacenters that coordinate and aggregate diverse customer,
geographic, and application demand patterns, improving the utilization of computing, storage, and network resources;
and multi-tenancy locations that lower application maintenance labor costs.

The Microsoft Cloud provides the best integration across the technology stack while offering openness, improving time
to value, reducing costs, and increasing agility. As the foundation of the Microsoft Cloud, Azure uniquely offers hybrid
consistency, developer productivity, data and Al capabilities, and trusted security and compliance.

We offer supercomputing power for AI at scale to run large workloads, complemented by our rapidly expanding portfolio
of Al cloud services and hardware, which includes custom-built silicon and strong partnerships with chip manufacturers.
We have introduced purpose-built cloud infrastructure for Al workloads including a custom Al accelerator, Azure Maia,
and a custom in-house central processing unit, Azure Cobalt.

Our AI platform, Azure AI, is helping organizations transform, bringing intelligence and insights to the hands of their
employees and customers to solve their most pressing challenges. We offer a wide selection of industry-leading frontier
and open models, including from partners, as well as state-of-the-art tooling, and AI-optimized infrastructure, delivering
the Copilot stack for Microsoft, enterprises, and developers. Organizations large and small are deploying Azure AI

<!-- PageNumber="4" -->
<!-- PageBreak -->

<!-- PageHeader="PARTI Item 1" -->

solutions to achieve more at scale, more easily, with the proper enterprise-level responsible AI and safety and security
protections. Azure Al Studio provides a full lifecycle toolchain customers can use to ground these models on their own
data, create prompt workflows, and help ensure they are deployed and used safely.

GitHub Copilot is at the forefront of AI-powered software development, giving developers a tool to write code easier
and faster. From GitHub to Visual Studio, we provide a developer tool chain for everyone, no matter the technical
experience, across all platforms.

We have a long-term partnership with OpenAI, a leading Al research and deployment company. We deploy OpenAl's
models across our consumer and enterprise products. As OpenAl's exclusive cloud provider, Azure powers all of
OpenAl's workloads. We have also increased our investments in the development and deployment of specialized
supercomputing systems to accelerate OpenAl's research.

Our hybrid infrastructure offers integrated, end-to-end security, compliance, identity, and management capabilities to
support the real-world needs and evolving regulatory requirements of commercial customers and enterprises. Our
industry clouds bring together capabilities across the entire Microsoft Cloud, along with industry-specific
customizations. Azure Arc simplifies governance and management by delivering a consistent multi-cloud and on-
premises management platform.

The Microsoft Intelligent Data Platform fully integrates databases, analytics, and governance. Microsoft Fabric is an
end-to-end, unified analytics platform that brings together all the data and analytics tools that organizations need.

Nuance is a leader in conversational AI and ambient intelligence across industries, including healthcare, financial
services, retail, and telecommunications. Microsoft and Nuance enable organizations to accelerate their business goals
with security-focused, cloud-based solutions infused with AI.

As the rate and pace of cyberthreats continue to accelerate, security is a top priority for every organization. Microsoft
offers customers integrated products addressing security, compliance, identity, management, and privacy across
customers' multi-cloud, application, and device assets. With Copilot for Security, Microsoft offers an Al cybersecurity
product that enables security professionals to respond to cyberthreats quickly.

Windows 365 enables users to stream a full Windows experience from the Microsoft Cloud to any device.


### Create More Personal Computing

We strive to make computing more personal, enabling users to interact with technology in more intuitive, engaging,
and dynamic ways.

Windows 11 offers innovations focused on performance, productivity, and creativity, including Copilot in Windows.
Windows 11 security and privacy features include operating system security, application security, and user and identity
security. Dev Home is an open-source experience in Windows to help developer productivity. We are committed to
designing and marketing first-party devices to help drive innovation, create new device categories, and stimulate
demand in the Windows ecosystem. The Surface family includes Surface Pro, Surface Laptop, and other Surface
products. Copilot+ PCs are a new class of Windows 11 PCs that are powered by a neural processing unit. These PCs
use on-device AI for enhanced performance and features.

Copilot is an AI assistant that helps users navigate the web, answer questions, and create content. Microsoft Edge is
our fast and secure browser that helps protect users' data and offers enhanced browsing capabilities including quick
access to Al-powered tools, apps, and more. The Al-powered Bing search engine with Copilot delivers better search,
more complete answers, and the ability to generate content.

Microsoft is expanding how billions of people globally access and play video games on PC, console, mobile, and cloud.
We put game development front and center, backed by innovative hardware, experiences, and a subscription service,
Xbox Game Pass, that allows those games to reach more players across more devices. Activision Blizzard, Inc.
("Activision Blizzard"), a leader in game development and an interactive entertainment content publisher, joined
Microsoft in October 2023.

<!-- PageNumber="5" -->
<!-- PageBreak -->

<!-- PageHeader="PARTI Item 1" -->


### Our Future Opportunity

We are focused on helping customers use the breadth and depth of the Microsoft Cloud to get the most value out of
their digital spend while leading the Al platform wave across our solution areas. We continue to develop complete,
intelligent solutions for our customers that empower people to be productive and collaborate, while safeguarding
businesses and simplifying IT management. Our goal is to lead the industry in several distinct areas of technology over
the long term, which we expect will translate to sustained growth. We are investing significant resources in:

· Transforming the workplace to deliver new modern, modular business applications, drive deeper insights,
and improve how people communicate, collaborate, learn, work, and interact with one another.

· Building and running cloud-based services in ways that utilize ubiquitous computing to unleash new
experiences and opportunities for businesses and individuals.

· Applying Al and ambient intelligence to drive insights, revolutionize many types of work and business
processes, and provide substantive productivity gains using natural methods of communication.

· Tackling security from all angles with our integrated, end-to-end solutions spanning security, compliance,
identity, and management, across all clouds and platforms.

· Inventing new gaming experiences that bring people together around their shared love for games on any
devices and pushing the boundaries of innovation with console and PC gaming.

. Using Windows to fuel our cloud business, grow our share of the PC market, and drive increased
engagement with our services like Microsoft Edge, Bing, Microsoft Teams, Microsoft 365 Consumer, Xbox
Game Pass, and more.

Our future growth depends on our ability to transcend current product category definitions, business models, and sales
motions.


### Corporate Social Responsibility


#### Commitment to Sustainability

Microsoft's approach to addressing climate change starts with the sustainability of our own business. In 2020, we
committed to being a carbon negative, water positive, and zero waste company by 2030.

Since announcing that commitment, we have seen major changes both in the technology sector and in our
understanding of what it will take to meet our climate goals. New technologies, including generative Al, hold promise
for new innovations that can help address the climate crisis. At the same time, the infrastructure and electricity needed
for these technologies create new challenges for meeting sustainability commitments across the tech sector.

In May 2024, we released our Environmental Sustainability Report which looked back at our progress in several areas
during fiscal year 2023. In four areas we are on track, and in each of these we see progress that has the potential to
have global impact beyond our own sustainability work. These are:

· Reducing our direct operational emissions (Scope 1 and 2).

· Accelerating carbon removal.

· Designing for circularity to minimize waste and reusing cloud hardware.

· Improving biodiversity and protecting more land than we use.

At the same time, there are two areas where we're not yet on track, and in each of these we are intensively engaged
in work to identify and pursue additional breakthroughs. These are:

· Reducing our indirect emissions (Scope 3).

· Reducing our water use and replenishing more water than we consume in our datacenter operations.

Even amid the challenges, we remain optimistic. We're encouraged by ongoing progress across our campuses and
datacenters, and throughout our value chain.

<!-- PageNumber="6" -->
<!-- PageBreak -->

<!-- PageHeader="PARTI Item 1" -->


#### Addressing Racial Injustice and Inequity

In June 2020, we outlined a series of multi-year commitments designed to address the racial injustice and inequity
experienced by racial and ethnic minorities in the United States, including Black and African American communities.
We remain committed to addressing racial injustice and inequity and helping improve lived experiences at Microsoft,
in employees' communities, and beyond.

In fiscal year 2024, we continued to collaborate with partners and worked within neighborhoods and communities to
advance projects and programs. We grew our Nonprofit Tech Acceleration for Black and African American
Communities program, to help more than 3,000 local organizations in nearly 1,900 Black and African American
communities use technical solutions to modernize and streamline operations. We also expanded our Technology
Education and Learning Support ("TEALS") program to reach nearly 550 high schools across 21 racial equity expansion
regions with the support of nearly 1,500 volunteers, 12% of whom identify as Black or African American.

We have committed $150 million in Minority Depository Institutions and funds supporting Black and African American-
owned small businesses. These commitments drive sustained impact by directly enabling an increase of funds into
local communities, improving diverse, small-business access to capital, and increasing skill development. We continue
to partner with diverse-owned banking partners and asset managers to catalyze growth and industry participation.
Additionally, we enriched our supplier pipeline, achieving our goal to spend $500 million with double the number of
Black- and African American-owned suppliers. We have also provided 162 low- or no-interest loans to our small to
medium-sized partners through our Partner Capital Fund.

We also continue to make progress toward our overall commitment to double the number of Black and African American
and Hispanic and Latinx leaders in the U.S. by 2025.


#### Investing in Digital Skills

Microsoft's Skills for Jobs initiative aims to support a more skills-based labor market, with greater flexibility and
accessible learning paths to develop the right skills needed for the most in-demand jobs. This initiative brings together
classes, Career Essentials Certificates, and other resources from LinkedIn, GitHub, and Microsoft Learn, and is built
on data insights drawn from LinkedIn's Economic Graph. Our goal was to train and certify 10 million learners by 2025.
As of May 2024, we have surpassed that goal, training and certifying 12.6 million learners. We also launched a
campaign in the United States in 2021 to help skill and recruit 250,000 people into the nation's cybersecurity workforce
by 2025, representing half of the country's workforce shortage. To that end, we are making curriculum available free
of charge to all of the nation's higher education institutions, providing training for new and existing faculty, and providing
scholarships and supplemental resources to 25,000 students. The cyber skills initiative has expanded to 27 additional
countries that show elevated cyberthreat risks coupled with significant gaps in their cybersecurity workforces, where
we've partnered with nonprofits and other educational institutions to train the next generation of cybersecurity workers.

Generative AI is creating unparalleled opportunities to empower workers globally, but only if everyone has the skills to
use it. In June 2023, we launched an AI Skills initiative to help everyone learn how to harness the power of AI. This
includes a new LinkedIn learning pathway offering new coursework on learning the foundations of generative Al. We
also launched a new global grant challenge to uncover new ways of training workers on generative AI and provide
greater access to digital learning events and resources. Additionally, we extended our reach in rural communities,
including through our TechSpark initiative in the United States. As of June 2024, we've helped more than 2.5 million
people in 92% of the world's countries learn how to use Al.


# HUMAN CAPITAL RESOURCES

Microsoft aims to recruit, develop, and retain world-changing talent from a diversity of backgrounds. To foster their and
our success, we seek to create an environment where people can thrive and do their best work. We strive to maximize
the potential of our human capital resources by creating a respectful, rewarding, and inclusive work environment that
enables our global employees to create products and services that further our mission. Microsoft's culture is grounded
in growth mindset. This means everyone is on a continuous journey to learn and grow, operating as one company
instead of multiple siloed businesses. Our culture also embeds the security of customers and Microsoft as a priority for
every employee and across all of our organizations.

As of June 30, 2024, we employed approximately 228,000 people on a full-time basis, 126,000 in the U.S. and 102,000
internationally. Of the total employed people, 86,000 were in operations, including product support and consulting

<!-- PageNumber="7" -->
<!-- PageBreak -->


# PARTI Item 1

services, datacenter operations, and manufacturing and distribution; 81,000 were in product research and
development; 45,000 were in sales and marketing; and 16,000 were in general and administration. Certain employees
are subject to collective bargaining agreements.

We design our programs to attract, reward, and retain top talent, enable our employees' continual growth, and reinforce
our culture and values. Our total compensation opportunity is highly differentiated and market competitive. Our intended
result is a global performance and development approach that fosters our culture, drives company performance, and
competitive compensation that ensures equitable pay by role while supporting pay for performance.

Diversity and inclusion are core to our business. As reported in our Global Diversity and Inclusion Reports, we monitor
pay equity and career progress across multiple dimensions. We encourage every person at Microsoft to play an active
role in creating an inclusive environment.

We have invested significantly in employee wellbeing and offer a differentiated benefits package which includes many
physical, emotional, and financial wellness programs. Our Occupational Health and Safety program helps to protect
employees' safety while they are working. We also have introduced Hybrid Workplace Flexibility guidance to better
support leaders, managers, and employees in hybrid work scenarios.

We believe providing employees with access to continual learning enables them to drive impact for the company. We
provide individuals and teams with access to first and third-party content resources across professions, disciplines,
and roles, and offer skilling opportunities to support employees' growth while driving organizations' needs.

Our employee listening systems enable us to gather feedback directly from our workforce to inform our programs and
employee needs globally, giving us real-time insights into ways we can support our employees. As a company, we will
continue to leverage data and research to inform decision making, balancing the needs of the business, team, and
individual.


# OPERATING SEGMENTS

We operate our business and report our financial performance using three segments: Productivity and Business
Processes, Intelligent Cloud, and More Personal Computing. Our segments provide management with a
comprehensive financial view of our key businesses. The segments enable the alignment of strategies and objectives
across the development, sales, marketing, and services organizations, and they provide a framework for timely and
rational allocation of resources within businesses.

Additional information on our operating segments and geographic and product information is contained in Note 19 -
Segment Information and Geographic Data of the Notes to Financial Statements (Part II, Item 8 of this Form 10-K).

Our reportable segments are described below.


## Productivity and Business Processes

Our Productivity and Business Processes segment consists of products and services in our portfolio of productivity,
communication, and information services, spanning a variety of devices and platforms. This segment primarily
comprises:

· Office Commercial (Office 365 subscriptions, the Office 365 portion of Microsoft 365 Commercial
subscriptions, and Office licensed on-premises), comprising Office, Exchange, SharePoint, Microsoft
Teams, Office 365 Security and Compliance, Microsoft Viva, and Copilot for Microsoft 365.

· Office Consumer, including Microsoft 365 Consumer and Copilot Pro subscriptions, Office licensed on-
premises, and other Office services.

· LinkedIn, including Talent Solutions, Marketing Solutions, Premium Subscriptions, and Sales Solutions.

· Dynamics business solutions, including Dynamics 365, comprising a set of intelligent, cloud-based
applications across ERP, CRM, Power Apps, and Power Automate; and on-premises ERP and CRM
applications.


# Office Commercial

<!-- PageNumber="8" -->
<!-- PageBreak -->


# PARTI Item 1

Office Commercial is designed to increase personal, team, and organizational productivity through a range of products
and services. Growth depends on our ability to reach new users in new markets such as frontline workers, small and
medium businesses, and growth markets, as well as add value to our core product and service offerings to span AI
and productivity categories such as communication, collaboration, analytics, security, and compliance. Office
Commercial revenue is mainly affected by a combination of continued installed base growth and average revenue per
user expansion, as well as the continued shift from Office licensed on-premises to Office 365.


# Office Consumer

Office Consumer is designed to increase personal productivity and creativity through a range of products and services.
Growth depends on our ability to reach new users, add value to our core product set with new features including Al
tools, and continue to expand our product and service offerings into new markets. Office Consumer revenue is mainly
affected by the percentage of customers that buy Office with their new devices and the continued shift from Office
licensed on-premises to Microsoft 365 Consumer subscriptions. Office Consumer Services revenue is mainly affected
by the demand for communication and storage through Skype, Outlook.com, and OneDrive, which is largely driven by
subscriptions, advertising, and the sale of minutes.


# Linkedin

LinkedIn connects the world's professionals to make them more productive and successful and transforms the way
companies hire, market, sell, and learn. Our vision is to create economic opportunity for every member of the global
workforce through the ongoing development of the world's first Economic Graph, a digital representation of the global
economy. In addition to LinkedIn's free services, LinkedIn offers monetized solutions designed to offer Al-enabled
insights and productivity: Talent Solutions, Marketing Solutions, Premium Subscriptions, and Sales Solutions. Talent
Solutions provide insights for workforce planning and tools to hire, nurture, and develop talent. Talent Solutions also
includes Learning Solutions, which help businesses close critical skills gaps in times where companies are having to
do more with existing talent. Marketing Solutions help companies reach, engage, and convert their audiences at scale.
Premium Subscriptions enable professionals to manage their professional identity, grow their network, find jobs, access
knowledge, and connect with talent through additional services like premium search. Sales Solutions help companies
strengthen customer relationships, empower teams with digital selling tools, and acquire new opportunities. Growth
will depend on our ability to increase the number of LinkedIn members and our ability to continue offering insight and
Al-enabled services that provide value for our members and increase their engagement. Linkedln revenue is mainly
affected by demand from enterprises and professionals for subscriptions to Talent Solutions, Sales Solutions, and
Premium Subscriptions offerings, as well as member engagement and the quality of the sponsored content delivered
to those members to drive Marketing Solutions.


# Dynamics

Dynamics provides cloud-based and on-premises business solutions for financial management, enterprise resource
planning ("ERP"), customer relationship management ("CRM"), and supply chain management, as well as other low
code application development platforms and AI offerings, for small and medium businesses, large organizations, and
divisions of global enterprises. Dynamics revenue is driven by the number of users licensed and applications
consumed, expansion of average revenue per user, and the continued shift to Dynamics 365, a unified set of cloud-
based intelligent business applications, including our low code development platforms, such as Power Apps and Power
Automate.


# Competition

Competitors to Office include software and global application vendors, such as Apple, Cisco Systems, Google, Meta,
Proofpoint, Slack, Symantec, Zoom, and numerous web-based and mobile application competitors as well as local
application developers. Apple distributes versions of its pre-installed application software, such as email and calendar
products, through its PCs, tablets, and phones. Cisco Systems is using its position in enterprise communications
equipment to grow its unified communications business. Google provides a hosted messaging and productivity suite.
Meta offers communication tools to enable productivity and engagement within organizations. Proofpoint and
Symantec provide security solutions across email security, information protection, and governance. Slack provides
teamwork and collaboration software. Zoom offers videoconferencing and cloud phone solutions. Web-based offerings
competing with individual applications have also positioned themselves as alternatives to our products and services.
We compete by providing powerful, flexible, secure, integrated industry-specific, and easy-to-use productivity and

<!-- PageNumber="9" -->
<!-- PageBreak -->


# PARTI Item 1

collaboration tools and services that create comprehensive solutions and work well with technologies our customers
already have both on-premises or in the cloud.

LinkedIn faces competition from online professional networks, recruiting companies, talent management companies,
and larger companies that are focusing on talent management and human resource services; job boards; traditional
recruiting firms; and companies that provide learning and development products and services. Marketing Solutions
competes with online and offline outlets that generate revenue from advertisers and marketers, and Sales Solutions
competes with online and offline outlets for companies with lead generation and customer intelligence and insights.

Dynamics competes with cloud-based and on-premises business solution providers such as Oracle, Salesforce, SAP,
Service Now, UI Path, and WorkDay.


## Intelligent Cloud

Our Intelligent Cloud segment consists of our public, private, and hybrid server products and cloud services that can
power modern business and developers. This segment primarily comprises:

· Server products and cloud services, including Azure and other cloud services; SQL Server, Windows
Server, Visual Studio, System Center, and related Client Access Licenses ("CALs"); and Nuance and
GitHub.

· Enterprise and partner services, including Enterprise Support Services, Industry Solutions, Nuance
professional services, Microsoft Partner Network, and Learning Experience.


### Server Products and Cloud Services

Azure is a comprehensive set of cloud services that offer developers, IT professionals, and enterprises freedom to
build, deploy, and manage applications on any platform or device. Customers can use Azure through our global network
of datacenters for computing, networking, storage, mobile and web application services, Al, Internet of Things ("loT"),
cognitive services, and machine learning. Azure enables customers to devote more resources to development and use
of applications that benefit their organizations, rather than managing on-premises hardware and software. Azure
revenue is mainly affected by infrastructure-as-a-service and platform-as-a-service consumption-based services, and
per user-based services such as Enterprise Mobility + Security.

Azure AI offerings provide a competitive advantage as companies seek ways to optimize and scale their business with
machine learning. With Azure's purpose-built, Al-optimized infrastructure, customers can use a variety of large
language models and developer tools to create the next generation of Al apps and services.

Our server products are designed to make IT professionals, developers, and their systems more productive and
efficient. Server software is integrated server infrastructure and middleware designed to support software applications
built on the Windows Server operating system. This includes the server platform, database, business intelligence,
storage, management and operations, virtualization, service-oriented architecture platform, security, and identity
software. We also license standalone and software development lifecycle tools for software architects, developers,
testers, and project managers. Server products revenue is mainly affected by purchases through volume licensing
programs, licenses sold to original equipment manufacturers ("OEM"), and retail packaged products. CALs provide
access rights to certain server products, including SQL Server and Windows Server, and revenue is reported along
with the associated server product.

Nuance and GitHub include both cloud and on-premises offerings. Nuance provides healthcare and enterprise AI
solutions. GitHub provides a collaboration platform and code hosting service for developers.


### Enterprise and Partner Services

Enterprise and Partner Services, including Enterprise Support Services, Industry Solutions, Nuance professional
services, Microsoft Partner Network, and Learning Experience, assist customers in developing, deploying, and
managing Microsoft server solutions, Microsoft desktop solutions, and Nuance conversational Al and ambient
intelligent solutions, along with providing training and certification to developers and IT professionals on various
Microsoft products.

<!-- PageNumber="10" -->
<!-- PageBreak -->

<!-- PageHeader="PARTI Item 1" -->


#### Competition

Azure faces diverse competition from companies such as Amazon, Broadcom, Google, IBM, Oracle, and open source
offerings. Azure's competitive advantage includes enabling a hybrid cloud, allowing deployment of existing datacenters
with our public cloud into a single, cohesive infrastructure, and the ability to run at a scale that meets the needs of
businesses of all sizes and complexities. Our AI offerings compete with AI products from hyperscalers such as Amazon
and Google, as well as products from other emerging competitors, including Anthropic, OpenAI, Meta, and other open
source offerings, many of which are also current or potential partners. Our Azure Security offerings include our cloud
security solution and security information and event management solution, which compete with companies such as
Palo Alto Networks and Cisco. Our Enterprise Mobility + Security offerings also compete with products from a range of
competitors including identity vendors, security solution vendors, and numerous other security point solution vendors.
We believe our cloud's global scale, coupled with our broad portfolio of identity and security solutions, allows us to
effectively solve complex cybersecurity challenges for our customers and differentiates us from the competition.

Our server products face competition from a wide variety of server operating systems and applications offered by
companies with a range of market approaches. Vertically integrated computer manufacturers such as Hewlett-Packard,
IBM, and Oracle offer their own versions of the Unix operating system preinstalled on server hardware and nearly all
computer manufacturers offer server hardware for the Linux operating system.

We compete to provide enterprise-wide computing and point solutions with numerous commercial software vendors
that offer solutions and middleware technology platforms, software applications for connectivity, security, hosting,
database, and e-business servers. IBM and Oracle lead a group of companies that compete with our enterprise-wide
computing solutions. Commercial competitors for our server applications for PC-based distributed client-server
environments include Broadcom, IBM, and Oracle. Our web application platform software competes with open source
software such as Apache, Linux, MySQL, and PHP. In middleware, we compete against Java vendors.

Our database, business intelligence, and data warehousing solutions offerings compete with products from Databricks,
IBM, Oracle, SAP, Snowflake, and other companies. Our system management solutions compete with server
management and server virtualization platform providers, such as BMC, Broadcom, Hewlett-Packard, and IBM. Our
products for software developers compete against offerings from Adobe, IBM, Oracle, and other companies, and also
against open source projects, including Eclipse (sponsored by IBM, Oracle, and SAP), PHP, and Ruby on Rails.

We believe our server products provide customers with advantages in performance, total costs of ownership, and
productivity by delivering superior applications, development tools, compatibility with a broad base of hardware and
software applications, security, and manageability.

Our Enterprise and Partner Services business competes with a wide range of companies that provide strategy and
business planning, application development, and infrastructure services, including multinational consulting firms and
small niche businesses focused on specific technologies.


### More Personal Computing

Our More Personal Computing segment consists of products and services that put customers at the center of the
experience with our technology. This segment primarily comprises:

· Windows, including Windows OEM licensing and other non-volume licensing of the Windows operating
system; Windows Commercial, comprising volume licensing of the Windows operating system, Windows
cloud services, and other Windows commercial offerings; patent licensing; and Windows Internet of Things.

· Devices, including Surface, HoloLens, and PC accessories.

· Gaming, including Xbox hardware and Xbox content and services, comprising first-party content (such as
Activision Blizzard) and third-party content, including games and in-game content; Xbox Game Pass and
other subscriptions; Xbox Cloud Gaming; advertising; third-party disc royalties; and other cloud services.

. Search and news advertising, comprising Bing (including Copilot), Microsoft News, Microsoft Edge, and
third-party affiliates.

<!-- PageNumber="11" -->
<!-- PageBreak -->

<!-- PageHeader="PARTI Item 1" -->


### Windows

The Windows operating system is designed to deliver a more personal computing experience for users by enabling
consistency of experience, applications, and information across their devices. Windows OEM revenue is impacted
significantly by the number of Windows operating system licenses purchased by OEMs, which they pre-install on the
devices they sell. In addition to computing device market volume, Windows OEM revenue is impacted by:

· The mix of computing devices based on form factor and screen size.

· Differences in device market demand between developed markets and growth markets.

· Growth of the AI PC category

· Attachment of Windows to devices shipped.

. Customer mix between consumer, small and medium businesses, and large enterprises.

· Changes in inventory levels in the OEM channel.

· Pricing changes and promotions, pricing variation that occurs when the mix of devices manufactured shifts
from local and regional system builders to large multinational OEMs, and different pricing of Windows
versions licensed.

· Constraints in the supply chain of device components.

· Piracy.

Windows Commercial revenue, which includes volume licensing of the Windows operating system and Windows cloud
services such as Microsoft Defender for Endpoint, is affected mainly by the demand from commercial customers for
Microsoft 365 and our advanced security offerings. Windows Commercial revenue often reflects the number of
information workers in a licensed enterprise and is relatively independent of the number of PCs sold in a given year.

Patent licensing includes our programs to license patents we own for use across a broad array of technology areas,
including mobile devices and cloud offerings.

Windows IoT extends the power of Windows and the cloud to intelligent systems by delivering specialized operating
systems, tools, and services for use in embedded devices.


### Devices

We design and sell devices, such as Surface (including Copilot+ PCs), HoloLens, and PC accessories. Our devices
are designed to enable people and organizations to connect to the people and content that matter most using
Windows and integrated Microsoft products and services. Surface is designed to help organizations, students, and
consumers be more productive. Growth in Devices is dependent on total PC shipments, the ability to attract new
customers, our product roadmap, and expanding into new categories.


### Gaming

Our gaming platform is designed to provide a variety of entertainment through a unique combination of content,
community, and cloud services. Our game content is developed through a collection of first-party studios creating iconic
and differentiated gaming experiences. We continue to invest in new gaming studios and content to expand our
intellectual property roadmap and leverage new content creators. These unique gaming experiences are the
cornerstone of Xbox Game Pass, a subscription service and gaming community with access to a curated library of over
400 first- and third-party console and PC titles.

The gamer remains at the heart of the Xbox ecosystem. We are identifying new opportunities to attract gamers across
a variety of different end points through our first- and third-party content and business diversification across
subscriptions, ads, and digital stores. We've seen new devices from third-party manufacturers along with key PC and
mobile end points that help us empower gamers to play in a way that is most convenient to them. We are focused on
growing the platform and expanding to new ecosystems to engage as many gamers as possible.

Xbox enables people to connect and share online gaming experiences that are accessible on Xbox consoles, Windows-
enabled devices, and other devices. Xbox is designed to benefit users by providing access to a network of certified
applications and services and to benefit our developer and partner ecosystems by providing access to a large customer
base. Xbox revenue is mainly affected by subscriptions and sales of first- and third-party content, as well as advertising.

<!-- PageNumber="12" -->
<!-- PageBreak -->


# PARTI Item 1

Growth of our Gaming business is determined by the overall active user base through Xbox enabled content, availability
of games, providing exclusive game content that gamers seek, the computational power and reliability of the devices
used to access our content and services, and the ability to create new experiences.


## Search and News Advertising

Our Search and news advertising business is designed to deliver relevant search, native, and display advertising to a
global audience. Our Microsoft Edge browser and Bing search engine with Copilot are key tools to enable user
acquisition and engagement, while our technology platform enables accelerated delivery of digital advertising solutions.
In addition to first-party tools, we have several partnerships with companies, such as Yahoo, through which we provide
and monetize search offerings. Growth depends on our ability to attract new users, understand intent, and match intent
with relevant content on advertising offerings.


### Competition

Windows faces competition from various software products and from alternative platforms and devices, mainly from
Apple and Google, and Microsoft Defender for Endpoint competes with CrowdStrike on endpoint security solutions.
We believe Windows competes effectively by giving customers choice, value, flexibility, security, an easy-to-use
interface, and compatibility with a broad range of hardware and software applications, including those that enable
productivity.

Devices face competition from various computer, tablet, and hardware manufacturers who offer a unique combination
of high-quality industrial design and innovative technologies across various price points. These manufacturers, many
of which are also current or potential partners and customers, include Apple and our Windows OEMs.

Xbox and our cloud gaming services face competition from various online gaming ecosystems and game streaming
services, including those operated by Amazon, Apple, Meta, and Tencent. We also compete with other providers of
entertainment services such as video streaming platforms. Our gaming platform competes with console platforms from
Nintendo and Sony, both of which have a large, established base of customers. We believe our gaming platform is
effectively positioned against, and uniquely differentiated from, competitive products and services based on significant
innovation in hardware architecture, user interface, developer tools, online gaming and entertainment services, and
continued strong content from our own first-party game franchises as well as other digital content offerings.

Our Search and news advertising business competes with Google, OpenAI, and a wide array of websites, social
platforms like Meta, and portals that provide content and online offerings to end users.


# OPERATIONS

We have regional operations service centers that support our operations, including customer contract and order
processing, billing, credit and collections, customer lifecycle operations, information processing, and vendor
management and logistics. The centers in Ireland and Romania support the African, European, and Middle East
regions; the centers in India and Ireland support the Asia-Pacific region; and the centers in Arlington, Virginia, Atlanta,
Georgia, Charlotte, North Carolina, Fargo, North Dakota, Fort Lauderdale, Florida, Redmond, Washington, Reno,
Nevada, and San Jose, Costa Rica support the Americas regions.

In addition to our operations centers, we also operate datacenters throughout each of these regions. We continue to
identify and evaluate opportunities to expand our datacenter locations and increase our server capacity to meet the
evolving needs of our customers, particularly given the growing demand for Al services. Our datacenters depend on
the availability of permitted and buildable land, predictable energy, networking supplies, and servers, including graphics
processing units ("GPUs") and other components.

Our devices are primarily manufactured by third-party contract manufacturers. For the majority of our products, we
have the ability to use other manufacturers if a current vendor becomes unavailable or unable to meet our requirements.
However, some of our products contain certain components for which there are very few qualified suppliers. Extended
disruptions at these suppliers could impact our ability to manufacture devices on time to meet consumer demand.

<!-- PageNumber="13" -->
<!-- PageBreak -->

<!-- PageHeader="PARTI Item 1" -->


# RESEARCH AND DEVELOPMENT


## Product and Service Development, and Intellectual Property

We develop most of our products and services internally through the following engineering groups.

· Cloud and Al - focuses on making IT professionals, developers, partners, independent software vendors,
and their systems more productive and efficient through development of Azure AI platform and cloud
infrastructure, server, database, CRM, ERP, software development tools and services, Al cognitive
services, and other business process applications and services for enterprises.

· Strategic Missions and Technologies - focuses on incubating technical products and support solutions with
transformative potential for the future of cloud computing and continued company growth, such as quantum
computing and advanced AI for science.

· Experiences and Devices - focuses on delivering high value end-user experiences across our products,
services, and devices, including Microsoft 365, Windows, Microsoft Teams, and the Surface line of devices.

· Microsoft AI - focuses on delivering online experiences targeted at consumers (including Bing, Copilot,
Start/MSN, and other advertising-based services) and developing advanced Al models.

· Microsoft Security - focuses on delivering a comprehensive portfolio of services that protect our customers'
digital infrastructure through cloud platform and application security, data protection and governance,
identity and network access, and device management.

· Technology and Research - focuses on fundamental research, product and business incubations, and
forward-looking Al innovations that span infrastructure, services, and applications. This engineering group
includes Microsoft Research, one of the world's largest corporate research organizations, which focuses on
fundamental research in Al, computer science, and a broad range of other disciplines.

· Linkedin - focuses on our services that transform the way professionals grow their network and find jobs
and the way businesses hire, market, sell, and learn.

· Gaming - focuses on developing hardware, content, and services across a large range of platforms to help
grow our user base through game experiences and social interaction.

Internal development allows us to maintain competitive advantages that come from product differentiation and closer
technical control over our products and services. It also gives us the freedom to decide which modifications and
enhancements are most important and when they should be implemented. We strive to obtain information as early as
possible about changing usage patterns and hardware advances that may affect software and hardware design. Before
releasing new software platforms, and as we make significant modifications to existing platforms, we provide
application vendors with a range of resources and guidelines for development, training, and testing. Generally, we also
create product documentation internally.

We protect our intellectual property investments in a variety of ways. We work actively in the U.S. and internationally
to ensure the enforcement of copyright, trademark, trade secret, and other protections that apply to our software and
hardware products, services, business plans, and branding. We are a leader among technology companies in pursuing
patents and currently have a portfolio of over 63,000 U.S. and international patents issued and over 23,000 pending
worldwide. While we employ much of our internally-developed intellectual property in our products and services, we
also engage in outbound licensing of specific patented technologies that are incorporated into licensees' products.
From time to time, we enter into broader cross-license agreements with other technology companies covering entire
groups of patents. We may also purchase or license technology that we incorporate into our products and services. At
times, we make select intellectual property broadly available at no or low cost to achieve a strategic objective, such as
promoting industry standards, advancing interoperability, supporting societal and/or environmental efforts, or attracting
and enabling our external development community. Our engagement with open source software also causes us to
license our intellectual property rights broadly in certain situations.

While it may be necessary in the future to seek or renew licenses relating to various aspects of our products and
services, we believe, based upon past experience and industry practice, such licenses generally can be obtained on
commercially reasonable terms. We believe our continuing research and product development are not materially
dependent on any single license or other agreement with a third party relating to the development of our products.

<!-- PageNumber="14" -->
<!-- PageBreak -->

<!-- PageHeader="PARTI Item 1" -->


## Investing in the Future

Our success is based on our ability to create new and compelling products, services, and experiences for our users,
to initiate and embrace disruptive technology trends, to enter new geographic and product markets, and to drive broad
adoption of our products and services. We invest in a range of emerging technology trends and breakthroughs that we
believe offer significant opportunities to deliver value to our customers and growth for the company. Based on our
assessment of key technology trends, we maintain our long-term commitment to research and development across a
wide spectrum of technologies, tools, and platforms spanning digital work and life experiences, cloud computing, AI,
devices, and operating systems.

While our main product research and development facilities are located in Redmond, Washington, we also operate
research and development facilities in other parts of the U.S. and around the world. This global approach helps us
remain competitive in local markets and enables us to continue to attract top talent from across the world.

We plan to continue to make significant investments in a broad range of product research and development activities,
and as appropriate we will coordinate our research and development across operating segments and leverage the
results across the company. This includes continuing to support fundamental research, which provides us with a unique
perspective on future trends and contributes to our innovation.


# DISTRIBUTION, SALES, AND MARKETING

We market and distribute our products and services through the following channels: OEMs, direct, and distributors and
resellers. Our sales organization performs a variety of functions, including working directly with commercial enterprises
and public-sector organizations worldwide to identify and meet their technology and digital transformation
requirements; managing OEM relationships; and supporting system integrators, independent software vendors, and
other partners who engage directly with our customers to perform sales, consulting, and fulfillment functions for our
products and services.


## OEMs

We distribute our products and services through OEMs that pre-install our software on new devices and servers they
sell. The largest component of the OEM business is the Windows operating system pre-installed on devices. OEMs
also sell devices pre-installed with other Microsoft products and services, including applications such as Office and the
capability to subscribe to Microsoft 365 Consumer.

There are two broad categories of OEMs. The largest category of OEMs are direct OEMs as our relationship with them
is managed through a direct agreement between Microsoft and the OEM. We have distribution agreements covering
one or more of our products with virtually all the multinational OEMs, including Dell, Hewlett-Packard, Lenovo, and with
many regional and local OEMs. The second broad category of OEMs are system builders consisting of lower-volume
PC manufacturers, which source Microsoft software for pre-installation and local redistribution primarily through the
Microsoft distributor channel rather than through a direct agreement or relationship with Microsoft.


## Direct

Many organizations that license our products and services transact directly with us through Enterprise Agreements
and Enterprise Services contracts, with sales support from system integrators, independent software vendors, web
agencies, and partners that advise organizations on licensing our products and services ("Enterprise Agreement
Software Advisors" or "ESA"). Microsoft offers direct sales programs targeted to reach small, medium, and corporate
customers, in addition to those offered through the reseller channel. A large network of partner advisors support many
of these sales.

We also sell commercial and consumer products and services directly to customers, such as cloud services, search,
and gaming, through our digital marketplaces and online stores. Additionally, our Microsoft Experience Centers are
designed to facilitate deeper engagement with our partners and customers across industries.


## Distributors and Resellers

Organizations also license our products and services indirectly, primarily through licensing solution partners ("LSP"),
distributors, value-added resellers ("VAR"), and retailers. Although each type of reselling partner may reach

<!-- PageNumber="15" -->
<!-- PageBreak -->


# PARTI Item 1

organizations of all sizes, LSPs are primarily engaged with large organizations, distributors resell primarily to VARs,
and VARs typically reach small and medium organizations. ESAs are also typically authorized as LSPs and operate
as resellers for our other volume licensing programs. Microsoft Cloud Solution Provider is our main partner program
for reselling cloud services.

We distribute our retail packaged products primarily through independent non-exclusive distributors, authorized
replicators, resellers, and retail outlets. Individual consumers obtain these products primarily through retail outlets. We
distribute our devices through third-party retailers. We have a network of field sales representatives and field support
personnel that solicit orders from distributors and resellers and provide product training and sales support.

Our Dynamics business solutions are also licensed to enterprises through a global network of channel partners
providing vertical solutions and specialized services.


## LICENSING OPTIONS

We offer options for organizations of varying sizes that want to purchase our cloud services and on-premise software.
We license these organizations under volume licensing agreements to allow the customer to acquire multiple licenses
of products and services instead of having to acquire separate licenses through retail channels. These volume licensing
programs have varying programmatic requirements and benefits to best meet the needs of our customers.

Software Assurance ("SA") conveys rights to new software and upgrades for perpetual licenses released over the
contract period. It also provides support, tools, training, and other licensing benefits to help customers deploy and use
software efficiently. SA is required to be purchased with certain volume licensing agreements and is an optional
purchase with others.


### Volume Licensing Programs


#### Enterprise Agreement

Enterprise Agreements offer large organizations a manageable volume licensing program that gives them the flexibility
to buy cloud services and software licenses under one agreement. Enterprise Agreements are designed for medium
or large organizations that want to license Microsoft products and services organization-wide over a three-year period.
Organizations can elect to purchase perpetual licenses (covered with $\mathrm { S A }$ and/or subscribe to cloud services.


#### Microsoft Customer Agreement

Microsoft Customer Agreements are simplified purchase agreements presented, accepted, and stored through a digital
experience. Microsoft Customer Agreements are non-expiring agreements that are designed to support all customers
over time, whether purchasing through a partner or directly from Microsoft.


#### Microsoft Online Subscription Agreement

Microsoft Online Subscription Agreements are designed for small and medium organizations that want to subscribe to,
activate, provision, and maintain cloud services seamlessly and directly via the web. These agreements allow
customers to acquire monthly or annual subscriptions for cloud-based services.


#### Microsoft Products and Services Agreement

Microsoft Products and Services Agreements are designed for medium and large organizations that want to license
cloud services and on-premises software as needed, with no organization-wide commitment, under a single, non-
expiring agreement. Organizations purchase perpetual licenses or subscribe to licenses. SA is optional for customers
that purchase perpetual licenses.


#### Open Value

Open Value agreements are a simple, cost-effective way to acquire the latest Microsoft technology. These agreements
are designed for small and medium organizations that want to license cloud services and on-premises software over
a three-year period. Under Open Value agreements, organizations can elect to purchase perpetual licenses or
subscribe to licenses and SA is included.

<!-- PageNumber="16" -->
<!-- PageBreak -->

<!-- PageHeader="PARTI Item 1" -->


#### Select Plus

A Select Plus agreement is designed for government and academic organizations to acquire on-premises licenses at
any affiliate or department level, while realizing advantages as one organization. Organizations purchase perpetual
licenses and SA is optional.


#### Partner Programs

The Microsoft Cloud Solution Provider Program offers customers an easy way to license the cloud services they need
in combination with the value-added services offered by their systems integrator, managed services provider, or cloud
reseller partner. Partners in this program can easily package their own products and services to directly provision,
manage, and support their customer subscriptions.

The Microsoft Services Provider License Agreement allows hosting service providers and independent software
vendors who want to license eligible Microsoft software products to provide hosted applications and software services
to their end customers. Partners license software over a three-year period and are billed monthly based on units
licensed.

The Independent Software Vendor Royalty Program enables partners to integrate Microsoft products into other
applications and then license the unified business solution to their end users.


## CUSTOMERS

Our customers include individual consumers, small and medium organizations, large global enterprises, public-sector
institutions, Internet service providers, application developers, and OEMs. Our practice is to ship our products promptly
upon receipt of purchase orders from customers; consequently, backlog is not significant.


## GOVERNMENT REGULATION

We are subject to a wide range of laws, regulations, and legal requirements in the U.S. and globally, including those
that may apply to our products and online services offerings, and those that impose requirements related to user
privacy, telecommunications, data storage and protection, advertising, and online content. How these laws and
regulations apply to our business is often unclear, subject to change over time, and sometimes may be inconsistent
from jurisdiction to jurisdiction. To comply with the accelerating global regulatory obligations, we established a
regulatory governance framework and to create a repeatable system-focused approach to regulatory governance with
an initial focus on four domains: Responsible Al, Privacy, Digital Safety, and Cybersecurity. The framework is designed
to help us maintain customer trust and confidence in our products, remain in compliance with regulators around the
globe, and effectively scale our capability to address the growing number of complex regulations. Through the
framework, our legal and regulatory subject matter experts ingest regulations, develop standards and implementation
guidance, and, when appropriate, work with our engineers to develop and implement products to monitor compliance.
Our business teams, with legal support, manage the compliance programs and prepare external regulatory and
commercial reporting, and our internal audit teams conduct reviews of our programs and processes. While we intended
to create a unified approach to regulatory compliance, some of the programs and processes established pursuant to
the framework are tailored to meet specific regulatory obligations, such as with the creation of independent compliance
functions required by the European Union ("EU") Digital Markets Act and the EU Digital Services Act, which oversee,
monitor, and assess the company's compliance with these acts.

For a description of the risks we face related to regulatory matters, refer to Risk Factors (Part I, Item 1A of this Form
10-K).

<!-- PageNumber="17" -->
<!-- PageBreak -->


# PARTI Item 1


## INFORMATION ABOUT OUR EXECUTIVE OFFICERS

Our executive officers as of July 30, 2024 were as follows:


<table>
<tr>
<th>Name</th>
<th>Age</th>
<th>Position with the Company</th>
</tr>
<tr>
<td>Satya Nadella</td>
<td>56</td>
<td>Chairman and Chief Executive Officer</td>
</tr>
<tr>
<td>Judson B. Althoff</td>
<td>51</td>
<td>Executive Vice President and Chief Commercial Officer</td>
</tr>
<tr>
<td>Kathleen T. Hogan</td>
<td>58</td>
<td>Executive Vice President and Chief Human Resources Officer</td>
</tr>
<tr>
<td>Amy E. Hood</td>
<td>52</td>
<td>Executive Vice President and Chief Financial Officer</td>
</tr>
<tr>
<td>Takeshi Numoto</td>
<td>53</td>
<td>Executive Vice President and Chief Marketing Officer</td>
</tr>
<tr>
<td>Bradford L. Smith</td>
<td>65</td>
<td>Vice Chair and President</td>
</tr>
<tr>
<td>Christopher D. Young</td>
<td>52</td>
<td>Executive Vice President, Business Development, Strategy, and Ventures</td>
</tr>
</table>


Mr. Nadella was appointed Chairman of the Board in June 2021 and Chief Executive Officer in February 2014. He
served as Executive Vice President, Cloud and Enterprise from July 2013 until that time. From 2011 to 2013,
Mr. Nadella served as President, Server and Tools. From 2009 to 2011, he was Senior Vice President, Online Services
Division. From 2008 to 2009, he was Senior Vice President, Search, Portal, and Advertising. Since joining Microsoft in
1992, Mr. Nadella's roles also included Vice President of the Business Division.

Mr. Althoff was appointed Executive Vice President and Chief Commercial Officer in July 2021. He served as Executive
Vice President, Worldwide Commercial Business from July 2017 until that time. Prior to that, Mr. Althoff served as the
President of Microsoft North America. Mr. Althoff joined Microsoft in March 2013 as President of Microsoft North
America. Mr. Althoff also serves on the Board of Directors of Ecolab Inc.

Ms. Hogan was appointed Executive Vice President and Chief Human Resources Officer in June 2023. Ms. Hogan
had been Executive Vice President, Human Resources since November 2014. Prior to that Ms. Hogan was Corporate
Vice President of Microsoft Services. She also served as Corporate Vice President of Customer Service and Support.
Ms. Hogan joined Microsoft in 2003. Ms. Hogan also serves on the Board of Directors of Alaska Air Group, Inc.

Ms. Hood was appointed Executive Vice President and Chief Financial Officer in July 2013, subsequent to her
appointment as Chief Financial Officer in May 2013. From 2010 to 2013, Ms. Hood was Chief Financial Officer of the
Microsoft Business Division. Since joining Microsoft in 2002, Ms. Hood has also held finance-related positions in the
Server and Tools Business and the corporate finance organization. Ms. Hood also serves on the Board of Directors of
3M Corporation.

Mr. Numoto was appointed Executive Vice President and Chief Marketing Officer in October 2023. He served as
Executive Vice President and Commercial Chief Marketing Officer from March 2020. Mr. Numoto served as a Corporate
Vice President, Cloud Marketing from January 2012. Prior to that, Mr. Numoto served as a Corporate Vice President
for Office 365 Marketing from 2004, where he led the transformation from traditional on-premises packaged software
to the introduction of Office 365. Since joining Microsoft in 1997, Mr. Numoto has held multiple roles in Windows
Program Management and Office Marketing.

Mr. Smith was appointed Vice Chair and President in September 2021. Prior to that, he served as President and Chief
Legal Officer since September 2015. He served as Executive Vice President, General Counsel, and Secretary from
2011 to 2015, and served as Senior Vice President, General Counsel, and Secretary from 2001 to 2011. Mr. Smith
was also named Chief Compliance Officer in 2002. Since joining Microsoft in 1993, he was Deputy General Counsel
for Worldwide Sales and previously was responsible for managing the European Law and Corporate Affairs Group,
based in Paris. Mr. Smith also serves on the Board of Directors of Netflix, Inc.

Mr. Young has served as Executive Vice President, Business Development, Strategy, and Ventures since joining
Microsoft in November 2020. Prior to Microsoft, he served as the Chief Executive Officer of McAfee, LLC from 2017 to
2020, and served as a Senior Vice President and General Manager of Intel Security Group from 2014 until 2017, when
he led the initiative to spin out McAfee into a standalone company. Mr. Young also serves on the Board of Directors of
American Express Company.

<!-- PageNumber="18" -->
<!-- PageBreak -->

<!-- PageHeader="PARTI Item 1" -->


### AVAILABLE INFORMATION

Our Internet address is www.microsoft.com. At our Investor Relations website, www.microsoft.com/investor, we make
available free of charge a variety of information for investors. Our goal is to maintain the Investor Relations website as
a portal through which investors can easily find or navigate to pertinent information about us, including:

. Our annual report on Form 10-K, quarterly reports on Form 10-Q, current reports on Form 8-K, and any
amendments to those reports, as soon as reasonably practicable after we electronically file that material
with or furnish it to the Securities and Exchange Commission ("SEC") at www.sec.gov.

· Information on our business strategies, financial results, and metrics for investors.

· Announcements of investor conferences, speeches, and events at which our executives talk about our
product, service, and competitive strategies. Archives of these events are also available.

· Press releases on quarterly earnings, product and service announcements, legal developments, and
international news.

· Corporate governance information including our articles of incorporation, bylaws, governance guidelines,
committee charters, codes of conduct and ethics, global corporate social responsibility initiatives, and other
governance-related policies.

· Other news and announcements that we may post from time to time that investors might find useful or
interesting.

· Opportunities to sign up for email alerts to have information pushed in real time.

We publish a variety of reports and resources related to our Corporate Social Responsibility programs and progress
on our Reports Hub website, www.microsoft.com/corporate-responsibility/reports-hub, including reports on
sustainability, responsible sourcing, accessibility, digital trust, and public policy engagement.

The information found on these websites is not part of, or incorporated by reference into, this or any other report we
file with, or furnish to, the SEC. In addition to these channels, we use social media to communicate to the public. It is
possible that the information we post on social media could be deemed to be material to investors. We encourage
investors, the media, and others interested in Microsoft to review the information we post on the social media channels
listed on our Investor Relations website.

<!-- PageNumber="19" -->
<!-- PageBreak -->

<!-- PageHeader="PARTI Item 1A" -->


## ITEM 1A. RISK FACTORS

Our operations and financial results are subject to various risks and uncertainties, including those described below,
that could adversely affect our business, operations, financial condition, results of operations, liquidity, and the trading
price of our common stock.


### STRATEGIC AND COMPETITIVE RISKS


#### We face intense competition across all markets for our products and services, which may adversely affect our results of operations.


##### Competition in the technology sector

Our competitors range in size from diversified global companies with significant research and development resources
to small, specialized firms whose narrower product lines may let them be more effective in deploying technical,
marketing, and financial resources. Barriers to entry in many of our businesses are low and many of the areas in which
we compete evolve rapidly with changing and disruptive technologies, shifting user needs, and frequent introductions
of new products and services. If we do not continue to innovate and provide products, devices, and services that appeal
to businesses and consumers, we may not remain competitive, which may adversely affect our business, financial
condition, and results of operations.


#### Competition among platform-based ecosystems

An important element of our business model has been to create platform-based ecosystems on which many
participants can build diverse solutions. A well-established ecosystem creates beneficial network effects among users,
application developers, and the platform provider that can accelerate growth. Establishing significant scale in the
marketplace is necessary to achieve and maintain attractive margins. We face significant competition from firms that
provide competing platforms.

· A competing vertically-integrated model, in which a single firm controls the software and hardware elements
of a product and related services, has succeeded with some consumer products such as PCs, tablets,
smartphones, gaming consoles, wearables, and other endpoint devices. Competitors pursuing this model
also earn revenue from services integrated with the hardware and software platform, including applications
and content sold through their integrated marketplaces. They may also be able to claim security and
performance benefits from their vertically integrated offer. We also offer some vertically-integrated hardware
and software products and services. Shifting a portion of our business to a vertically integrated model may
increase our cost of revenue and reduce our operating margins.

· We derive substantial revenue from licenses of Windows operating systems on PCs. We face significant
competition from competing platforms developed for new devices and form factors such as smartphones
and tablets. These devices compete on multiple bases including price and the perceived utility of the device
and its platform. Users continue to turn to these devices to perform functions that in the past were performed
by PCs. Even if many users view these devices as complementary to a PC, the prevalence of these devices
may make it more difficult to attract application developers to our PC operating system platforms. Competing
with operating systems licensed at low or no cost may decrease our PC operating system margins. Popular
products or services offered on competing platforms could increase their competitive strength. In addition,
some of our devices compete with products made by our original equipment manufacturer ("OEM") partners,
which may affect their commitment to our platform.

· Competing platforms have content and application marketplaces with scale and significant installed bases.
The variety and utility of content and applications available on a platform are important to device purchasing
decisions. Users may incur costs to move data and buy new content and applications when switching
platforms. To compete, we must successfully enlist developers to write applications for our platform and
ensure that these applications have high quality, security, customer appeal, and value. Efforts to compete
with competitors' content and application marketplaces may increase our cost of revenue and lower our
operating margins. Competitors' rules governing their content and applications marketplaces may restrict
our ability to distribute products and services through them in accordance with our technical and business
model objectives.

<!-- PageNumber="20" -->
<!-- PageBreak -->


# PARTI Item 1A

For all of these reasons, we may not be able to compete successfully against our current and future competitors, which
may adversely affect our business, operations, financial condition, and results of operations.


## Business model competition

Companies compete with us based on a growing variety of business models.

· A material part of our business involves cloud-based services available across the spectrum of computing
devices. Our competitors continue to develop and deploy cloud-based services for consumers and business
customers, and pricing and delivery models are evolving. We and our competitors are devoting significant
resources to develop and deploy our cloud-based strategies.

. We are investing in artificial intelligence ("Al") across the entire company and infusing generative Al
capabilities into our consumer and commercial offerings. We expect Al technology and services to be a
highly competitive and rapidly evolving market, and new competitors continue to enter the market. We will
bear significant development and operational costs to build and support the AI models, services, platforms,
and infrastructure necessary to meet the needs of our customers. To compete effectively we must also be
responsive to technological change, new and potential regulatory developments, and public scrutiny.

· Even as we transition more of our business to infrastructure-, platform-, and software-as-a-service business
model, the license-based proprietary software model generates a substantial portion of our software
revenue. We bear the costs of converting original ideas into software products through investments in
research and development, offsetting these costs with the revenue received from licensing our products.
Many of our competitors also develop and sell software to businesses and consumers under this model.

· Other competitors develop and offer free applications, online services, and content, and make money by
selling third-party advertising. Advertising revenue funds development of products and services these
competitors provide to users at little or no cost, competing directly with our revenue-generating products.

. Some companies compete with us by modifying and then distributing open source software at little or no
cost to end users, using open source AI models, and earning revenue on advertising or integrated products
and services. These firms do not bear the full costs of research and development for the open source
products. Some open source products mimic the features and functionality of our products.

The competitive pressures described above may cause decreased sales volumes, price reductions, and/or increased
operating costs, such as for research and development, marketing, and sales incentives, which may adversely affect
our financial condition and results of operations.

Our focus on cloud-based and AI services presents execution and competitive risks. We are incurring significant
costs to build and maintain infrastructure to support cloud computing and Al services. These costs will reduce the
operating margins. Whether we succeed in cloud-based and Al services depends on our execution in several areas,
including:

. Continuing to bring to market compelling cloud-based and Al experiences and products that generate
increasing traffic and market share.

· Maintaining the utility, compatibility, and performance of our cloud-based and Al services on the growing
array of computing devices, including PCs, smartphones, tablets, gaming consoles, and other devices.

. Continuing to enhance the attractiveness of our cloud platforms to third-party developers.

· Ensuring our cloud-based services meet the reliability expectations and specific requirements of our
customers and maintain the security of their data as well as help them meet their own compliance needs.

· Making our suite of cloud-based services platform-agnostic, available on a wide range of devices and
ecosystems, including those of our competitors.

It is uncertain whether our strategies will continue to attract users or generate the revenue required to succeed. If we
are not effective in executing organizational and technical changes to increase efficiency and accelerate innovation, or
if we fail to generate sufficient usage of our new products and services, we may not grow revenue in line with the
infrastructure and development investments described above. This may adversely affect our operations, financial
condition, and results of operations.

Our AI systems offer users powerful tools and capabilities. However, there may be instances where these systems are
used in ways that are unintended or inappropriate. In addition, some users may also engage in fraudulent or abusive

<!-- PageNumber="21" -->
<!-- PageBreak -->


# PARTI Item 1A

activities through our cloud-based services, such as unauthorized account access, payment fraud, or terms of service
violations including cryptocurrency mining or launching cyberattacks. While are committed to detecting and controlling
such misuse of our cloud-based and Al services, our efforts may not be effective, and we may incur reputational
damage or experience adverse impacts to our business and results of operations.


# RISKS RELATING TO THE EVOLUTION OF OUR BUSINESS

We make significant investments in products and services that may not achieve expected returns. We will
continue to make significant investments in research, development, and marketing for existing products, services, and
technologies. In addition, we are focused on developing new Al platform services and incorporating Al into existing
products and services. We also invest in the development and acquisition of a variety of hardware for productivity,
communication, and entertainment, including PCs, tablets, and gaming devices. Investments in new technology are
speculative. Commercial success depends on many factors, including innovation, developer support, and effective
distribution and marketing. If customers do not perceive our latest offerings as providing significant new functionality
or other value, they may reduce their purchases of new software and hardware products or upgrades, unfavorably
affecting revenue. We may not achieve significant revenue from new product, service, and distribution channel
investments for several years, if at all. New products and services may not be profitable or may not achieve operating
margins as high as we have experienced historically. We may not get engagement in certain features that drive post-
sale monetization opportunities. Our data-handling practices across our products and services will continue to be under
scrutiny. Perceptions of mismanagement, driven by regulatory activity or negative public reaction to our practices or
product experiences, could negatively impact product and feature adoption. Developing new technologies is complex.
It can require long development and testing periods. We could experience significant delays in new releases or
significant problems in creating new products or services. These factors could adversely affect our business, financial
condition, and results of operations.

Acquisitions, joint ventures, and strategic alliances may have an adverse effect on our business. We expect to
continue making acquisitions and entering into joint ventures and strategic alliances as part of our long-term business
strategy. For example, in March 2022 we completed our acquisition of Nuance Communications, Inc., and in October
2023 we completed our acquisition of Activision Blizzard, Inc. ("Activision Blizzard"). In January 2023 we announced
the third phase of our OpenAl strategic partnership. Acquisitions and other transactions and arrangements involve
significant challenges and risks, including that they do not advance our business strategy, that we get an unsatisfactory
return on our investment, that they raise new compliance-related obligations and challenges, that we have difficulty
integrating and retaining new employees, business systems, and technology, that they distract management from our
other businesses, or that announced transactions may not be completed. If an arrangement fails to adequately
anticipate changing circumstances and interests of a party, it may result in early termination or renegotiation of the
arrangement. We also have limited ability to control or influence third parties with whom we have arrangements, which
may impact our ability to realize the anticipated benefits. The success of these transactions and arrangements depend
in part on our ability to leverage them to enhance our existing products and services or develop compelling new ones,
as well as the acquired companies' ability to meet our policies and processes in areas such as data governance,
privacy, and cybersecurity. It may take longer than expected to realize the full benefits from these transactions and
arrangements, such as increased revenue or enhanced efficiencies, or the benefits may ultimately be smaller than we
expected. In addition, an acquisition may be subject to challenge even after it has been completed. For example, the
Federal Trade Commission continues to challenge our Activision Blizzard acquisition and could, if successful, alter or
unwind the transaction. These events could adversely affect our business, operations, financial condition, and results
of operations.

If our goodwill or amortizable intangible assets become impaired, we may be required to record a significant
charge to earnings. We acquire other companies and intangible assets and may not realize all the economic benefit
from those acquisitions, which could cause an impairment of goodwill or intangibles. We review our amortizable
intangible assets for impairment when events or changes in circumstances indicate the carrying value may not be
recoverable. We test goodwill for impairment at least annually. Factors that may be a change in circumstances,
indicating that the carrying value of our goodwill or amortizable intangible assets may not be recoverable, include a
decline in our stock price and market capitalization, reduced future cash flow estimates, and slower growth rates in
industry segments in which we participate. We have in the past recorded, and may in the future be required to record,
a significant charge in our consolidated financial statements during the period in which any impairment of our goodwill
or amortizable intangible assets is determined, negatively affecting our results of operations.


# CYBERSECURITY, DATA PRIVACY, AND PLATFORM ABUSE RISKS

<!-- PageNumber="22" -->
<!-- PageBreak -->


# PARTI Item 1A


## Cyberattacks and security vulnerabilities could lead to reduced revenue, increased costs, liability claims, or harm to our reputation or competitive position.


### Security of our information technology

Threats to IT security can take a variety of forms. Individual and groups of hackers and sophisticated organizations,
including state-sponsored organizations or nation-states, continuously undertake attacks that pose threats to our
customers and our IT, and we have experienced cybersecurity incidents in which such actors have gained unauthorized
access to our IT systems and data, including customer systems and data. These actors use a wide variety of methods,
which include developing and deploying malicious software; exploiting known and potential vulnerabilities or
intentionally designed processes in hardware, software, or other infrastructure to attack our products and services or
gain access to our networks and datacenters; using social engineering techniques to induce our employees, users,
partners, or customers to disclose sensitive information, such as passwords, or take other actions to gain access to
our data or our users' or customers' data; or acting in a coordinated manner or conducting coordinated attacks. For
example, as previously disclosed in our Form 8-K filed with the Securities and Exchange Commission on January 19,
2024 and amended on March 8, 2024, beginning in late November 2023, a nation-state associated threat actor used
a password spray attack to compromise a legacy test account and, in turn, gain access to Microsoft email accounts.
The threat actor used and may continue to use information it obtained to gain, or attempt to gain, unauthorized access
to some of our source code repositories and internal systems, and the threat actor may utilize this information to
otherwise adversely affect our business and results of operations. This incident has and may continue to result in harm
to our reputation and customer relationships. Additionally, we may discover additional impacts of this or other incidents
as part of our ongoing examination of this incident. Nation-state and state-sponsored actors can sustain malicious
activities for extended periods and deploy significant resources to plan and carry out attacks. Nation-state attacks
against us, our customers, or our partners have and may continue to intensify during periods of intense diplomatic or
armed conflict, such as the ongoing conflict in Ukraine. Cyber incidents and attacks, individually or in the aggregate,
could adversely affect our financial condition, results of operations, competitive position, and reputation, or expose us
to legal or regulatory risk.

Inadequate account security or organizational security practices, including those of companies we have acquired or
those of the third parties we utilize, have resulted and may result in unauthorized access to our IT systems and data,
including customer systems and data, in the future. For example, system administrators may fail to timely remove
employee account access when no longer appropriate. Employees or third parties may intentionally compromise our
or our users' security or systems or reveal confidential information. Malicious actors may employ the IT supply chain
to introduce malware through software updates or compromised supplier accounts or hardware.

Cyberthreats are constantly evolving and becoming increasingly sophisticated and complex, increasing the difficulty of
detecting and successfully defending against them. Threat actors may also utilize emerging technologies, such as AI
and machine learning. We may have no current capability to detect certain vulnerabilities or new attack methods, which
may allow them to persist in the environment over long periods of time. It may be difficult to determine the best way to
investigate, mitigate, contain, and remediate the harm caused by a cyber incident. Such efforts may not be successful,
and we may make errors or fail to take necessary actions. It is possible that threat actors may gain undetected access
to other networks and systems after establishing a foothold on an internal system. Cyber incidents and attacks can
have cascading impacts that unfold with increasing speed across our internal networks and systems, as well as those
of our partners and customers. In addition, it may take considerable time for us to investigate and evaluate the full
impact of incidents, particularly for sophisticated attacks. These factors may inhibit our ability to provide prompt, full,
and reliable information about the incident to our customers, partners, regulators, and the public. Breaches of our
facilities, network, or data security can disrupt the security of our systems and business applications, impair our ability
to provide services to our customers and protect the privacy of their data, result in product development delays,
compromise confidential or technical business information, result in theft or misuse of our intellectual property or other
assets, subject us to ransomware attacks, require us to allocate more resources to improve technologies or remediate
the impacts of attacks, or otherwise adversely affect our business. In addition, actions taken to remediate an incident
could result in outages, data losses, and disruptions of our services.

<!-- PageNumber="23" -->
<!-- PageBreak -->


# PARTI Item 1A

Our internal IT environment continues to evolve. Often, we are early adopters of new devices and technologies. We
embrace new ways of sharing data and communicating internally and with partners and customers using methods such
as social networking and other consumer-oriented technologies. Increasing use of generative AI models in our internal
systems may create new attack methods for adversaries. Our business policies and internal security controls may not
keep pace with these changes as new threats emerge or the emerging cybersecurity regulations in jurisdictions
worldwide.


## Security of our products, services, devices, and customers' data

The security of our products and services is important in our customers' decisions to purchase or use our products or
services across cloud and on-premises environments. Security threats are a significant challenge to companies like
us, whose business is providing technology products and services to others. Threats to or attacks on our own IT
infrastructure, such as the nation-state attack described in the prior risk factor, have also affected our customers and
may do so in the future. Customers using our cloud-based services rely on the security of our infrastructure, including
hardware and other elements provided by third parties, to ensure the reliability of our services and the protection of
their data. Adversaries tend to focus their efforts on the most popular operating systems, programs, and services,
including many of ours, and we expect that to continue. In addition, adversaries can attack our customers' on-premises
or cloud environments, sometimes exploiting previously unknown ("zero-day") vulnerabilities, such as the attack in
early calendar year 2021 with several of our Exchange Server on-premises products. Vulnerabilities in these or any
product can persist even after we have issued security patches if customers have not installed the most recent updates,
or if the attackers exploited the vulnerabilities before patching to install additional malware to further compromise
customers' systems. Adversaries will continue to attack customers using our cloud services as customers embrace
digital transformation. Adversaries that acquire user account information can use that information to compromise our
users' accounts, including where accounts share the same attributes such as passwords. Inadequate account security
practices may also result in unauthorized access, and user activity may result in ransomware or other malicious
software impacting a customer's use of our products or services. There may be vulnerabilities in open source software
that may make our products susceptible to cyberattacks as we increasingly incorporate open source software into our
products. Additionally, features that rely on generative Al may be susceptible to unanticipated security threats from
adversaries as we add new generative Al features to our services while continuously developing our understanding of
security risks and protection methods in the new field of generative AI.

Our customers operate complex IT systems with third-party hardware and software from multiple vendors that may
include systems acquired over many years. They expect our products and services to support all these systems and
products, including those that no longer incorporate the strongest current security advances or standards. As a result,
we may not be able to discontinue support in our services for a product, service, standard, or feature solely because a
more secure alternative is available. Failure to utilize the most current security advances and standards can increase
our customers' vulnerability to attack. Further, customers of widely varied sizes and technical sophistication use our
technology, and consequently may still have limited capabilities and resources to help them adopt and implement state-
of-the-art cybersecurity practices and technologies. In addition, we must account for this wide variation of technical
sophistication when defining default settings for our products and services, including security default settings, as these
settings may limit or otherwise impact other aspects of IT operations and some customers may have limited capability
to review and reset these defaults.

Cyberattacks may adversely impact our customers even if our production services are not directly compromised. We
are committed to notifying our customers whose systems have been impacted as we become aware and have
actionable information for customers to help protect themselves. We are also committed to providing guidance and
support on detection, tracking, and remediation. We may not be able to detect the existence or extent of these attacks
for all of our customers or have information on how to detect or track an attack, especially where an attack involves
on-premises software such as Exchange Server where we may have no or limited visibility into our customers'
computing environments.

Any of the foregoing events could result in reputational harm, loss of revenue, increased costs, or otherwise adversely
affect our business, financial condition, and results of operations.

<!-- PageNumber="24" -->
<!-- PageBreak -->


# PARTI Item 1A


## Development and deployment of defensive measures

To defend against security threats to our internal IT systems, our cloud-based services, and our customers' systems,
we must continuously engineer more secure products and services, enhance security, threat detection, and reliability
features, escalate and improve the deployment of software updates to address security vulnerabilities in our own
products as well as those provided by others in a timely manner, develop mitigation technologies that help to secure
customers from attacks even when software updates are not deployed, maintain the digital security infrastructure that
protects the integrity of our network, products, and services, and provide security tools such as firewalls, anti-virus
software, and advanced security and information about the need to deploy security measures and the impact of doing
SO.

The cost of measures to protect products and customer-facing services could reduce our operating margins. If we fail
to do these things well, actual or perceived security vulnerabilities in our products and services, data corruption issues,
or reduced performance could harm our reputation and lead customers to reduce or delay future purchases of products
or subscriptions to services, or to use competing products or services. Customers may also spend more on protecting
their existing computer systems from attack, which could delay adoption of additional products or services. Customers
in certain industries such as financial services, health care, and government may have enhanced or specialized
expectations and requirements to which we must engineer our products and services. Customers and third parties
granted access to their systems may fail to update their systems, continue to run software or operating systems we no
longer support, or may fail timely to install or enable security patches, or may otherwise fail to adopt adequate security
practices Any of these could adversely affect our reputation and results of operations. Actual or perceived vulnerabilities
may lead to claims against us. Our license agreements typically contain provisions that eliminate or limit our exposure
to liability, but there is no assurance these provisions will withstand legal challenges. At times, to achieve commercial
objectives, we may enter into agreements with larger liability exposure to customers.

Our products operate in conjunction with and are dependent on products and components across a broad ecosystem
of third parties. If there is a security vulnerability in one of these components, and if there is a security exploit targeting
it, we may experience adverse impacts to our results of operations, reputation, or competitive position.

Disclosure and misuse of personal data could result in liability and harm our reputation. As we continue to grow
the number, breadth, and scale of our cloud-based offerings, we store and process increasingly large amounts of
personal data of our customers and users. The continued occurrence of high-profile data breaches provides evidence
of an external environment increasingly hostile to information security. Despite our efforts to improve the security
controls across our business groups and geographies, it is possible our security controls over personal data, our
training of employees and third parties on data security, and other practices we follow may not prevent the improper
disclosure or misuse of customer or user data we or our vendors store and manage. Relatedly, despite our efforts to
continuously improve security controls, it is possible that we may fail to identify or mitigate insider threat activities that
could lead to the misuse of our systems or customer and user data. In addition, third parties who have limited access
to our customer or user data may use this data in unauthorized ways. Improper disclosure or misuse could harm our
reputation, lead to legal exposure to customers or users, or subject us to liability under laws that protect personal data,
resulting in increased costs or loss of revenue. Our software products and services also enable our customers and
users to store and process personal data on-premises or in a cloud-based environment we host. Government
authorities can sometimes require us to produce customer or user data in response to valid legal orders. In the U.S.
and elsewhere, we advocate for transparency concerning these requests and appropriate limitations on government
authority to compel disclosure. Despite our efforts to protect customer and user data, perceptions that the collection,
use, and retention of personal information is not satisfactorily protected could inhibit sales of our products or services
and could limit adoption of our cloud-based solutions by consumers, businesses, and government entities. Additional
security measures we may take to address customer or user concerns, or constraints on our flexibility to determine
where and how to operate datacenters in response to customer or user expectations or governmental rules or actions,
may increase costs or hinder sales of our products and services.

We may not be able to protect information in our products and services from use by others. LinkedIn and other
Microsoft products and services contain valuable information and content protected by contractual restrictions or
technical measures. In certain cases, we have made commitments to our members and users to limit access to or use
of this information. Changes in the law or interpretations of the law may weaken our ability to prevent third parties from
scraping or gathering information or content through use of bots or other measures and using it for their own benefit
which could adversely affect our business, financial condition, and results of operations.

<!-- PageNumber="25" -->
<!-- PageBreak -->


# PARTI Item 1A


## Abuse of our platforms may harm our reputation or user engagement.


### Advertising, professional, marketplace, and gaming platform abuses

For platform products and services that provide content or host ads that come from or can be influenced by third parties,
our reputation or user engagement may be negatively affected by activity that is hostile or inappropriate. This activity
may come from users impersonating other people or organizations, including through the use of AI technologies,
dissemination of information that may be viewed as misleading or intended to manipulate the opinions of our users, or
the use of our products or services that violates our terms of service or otherwise for objectionable or illegal ends.
Preventing or responding to these actions may require us to make substantial investments in people and technology
and these investments may not be successful, adversely affecting our business, financial condition, and results of
operations.


### Other digital safety abuses

Our hosted consumer services as well as our enterprise services may be used to generate or disseminate harmful or
illegal content in violation of our terms or applicable law. We may not proactively discover such content due to scale,
the limitations of existing technologies, and conflicting legal frameworks. When discovered by users and others, such
content may negatively affect our reputation, our brands, and user engagement. Regulations and other initiatives to
make platforms responsible for preventing or eliminating harmful content online have been enacted, and we expect
this to continue. We may be subject to enhanced regulatory oversight, civil or criminal liability, or reputational damage
if we fail to comply with content moderation regulations, adversely affecting our business, financial condition, and
results of operations.

Our products and services, how they are used by customers, and how third-party products and services
interact with them, may present security, privacy, and execution risks. Our products and services may contain
defects in design, manufacture, or operation that make them insecure or ineffective for their intended purposes. For
example, an Internet of Things solution may have multiple layers of hardware, sensors, processors, software, and
firmware, several of which we may not develop or control, and may have limited ability to be updated or patched.
Further, customers control our products and services, including our Al products, within their environments, and may
deploy them in high-risk scenarios or utilize them inappropriately. As a result, our products and services may
increasingly affect personal health and safety. Our products may also collect large amounts of data in manners which
may not satisfy customers or regulatory requirements. Our customers also operate complex IT systems with third-party
hardware and software from multiple vendors whose products or personnel may take or fail to take actions which
impact the reliability or security of our products and services. If our products and services do not work as intended, are
utilized in methods not intended, violate the law, or harm individuals or businesses, we may be subject to legal claims
or enforcement actions. These risks, if realized, may increase our costs, damage our reputation, or adversely affect
our results of operations.

Issues in the development and use of AI may result in reputational or competitive harm or liability. We are
building Al into many of our offerings, including our productivity services, and we are also making Al available for our
customers to use in solutions that they build. This AI may be developed by Microsoft or others, including our strategic
partner, OpenAI. We expect these elements of our business to grow. We envision a future in which Al operating in
devices, applications, and the cloud helps our customers be more productive in their work and personal lives. As with
many innovations, AI presents risks and challenges that could affect its adoption, and therefore our business. AI
algorithms or training methodologies may be flawed. Datasets may be overbroad, insufficient, or contain biased
information. Content generated by Al systems may be offensive, illegal, inaccurate, or otherwise harmful. Ineffective
or inadequate Al development or deployment practices by Microsoft or others could result in incidents that impair the
acceptance of Al solutions, cause harm to individuals, customers, or society, or result in our products and services not
working as intended. Human review of certain outputs may be required. Our implementation of AI systems could result
in legal liability, regulatory action, brand, reputational, or competitive harm, or other adverse impacts. These risks may
arise from current copyright infringement and other claims related to Al training and output, new and proposed
legislation and regulations, such as the European Union's ("EU") Al Act and the U.S.'s Al Executive Order, and new
applications of data protection, privacy, consumer protection, intellectual property, and other laws. Some Al scenarios
present ethical issues or may have broad impacts on society. If we enable or offer Al solutions that have unintended
consequences, unintended usage or customization by our customers and partners, are contrary to our responsible AI
policies and practices, or are otherwise controversial because of their impact on human rights, privacy, employment,
or other social, economic, or political issues, our reputation, competitive position, business, financial condition, and
results of operations may be adversely affected.

<!-- PageNumber="26" -->
<!-- PageBreak -->


# PARTI Item 1A


## OPERATIONAL RISKS

We may have excessive outages, data losses, and disruptions of our online services if we fail to maintain an
adequate operations infrastructure. Our increasing user traffic, growth in services, and the complexity of our
products and services demand more computing power. We spend substantial amounts to build, purchase, or lease
datacenters and equipment and to upgrade our technology and network infrastructure to handle more traffic on our
websites and in our datacenters. Our datacenters depend on the availability of permitted and buildable land, predictable
energy, networking supplies, and servers, including graphics processing units and other components. The cost or
availability of these dependencies could be adversely affected by a variety of factors, including the transition to a clean
energy economy, local and regional environmental regulations, and geopolitical disruptions. These demands continue
to increase as we introduce new products and services and support the growth and the augmentation of existing
services, including through the incorporation of Al features and/or functionality. We are rapidly growing our business
of providing a platform and back-end hosting for services provided by third parties to their end users. Maintaining,
securing, and expanding this infrastructure is expensive and complex, and requires development of principles for
datacenter builds in geographies with higher safety and reliability risks. It requires that we maintain an Internet
connectivity infrastructure and storage and compute capacity that is robust and reliable within competitive and
regulatory constraints that continue to evolve. Inefficiencies or operational failures, including temporary or permanent
loss of customer data, outages, insufficient Internet connectivity, insufficient or unavailable power or water supply, or
inadequate storage and compute capacity could diminish the quality of our products, services, and user experience,
resulting in contractual liability, claims by customers and other third parties, regulatory actions, damage to our
reputation, and loss of current and potential users, subscribers, and advertisers, each of which may adversely affect
our business, operations, financial condition, and results of operations.

We may experience quality or supply problems. There are limited suppliers for certain device and datacenter
components. We continue to identify and evaluate opportunities to expand our datacenter locations and increase our
server capacity to meet the evolving needs of our customers, particularly given the growing demand for Al services.
Capacity available to us may be affected as competitors use some of the same suppliers and materials for hardware
components. If components are delayed or become unavailable, whether because of supplier capacity constraint,
industry shortages, legal or regulatory changes that restrict supply sources, or other reasons, we may not obtain timely
replacement supplies, resulting in reduced sales or inadequate datacenter capacity to support the delivery and
continued development of our products and services. Component shortages, excess or obsolete inventory, or price
reductions resulting in inventory adjustments may increase our cost of revenue. Datacenter servers, Xbox consoles,
Surface devices, and other hardware are assembled in Asia and other geographies that may be subject to disruptions
in the supply chain, resulting in shortages which may adversely affect our business, operations, financial condition,
and results of operations.

Our software products and services also may experience quality or reliability problems. The highly sophisticated
software we develop may contain bugs and other defects that interfere with their intended operation. Our customers
increasingly rely on us for critical business functions and multiple workloads. Many of our products and services are
interdependent on one another. Our products and services may be impacted by interaction with third-party products
and services. Our customers may also utilize their own or third-party products and services whose reliability is
dependent on interaction with our products and services. Each of these circumstances potentially magnifies the impact
of quality or reliability issues. Any defects we do not detect and fix in pre-release testing could cause reduced sales,
damage to our reputation, repair or remediation costs, delays in the release of new products or versions, or legal
liability, which could adversely affect our business, financial condition, and results of operations. Although our license
agreements typically contain provisions that eliminate or limit our exposure to liability, there is no assurance these
provisions will withstand legal challenge.

Our hardware products such as Xbox consoles, Surface devices, and other devices we design and market are highly
complex. Failure to prevent, detect, or address defects in design, manufacture, or associated software could result in
recalls, safety alerts, or product liability claims, which could adversely affect our business and results of operations.


### LEGAL, REGULATORY, AND LITIGATION RISKS

Government enforcement under competition laws and new market regulation may limit how we design and
market our products. Government agencies closely scrutinize us under U.S. and foreign competition laws.
Governments are actively enforcing competition laws and regulations and enacting new regulations to intervene in
digital markets, and this includes markets such as the EU, the United Kingdom, the U.S., and China. Some jurisdictions

<!-- PageNumber="27" -->
<!-- PageBreak -->


# PARTI Item 1A

also allow competitors or consumers to assert claims of anti-competitive conduct. U.S. and foreign antitrust authorities
have previously brought enforcement actions and continue to scrutinize our business.

For example, the European Commission ("the Commission") has designated Windows and Linkedln as core platform
services subject to obligations under the EU Digital Markets Act, which prohibits certain self-preferencing behaviors
and places limitations on certain data use among other obligations. The Commission also continues to closely
scrutinize the design of high-volume Microsoft products and the terms on which we make certain technologies used in
these products, such as file formats, programming interfaces, and protocols, available to other companies. Flagship
product releases such as Microsoft 365 and Windows can receive significant scrutiny under EU or other competition
laws.

Our portfolio of first-party devices continues to grow; at the same time, our OEM partners offer a large variety of devices
for our platforms. As a result, we increasingly both cooperate and compete with our OEM partners, creating a risk that
we fail to do so in compliance with competition rules. Regulatory scrutiny in this area may increase. Certain foreign
governments, particularly in China and other countries in Asia, have advanced arguments under their competition laws
that exert downward pressure on royalties for our intellectual property.

Competition law enforcement actions and court decisions along with new market regulations may result in fines or
hinder our ability to provide the benefits of our software to consumers and businesses, reducing the attractiveness of
our products and the revenue that comes from them. New competition law actions or obligations under market
regulation schemes could be initiated, potentially using previous actions as precedent. The outcome of such actions,
or steps taken to avoid them, could adversely affect us in a variety of ways, including causing us to withdraw products
from or modify products for certain markets, decreasing the value of our assets, adversely affecting our ability to
monetize our products, or inhibiting our ability to consummate acquisition or impose conditions on acquisitions that
may reduce their value, which may adversely affect our business, financial condition, and results of operations.

Laws and regulations relating to anti-corruption and trade could result in increased costs, fines, criminal
penalties, or reputational damage. The Foreign Corrupt Practices Act ("FCPA") and other anti-corruption laws and
regulations ("Anti-Corruption Laws") prohibit corrupt payments by our employees, vendors, or agents, and the
accounting provisions of the FCPA require us to maintain accurate books and records and adequate internal controls.
From time to time, we receive inquiries from authorities in the U.S. and elsewhere which may be based on reports from
employees and others about our business activities outside the U.S. and our compliance with Anti-Corruption Laws.
Periodically, we receive such reports directly and investigate them, and also cooperate with investigations by U.S. and
foreign law enforcement authorities. An example of increasing international regulatory complexity is the EU
Whistleblower Directive, initiated in 2021, which presents compliance challenges as it is implemented in different forms
by EU member states. Most countries in which we operate also have competition laws that prohibit competitors from
colluding or otherwise attempting to reduce competition between themselves. While we devote substantial resources
to our U.S. and international compliance programs and have implemented policies, training, and internal controls
designed to reduce the risk of corrupt payments and collusive activity, our employees, partners, vendors, or agents
may violate our policies. Our failure to comply with Anti-Corruption Laws or competition laws could result in significant
fines and penalties, criminal sanctions against us, our officers, or our employees, prohibitions on the conduct of our
business, and damage to our reputation, which could adversely affect our business, financial condition, and results of
operations.

Increasing trade laws, policies, sanctions, and other regulatory requirements also affect our operations in and outside
the U.S. relating to trade and investment. Economic sanctions in the U.S., the EU, and other countries prohibit most
business with restricted entities or countries. U.S. export controls restrict Microsoft from offering many of its products
and services to, or making investments in, certain entities in specified countries. U.S. import controls restrict us from
integrating certain information and communication technologies into our supply chain and allow for government review
of transactions involving information and communications technology from countries determined to be foreign
adversaries. Supply chain regulations may impact the availability of goods or result in additional regulatory scrutiny.
Periods of intense diplomatic or armed conflict, such as the ongoing conflict in Ukraine, may result in (1) new and
rapidly evolving sanctions and trade restrictions, which may impair trade with sanctioned individuals and countries, and
(2) negative impacts to regional trade ecosystems among our customers, partners, and us. Non-compliance with
sanctions as well as general ecosystem disruptions could result in reputational harm, operational delays, monetary
fines, loss of revenue, increased costs, loss of export privileges, or criminal sanctions, which could adversely affect our
business, financial condition, and results of operations.

<!-- PageNumber="28" -->
<!-- PageBreak -->

<!-- PageHeader="PARTI Item 1A" -->

Laws and regulations relating to the handling of personal data may impede the adoption of our services or
result in increased costs, legal claims, fines against us, or reputational damage. The growth of our Internet- and
cloud-based services internationally relies increasingly on the movement of data across national boundaries. Legal
requirements relating to the collection, storage, handling, and transfer of personal data continue to evolve. For example,
while the EU-U.S. Data Privacy Framework ("DPF") has been recognized as adequate under EU law to allow transfers
of personal data from the EU to certified companies in the U.S., the DPF is subject to further legal challenge which
could cause the legal requirements for data transfers from the EU to be uncertain. EU data protection authorities have
and may again block the use of certain U.S .- based services that involve the transfer of data to the U.S. In the EU and
other markets, potential new rules and restrictions on the flow of data across borders could increase the cost and
complexity of delivering our products and services. In addition, the EU General Data Protection Regulation ("GDPR"),
which applies to all of our activities conducted from an establishment in the EU or related to products and services
offered in the EU, imposes a range of compliance obligations regarding the handling of personal data. More recently,
the EU has been developing new requirements related to the use of data, including in the Digital Markets Act, the
Digital Services Act, and the Data Act, that add additional rules and restriction on the use of data in our products and
services. Engineering efforts to build and maintain capabilities to facilitate compliance with these laws involve
substantial expense and the diversion of engineering resources from other projects. We might experience reduced
demand for our offerings if we are unable to engineer products that meet our legal duties or help our customers meet
their obligations under these and other data regulations, or if our implementation to comply makes our offerings less
attractive. Compliance with these obligations depends in part on how particular regulators interpret and apply them. If
we fail to comply, or if regulators assert we have failed to comply (including in response to complaints made by
customers), it may lead to regulatory enforcement actions, which can result in significant monetary penalties, private
lawsuits, reputational damage, blockage of product offerings or of international data transfers, and loss of customers.
The highest fines assessed under GDPR have recently been increasing, especially against large technology
companies, and European data protection authorities have taken action to block or remove services from their markets.
Jurisdictions around the world, such as China, India, and states in the U.S. have adopted, or are considering adopting
or expanding, laws and regulations imposing obligations regarding the collection, handling, and transfer of personal
data.

Our investment in gaining insights from data is becoming central to the value of the services we deliver to customers,
including Al services, to operational efficiency and key opportunities in monetization, and to customer perceptions of
quality. Our ability to use data in this way may be constrained by regulatory developments that impede realizing the
expected return from this investment. Ongoing legal analyses, reviews, and inquiries by regulators of Microsoft
practices, or relevant practices of other organizations, may result in burdensome or inconsistent requirements,
including data sovereignty and localization requirements, affecting the location, movement, collection, and use of our
customer and internal employee data as well as the management of that data. Compliance with applicable laws and
regulations regarding personal data may require changes in services, business practices, or internal systems that result
in increased costs, lower revenue, reduced efficiency, or greater difficulty in competing with foreign-based firms.
Compliance with data regulations might limit our ability to innovate or offer certain features and functionality in some
jurisdictions where we operate. Failure to comply with existing or new rules may result in significant penalties or orders
to stop the alleged noncompliant activity, negative publicity, and diversion of management time and effort.

Existing and increasing legal and regulatory requirements could adversely affect our results of operations.
We are subject to a wide range of laws, regulations, and legal requirements in the U.S. and globally, including those
that may apply to our products and online services offerings, and those that impose requirements related to user
privacy, telecommunications, data storage and protection, digital accessibility, advertising, and online content. Laws in
several jurisdictions, including EU Member State laws under the European Electronic Communications Code,
increasingly define certain of our services as regulated telecommunications services. This trend may continue and will
result in these offerings being subject to additional data protection, security, law enforcement surveillance, and other
obligations. Regulators and private litigants may assert that our collection, use, and management of customer data
and other information is inconsistent with their laws and regulations, including laws that apply to the tracking of users
via technology such as cookies. In addition, laws requiring us to retrieve and produce customer data in response to
compulsory legal demands from law enforcement and governmental authorities are expanding and the requests we
are experiencing are increasing in volume and complexity. New environmental, social, and governance laws and
regulations are expanding mandatory disclosure, reporting, and diligence requirements. Legislative or regulatory action
relating to cybersecurity requirements may increase the costs to develop, implement, or secure our products and
services. Legislative and regulatory action is emerging in the areas of Al and content moderation, which could increase
costs or restrict opportunity. For example, the EU's Al Act may increase costs or impact the provision or operation of
our AI models and services in the European market.

<!-- PageNumber="29" -->
<!-- PageBreak -->


# PARTI Item 1A

How these laws and regulations apply to our business is often unclear, subject to change over time, and sometimes
may be inconsistent from jurisdiction to jurisdiction. In addition, governments' approach to enforcement, and our
products and services, are continuing to evolve. Compliance with existing, expanding, or new laws and regulations
may involve significant costs or require changes in products or business practices that could adversely affect our results
of operations. Noncompliance could result in the imposition of penalties, criminal sanctions, or orders we cease the
alleged noncompliant activity. In addition, there is increasing pressure from advocacy groups, regulators, competitors,
customers, and other stakeholders across many of these areas. If our products do not meet customer expectations or
legal requirements, we could face regulatory or legal actions, and our business, operations, financial condition, and
results of operations could be adversely affected.

We have claims and lawsuits against us that may result in adverse outcomes. We are subject to a variety of
claims and lawsuits. These claims may arise from a wide variety of business practices and initiatives, including major
new product releases, AI services, significant business transactions, warranty or product claims, employment practices,
and regulation. As we continue to expand our business and offerings, we may experience new and novel legal claims.
Adverse outcomes in some or all of these claims may result in significant monetary damages or injunctive relief that
could adversely affect our ability to conduct our business. Litigation and other claims are subject to inherent
uncertainties and management's view of these matters may change in the future. A material adverse impact to our
financial condition and results of operations could occur for the period in which the effect of an unfavorable outcome
becomes probable and reasonably estimable.

Our business with government customers may present additional uncertainties. We derive substantial revenue
from government contracts. Government contracts generally can present risks and challenges not present in private
commercial agreements. For instance, we may be subject to government audits and investigations relating to these
contracts, we could be suspended or debarred as a governmental contractor, we could incur civil and criminal fines and
penalties, and under certain circumstances contracts may be rescinded. Some agreements may allow a government to
terminate without cause and provide for higher liability limits for certain losses. Some contracts may be subject to periodic
funding approval, reductions, cancellations, or delays which could adversely impact public-sector demand for our products
and services. These events could negatively impact our financial condition, results of operations, and reputation.

We may have additional tax liabilities. We are subject to income taxes in the U.S. and many foreign jurisdictions.
Significant judgment is required in determining our worldwide provision for income taxes. In the course of our business,
there are many transactions and calculations where the ultimate tax determination is uncertain. We may recognize
additional tax expense and be subject to additional tax liabilities due to changes in tax laws, regulations, and
administrative practices and principles, including changes to the global tax framework, in various jurisdictions. In recent
years, multiple domestic and international tax proposals were proposed to impose greater tax burdens on large
multinational enterprises. For example, the Organisation for Economic Co-operation and Development continues to
advance proposals or guidance in international taxation, including the establishment of a global minimum tax.

We are regularly under audit by tax authorities in different jurisdictions. Although we believe that our provision for
income taxes and our tax estimates are reasonable, tax authorities may disagree with certain positions we have taken.
In addition, economic and political pressures to increase tax revenue in various jurisdictions may make resolving tax
disputes favorably more difficult. We are currently under Internal Revenue Service ("IRS") audit for prior tax years and
have received Notices of Proposed Adjustment ("NOPAs") from the IRS for the tax years 2004 to 2013. The primary
issues in the NOPAs relate to intercompany transfer pricing. In the NOPAs, the IRS is seeking an additional tax
payment of $28.9 billion plus penalties and interest. The final resolution of the proposed adjustments, and other audits
or litigation, may differ from the amounts recorded in our consolidated financial statements and adversely affect our
results of operations in the period or periods in which that determination is made.

We earn a significant amount of our operating income outside the U.S. A change in the mix of earnings and losses in
countries with differing statutory tax rates, changes in our business or structure, or the expiration of or disputes about
certain tax agreements in a particular country may result in higher effective tax rates for the company. In addition,
changes in U.S. federal and state or international tax laws applicable to corporate multinationals, other global
fundamental law changes currently being considered by many countries, including in the U.S., and changes in taxing
jurisdictions' administrative interpretations, decisions, policies, and positions may materially adversely affect our
financial condition and results of operations.

We are subject to evolving sustainability regulatory requirements and expectations, which exposes us to
increased costs and legal and reputational risks. Laws, regulations, and policies relating to environmental, social,
and governance matters are being developed and formalized in Europe, the U.S., and elsewhere, which may include

<!-- PageNumber="30" -->
<!-- PageBreak -->

<!-- PageHeader="PARTI Item 1A" -->

specific, target-driven frameworks and disclosure requirements. In addition, we have established and publicly
announced goals and commitments to become carbon negative, water positive, zero waste, and protect more land
than we use. Any failure or perceived failure to pursue or fulfill our sustainability goals and commitments or to satisfy
various sustainability reporting standards or regulatory requirements within the timelines we announce, or at all, could
result in claims and lawsuits, regulatory actions, or damage to our reputation, each of which may adversely affect our
business, operations, financial condition, and results of operations.


# INTELLECTUAL PROPERTY RISKS

We face risks related to the protection and utilization of our intellectual property that may result in our
business and operating results being harmed. Protecting our intellectual property rights and combating unlicensed
copying and use of our software, source code, and other intellectual property on a global basis is difficult. Similarly, the
absence of harmonized patent laws makes it more difficult to ensure consistent respect for patent rights.

Changes in the law may continue to weaken our ability to prevent the use of patented technology. Our increasing
engagement with open source software will also cause us to license our intellectual property rights broadly in certain
situations. If we are unable to protect our intellectual property, our results of operations may be adversely affected.

Source code, the detailed program commands for our operating systems and other software programs, is critical to our
business. If our source code leaks, we might lose future trade secret protection for that code. It may then become
easier for third parties to compete with our products by copying functionality, which could adversely affect our results
of operations. Unauthorized access to or disclosure of source code or other intellectual property also could increase
the security risks described elsewhere in these risk factors.

Third parties may claim that we infringe their intellectual property. From time to time, others claim we infringe
their intellectual property rights, including current copyright infringement and other claims arising from Al training and
output. To resolve these claims, we may enter into royalty-bearing data access or licensing agreements on terms that
are less favorable than currently available, stop selling or redesign affected products or services, or pay damages to
satisfy indemnification commitments with our customers. Adverse outcomes could also include monetary damages or
injunctive relief that may limit or prevent importing, marketing, and selling our products or services that have infringing
technologies. We have paid significant amounts to settle claims related to the use of technology and intellectual
property rights and to procure intellectual property rights as part of our strategy to manage this risk, and may continue
to do so, which could adversely affect our results of operations.


# GENERAL RISKS

If our reputation or our brands are damaged, our business and results of operations may be harmed. Our
reputation and brands are globally recognized and are important to our business. Our reputation and brands affect our
ability to attract and retain consumer, business, and public-sector customers. There are numerous ways our reputation
or brands could be damaged. These include product safety or quality issues, our environmental impact and
sustainability, supply chain practices, or human rights record. We may experience backlash from customers,
government entities, advocacy groups, employees, and other stakeholders that disagree with our product offering
decisions, public policy positions, or corporate philanthropic initiatives. Damage to our reputation or our brands may
occur from, among other things:

· The introduction of new features, products, services, or terms of service that customers, users, or partners
do not like.

· Public scrutiny of our decisions regarding user privacy, data practices, content, or development and
deployment of AI.

· Data security breaches, cybersecurity incidents, responsible Al failures, compliance failures, or actions of
partners or individual employees.

Social media may increase the likelihood, speed, and magnitude of negative brand events. If our brands or reputation
are damaged, it could adversely affect our business, results of operations, or ability to attract the most highly qualified
employees.

Adverse economic or market conditions may harm our business. Worsening economic conditions, including
inflation, recession, pandemic, or other changes in economic conditions, may cause lower IT spending and adversely

<!-- PageNumber="31" -->
<!-- PageBreak -->


# PARTI Item 1A

affect our results of operations. If demand for PCs, servers, and other computing devices declines, or consumer or
business spending for those products declines, our results of operations may be adversely affected.

Our product distribution system relies on an extensive partner and retail network. OEMs building devices that run our
software have also been a significant means of distribution. The impact of economic conditions on our partners, such
as the bankruptcy of a major distributor, OEM, or retailer, could cause sales channel disruption.

Challenging economic conditions also may impair the ability of our customers to pay for products and services they
have purchased. As a result, allowances for doubtful accounts and write-offs of accounts receivable may increase.

We maintain an investment portfolio of various holdings, types, and maturities. These investments are subject to
general credit, liquidity, market, and interest rate risks, which may be exacerbated by market downturns or events that
affect global financial markets. A significant part of our investment portfolio comprises U.S. government securities. If
global financial markets decline for long periods, or if there is a downgrade of the U.S. government credit rating due to
an actual or threatened default on government debt, our investment portfolio may be adversely affected and we could
determine that more of our investments have experienced a decline in fair value, requiring impairment charges that
could adversely affect our financial condition and results of operations.

Catastrophic events or geopolitical conditions may disrupt our business. A disruption or failure of our systems,
operations, or supply chain because of a major earthquake, weather event, cyberattack, terrorist attack, pandemic, or
other catastrophic event could cause delays in completing sales, providing services, or performing other critical
functions. Our corporate headquarters, a significant portion of our research and development activities, and certain
other essential business operations are in the Seattle, Washington area, and we have other business operations in the
Silicon Valley area of California, both of which are seismically active regions. A catastrophic event that results in the
destruction or disruption of any of our critical business or IT systems, or the infrastructure or systems they rely on, such
as power grids, could harm our ability to conduct normal business operations or adversely affect our results of
operations. Providing our customers with more services and solutions in the cloud puts a premium on the resilience of
our systems and strength of our business continuity management plans and magnifies the potential negative
consequences of prolonged service outages.

Abrupt political change, terrorist activity, and armed conflict, such as the ongoing conflict in Ukraine, pose economic
and other risks, which may negatively impact our ability to sell to and collect from customers, increase our operating
costs, or otherwise disrupt our operations in markets both directly and indirectly impacted by such events. These
conditions also may add uncertainty to the timing and budget for technology investment decisions by our customers

regulatory systems and requirements and market interventions that could impact our operating strategies, access to
national, regional, and global markets, hiring, and profitability. Geopolitical instability may lead to sanctions and impact
our ability to do business in some markets or with some public-sector customers. Any of these changes may negatively
affect our results of operations.

The occurrence of regional epidemics or a global pandemic, such as COVID-19, may adversely affect our business,
operations, financial condition, and results of operations. The extent to which global pandemics impact our business
going forward will depend on factors such as the duration and scope of the pandemic; governmental, business, and
individuals' actions in response to the pandemic; and the impact on economic activity, including the possibility of
recession or financial market instability. Measures to contain a global pandemic may intensify other risks described in
these Risk Factors.

The long-term effects of climate change on the global economy and the IT industry in particular are unclear.
Environmental regulations or changes in the supply, demand, or available sources of energy or other resources may
affect the availability or cost of goods and services, including natural resources, necessary to run our business.
Changes in climate where we operate may increase the costs of powering and cooling computer hardware we use to
develop software and provide cloud-based services.

Our global business exposes us to operational and economic risks. Our customers are located throughout the world
and a significant part of our revenue comes from international sales. The global nature of our business creates operational,
economic, and geopolitical risks. Global, regional, and local economic developments, monetary policy, inflation, and
recession, as well as political and military disputes, may adversely affect our results of operations. In addition, our
international growth strategy includes certain markets, the developing nature of which presents several risks, including
deterioration of social, political, labor, or economic conditions in a country or region, and difficulties in staffing and
managing foreign operations. Emerging nationalist and protectionist trends and concerns about human rights, the

<!-- PageNumber="32" -->
<!-- PageBreak -->


# PARTI Item 1A

environment, and political expression in specific countries may significantly alter the trade and commercial environments.
Changes to trade policy or agreements as a result of populism, protectionism, or economic nationalism may result in
higher tariffs, local sourcing initiatives, and non-local sourcing restrictions, export controls, investment restrictions, or other
developments that make it more difficult to sell our products in foreign countries. Disruptions of these kinds in developed
or emerging markets could negatively impact demand for our products and services, impair our ability to operate in certain
regions, or increase operating costs. Although we hedge a portion of our international currency exposure, significant
fluctuations in foreign exchange rates between the U.S. dollar and foreign currencies may adversely affect our results of
operations.

Our business depends on our ability to attract and retain talented employees. Our business is based on
successfully attracting, training, and retaining talented employees representing diverse backgrounds, experiences, and
skill sets. The market for highly skilled workers and leaders in our industry is extremely competitive. Maintaining our
brand and reputation, as well as a diverse and inclusive work environment that enables all our employees to thrive, are
important to our ability to recruit and retain employees. We are also limited in our ability to recruit internationally by
restrictive domestic immigration laws. Restraints on the flow of technical and professional talent, including as a result
of changes to U.S. immigration policies or laws, may inhibit our ability to adequately staff our research and development
efforts. If we are less successful in our recruiting efforts, or if we cannot retain highly skilled workers and key leaders,
our ability to develop and deliver successful products and services may be adversely affected. Effective succession
planning is also important to our long-term success. Failure to ensure effective transfer of knowledge and smooth
transitions involving key employees could hinder our strategic planning and execution. How employment-related laws
are interpreted and applied to our workforce practices may result in increased operating costs and less flexibility in how
we meet our workforce needs. Our global workforce is predominantly non-unionized, although we do have some
employees in the U.S. and internationally who are represented by unions or works councils. In the U.S., there has been
a general increase in workers exercising their right to form or join a union. The unionization of significant employee
populations could result in higher costs and other operational changes necessary to respond to changing conditions
and to establish new relationships with worker representatives.

<!-- PageNumber="33" -->
<!-- PageBreak -->

<!-- PageHeader="PARTI Item 1B, 1C" -->


## ITEM 1B. UNRESOLVED STAFF COMMENTS

We have received no written comments regarding our periodic or current reports from the staff of the Securities and
Exchange Commission that were issued 180 days or more preceding the end of our fiscal year 2024 that remain
unresolved.


## ITEM 1C. CYBERSECURITY


### RISK MANAGEMENT AND STRATEGY

Microsoft plays a central role in the world's digital ecosystem. We have made it the top corporate priority to protect the
computing environment used by our customers and employees and to support the resiliency of our cloud infrastructure
and services, products, devices, and our internal corporate resources from determined adversaries. In response to the
evolving cybersecurity threat landscape, we launched the Secure Future Initiative ("SFI") in November 2023 and
expanded the scope of SFI in May 2024. The SFI focuses our business strategy and efforts on continual improvement
in cybersecurity protection, and is aligned around three security principles:

· Secure by Design: Security comes first when designing any product or service.

· Secure by Default: Security protections are enabled and enforced by default, require no extra effort, and
are not optional.

· Secure Operations: Security controls and monitoring will continuously be improved to meet current and
future threats.

We operate a cybersecurity program and governance framework designed to protect our computing environments
against cybersecurity threats, and we have controls, policies, and procedures to identify, manage, and mitigate
cybersecurity threats. Annually, we assess our cybersecurity program's alignment with the National Institute of
Standards & Technology's Cyber Security Framework ("NIST") and other applicable industry standards. We also
undertake integrated planning and preparedness activities to support business continuity and operational resiliency.
We assess our program's effectiveness through various exercises, including tabletop simulations and production
environment tests, penetration and vulnerability tests, red team exercises, and other related activities. We conduct
mandatory cybersecurity training, provide employees with tools to report suspected incidents and assess their own
security posture, and conduct real-time simulated employee education exercises, such as phishing email campaigns
designed to emulate real-world attacks. We also engage in robust cybersecurity assessments and remediation efforts
for acquired companies.

Our computing environments, products, and services are reviewed by our internal audit teams as well as independent
third-party assessors. We are committed to managing the most significant risks to our strategies and ambitions,
including cybersecurity risks. The Enterprise Risk Management ("ERM") organization supports management in this
commitment by facilitating the semiannual risk assessment, which documents the priority and status of these risks and
aligns them with our strategic mitigation efforts. ERM is structured using a framework based on the Committee of
Sponsoring Organization ("COSO") guidance on Enterprise Risk Management Integrating Strategy with Performance
and it also aligns with the International Organization for Standardization 31000:2018 Risk Management Standard.

We continuously monitor our computing environments, products, and services for vulnerabilities and signs of
compromise, and we utilize our own security products to combat cybersecurity threats. We integrate security into our
computing environments, products, and services through our Security Development Lifecycle ("SDL"). Our SDL
introduces security and privacy considerations throughout all phases of our development process and through the
adoption of zero-trust end-to-end architecture. We utilize machine learning and AI-powered security tools to gain
insights from over 78 trillion signals per day and over 135 million managed devices. We track over 300 unique threat
actors, including 160 nation-state actors and 50 ransomware groups. To support our efforts, we operate a Cyber
Defense Operations Center connected to over 10,000 security and threat intelligence experts, including engineers,
researchers, data scientists, cybersecurity experts, threat hunters, geopolitical analysts, investigators, and frontline
responders across the globe.

<!-- PageNumber="34" -->
<!-- PageBreak -->


# PARTI Item 1C

When appropriate, we utilize external service providers to assess, test, or otherwise assist our program. We also
leverage third parties by working with external researchers, operating bug bounty programs, and managing coordinated
vulnerability disclosure programs with security organizations. We maintain a systematic approach to assessing and
controlling the cybersecurity risks presented by third-party service providers. We require third-party service providers
to manage their cybersecurity risks in defined ways, undergo cybersecurity reviews, notify us of cyber events, and
satisfy additional contractual requirements.

We seek to improve the entire cybersecurity ecosystem through multistakeholder diplomacy to set and uphold
expectations for state behavior, advancement of government policy that strengthens cybersecurity and resiliency,
disruption and deterrence of cybercrime, protection of national security interests, and disruption of digital threats to
democracies. We also establish processes and innovate solutions for us and our customers to address the growing
number and complexity of cybersecurity regulations.

When we experience a cybersecurity incident, we utilize our well-established incident response plans that operate both
across the company and at the product and services level. Incidents are first triaged for severity, and then more deeply
assessed to establish a plan of record and activate internal and external notification, disclosure, and communication
plans, as applicable. Engineering and development resources are mobilized to resolve or remediate the incident. After
the incident is resolved, a comprehensive post-incident review process is conducted.

We describe the risks from cybersecurity threats, including previous cybersecurity incidents, in section "Risk Factors"
(Part I, Item 1A of this Form 10-K). As of the date of this Form 10-K, we do not believe any risks from cybersecurity
threats have materially affected or are reasonably likely to materially affect us, including our results of operations or
financial condition. However, the cybersecurity threat environment is increasingly challenging, and we, along with the
entire digital ecosystem, are under constant and increasing threat. As discussed above, our business strategy is tied
to the SFI and we are committed to continuously monitoring cybersecurity threats, enhancing the security of our
products, investing in our cybersecurity infrastructure, and collaborating with peers, customers, service providers,
regulators, and governments to advance our and the entire digital ecosystem's cybersecurity defenses and resiliency.


# GOVERNANCE

Our Board of Directors oversees cybersecurity risk. Cybersecurity reviews by the Board are scheduled to occur at least
quarterly, or more frequently as determined to be necessary or advisable. Presentations to the Board of Directors are
made by senior management, including our Chief Information Security Officer ("CISO"), our EVP of Microsoft Security,
and the head of our Customer Security and Trust organization. The presentations address topics such as cybersecurity
threats, incidents, top risks and related remediation efforts, results from internal and third-party assessments, progress
towards risk-mitigation goals, the functioning of our incident response program, regulatory developments, and digital
diplomacy efforts. In addition, we have an escalation process in place to inform senior management and the Board of
significant issues. Cybersecurity issues are also considered during separate Board meeting discussions regarding
important matters like ERM, audit issues, operational budgeting, business continuity planning, mergers and
acquisitions, brand management, and other relevant matters.

Our CISO leads the strategy, engineering, and operations of cybersecurity across the company, and reports to the
EVP of Microsoft Security. Our CISO has extensive experience assessing and managing cybersecurity programs and
cybersecurity risk. Before joining Microsoft, our CISO served in a prior Chief Technology Officer role as well as in senior
leadership, engineering, and operational roles within multiple organizations. In addition to the Board's oversight of
cybersecurity risk, to support the CISO, we have established a Cybersecurity Governance Council ("CGC") charged
with overseeing initiatives that safeguard Microsoft's infrastructure. The CGC is comprised of an executive-level team
of Deputy CISOs with cybersecurity backgrounds and expertise relevant to their roles. The CGC responsibilities include
approving our enterprise security risk assessment process and results, determining the appropriate cybersecurity risk
level and mitigations, reviewing the NIST CSF alignment, and supporting compliance with cybersecurity regulations.
Our cybersecurity efforts are supported directly by Microsoft's security and threat intelligence experts and our
employees across the company, all of whom receive cybersecurity awareness training and education and are expected
to support our efforts.

<!-- PageNumber="35" -->
<!-- PageBreak -->

<!-- PageHeader="PARTI Item 2, 3, 4" -->


# ITEM 2. PROPERTIES

Our corporate headquarters are located in Redmond, Washington. We have approximately 15 million square feet of
space located in King County, Washington that is used for engineering, sales, marketing, and operations, among other
general and administrative purposes. These facilities include approximately 12 million square feet of owned space
situated on approximately 530 acres of land we own at our corporate headquarters, and approximately 3 million square
feet of space we lease.

We own and lease other facilities domestically and internationally, primarily for offices, datacenters, and research and
development. The largest owned international properties include space in the following locations: China, India, Ireland,
and the Netherlands. The largest leased international properties include space in the following locations: Australia,
Canada, China, France, Germany, India, Ireland, Israel, Japan, the Netherlands, and the United Kingdom. Refer to
Research and Development (Part I, Item 1 of this Form 10-K) for further discussion of our research and development
facilities.

The table below shows a summary of the square footage of our properties owned and leased domestically and
internationally as of June 30, 2024:

(Square feet in millions)


<table>
<tr>
<th>Location</th>
<th>Owned</th>
<th>Leased</th>
<th>Total</th>
</tr>
<tr>
<td>U.S.</td>
<td>30</td>
<td>20</td>
<td>50</td>
</tr>
<tr>
<td>International</td>
<td>10</td>
<td>25</td>
<td>35</td>
</tr>
<tr>
<td>Total</td>
<td>40</td>
<td>45</td>
<td>85</td>
</tr>
</table>


# ITEM 3. LEGAL PROCEEDINGS

Refer to Note 15 - Contingencies of the Notes to Financial Statements (Part II, Item 8 of this Form 10-K) for information
regarding legal proceedings in which we are involved.


# ITEM 4. MINE SAFETY DISCLOSURES

Not applicable.

<!-- PageNumber="36" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 5, 6" -->


# PART II


## ITEM 5. MARKET FOR REGISTRANT'S COMMON EQUITY, RELATED STOCKHOLDER MATTERS, AND ISSUER PURCHASES OF EQUITY SECURITIES


### MARKET AND STOCKHOLDERS

Our common stock is traded on the NASDAQ Stock Market under the symbol MSFT. On July 25, 2024, there were
81,346 registered holders of record of our common stock.


### SHARE REPURCHASES AND DIVIDENDS

Following are our monthly share repurchases for the fourth quarter of fiscal year 2024:


<table>
<tr>
<th>Period</th>
<th>Total Number of Shares Purchased</th>
<th>Average Price Paid Per Share</th>
<th>Total Number of Shares Purchased as Part of Publicly Announced Plans or Programs</th>
<th>Approximate Dollar Value of Shares That May Yet Be Purchased Under the Plans or Programs</th>
</tr>
<tr>
<td></td>
<td></td>
<td></td>
<td></td>
<td>(In millions)</td>
</tr>
<tr>
<td>April 1, 2024 - April 30, 2024</td>
<td>2,444,905</td>
<td>413.75</td>
<td>2,444,905</td>
<td>$ 12,138</td>
</tr>
<tr>
<td>May 1, 2024 - May 31, 2024</td>
<td>2,233,450</td>
<td>416.85</td>
<td>2,233,450</td>
<td>11,207</td>
</tr>
<tr>
<td>June 1, 2024 - June 30, 2024</td>
<td>1,963,873</td>
<td>436.58</td>
<td>1,963,873</td>
<td>10,349</td>
</tr>
<tr>
<td></td>
<td>6,642,228</td>
<td></td>
<td>6,642,228</td>
<td></td>
</tr>
</table>


All share repurchases were made using cash resources. Our share repurchases may occur through open market
purchases or pursuant to a Rule 10b5-1 trading plan. The above table excludes shares repurchased to settle employee
tax withholding related to the vesting of stock awards.

Our Board of Directors declared the following dividends during the fourth quarter of fiscal year 2024:


<table>
<tr>
<th>Declaration Date</th>
<th>Record Date</th>
<th>Payment Date</th>
<th>Dividend Per Share</th>
<th>Amount</th>
</tr>
<tr>
<td></td>
<td></td>
<td></td>
<td></td>
<td>(In millions)</td>
</tr>
<tr>
<td>June 12, 2024</td>
<td>August 15, 2024</td>
<td>September 12, 2024</td>
<td>$ 0.75</td>
<td>$ 5,575</td>
</tr>
</table>


We returned $8.4 billion to shareholders in the form of share repurchases and dividends in the fourth quarter of fiscal
year 2024. Refer to Note 16 - Stockholders' Equity of the Notes to Financial Statements (Part II, Item 8 of this Form
10-K) for further discussion regarding share repurchases and dividends.

ITEM 6. [RESERVED]

<!-- PageNumber="37" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 7" -->


## ITEM 7. MANAGEMENT'S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS

The following Management's Discussion and Analysis of Financial Condition and Results of Operations ("MD&A") is
intended to help the reader understand the results of operations and financial condition of Microsoft Corporation. MD&A
is provided as a supplement to, and should be read in conjunction with, our consolidated financial statements and the
accompanying Notes to Financial Statements (Part II, Item 8 of this Form 10-K). This section generally discusses the
results of our operations for the year ended June 30, 2024 compared to the year ended June 30, 2023. For a discussion
of the year ended June 30, 2023 compared to the year ended June 30, 2022, please refer to Part II, Item 7,
"Management's Discussion and Analysis of Financial Condition and Results of Operations" in our Annual Report on
Form 10-K for the year ended June 30, 2023.


### OVERVIEW

Microsoft is a technology company committed to making digital technology and artificial intelligence ("Al") available
broadly and doing so responsibly, with a mission to empower every person and every organization on the planet to
achieve more. We create platforms and tools, powered by Al, that deliver innovative solutions that meet the evolving
needs of our customers.

We generate revenue by offering a wide range of cloud-based solutions, content, and other services to people and
businesses; licensing and supporting an array of software products; delivering relevant online advertising to a global
audience; and designing and selling devices. Our most significant expenses are related to compensating employees;
supporting and investing in our cloud-based services, including datacenter operations; designing, manufacturing,
marketing, and selling our other products and services; and income taxes.

Highlights from fiscal year 2024 compared with fiscal year 2023 included:

· Microsoft Cloud revenue increased 23% to $137.4 billion.

· Office Commercial products and cloud services revenue increased 14% driven by Office 365 Commercial
growth of 16%.

· Office Consumer products and cloud services revenue increased 4% and Microsoft 365 Consumer
subscribers grew to 82.5 million.

· LinkedIn revenue increased 9%.

· Dynamics products and cloud services revenue increased 19% driven by Dynamics 365 growth of 24%.

· Server products and cloud services revenue increased 22% driven by Azure and other cloud services
growth of 30%.

· Windows revenue increased 8% with Windows original equipment manufacturer licensing ("Windows OEM")
revenue growth of 7% and Windows Commercial products and cloud services revenue growth of 11%.

· Devices revenue decreased 15%.

· Xbox content and services revenue increased 50% driven by 44 points of net impact from the Activision
Blizzard Inc. ("Activision Blizzard") acquisition. The net impact reflects the change of Activision Blizzard
content from third-party to first-party.

. Search and news advertising revenue excluding traffic acquisition costs increased 12%.

On October 13, 2023, we completed our acquisition of Activision Blizzard for a total purchase price of $75.4 billion,
consisting primarily of cash. The financial results of Activision Blizzard have been included in our consolidated financial
statements since the date of the acquisition. Activision Blizzard is reported as part of our More Personal Computing
segment. Refer to Note 8 - Business Combinations of the Notes to Financial Statements (Part II, Item 8 of this Form
10-K) for further discussion.


#### Industry Trends

Our industry is dynamic and highly competitive, with frequent changes in both technologies and business models. Each
industry shift is an opportunity to conceive new products, new technologies, or new ideas that can further transform
the industry and our business. At Microsoft, we push the boundaries of what is possible through a broad range of

<!-- PageNumber="38" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 7" -->

research and development activities that seek to identify and address the changing demands of customers and users,
industry trends, and competitive forces.


#### Economic Conditions, Challenges, and Risks

The markets for software, devices, and cloud-based services are dynamic and highly competitive. Our competitors are
developing new software and devices, while also deploying competing cloud-based services for consumers and
businesses. The devices and form factors customers prefer evolve rapidly, influencing how users access services in
the cloud and, in some cases, the user's choice of which suite of cloud-based services to use. Aggregate demand for
our software, services, and devices is also correlated to global macroeconomic and geopolitical factors, which remain
dynamic. We must continue to evolve and adapt over an extended time in pace with this changing environment.

The investments we are making in cloud and AI infrastructure and devices will continue to increase our operating costs
and may decrease our operating margins. We continue to identify and evaluate opportunities to expand our datacenter
locations and increase our server capacity to meet the evolving needs of our customers, particularly given the growing
demand for Al services. Our datacenters depend on the availability of permitted and buildable land, predictable energy,
networking supplies, and servers, including graphics processing units ("GPUs") and other components. Our devices
are primarily manufactured by third-party contract manufacturers. For the majority of our products, we have the ability
to use other manufacturers if a current vendor becomes unavailable or unable to meet our requirements. However,
some of our products contain certain components for which there are very few qualified suppliers. Extended disruptions
at these suppliers could impact our ability to manufacture devices on time to meet consumer demand.

Our success is highly dependent on our ability to attract and retain qualified employees. We hire a mix of university
and industry talent worldwide. We compete for talented individuals globally by offering an exceptional working
environment, broad customer reach, scale in resources, the ability to grow one's career across many different products
and businesses, and competitive compensation and benefits.

Our international operations provide a significant portion of our total revenue and expenses. Many of these revenue
and expenses are denominated in currencies other than the U.S. dollar. As a result, changes in foreign exchange rates
may significantly affect revenue and expenses. Fluctuations in the U.S. dollar relative to certain foreign currencies did
not have a material impact on reported revenue and expenses from our international operations in fiscal year 2024.

Refer to Risk Factors (Part I, Item 1A of this Form 10-K) for a discussion of these factors and other risks.


#### Seasonality

Our revenue fluctuates quarterly and is generally higher in the fourth quarter of our fiscal year. Fourth quarter revenue
is driven by a higher volume of multi-year contracts executed during the period.


#### Change in Accounting Estimate

In July 2022, we completed an assessment of the useful lives of our server and network equipment. Due to investments
in software that increased efficiencies in how we operate our server and network equipment, as well as advances in
technology, we determined we should increase the estimated useful lives of both server and network equipment from
four years to six years. This change in accounting estimate was effective beginning fiscal year 2023.


#### Reportable Segments

We report our financial performance based on the following segments: Productivity and Business Processes, Intelligent
Cloud, and More Personal Computing. The segment amounts included in MD&A are presented on a basis consistent
with our internal management reporting.

Additional information on our reportable segments is contained in Note 19 - Segment Information and Geographic
Data of the Notes to Financial Statements (Part II, Item 8 of this Form 10-K).


#### Metrics

<!-- PageNumber="39" -->
<!-- PageBreak -->


# PART II Item 7

We use metrics in assessing the performance of our business and to make informed decisions regarding the allocation
of resources. We disclose metrics to enable investors to evaluate progress against our ambitions, provide transparency
into performance trends, and reflect the continued evolution of our products and services. Our commercial and other
business metrics are fundamentally connected based on how customers use our products and services. The metrics
are disclosed in the MD&A or the Notes to Financial Statements (Part II, Item 8 of this Form 10-K). Financial metrics
are calculated based on financial results prepared in accordance with accounting principles generally accepted in the
United States of America ("GAAP"), and growth comparisons relate to the corresponding period of last fiscal year.

In the first quarter of fiscal year 2024, we made updates to the presentation and method of calculation for certain
metrics, revising our Microsoft Cloud revenue metric to include revenue growth and expanding our Microsoft 365
Consumer subscribers metric to include Microsoft 365 Basic subscribers, aligning with how we manage our business.


## Commercial

Our commercial business primarily consists of Server products and cloud services, Office Commercial, Windows
Commercial, the commercial portion of LinkedIn, Enterprise and partner services, and Dynamics. Our commercial
metrics allow management and investors to assess the overall health of our commercial business and include leading
indicators of future performance.

Commercial remaining performance
obligation

Commercial portion of revenue allocated to remaining performance
obligations, which includes unearned revenue and amounts that will be
invoiced and recognized as revenue in future periods

Microsoft Cloud revenue and revenue
growth

Revenue from Azure and other cloud services, Office 365 Commercial,
the commercial portion of LinkedIn, Dynamics 365, and other
commercial cloud properties

Microsoft Cloud gross margin percentage

Gross margin percentage for our Microsoft Cloud business


### Productivity and Business Processes and Intelligent Cloud

Metrics related to our Productivity and Business Processes and Intelligent Cloud segments assess the health of our
core businesses within these segments. The metrics reflect our cloud and on-premises product strategies and trends.

Office Commercial products and cloud
services revenue growth

Revenue from Office Commercial products and cloud services (Office
365 subscriptions, the Office 365 portion of Microsoft 365 Commercial
subscriptions, and Office licensed on-premises), comprising Office,
Exchange, SharePoint, Microsoft Teams, Office 365 Security and
Compliance, Microsoft Viva, and Copilot for Microsoft 365

Office Consumer products and cloud
services revenue growth

Revenue from Office Consumer products and cloud services, including
Microsoft 365 Consumer and Copilot Pro subscriptions, Office licensed
on-premises, and other Office services

Office 365 Commercial seat growth

The number of Office 365 Commercial seats at end of period where
seats are paid users covered by an Office 365 Commercial
subscription

Microsoft 365 Consumer subscribers

The number of Microsoft 365 Consumer and Copilot Pro subscribers at
end of period

Dynamics products and cloud services
revenue growth

Revenue from Dynamics products and cloud services, including
Dynamics 365, comprising a set of intelligent, cloud-based applications
across ERP, CRM, Power Apps, and Power Automate; and on-
premises ERP and CRM applications


### LinkedIn revenue growth

Revenue from LinkedIn, including Talent Solutions, Marketing
Solutions, Premium Subscriptions, and Sales Solutions

Server products and cloud services
revenue growth

Revenue from Server products and cloud services, including Azure
and other cloud services; SQL Server, Windows Server, Visual Studio,
System Center, and related Client Access Licenses ("CALs"); and
Nuance and GitHub

<!-- PageNumber="40" -->
<!-- PageBreak -->


# PART II Item 7


## More Personal Computing

Metrics related to our More Personal Computing segment assess the performance of key lines of business within this
segment. These metrics provide strategic product insights which allow us to assess the performance across our
commercial and consumer businesses. As we have diversity of target audiences and sales motions within the Windows
business, we monitor metrics that are reflective of those varying motions.


<table>
<tr>
<td>Windows OEM revenue growth</td>
<td>Revenue from sales of Windows Pro and non-Pro licenses sold through the OEM channel</td>
</tr>
<tr>
<td>Windows Commercial products and cloud services revenue growth</td>
<td>Revenue from Windows Commercial products and cloud services, comprising volume licensing of the Windows operating system, Windows cloud services, and other Windows commercial offerings</td>
</tr>
<tr>
<td>Devices revenue growth</td>
<td>Revenue from Devices, including Surface, HoloLens, and PC accessories</td>
</tr>
<tr>
<td>Xbox content and services revenue growth</td>
<td>Revenue from Xbox content and services, comprising first-party content (such as Activision Blizzard) and third-party content, including games and in-game content; Xbox Game Pass and other subscriptions; Xbox Cloud Gaming; advertising; third-party disc royalties; and other cloud services</td>
</tr>
<tr>
<td>Search and news advertising revenue (ex TAC) growth</td>
<td>Revenue from search and news advertising excluding traffic acquisition costs ("TAC") paid to Bing Ads network publishers and news partners</td>
</tr>
</table>


<!-- PageNumber="41" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 7" -->


### SUMMARY RESULTS OF OPERATIONS


<table>
<tr>
<th>(In millions, except percentages and per share amounts)</th>
<th>2024</th>
<th>2023</th>
<th>Percentage Change</th>
</tr>
<tr>
<td>Revenue</td>
<td>$ 245,122</td>
<td>$ 211,915</td>
<td>16%</td>
</tr>
<tr>
<td>Gross margin</td>
<td>171,008</td>
<td>146,052</td>
<td>17%</td>
</tr>
<tr>
<td>Operating income</td>
<td>109,433</td>
<td>88,523</td>
<td>24%</td>
</tr>
<tr>
<td>Net income</td>
<td>88,136</td>
<td>72,361</td>
<td>22%</td>
</tr>
<tr>
<td>Diluted earnings per share</td>
<td>11.80</td>
<td>9.68</td>
<td>22%</td>
</tr>
<tr>
<td>Adjusted gross margin (non-GAAP)</td>
<td>171,008</td>
<td>146,204</td>
<td>17%</td>
</tr>
<tr>
<td>Adjusted operating income (non-GAAP)</td>
<td>109,433</td>
<td>89,694</td>
<td>22%</td>
</tr>
<tr>
<td>Adjusted net income (non-GAAP)</td>
<td>88,136</td>
<td>73,307</td>
<td>20%</td>
</tr>
<tr>
<td>Adjusted diluted earnings per share (non-GAAP)</td>
<td>11.80</td>
<td>9.81</td>
<td>20%</td>
</tr>
</table>


Adjusted gross margin, operating income, net income, and diluted earnings per share ("EPS") are non-GAAP financial
measures. Prior year non-GAAP financial measures exclude the impact of a $1.2 billion charge in the second quarter
of fiscal year 2023 ("Q2 charge"), which included employee severance expenses, impairment charges resulting from
changes to our hardware portfolio, and costs related to lease consolidation activities. Refer to the Non-GAAP Financial
Measures section below for a reconciliation of our financial results reported in accordance with GAAP to non-GAAP
financial results.


#### Fiscal Year 2024 Compared with Fiscal Year 2023

Revenue increased $33.2 billion or 16% driven by growth across each of our segments. Intelligent Cloud revenue
increased driven by Azure. Productivity and Business Processes revenue increased driven by Office 365 Commercial.
More Personal Computing revenue increased driven by Gaming.

Cost of revenue increased $8.3 billion or 13% driven by growth in Microsoft Cloud and Gaming, offset in part by a
decline in Devices.

Gross margin increased $25.0 billion or 17% driven by growth across each of our segments.

. Gross margin percentage increased slightly. Excluding the impact of the change in accounting estimate for
the useful lives of our server and network equipment, gross margin percentage increased 2 points driven
by improvement in More Personal Computing.

· Microsoft Cloud gross margin percentage decreased slightly to 71%. Excluding the impact of the change in
accounting estimate, Microsoft Cloud gross margin percentage increased slightly driven by improvements
in Azure and Office 365 Commercial, inclusive of scaling our AI infrastructure, offset in part by sales mix
shift to Azure.

Operating expenses increased $4.0 billion or 7% driven by Gaming, with 7 points of growth from the Activision Blizzard
acquisition, and investments in cloud engineering, offset in part by the prior year Q2 charge.

Operating income increased $20.9 billion or 24% driven by growth across each of our segments.

Prior year gross margin, operating income, net income, and diluted EPS were negatively impacted by the Q2 charge,
which resulted in decreases of $152 million, $1.2 billion, $946 million, and $0.13, respectively.

<!-- PageNumber="42" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 7" -->


# SEGMENT RESULTS OF OPERATIONS


<table>
<tr>
<th>(In millions, except percentages)</th>
<th>2024</th>
<th>2023</th>
<th>Percentage Change</th>
</tr>
<tr>
<td>Revenue</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Productivity and Business Processes</td>
<td>$ 77,728</td>
<td>$ 69,274</td>
<td>12%</td>
</tr>
<tr>
<td>Intelligent Cloud</td>
<td>105,362</td>
<td>87,907</td>
<td>20%</td>
</tr>
<tr>
<td>More Personal Computing</td>
<td>62,032</td>
<td>54,734</td>
<td>13%</td>
</tr>
<tr>
<td>Total</td>
<td>$ 245,122</td>
<td>$ 211,915</td>
<td>16%</td>
</tr>
<tr>
<td>Operating Income</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Productivity and Business Processes</td>
<td>$ 40,540</td>
<td>$ 34,189</td>
<td>19%</td>
</tr>
<tr>
<td>Intelligent Cloud</td>
<td>49,584</td>
<td>37,884</td>
<td>31%</td>
</tr>
<tr>
<td>More Personal Computing</td>
<td>19,309</td>
<td>16,450</td>
<td>17%</td>
</tr>
<tr>
<td>Total</td>
<td>$ 109,433</td>
<td>$ 88,523</td>
<td>24%</td>
</tr>
</table>


## Reportable Segments


### Fiscal Year 2024 Compared with Fiscal Year 2023


### Productivity and Business Processes

Revenue increased $8.5 billion or 12%.

· Office Commercial products and cloud services revenue increased $5.8 billion or 14%. Office 365
Commercial revenue grew 16% with seat growth of 7%, driven by small and medium business and frontline
worker offerings, as well as growth in revenue per user. Office Commercial products revenue declined 16%
driven by continued customer shift to cloud offerings.

· Office Consumer products and cloud services revenue increased $237 million or 4%. Microsoft 365
Consumer subscribers grew 10% to 82.5 million.

· Linkedln revenue increased $1.4 billion or 9% driven by growth across all lines of business - Talent
Solutions, Premium Subscriptions, Marketing Solutions, and Sales Solutions.

· Dynamics products and cloud services revenue increased $1.0 billion or 19% driven by Dynamics 365.
Dynamics 365 revenue grew 24% driven by growth across all workloads.

Operating income increased $6.4 billion or 19%.

. Gross margin increased $6.5 billion or 12% driven by growth in Office 365 Commercial. Gross margin
percentage decreased slightly. Excluding the impact of the change in accounting estimate, gross margin
percentage increased slightly driven by improvement in Office 365 Commercial.

· Operating expenses increased $159 million or 1%.


### Intelligent Cloud

Revenue increased $17.5 billion or 20%.

· Server products and cloud services revenue increased $17.8 billion or 22% driven by Azure and other cloud
services. Azure and other cloud services revenue grew 30% driven by growth in our consumption-based
services. Server products revenue increased 3% driven by continued demand for our hybrid solutions,
including Windows Server and SQL Server running in multi-cloud environments.

· Enterprise and partner services revenue decreased $306 million or 4% driven by declines in Enterprise
Support Services and Industry Solutions.

Operating income increased $11.7 billion or 31%.

. Gross margin increased $11.6 billion or 19% driven by growth in Azure. Gross margin percentage

<!-- PageNumber="43" -->
<!-- PageBreak -->


# PART II Item 7

decreased slightly. Excluding the impact of the change in accounting estimate, gross margin percentage
increased slightly primarily driven by improvement in Azure, inclusive of scaling our Al infrastructure, offset
in part by sales mix shift to Azure.

· Operating expenses decreased slightly primarily driven by the prior year Q2 charge, offset in part by
investments in Azure.


# More Personal Computing

Revenue increased $7.3 billion or 13%.

· Windows revenue increased $1.7 billion or 8% driven by growth in Windows Commercial and Windows
OEM. Windows Commercial products and cloud services revenue increased 11% driven by demand for
Microsoft 365. Windows OEM revenue increased 7%.

· Gaming revenue increased $6.0 billion or 39% driven by growth in Xbox content and services. Xbox content
and services revenue increased 50% driven by 44 points of net impact from the Activision Blizzard
acquisition. Xbox hardware revenue decreased 13% driven by lower volume of consoles sold.

. Search and news advertising revenue increased $418 million or 3%. Search and news advertising revenue
excluding traffic acquisition costs increased 12% driven by higher search volume.

· Devices revenue decreased $815 million or 15%.

Operating income increased $2.9 billion or 17%.

. Gross margin increased $6.8 billion or 23% driven by growth in Gaming, with 10 points of net impact from
the Activision Blizzard acquisition, as well as growth in Windows. Gross margin percentage increased driven
by sales mix shift to higher margin businesses and improvement in Devices.

· Operating expenses increased $3.9 billion or 31% driven by Gaming, with 34 points of growth from the
Activision Blizzard acquisition.


## OPERATING EXPENSES


### Research and Development


<table>
<tr>
<th>(In millions, except percentages)</th>
<th>2024</th>
<th>2023</th>
<th>Percentage Change</th>
</tr>
<tr>
<td>Research and development</td>
<td>$ 29,510</td>
<td>$ 27,195</td>
<td>9%</td>
</tr>
<tr>
<td>As a percent of revenue</td>
<td>12%</td>
<td>13%</td>
<td>(1)ppt</td>
</tr>
</table>


Research and development expenses include payroll, employee benefits, stock-based compensation expense, and
other headcount-related expenses associated with product development. Research and development expenses also
include third-party development and programming costs and the amortization of purchased software code and services
content.


#### Fiscal Year 2024 Compared with Fiscal Year 2023

Research and development expenses increased $2.3 billion or 9% driven by Gaming, with 7 points of growth from the
Activision Blizzard acquisition, and investments in cloud engineering.

<!-- PageNumber="44" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 7" -->


### Sales and Marketing


<table>
<tr>
<th>(In millions, except percentages)</th>
<th>2024</th>
<th>2023</th>
<th>Percentage Change</th>
</tr>
<tr>
<td>Sales and marketing</td>
<td>$ 24,456</td>
<td>$ 22,759</td>
<td>7%</td>
</tr>
<tr>
<td>As a percent of revenue</td>
<td>10%</td>
<td>11%</td>
<td>(1)ppt</td>
</tr>
</table>


Sales and marketing expenses include payroll, employee benefits, stock-based compensation expense, and other
headcount-related expenses associated with sales and marketing personnel, and the costs of advertising, promotions,
trade shows, seminars, and other programs.


#### Fiscal Year 2024 Compared with Fiscal Year 2023

Sales and marketing expenses increased $1.7 billion or 7% driven by Gaming, with 6 points of growth from the
Activision Blizzard acquisition.


### General and Administrative


<table>
<tr>
<th>(In millions, except percentages)</th>
<th>2024</th>
<th>2023</th>
<th>Percentage Change</th>
</tr>
<tr>
<td>General and administrative</td>
<td>$ 7,609</td>
<td>$ 7,575</td>
<td>0%</td>
</tr>
<tr>
<td>As a percent of revenue</td>
<td>3%</td>
<td>4%</td>
<td>(1)ppt</td>
</tr>
</table>


General and administrative expenses include payroll, employee benefits, stock-based compensation expense,
employee severance expense incurred as part of a corporate program, and other headcount-related expenses
associated with finance, legal, facilities, certain human resources and other administrative personnel, certain taxes,
and legal and other administrative fees.


#### Fiscal Year 2024 Compared with Fiscal Year 2023

General and administrative expenses increased slightly as growth from the Activision Blizzard acquisition was offset in
part by the prior year Q2 charge.


### OTHER INCOME (EXPENSE), NET

The components of other income (expense), net were as follows:

(In millions)


<table>
<tr>
<th>Year Ended June 30,</th>
<th>2024</th>
<th>2023</th>
</tr>
<tr>
<td>Interest and dividends income</td>
<td>$ 3,157</td>
<td>$ 2,994</td>
</tr>
<tr>
<td>Interest expense</td>
<td>(2,935)</td>
<td>(1,968)</td>
</tr>
<tr>
<td>Net recognized gains (losses) on investments</td>
<td>(118)</td>
<td>260</td>
</tr>
<tr>
<td>Net losses on derivatives</td>
<td>(187)</td>
<td>(456)</td>
</tr>
<tr>
<td>Net gains (losses) on foreign currency remeasurements</td>
<td>(244)</td>
<td>181</td>
</tr>
<tr>
<td>Other, net</td>
<td>(1,319)</td>
<td>(223)</td>
</tr>
<tr>
<td>Total</td>
<td>$ (1,646)</td>
<td>$ 788</td>
</tr>
</table>


We use derivative instruments to manage risks related to foreign currencies, interest rates, equity prices, and credit;
to enhance investment returns; and to facilitate portfolio diversification. Gains and losses from changes in fair values
of derivatives that are not designated as hedging instruments are primarily recognized in other income (expense), net.

<!-- PageNumber="45" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 7" -->


#### Fiscal Year 2024 Compared with Fiscal Year 2023

Interest and dividends income increased due to higher yields. Interest expense increased due to the issuance of
commercial paper. Net recognized losses on investments increased primarily due to higher equity impairments and
lower gains on equity investments. Net losses on derivatives decreased primarily due to lower losses on equity
derivatives. Other, net primarily reflects net recognized losses on equity method investments.


##### INCOME TAXES


###### Effective Tax Rate

Our effective tax rate for fiscal years 2024 and 2023 was 18% and 19%, respectively. The decrease in our effective
tax rate was primarily due to tax benefits from tax law changes, including the impact from the issuance of Notice 2023-
55 and Notice 2023-80 by the Internal Revenue Service ("IRS") and U.S. Treasury Department. Notice 2023-55, issued
in the first quarter of fiscal year 2024, delayed the effective date of final foreign tax credit regulations to fiscal year 2024
for Microsoft. Notice 2023-80, issued in the second quarter of fiscal year 2024, further delayed the effective date of
final foreign tax credit regulations indefinitely.

Our effective tax rate was lower than the U.S. federal statutory rate, primarily due to earnings taxed at lower rates in
foreign jurisdictions resulting from producing and distributing our products and services through our foreign regional
operations center in Ireland.

The mix of income before income taxes between the U.S. and foreign countries impacted our effective tax rate as a
result of the geographic distribution of, and customer demand for, our products and services. In fiscal year 2024, our
U.S. income before income taxes was $62.9 billion and our foreign income before income taxes was $44.9 billion. In
fiscal year 2023, our U.S. income before income taxes was $52.9 billion and our foreign income before income taxes
was $36.4 billion.

The Organisation for Economic Co-operation and Development ("OECD") published its model rules "Tax Challenges
Arising From the Digitalisation of the Economy - Global Anti-Base Erosion Model Rules (Pillar Two)" which established
a global minimum corporate tax rate of 15% for certain multinational enterprises. Many countries have implemented or
are in the process of implementing the Pillar Two legislation, which will apply to Microsoft beginning in fiscal year 2025.
While we do not currently estimate a material impact to our consolidated financial statements, we continue to monitor
the impact as countries implement legislation and the OECD provides additional guidance.


###### Uncertain Tax Positions

We remain under audit by the IRS for tax years 2014 to 2017. With respect to the audit for tax years 2004 to 2013, on
September 26, 2023, we received Notices of Proposed Adjustment ("NOPAs") from the IRS. The primary issues in the
NOPAs relate to intercompany transfer pricing. In the NOPAs, the IRS is seeking an additional tax payment of $28.9
billion plus penalties and interest. As of June 30, 2024, we believe our allowances for income tax contingencies are
adequate. We disagree with the proposed adjustments and will vigorously contest the NOPAs through the IRS's
administrative appeals office and, if necessary, judicial proceedings. We do not expect a final resolution of these issues
in the next 12 months. Based on the information currently available, we do not anticipate a significant increase or
decrease to our income tax contingencies for these issues within the next 12 months.

We are subject to income tax in many jurisdictions outside the U.S. Our operations in certain jurisdictions remain
subject to examination for tax years 1996 to 2023, some of which are currently under audit by local tax authorities. The
resolution of each of these audits is not expected to be material to our consolidated financial statements.


##### NON-GAAP FINANCIAL MEASURES

Adjusted gross margin, operating income, net income, and diluted EPS are non-GAAP financial measures. Prior year
non-GAAP financial measures exclude the impact of the Q2 charge, which includes employee severance expenses,
impairment charges resulting from changes to our hardware portfolio, and costs related to lease consolidation activities.
We believe these non-GAAP measures aid investors by providing additional insight into our operational performance
and help clarify trends affecting our business. For comparability of reporting, management considers non-GAAP
measures in conjunction with GAAP financial results in evaluating business performance. These non-GAAP financial

<!-- PageNumber="46" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 7" -->

measures presented should not be considered a substitute for, or superior to, the measures of financial performance
prepared in accordance with GAAP.

The following table reconciles our financial results reported in accordance with GAAP to non-GAAP financial results:


<table>
<tr>
<th>(In millions, except percentages and per share amounts)</th>
<th>2024</th>
<th>2023</th>
<th>Percentage Change</th>
</tr>
<tr>
<td>Gross margin</td>
<td>$ 171,008</td>
<td>$ 146,052</td>
<td>17%</td>
</tr>
<tr>
<td>Severance, hardware-related impairment, and lease consolidation costs</td>
<td>0</td>
<td>152</td>
<td>*</td>
</tr>
<tr>
<td>Adjusted gross margin (non-GAAP)</td>
<td>$ 171,008</td>
<td>$ 146,204</td>
<td>17%</td>
</tr>
<tr>
<td>Operating income</td>
<td>$ 109,433</td>
<td>$ 88,523</td>
<td>24%</td>
</tr>
<tr>
<td>Severance, hardware-related impairment, and lease consolidation costs</td>
<td>0</td>
<td>1,171</td>
<td>*</td>
</tr>
<tr>
<td>Adjusted operating income (non-GAAP)</td>
<td>$ 109,433</td>
<td>$ 89,694</td>
<td>22%</td>
</tr>
<tr>
<td>Net income</td>
<td>$ 88,136</td>
<td>$ 72,361</td>
<td>$2 2$</td>
</tr>
<tr>
<td>Severance, hardware-related impairment, and lease consolidation costs</td>
<td>0</td>
<td>946</td>
<td>*</td>
</tr>
<tr>
<td>Adjusted net income (non-GAAP)</td>
<td>$ 88,136</td>
<td>$ 73,307</td>
<td>20%</td>
</tr>
<tr>
<td>Diluted earnings per share</td>
<td>$ 11.80</td>
<td>$ 9.68</td>
<td>$2 2$</td>
</tr>
<tr>
<td>Severance, hardware-related impairment, and lease consolidation costs</td>
<td>0</td>
<td>0.13</td>
<td>*</td>
</tr>
<tr>
<td>Adjusted diluted earnings per share (non-GAAP)</td>
<td>$ 11.80</td>
<td>$ 9.81</td>
<td>20%</td>
</tr>
</table>

\*
Not meaningful.


### LIQUIDITY AND CAPITAL RESOURCES

We expect existing cash, cash equivalents, short-term investments, cash flows from operations, and access to capital
markets to continue to be sufficient to fund our operating activities and cash commitments for investing and financing
activities, such as dividends, share repurchases, debt maturities, material capital expenditures, and the transition tax
related to the Tax Cuts and Jobs Act ("TCJA"), for at least the next 12 months and thereafter for the foreseeable future.


#### Cash, Cash Equivalents, and Investments

Cash, cash equivalents, and short-term investments totaled $75.5 billion and $111.3 billion as of June 30, 2024 and
2023, respectively. Equity and other investments were $14.6 billion and $9.9 billion as of June 30, 2024 and 2023,
respectively. Our short-term investments are primarily intended to facilitate liquidity and capital preservation. They
consist predominantly of highly liquid investment-grade fixed-income securities, diversified among industries and
individual issuers. The investments are predominantly U.S. dollar-denominated securities, but also include foreign
currency-denominated securities to diversify risk. Our fixed-income investments are exposed to interest rate risk and
credit risk. The credit risk and average maturity of our fixed-income portfolio are managed to achieve economic returns
that correlate to certain fixed-income indices. The settlement risk related to these investments is insignificant given that
the short-term investments held are primarily highly liquid investment-grade fixed-income securities.


#### Valuation

In general, and where applicable, we use quoted prices in active markets for identical assets or liabilities to determine
the fair value of our financial instruments. This pricing methodology applies to our Level 1 investments, such as U.S.
government securities, common and preferred stock, and mutual funds. If quoted prices in active markets for identical
assets or liabilities are not available to determine fair value, then we use quoted prices for similar assets and liabilities
or inputs other than the quoted prices that are observable either directly or indirectly. This pricing methodology applies
to our Level 2 investments, such as commercial paper, certificates of deposit, U.S. agency securities, foreign
government bonds, mortgage- and asset-backed securities, corporate notes and bonds, and municipal securities. Level

<!-- PageNumber="47" -->
<!-- PageBreak -->


# PART II Item 7

3 investments are valued using internally-developed models with unobservable inputs. Assets and liabilities measured
at fair value on a recurring basis using unobservable inputs are an immaterial portion of our portfolio.

A majority of our investments are priced by pricing vendors and are generally Level 1 or Level 2 investments as these
vendors either provide a quoted market price in an active market or use observable inputs for their pricing without
applying significant adjustments. Broker pricing is used mainly when a quoted price is not available, the investment is
not priced by our pricing vendors, or when a broker price is more reflective of fair values in the market in which the
investment trades. Our broker-priced investments are generally classified as Level 2 investments because the broker
prices these investments based on similar assets without applying significant adjustments. In addition, all our broker-
priced investments have a sufficient level of trading volume to demonstrate that the fair values used are appropriate
for these investments. Our fair value processes include controls that are designed to ensure appropriate fair values
are recorded. These controls include model validation, review of key model inputs, analysis of period-over-period
fluctuations, and independent recalculation of prices where appropriate.


## Cash Flows

Cash from operations increased $31.0 billion to $118.5 billion for fiscal year 2024, primarily due to an increase in cash
received from customers. Cash used in financing decreased $6.2 billion to $37.8 billion for fiscal year 2024, primarily
due to a $5.0 billion decrease in common stock repurchases and a $3.3 billion increase in proceeds from issuance of
debt, net of repayments, offset in part by a $2.0 billion increase in dividends paid. Cash used in investing increased
$74.3 billion to $97.0 billion for fiscal year 2024, primarily due to a $67.5 billion increase in cash used for acquisitions
of companies, net of cash acquired, and purchases of intangible and other assets and a $16.4 billion increase in
additions to property and equipment.


## Debt Proceeds

We issue debt to take advantage of favorable pricing and liquidity in the debt markets, reflecting our credit rating. The
proceeds of these issuances were or will be used for general corporate purposes, which may include, among other
things, funding for working capital, capital expenditures, repurchases of capital stock, acquisitions, and repayment of
existing debt. Refer to Note 11 - Debt of the Notes to Financial Statements (Part II, Item 8 of this Form 10-K) for further
discussion.


## Unearned Revenue

Unearned revenue comprises mainly unearned revenue related to volume licensing programs, which may include
Software Assurance ("SA") and cloud services. Unearned revenue is generally invoiced annually at the beginning of
each contract period for multi-year agreements and recognized ratably over the coverage period. Unearned revenue
also includes payments for other offerings for which we have been paid in advance and earn the revenue when we
transfer control of the product or service. Refer to Note 1 - Accounting Policies of the Notes to Financial Statements
(Part II, Item 8 of this Form 10-K) for further discussion.

The following table outlines the expected future recognition of unearned revenue as of June 30, 2024:

(In millions)


<table>
<tr>
<td>Three Months Ending</td>
<td></td>
</tr>
<tr>
<td>September 30, 2024</td>
<td>$ 22,529</td>
</tr>
<tr>
<td>December 31, 2024</td>
<td>17,664</td>
</tr>
<tr>
<td>March 31, 2025</td>
<td>12,076</td>
</tr>
<tr>
<td>June 30, 2025</td>
<td>5,313</td>
</tr>
<tr>
<td>Thereafter</td>
<td>2,602</td>
</tr>
<tr>
<td>Total</td>
<td>$ 60,184</td>
</tr>
</table>


If our customers choose to license cloud-based versions of our products and services rather than licensing transaction-
based products and services, the associated revenue will shift from being recognized at the time of the transaction to

<!-- PageNumber="48" -->
<!-- PageBreak -->


# PART II Item 7

being recognized over the subscription period or upon consumption, as applicable. Refer to Note 13 - Unearned
Revenue of the Notes to Financial Statements (Part II, Item 8 of this Form 10-K) for further discussion.


## Material Cash Requirements and Other Obligations


### Contractual Obligations

The following table summarizes the payments due by fiscal year for our outstanding contractual obligations as of June
30, 2024:


<table>
<tr>
<th>(In millions)</th>
<th>2025</th>
<th>Thereafter</th>
<th>Total</th>
</tr>
<tr>
<td>Long-term debt: (a)</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Principal payments</td>
<td>$ 2,250</td>
<td>$ 48,971</td>
<td>$ 51,221</td>
</tr>
<tr>
<td>Interest payments</td>
<td>1,618</td>
<td>27,041</td>
<td>28,659</td>
</tr>
<tr>
<td>Construction commitments (b)</td>
<td>29,892</td>
<td>5,499</td>
<td>35,391</td>
</tr>
<tr>
<td>Operating and finance leases, including imputed interest (c)</td>
<td>12,250</td>
<td>160,475</td>
<td>172,725</td>
</tr>
<tr>
<td>Purchase commitments (d)</td>
<td>68,280</td>
<td>3,742</td>
<td>72,022</td>
</tr>
<tr>
<td>Total</td>
<td>$ 114,290</td>
<td>$ 245,728</td>
<td>$ 360,018</td>
</tr>
</table>

(a) Refer to Note 11 - Debt of the Notes to Financial Statements (Part II, Item 8 of this Form 10-K).

(b)
Refer to Note 7 - Property and Equipment of the Notes to Financial Statements (Part II, Item 8 of this Form 10-K).

(c)
Refer to Note 14 - Leases of the Notes to Financial Statements (Part II, Item 8 of this Form 10-K).

(d)
Purchase commitments primarily relate to datacenters and include open purchase orders and take-or-pay
contracts that are not presented as construction commitments above.


### Income Taxes

As a result of the TCJA, we are required to pay a one-time transition tax on deferred foreign income not previously
subject to U.S. income tax. Under the TCJA, the transition tax is payable in interest-free installments over eight years,
with 8% due in each of the first five years, 15% in year six, 20% in year seven, and 25% in year eight. As of June 30,
2024, we had a remaining transition tax liability of $7.6 billion, of which $3.8 billion is short-term and payable in the first
quarter of fiscal year 2025.


### Share Repurchases

During fiscal years 2024 and 2023, we repurchased 32 million shares and 69 million shares of our common stock for
$12.0 billion and $18.4 billion, respectively, through our share repurchase program. All repurchases were made using
cash resources. As of June 30, 2024, $10.3 billion remained of our $60 billion share repurchase program. Refer to
Note 16 - Stockholders' Equity of the Notes to Financial Statements (Part II, Item 8 of this Form 10-K) for further
discussion.


### Dividends

During fiscal years 2024 and 2023, our Board of Directors declared dividends totaling $22.3 billion and $20.2 billion,
respectively. We intend to continue returning capital to shareholders in the form of dividends, subject to declaration by
our Board of Directors. Refer to Note 16 - Stockholders' Equity of the Notes to Financial Statements (Part II, Item 8 of
this Form 10-K) for further discussion.


### Other Planned Uses of Capital

We will continue to invest in sales, marketing, product support infrastructure, and existing and advanced areas of
technology, as well as acquisitions that align with our business strategy. Additions to property and equipment will
continue, including new facilities, datacenters, and computer systems for research and development, sales and
marketing, support, and administrative staff. We expect capital expenditures to increase in coming years to support
growth in our cloud offerings and our investments in AI infrastructure and training. We have operating and finance

<!-- PageNumber="49" -->
<!-- PageBreak -->


# PART II Item 7

leases for datacenters, corporate offices, research and development facilities, Microsoft Experience Centers, and
certain equipment. We have not engaged in any related party transactions or arrangements with unconsolidated entities
or other persons that are reasonably likely to materially affect liquidity or the availability of capital resources.


## RECENT ACCOUNTING GUIDANCE

Refer to Note 1 - Accounting Policies of the Notes to Financial Statements (Part II, Item 8 of this Form 10-K) for further
discussion.


## CRITICAL ACCOUNTING ESTIMATES

Our consolidated financial statements and accompanying notes are prepared in accordance with GAAP. Preparing
consolidated financial statements requires management to make estimates and assumptions that affect the reported
amounts of assets, liabilities, revenue, and expenses. Critical accounting estimates are those estimates that involve a
significant level of estimation uncertainty and could have a material impact on our financial condition or results of
operations. We have critical accounting estimates in the areas of revenue recognition, impairment of investment
securities, goodwill, research and development costs, legal and other contingencies, income taxes, and business
combinations - valuation of intangible assets.


### Revenue Recognition

Our contracts with customers often include promises to transfer multiple products and services to a customer.
Determining whether products and services are considered distinct performance obligations that should be accounted
for separately versus together may require significant judgment. When a cloud-based service includes both on-
premises software licenses and cloud services, judgment is required to determine whether the software license is
considered distinct and accounted for separately, or not distinct and accounted for together with the cloud service and
recognized over time. Certain cloud services, primarily Office 365, depend on a significant level of integration,
interdependency, and interrelation between the desktop applications and cloud services, and are accounted for
together as one performance obligation. Revenue from Office 365 is recognized ratably over the period in which the
cloud services are provided.

Judgment is required to determine the standalone selling price ("SSP") for each distinct performance obligation. We
use a single amount to estimate SSP for items that are not sold separately, including on-premises licenses sold with
SA or software updates provided at no additional charge. We use a range of amounts to estimate SSP when we sell
each of the products and services separately and need to determine whether there is a discount to be allocated based
on the relative SSP of the various products and services.

In instances where SSP is not directly observable, such as when we do not sell the product or service separately, we
determine the SSP using information that may include market conditions and other observable inputs. We typically
have more than one SSP for individual products and services due to the stratification of those products and services
by customers and circumstances. In these instances, we may use information such as the size of the customer and
geographic region in determining the SSP.

Due to the various benefits from and the nature of our SA program, judgment is required to assess the pattern of
delivery, including the exercise pattern of certain benefits across our portfolio of customers.

Our products are generally sold with a right of return, we may provide other credits or incentives, and in certain
instances we estimate customer usage of our products and services, which are accounted for as variable consideration
when determining the amount of revenue to recognize. Returns and credits are estimated at contract inception and
updated at the end of each reporting period if additional information becomes available. Changes to our estimated
variable consideration were not material for the periods presented.


### Impairment of Investment Securities

We review debt investments quarterly for credit losses and impairment. If the cost of an investment exceeds its fair
value, we evaluate, among other factors, general market conditions, credit quality of debt instrument issuers, and the
extent to which the fair value is less than cost. This determination requires significant judgment. In making this
judgment, we employ a systematic methodology that considers available quantitative and qualitative evidence in

<!-- PageNumber="50" -->
<!-- PageBreak -->


# PART II Item 7

evaluating potential impairment of our investments. In addition, we consider specific adverse conditions related to the
financial health of, and business outlook for, the investee. If we have plans to sell the security or it is more likely than
not that we will be required to sell the security before recovery, then a decline in fair value below cost is recorded as
an impairment charge in other income (expense), net and a new cost basis in the investment is established. If market,
industry, and/or investee conditions deteriorate, we may incur future impairments.

Equity investments without readily determinable fair values are written down to fair value if a qualitative assessment
indicates that the investment is impaired and the fair value of the investment is less than carrying value. We perform a
qualitative assessment on a periodic basis. We are required to estimate the fair value of the investment to determine
the amount of the impairment loss. Once an investment is determined to be impaired, an impairment charge is recorded
in other income (expense), net.


## Goodwill

We allocate goodwill to reporting units based on the reporting unit expected to benefit from the business combination.
We evaluate our reporting units on an annual basis and, if necessary, reassign goodwill using a relative fair value
allocation approach. Goodwill is tested for impairment at the reporting unit level (operating segment or one level below
an operating segment) on an annual basis (May 1) and between annual tests if an event occurs or circumstances
change that would more likely than not reduce the fair value of a reporting unit below its carrying value. These events
or circumstances could include a significant change in the business climate, legal factors, operating performance
indicators, competition, or sale or disposition of a significant portion of a reporting unit.

Application of the goodwill impairment test requires judgment, including the identification of reporting units, assignment
of assets and liabilities to reporting units, assignment of goodwill to reporting units, and determination of the fair value
of each reporting unit. The fair value of each reporting unit is estimated primarily through the use of a discounted cash
flow methodology. This analysis requires significant judgments, including estimation of future cash flows, which is
dependent on internal forecasts, estimation of the long-term rate of growth for our business, estimation of the useful
life over which cash flows will occur, and determination of our weighted average cost of capital.

The estimates used to calculate the fair value of a reporting unit change from year to year based on operating results,
market conditions, and other factors. Changes in these estimates and assumptions could materially affect the
determination of fair value and goodwill impairment for each reporting unit.


## Research and Development Costs

Costs incurred internally in researching and developing a computer software product are charged to expense until
technological feasibility has been established for the product. Once technological feasibility is established, software
costs are capitalized until the product is available for general release to customers. Judgment is required in determining
when technological feasibility of a product is established. We have determined that technological feasibility for our
software products is reached after all high-risk development issues have been resolved through coding and testing.
Generally, this occurs shortly before the products are released to production. The amortization of these costs is
included in cost of revenue over the estimated life of the products.


## Legal and Other Contingencies

The outcomes of legal proceedings and claims brought against us are subject to significant uncertainty. An estimated
loss from a loss contingency such as a legal proceeding or claim is accrued by a charge to income if it is probable that
an asset has been impaired or a liability has been incurred and the amount of the loss can be reasonably estimated.
In determining whether a loss should be accrued we evaluate, among other factors, the degree of probability of an
unfavorable outcome and the ability to make a reasonable estimate of the amount of loss. Changes in these factors
could materially impact our consolidated financial statements.


## Income Taxes

The objectives of accounting for income taxes are to recognize the amount of taxes payable or refundable for the
current year, and deferred tax liabilities and assets for the future tax consequences of events that have been recognized
in an entity's financial statements or tax returns. We recognize the tax benefit from an uncertain tax position only if it is
more likely than not that the tax position will be sustained on examination by the taxing authorities, based on the

<!-- PageNumber="51" -->
<!-- PageBreak -->


# PART II Item 7

technical merits of the position. The tax benefits recognized in the financial statements from such a position are
measured based on the largest benefit that has a greater than 50% likelihood of being realized upon ultimate
settlement. Accounting literature also provides guidance on derecognition of income tax assets and liabilities,
classification of deferred income tax assets and liabilities, accounting for interest and penalties associated with tax
positions, and income tax disclosures. Judgment is required in assessing the future tax consequences of events that
have been recognized in our consolidated financial statements or tax returns. Variations in the actual outcome of these
future tax consequences could materially impact our consolidated financial statements.


## Business Combinations - Valuation of Intangible Assets

Accounting for business combinations requires significant judgments when allocating the purchase price to the
estimated fair values of assets acquired and liabilities assumed at the acquisition date. Determination of fair value
involves estimates and assumptions which can be complex, most notably with respect to intangible assets. Critical
estimates used in the valuation of intangible assets include, but are not limited to, the amount and timing of projected
cash flows, useful lives, and discount rates. While management's estimates of fair value are based on assumptions
that are believed to be reasonable, these assumptions are inherently uncertain as they pertain to forward-looking views
of our business and market conditions. The judgments made in this valuation process could materially impact our
consolidated financial statements.

<!-- PageNumber="52" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 7" -->


## STATEMENT OF MANAGEMENT'S RESPONSIBILITY FOR FINANCIAL STATEMENTS

Management is responsible for the preparation of the consolidated financial statements and related information that
are presented in this report. The consolidated financial statements, which include amounts based on management's
estimates and judgments, have been prepared in conformity with accounting principles generally accepted in the United
States of America.

The Company designs and maintains accounting and internal control systems to provide reasonable assurance at
reasonable cost that assets are safeguarded against loss from unauthorized use or disposition, and that the financial
records are reliable for preparing consolidated financial statements and maintaining accountability for assets. These
systems are augmented by written policies, an organizational structure providing division of responsibilities, careful
selection and training of qualified personnel, and a program of internal audits.

The Company engaged Deloitte & Touche LLP, an independent registered public accounting firm, to audit and render
an opinion on the consolidated financial statements and internal control over financial reporting in accordance with the
standards of the Public Company Accounting Oversight Board (United States).

The Board of Directors, through its Audit Committee, consisting solely of independent directors of the Company, meets
periodically with management, internal auditors, and our independent registered public accounting firm to ensure that
each is meeting its responsibilities and to discuss matters concerning internal controls and financial reporting.
Deloitte & Touche LLP and the internal auditors each have full and free access to the Audit Committee.

Satya Nadella
Chief Executive Officer

Amy E. Hood
Executive Vice President and Chief Financial Officer

Alice L. Jolla
Corporate Vice President and Chief Accounting Officer

<!-- PageNumber="53" -->
<!-- PageBreak -->


# PART II Item 7A


## ITEM 7A. QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK


### RISKS

We are exposed to economic risk from foreign exchange rates, interest rates, credit risk, and equity prices. We use
derivatives instruments to manage these risks, however, they may still impact our consolidated financial statements.


#### Foreign Currencies

Certain forecasted transactions, assets, and liabilities are exposed to foreign currency risk. We monitor our foreign
currency exposures daily to maximize the economic effectiveness of our foreign currency positions, including hedges.
Principal currency exposures include the Euro, Japanese yen, British pound, Canadian dollar, and Australian dollar.


#### Interest Rate

Securities held in our fixed-income portfolio are subject to different interest rate risks based on their maturities. We
manage the average maturity of the fixed-income portfolio to achieve economic returns that correlate to certain global
fixed-income indices.


#### Credit

Our fixed-income portfolio is diversified and consists primarily of investment-grade securities. We manage credit
exposures relative to broad-based indices to facilitate portfolio diversification.


#### Equity

Securities held in our equity investments portfolio are subject to price risk.


### SENSITIVITY ANALYSIS

The following table sets forth the potential loss in future earnings or fair values, including associated derivatives,
resulting from hypothetical changes in relevant market rates or prices:

(In millions)


<table>
<tr>
<th>Risk Categories</th>
<th>Hypothetical Change</th>
<th>June 30, 2024</th>
<th>Impact</th>
</tr>
<tr>
<td>Foreign currency - Revenue</td>
<td>10% decrease in foreign exchange rates</td>
<td>$ (9,605)</td>
<td>Earnings</td>
</tr>
<tr>
<td>Foreign currency - Investments</td>
<td>10% decrease in foreign exchange rates</td>
<td>(38)</td>
<td>Fair Value</td>
</tr>
<tr>
<td>Interest rate</td>
<td>100 basis point increase in U.S. treasury interest rates</td>
<td>(1,343)</td>
<td>Fair Value</td>
</tr>
<tr>
<td>Credit</td>
<td>100 basis point increase in credit spreads</td>
<td>(318)</td>
<td>Fair Value</td>
</tr>
<tr>
<td>Equity</td>
<td>10% decrease in equity market prices</td>
<td>(1,078)</td>
<td>Earnings</td>
</tr>
</table>


<!-- PageNumber="54" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->


# ITEM 8. FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA INCOME STATEMENTS


<table>
<tr>
<th colspan="4">(In millions, except per share amounts)</th>
</tr>
<tr>
<th>Year Ended June 30,</th>
<th>2024</th>
<th>2023</th>
<th>2022</th>
</tr>
<tr>
<td>Revenue:</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Product</td>
<td>$ 64,773</td>
<td>$ 64,699</td>
<td>$ 72,732</td>
</tr>
<tr>
<td>Service and other</td>
<td>180,349</td>
<td>147,216</td>
<td>125,538</td>
</tr>
<tr>
<td>Total revenue</td>
<td>245,122</td>
<td>211,915</td>
<td>198,270</td>
</tr>
<tr>
<td>Cost of revenue:</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Product</td>
<td>15,272</td>
<td>17,804</td>
<td>19,064</td>
</tr>
<tr>
<td>Service and other</td>
<td>58,842</td>
<td>48,059</td>
<td>43,586</td>
</tr>
<tr>
<td>Total cost of revenue</td>
<td>74,114</td>
<td>65,863</td>
<td>62,650</td>
</tr>
<tr>
<td>Gross margin</td>
<td>171,008</td>
<td>146,052</td>
<td>135,620</td>
</tr>
<tr>
<td>Research and development</td>
<td>29,510</td>
<td>27,195</td>
<td>24,512</td>
</tr>
<tr>
<td>Sales and marketing</td>
<td>24,456</td>
<td>22,759</td>
<td>21,825</td>
</tr>
<tr>
<td>General and administrative</td>
<td>7,609</td>
<td>7,575</td>
<td>5,900</td>
</tr>
<tr>
<td>Operating income</td>
<td>109,433</td>
<td>88,523</td>
<td>83,383</td>
</tr>
<tr>
<td>Other income (expense), net</td>
<td>(1,646)</td>
<td>788</td>
<td>333</td>
</tr>
<tr>
<td>Income before income taxes</td>
<td>107,787</td>
<td>89,311</td>
<td>83,716</td>
</tr>
<tr>
<td>Provision for income taxes</td>
<td>19,651</td>
<td>16,950</td>
<td>10,978</td>
</tr>
<tr>
<td>Net income</td>
<td>$ 88,136</td>
<td>$ 72,361</td>
<td>$ 72,738</td>
</tr>
<tr>
<td>Earnings per share:</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Basic</td>
<td>$ 11.86</td>
<td>$ 9.72</td>
<td>$ 9.70</td>
</tr>
<tr>
<td>Diluted</td>
<td>$ 11.80</td>
<td>$ 9.68</td>
<td>$ 9.65</td>
</tr>
<tr>
<td>Weighted average shares outstanding:</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Basic</td>
<td>7,431</td>
<td>7,446</td>
<td>7,496</td>
</tr>
<tr>
<td>Diluted</td>
<td>7,469</td>
<td>7,472</td>
<td>7,540</td>
</tr>
</table>


Refer to accompanying notes.

<!-- PageNumber="55" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->


## COMPREHENSIVE INCOME STATEMENTS

(In millions)


<table>
<tr>
<th>Year Ended June 30,</th>
<th>2024</th>
<th>2023</th>
<th>2022</th>
</tr>
<tr>
<td>Net income</td>
<td>$ 88,136</td>
<td>$ 72,361</td>
<td>$ 72,738</td>
</tr>
<tr>
<td>Other comprehensive income (loss), net of tax:</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Net change related to derivatives</td>
<td>24</td>
<td>(14)</td>
<td>6</td>
</tr>
<tr>
<td>Net change related to investments</td>
<td>957</td>
<td>(1,444)</td>
<td>(5,360)</td>
</tr>
<tr>
<td>Translation adjustments and other</td>
<td>(228)</td>
<td>(207)</td>
<td>(1,146)</td>
</tr>
<tr>
<td>Other comprehensive income (loss)</td>
<td>753</td>
<td>(1,665)</td>
<td>(6,500)</td>
</tr>
<tr>
<td>Comprehensive income</td>
<td>$ 88,889</td>
<td>$ 70,696</td>
<td>$ 66,238</td>
</tr>
</table>


Refer to accompanying notes.

<!-- PageNumber="56" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->


## BALANCE SHEETS

(In millions)


<table>
<tr>
<th>June 30,</th>
<th>2024</th>
<th>2023</th>
</tr>
<tr>
<td>Assets</td>
<td></td>
<td></td>
</tr>
<tr>
<td>Current assets:</td>
<td></td>
<td></td>
</tr>
<tr>
<td>Cash and cash equivalents</td>
<td>$ 18,315</td>
<td>$ 34,704</td>
</tr>
<tr>
<td>Short-term investments</td>
<td>57,228</td>
<td>76,558</td>
</tr>
<tr>
<td>Total cash, cash equivalents, and short-term investments</td>
<td>75,543</td>
<td>111,262</td>
</tr>
<tr>
<td>Accounts receivable, net of allowance for doubtful accounts of $830 and $650</td>
<td>56,924</td>
<td>48,688</td>
</tr>
<tr>
<td>Inventories</td>
<td>1,246</td>
<td>2,500</td>
</tr>
<tr>
<td>Other current assets</td>
<td>26,021</td>
<td>21,807</td>
</tr>
<tr>
<td>Total current assets</td>
<td>159,734</td>
<td>184,257</td>
</tr>
<tr>
<td>Property and equipment, net of accumulated depreciation of $76,421 and $68,251</td>
<td>135,591</td>
<td>95,641</td>
</tr>
<tr>
<td>Operating lease right-of-use assets</td>
<td>18,961</td>
<td>14,346</td>
</tr>
<tr>
<td>Equity and other investments</td>
<td>14,600</td>
<td>9,879</td>
</tr>
<tr>
<td>Goodwill</td>
<td>119,220</td>
<td>67,886</td>
</tr>
<tr>
<td>Intangible assets, net</td>
<td>27,597</td>
<td>9,366</td>
</tr>
<tr>
<td>Other long-term assets</td>
<td>36,460</td>
<td>30,601</td>
</tr>
<tr>
<td>Total assets</td>
<td>$ 512,163</td>
<td>$ 411,976</td>
</tr>
<tr>
<td>Liabilities and stockholders' equity</td>
<td></td>
<td></td>
</tr>
<tr>
<td>Current liabilities:</td>
<td></td>
<td></td>
</tr>
<tr>
<td>Accounts payable</td>
<td>$ 21,996</td>
<td>$ 18,095</td>
</tr>
<tr>
<td>Short-term debt</td>
<td>6,693</td>
<td>0</td>
</tr>
<tr>
<td>Current portion of long-term debt</td>
<td>2,249</td>
<td>5,247</td>
</tr>
<tr>
<td>Accrued compensation</td>
<td>12,564</td>
<td>11,009</td>
</tr>
<tr>
<td>Short-term income taxes</td>
<td>5,017</td>
<td>4,152</td>
</tr>
<tr>
<td>Short-term unearned revenue</td>
<td>57,582</td>
<td>50,901</td>
</tr>
<tr>
<td>Other current liabilities</td>
<td>19,185</td>
<td>14,745</td>
</tr>
<tr>
<td>Total current liabilities</td>
<td>125,286</td>
<td>104,149</td>
</tr>
<tr>
<td>Long-term debt</td>
<td>42,688</td>
<td>41,990</td>
</tr>
<tr>
<td>Long-term income taxes</td>
<td>27,931</td>
<td>25,560</td>
</tr>
<tr>
<td>Long-term unearned revenue</td>
<td>2,602</td>
<td>2,912</td>
</tr>
<tr>
<td>Deferred income taxes</td>
<td>2,618</td>
<td>433</td>
</tr>
<tr>
<td>Operating lease liabilities</td>
<td>15,497</td>
<td>12,728</td>
</tr>
<tr>
<td>Other long-term liabilities</td>
<td>27,064</td>
<td>17,981</td>
</tr>
<tr>
<td>Total liabilities</td>
<td>243,686</td>
<td>205,753</td>
</tr>
<tr>
<td>Commitments and contingencies</td>
<td></td>
<td></td>
</tr>
<tr>
<td>Stockholders' equity:</td>
<td></td>
<td></td>
</tr>
<tr>
<td>Common stock and paid-in capital - shares authorized 24,000; outstanding 7,434 and 7,432</td>
<td>100,923</td>
<td>93,718</td>
</tr>
<tr>
<td>Retained earnings</td>
<td>173,144</td>
<td>118,848</td>
</tr>
<tr>
<td>Accumulated other comprehensive loss</td>
<td>(5,590)</td>
<td>(6,343)</td>
</tr>
<tr>
<td>Total stockholders' equity</td>
<td>268,477</td>
<td>206,223</td>
</tr>
<tr>
<td>Total liabilities and stockholders' equity</td>
<td>$ 512,163</td>
<td>$ 411,976</td>
</tr>
</table>


Refer to accompanying notes.

<!-- PageNumber="57" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->


## CASH FLOWS STATEMENTS

(In millions)


<table>
<tr>
<th>Year Ended June 30,</th>
<th>2024</th>
<th>2023</th>
<th>2022</th>
</tr>
<tr>
<td>Operations</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Net income</td>
<td>$ 88,136</td>
<td>$ 72,361</td>
<td>$ 72,738</td>
</tr>
<tr>
<td>Adjustments to reconcile net income to net cash from operations:</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Depreciation, amortization, and other</td>
<td>22,287</td>
<td>13,861</td>
<td>14,460</td>
</tr>
<tr>
<td>Stock-based compensation expense</td>
<td>10,734</td>
<td>9,611</td>
<td>7,502</td>
</tr>
<tr>
<td>Net recognized losses (gains) on investments and derivatives</td>
<td>305</td>
<td>196</td>
<td>(409)</td>
</tr>
<tr>
<td>Deferred income taxes</td>
<td>(4,738)</td>
<td>(6,059)</td>
<td>(5,702)</td>
</tr>
<tr>
<td>Changes in operating assets and liabilities:</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Accounts receivable</td>
<td>(7,191)</td>
<td>(4,087)</td>
<td>(6,834)</td>
</tr>
<tr>
<td>Inventories</td>
<td>1,284</td>
<td>1,242</td>
<td>(1,123)</td>
</tr>
<tr>
<td>Other current assets</td>
<td>(1,648)</td>
<td>(1,991)</td>
<td>(709)</td>
</tr>
<tr>
<td>Other long-term assets</td>
<td>(6,817)</td>
<td>(2,833)</td>
<td>(2,805)</td>
</tr>
<tr>
<td>Accounts payable</td>
<td>3,545</td>
<td>(2,721)</td>
<td>2,943</td>
</tr>
<tr>
<td>Unearned revenue</td>
<td>5,348</td>
<td>5,535</td>
<td>5,109</td>
</tr>
<tr>
<td>Income taxes</td>
<td>1,687</td>
<td>(358)</td>
<td>696</td>
</tr>
<tr>
<td>Other current liabilities</td>
<td>4,867</td>
<td>2,272</td>
<td>2,344</td>
</tr>
<tr>
<td>Other long-term liabilities</td>
<td>749</td>
<td>553</td>
<td>825</td>
</tr>
<tr>
<td>Net cash from operations</td>
<td>118,548</td>
<td>87,582</td>
<td>89,035</td>
</tr>
<tr>
<td>Financing</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Proceeds from issuance of debt, maturities of 90 days or less, net</td>
<td>5,250</td>
<td>0</td>
<td>0</td>
</tr>
<tr>
<td>Proceeds from issuance of debt</td>
<td>24,395</td>
<td>0</td>
<td>0</td>
</tr>
<tr>
<td>Repayments of debt</td>
<td>(29,070)</td>
<td>(2,750)</td>
<td>(9,023)</td>
</tr>
<tr>
<td>Common stock issued</td>
<td>2,002</td>
<td>1,866</td>
<td>1,841</td>
</tr>
<tr>
<td>Common stock repurchased</td>
<td>(17,254)</td>
<td>(22,245)</td>
<td>(32,696)</td>
</tr>
<tr>
<td>Common stock cash dividends paid</td>
<td>(21,771)</td>
<td>(19,800)</td>
<td>(18,135)</td>
</tr>
<tr>
<td>Other, net</td>
<td>(1,309)</td>
<td>(1,006)</td>
<td>(863)</td>
</tr>
<tr>
<td>Net cash used in financing</td>
<td>(37,757)</td>
<td>(43,935)</td>
<td>(58,876)</td>
</tr>
<tr>
<td>Investing</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Additions to property and equipment</td>
<td>(44,477)</td>
<td>(28,107)</td>
<td>(23,886)</td>
</tr>
<tr>
<td>Acquisition of companies, net of cash acquired, and purchases of intangible and other assets</td>
<td>(69,132)</td>
<td>(1,670)</td>
<td>(22,038)</td>
</tr>
<tr>
<td>Purchases of investments</td>
<td>(17,732)</td>
<td>(37,651)</td>
<td>(26,456)</td>
</tr>
<tr>
<td>Maturities of investments</td>
<td>24,775</td>
<td>33,510</td>
<td>16,451</td>
</tr>
<tr>
<td>Sales of investments</td>
<td>10,894</td>
<td>14,354</td>
<td>28,443</td>
</tr>
<tr>
<td>Other, net</td>
<td>(1,298)</td>
<td>(3,116)</td>
<td>(2,825)</td>
</tr>
<tr>
<td>Net cash used in investing</td>
<td>(96,970)</td>
<td>(22,680)</td>
<td>(30,311)</td>
</tr>
<tr>
<td>Effect of foreign exchange rates on cash and cash equivalents</td>
<td>(210)</td>
<td>(194)</td>
<td>(141)</td>
</tr>
<tr>
<td>Net change in cash and cash equivalents</td>
<td>(16,389)</td>
<td>20,773</td>
<td>(293)</td>
</tr>
<tr>
<td>Cash and cash equivalents, beginning of period</td>
<td>34,704</td>
<td>13,931</td>
<td>14,224</td>
</tr>
<tr>
<td>Cash and cash equivalents, end of period</td>
<td>$ 18,315</td>
<td>$ 34,704</td>
<td>$ 13,931</td>
</tr>
</table>


Refer to accompanying notes.

<!-- PageNumber="58" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->


## STOCKHOLDERS' EQUITY STATEMENTS

(In millions, except per share amounts)


<table>
<tr>
<th>Year Ended June 30,</th>
<th>2024</th>
<th>2023</th>
<th>2022</th>
</tr>
<tr>
<td>Common stock and paid-in capital</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Balance, beginning of period</td>
<td>$ 93,718</td>
<td>$ 86,939</td>
<td>$ 83,111</td>
</tr>
<tr>
<td>Common stock issued</td>
<td>2,002</td>
<td>1,866</td>
<td>1,841</td>
</tr>
<tr>
<td>Common stock repurchased</td>
<td>(5,712)</td>
<td>(4,696)</td>
<td>(5,688)</td>
</tr>
<tr>
<td>Stock-based compensation expense</td>
<td>10,734</td>
<td>9,611</td>
<td>7,502</td>
</tr>
<tr>
<td>Other, net</td>
<td>181</td>
<td>(2)</td>
<td>173</td>
</tr>
<tr>
<td>Balance, end of period</td>
<td>100,923</td>
<td>93,718</td>
<td>86,939</td>
</tr>
<tr>
<td>Retained earnings</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Balance, beginning of period</td>
<td>118,848</td>
<td>84,281</td>
<td>57,055</td>
</tr>
<tr>
<td>Net income</td>
<td>88,136</td>
<td>72,361</td>
<td>72,738</td>
</tr>
<tr>
<td>Common stock cash dividends</td>
<td>(22,293)</td>
<td>(20,226)</td>
<td>(18,552)</td>
</tr>
<tr>
<td>Common stock repurchased</td>
<td>(11,547)</td>
<td>(17,568)</td>
<td>(26,960)</td>
</tr>
<tr>
<td>Balance, end of period</td>
<td>173,144</td>
<td>118,848</td>
<td>84,281</td>
</tr>
<tr>
<td>Accumulated other comprehensive loss</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Balance, beginning of period</td>
<td>(6,343)</td>
<td>(4,678)</td>
<td>1,822</td>
</tr>
<tr>
<td>Other comprehensive income (loss)</td>
<td>753</td>
<td>(1,665)</td>
<td>(6,500)</td>
</tr>
<tr>
<td>Balance, end of period</td>
<td>(5,590)</td>
<td>(6,343)</td>
<td>(4,678)</td>
</tr>
<tr>
<td>Total stockholders' equity</td>
<td>$ 268,477</td>
<td>$ 206,223</td>
<td>$ 166,542</td>
</tr>
<tr>
<td>Cash dividends declared per common share</td>
<td>$ 3.00</td>
<td>$ 2.72</td>
<td>$ 2.48</td>
</tr>
</table>


Refer to accompanying notes.

<!-- PageNumber="59" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->


# NOTES TO FINANCIAL STATEMENTS NOTE 1 - ACCOUNTING POLICIES


## Accounting Principles

Our consolidated financial statements and accompanying notes are prepared in accordance with accounting principles
generally accepted in the United States of America ("GAAP").

We have recast certain prior period amounts to conform to the current period presentation. The recast of these prior
period amounts had no impact on our consolidated balance sheets, consolidated income statements, or consolidated
cash flows statements.


## Principles of Consolidation

The consolidated financial statements include the accounts of Microsoft Corporation and its subsidiaries. Intercompany
transactions and balances have been eliminated.


## Estimates and Assumptions

Preparing financial statements requires management to make estimates and assumptions that affect the reported
amounts of assets, liabilities, revenue, and expenses. Examples of estimates and assumptions include: for revenue
recognition, determining the nature and timing of satisfaction of performance obligations, and determining the
standalone selling price ("SSP") of performance obligations, variable consideration, and other obligations such as
product returns and refunds; loss contingencies; product warranties; the fair value of and/or potential impairment of
goodwill and intangible assets for our reporting units; product life cycles; useful lives of our tangible and intangible
assets; allowances for doubtful accounts; the market value of, and demand for, our inventory; stock-based
compensation forfeiture rates; when technological feasibility is achieved for our products; the potential outcome of
uncertain tax positions that have been recognized in our consolidated financial statements or tax returns; and
determining the timing and amount of impairments for investments. Actual results and outcomes may differ from
management's estimates and assumptions due to risks and uncertainties.

In July 2022, we completed an assessment of the useful lives of our server and network equipment. Due to investments
in software that increased efficiencies in how we operate our server and network equipment, as well as advances in
technology, we determined we should increase the estimated useful lives of both server and network equipment from
four years to six years. This change in accounting estimate was effective beginning fiscal year 2023.


## Foreign Currencies

Assets and liabilities recorded in foreign currencies are translated at the exchange rate on the balance sheet date.
Revenue and expenses are translated at average rates of exchange prevailing during the year. Translation adjustments
resulting from this process are recorded to other comprehensive income.


## Revenue


### Product Revenue and Service and Other Revenue

Product revenue includes sales from operating systems, cross-device productivity and collaboration applications,
server applications, business solution applications, desktop and server management tools, software development
tools, video games, and hardware such as PCs, tablets, gaming and entertainment consoles, other intelligent devices,
and related accessories.

Service and other revenue includes sales from cloud-based solutions that provide customers with software, services,
platforms, and content such as Office 365, Azure, Dynamics 365, and gaming; solution support; and consulting
services. Service and other revenue also includes sales from online advertising and LinkedIn.

<!-- PageNumber="60" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->


### Revenue Recognition

Revenue is recognized upon transfer of control of promised products or services to customers in an amount that reflects
the consideration we expect to receive in exchange for those products or services. We enter into contracts that can
include various combinations of products and services, which are generally capable of being distinct and accounted
for as separate performance obligations. Revenue is recognized net of allowances for returns and any taxes collected
from customers, which are subsequently remitted to governmental authorities.


#### Nature of Products and Services

Licenses for on-premises software provide the customer with a right to use the software as it exists when made
available to the customer. Customers may purchase perpetual licenses or subscribe to licenses, which provide
customers with the same functionality and differ mainly in the duration over which the customer benefits from the
software. Revenue from distinct on-premises licenses is recognized upfront at the point in time when the software is
made available to the customer. In cases where we allocate revenue to software updates, primarily because the
updates are provided at no additional charge, revenue is recognized as the updates are provided, which is generally
ratably over the estimated life of the related device or license.

Certain volume licensing programs, including Enterprise Agreements, include on-premises licenses combined with
Software Assurance ("SA"). SA conveys rights to new software and upgrades released over the contract period and
provides support, tools, and training to help customers deploy and use products more efficiently. On-premises licenses
are considered distinct performance obligations when sold with SA. Revenue allocated to SA is generally recognized
ratably over the contract period as customers simultaneously consume and receive benefits, given that SA comprises
distinct performance obligations that are satisfied over time.

Cloud services, which allow customers to use hosted software over the contract period without taking possession of
the software, are provided on either a subscription or consumption basis. Revenue related to cloud services provided
on a subscription basis is recognized ratably over the contract period. Revenue related to cloud services provided on
a consumption basis, such as the amount of storage used in a period, is recognized based on the customer utilization
of such resources. When cloud services require a significant level of integration and interdependency with software
and the individual components are not considered distinct, all revenue is recognized over the period in which the cloud
services are provided.

Revenue from search advertising is recognized when the advertisement appears in the search results or when the
action necessary to earn the revenue has been completed. Revenue from consulting services is recognized as services
are provided.

Our hardware is generally highly dependent on, and interrelated with, the underlying operating system and cannot
function without the operating system. In these cases, the hardware and software license are accounted for as a single
performance obligation and revenue is recognized at the point in time when ownership is transferred to resellers or
directly to end customers through retail stores and online marketplaces.

Refer to Note 19 - Segment Information and Geographic Data for further information, including revenue by significant
product and service offering.


#### Significant Judgments

Our contracts with customers often include promises to transfer multiple products and services to a customer.
Determining whether products and services are considered distinct performance obligations that should be accounted
for separately versus together may require significant judgment. When a cloud-based service includes both on-
premises software licenses and cloud services, judgment is required to determine whether the software license is
considered distinct and accounted for separately, or not distinct and accounted for together with the cloud service and
recognized over time. Certain cloud services, primarily Office 365, depend on a significant level of integration,
interdependency, and interrelation between the desktop applications and cloud services, and are accounted for
together as one performance obligation. Revenue from Office 365 is recognized ratably over the period in which the
cloud services are provided.

Judgment is required to determine the SSP for each distinct performance obligation. We use a single amount to
estimate SSP for items that are not sold separately, including on-premises licenses sold with SA or software updates

<!-- PageNumber="61" -->
<!-- PageBreak -->


# PART II Item 8

provided at no additional charge. We use a range of amounts to estimate SSP when we sell each of the products and
services separately and need to determine whether there is a discount to be allocated based on the relative SSP of
the various products and services.

In instances where SSP is not directly observable, such as when we do not sell the product or service separately, we
determine the SSP using information that may include market conditions and other observable inputs. We typically
have more than one SSP for individual products and services due to the stratification of those products and services
by customers and circumstances. In these instances, we may use information such as the size of the customer and
geographic region in determining the SSP.

Due to the various benefits from and the nature of our SA program, judgment is required to assess the pattern of
delivery, including the exercise pattern of certain benefits across our portfolio of customers.

Our products are generally sold with a right of return, we may provide other credits or incentives, and in certain
instances we estimate customer usage of our products and services, which are accounted for as variable consideration
when determining the amount of revenue to recognize. Returns and credits are estimated at contract inception and
updated at the end of each reporting period if additional information becomes available. Changes to our estimated
variable consideration were not material for the periods presented.


## Contract Balances and Other Receivables

Timing of revenue recognition may differ from the timing of invoicing to customers. We record a receivable when
revenue is recognized prior to invoicing, or unearned revenue when revenue is recognized subsequent to invoicing.
For multi-year agreements, we generally invoice customers annually at the beginning of each annual coverage period.
We record a receivable related to revenue recognized for multi-year on-premises licenses as we have an unconditional
right to invoice and receive payment in the future related to those licenses.

Unearned revenue comprises mainly unearned revenue related to volume licensing programs, which may include $\mathrm { S A }$
and cloud services. Unearned revenue is generally invoiced annually at the beginning of each contract period for multi-
year agreements and recognized ratably over the coverage period. Unearned revenue also includes payments for
consulting services to be performed in the future, Linkedln subscriptions, Office 365 subscriptions, Xbox subscriptions,
Windows post-delivery support, Dynamics business solutions, and other offerings for which we have been paid in
advance and earn the revenue when we transfer control of the product or service.

Refer to Note 13 - Unearned Revenue for further information, including unearned revenue by segment and changes
in unearned revenue during the period.

Payment terms and conditions vary by contract type, although terms generally include a requirement of payment within
30 to 60 days. In instances where the timing of revenue recognition differs from the timing of invoicing, we have
determined our contracts generally do not include a significant financing component. The primary purpose of our
invoicing terms is to provide customers with simplified and predictable ways of purchasing our products and services,
not to receive financing from our customers or to provide customers with financing. Examples include invoicing at the
beginning of a subscription term with revenue recognized ratably over the contract period, and multi-year on-premises
licenses that are invoiced annually with revenue recognized upfront.

As of June 30, 2024 and 2023, long-term accounts receivable, net of allowance for doubtful accounts, was $4.9 billion
and $4.5 billion, respectively, and is included in other long-term assets in our consolidated balance sheets.

The allowance for doubtful accounts reflects our best estimate of probable losses inherent in the accounts receivable
balance. We determine the allowance based on known troubled accounts, historical experience, and other currently
available evidence.

<!-- PageNumber="62" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->


# Activity in the allowance for doubtful accounts was as follows:


## (In millions)


<table>
<tr>
<th>Year Ended June 30,</th>
<th>2024</th>
<th>2023</th>
<th>2022</th>
</tr>
<tr>
<td>Balance, beginning of period</td>
<td>$ 716</td>
<td>$ 710</td>
<td>$ 798</td>
</tr>
<tr>
<td>Charged to costs and other</td>
<td>386</td>
<td>258</td>
<td>157</td>
</tr>
<tr>
<td>Write-offs</td>
<td>(218)</td>
<td>(252)</td>
<td>(245)</td>
</tr>
<tr>
<td>Balance, end of period</td>
<td>$ 884</td>
<td>$ 716</td>
<td>$ 710</td>
</tr>
</table>


Allowance for doubtful accounts included in our consolidated balance sheets:

(In millions)


<table>
<tr>
<th>June 30,</th>
<th>2024</th>
<th>2023</th>
<th>2022</th>
</tr>
<tr>
<td>Accounts receivable, net of allowance for doubtful accounts</td>
<td>$ 830</td>
<td>$ 650</td>
<td>$ 633</td>
</tr>
<tr>
<td>Other long-term assets</td>
<td>54</td>
<td>66</td>
<td>77</td>
</tr>
<tr>
<td>Total</td>
<td>$ 884</td>
<td>$ 716</td>
<td>$ 710</td>
</tr>
</table>


As of June 30, 2024 and 2023, other receivables related to activities to facilitate the purchase of server components
were $10.5 billion and $9.2 billion, respectively, and are included in other current assets in our consolidated balance
sheets.

We record financing receivables when we offer certain customers the option to acquire our software products and
services offerings through a financing program in a limited number of countries. As of June 30, 2024 and 2023, our
financing receivables, net were $4.5 billion and $5.3 billion, respectively, for short-term and long-term financing
receivables, which are included in other current assets and other long-term assets in our consolidated balance sheets.
We record an allowance to cover expected losses based on troubled accounts, historical experience, and other
currently available evidence.


### Assets Recognized from Costs to Obtain a Contract with a Customer

We recognize an asset for the incremental costs of obtaining a contract with a customer if we expect the benefit of
those costs to be longer than one year. We have determined that certain sales incentive programs meet the
requirements to be capitalized. Total capitalized costs to obtain a contract were immaterial during the periods presented
and are included in other current and long-term assets in our consolidated balance sheets.

We apply a practical expedient to expense costs as incurred for costs to obtain a contract with a customer when the
amortization period would have been one year or less. These costs include our internal sales organization
compensation program and certain partner sales incentive programs as we have determined annual compensation is
commensurate with annual sales activities.


#### Cost of Revenue

Cost of revenue includes: manufacturing and distribution costs for products sold and programs licensed; operating
costs related to product support service centers and product distribution centers; costs incurred to include software on
PCs sold by original equipment manufacturers ("OEM"), to drive traffic to our websites, and to acquire online advertising
space; costs incurred to support and maintain cloud-based and other online products and services, including datacenter
costs and royalties; warranty costs; inventory valuation adjustments; costs associated with the delivery of consulting
services; and the amortization of capitalized software development costs. Capitalized software development costs are
amortized over the estimated lives of the products.


#### Product Warranty

We provide for the estimated costs of fulfilling our obligations under hardware and software warranties at the time the
related revenue is recognized. For hardware warranties, we estimate the costs based on historical and projected
product failure rates, historical and projected repair costs, and knowledge of specific product failures (if any). The

<!-- PageNumber="63" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->

specific hardware warranty terms and conditions vary depending upon the product sold and the country in which we
do business, but generally include parts and labor over a period generally ranging from 90 days to three years. For
software warranties, we estimate the costs to provide bug fixes, such as security patches, over the estimated life of the
software. We regularly reevaluate our estimates to assess the adequacy of the recorded warranty liabilities and adjust
the amounts as necessary.


#### Research and Development

Research and development expenses include payroll, employee benefits, stock-based compensation expense, and
other headcount-related expenses associated with product development. Research and development expenses also
include third-party development and programming costs and the amortization of purchased software code and services
content. Such costs related to software development are included in research and development expense until the point
that technological feasibility is reached, which for our software products, is generally shortly before the products are
released to production. Once technological feasibility is reached, such costs are capitalized and amortized to cost of
revenue over the estimated lives of the products.


#### Sales and Marketing

Sales and marketing expenses include payroll, employee benefits, stock-based compensation expense, and other
headcount-related expenses associated with sales and marketing personnel, and the costs of advertising, promotions,
trade shows, seminars, and other programs. Advertising costs are expensed as incurred. Advertising expense was
$1.7 billion, $904 million, and $1.5 billion in fiscal years 2024, 2023, and 2022, respectively.


#### Stock-Based Compensation

Compensation cost for stock awards, which include restricted stock units ("RSUs") and performance stock units
("PSUs"), is measured at the fair value on the grant date and recognized as expense, net of estimated forfeitures, over
the related service or performance period. The fair value of stock awards is based on the quoted price of our common
stock on the grant date less the present value of expected dividends not received during the vesting period. We
measure the fair value of PSUs using a Monte Carlo valuation model. Compensation cost for RSUs is recognized using
the straight-line method and for PSUs is recognized using the accelerated method.

Compensation expense for the employee stock purchase plan ("ESPP") is measured as the discount the employee is
entitled to upon purchase and is recognized in the period of purchase.


#### Income Taxes

Income tax expense includes U.S. and international income taxes, and interest and penalties on uncertain tax positions.
Certain income and expenses are not reported in tax returns and financial statements in the same year. The tax effect
of such temporary differences is reported as deferred income taxes. Deferred tax assets are reported net of a valuation
allowance when it is more likely than not that a tax benefit will not be realized. All deferred income taxes are classified
as long-term in our consolidated balance sheets.


##### Financial Instruments


###### Investments

We consider all highly liquid interest-earning investments with a maturity of three months or less at the date of purchase
to be cash equivalents. The fair values of these investments approximate their carrying values. In general, investments
with original maturities of greater than three months and remaining maturities of less than one year are classified as
short-term investments. Investments with maturities beyond one year may be classified as short-term based on their
highly liquid nature and because such marketable securities represent the investment of cash that is available for
current operations.

Debt investments are classified as available-for-sale and realized gains and losses are recorded using the specific
identification method. Changes in fair value, excluding credit losses and impairments, are recorded in other
comprehensive income. Fair value is calculated based on publicly available market information or other estimates
determined by management. If the cost of an investment exceeds its fair value, we evaluate, among other factors,

<!-- PageNumber="64" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->

general market conditions, credit quality of debt instrument issuers, and the extent to which the fair value is less than
cost. To determine credit losses, we employ a systematic methodology that considers available quantitative and
qualitative evidence. In addition, we consider specific adverse conditions related to the financial health of, and business
outlook for, the investee. If we have plans to sell the security or it is more likely than not that we will be required to sell
the security before recovery, then a decline in fair value below cost is recorded as an impairment charge in other
income (expense), net and a new cost basis in the investment is established. If market, industry, and/or investee
conditions deteriorate, we may incur future impairments.

Equity investments with readily determinable fair values are measured at fair value. Equity investments without readily
determinable fair values are measured using the equity method or measured at cost with adjustments for observable
changes in price or impairments (referred to as the measurement alternative). We perform a qualitative assessment
on a periodic basis and recognize an impairment if there are sufficient indicators that the fair value of the investment
is less than carrying value. Changes in value are recorded in other income (expense), net.

Investments that are considered variable interest entities ("VIEs") are evaluated to determine whether we are the
primary beneficiary of the VIE, in which case we would be required to consolidate the entity. We evaluate whether we
have (1) the power to direct the activities that most significantly impact the VIE's economic performance, and (2) the
obligation to absorb losses or the right to receive benefits from the VIE that could potentially be significant to the VIE.
We have determined we are not the primary beneficiary of any of our VIE investments. Therefore, our VIE investments
are not consolidated and the majority are accounted for under the equity method of accounting.


###### Derivatives

Derivative instruments are recognized as either assets or liabilities and measured at fair value. The accounting for
changes in the fair value of a derivative depends on the intended use of the derivative and the resulting designation.

For derivative instruments designated as fair value hedges, gains and losses are recognized in other income (expense),
net with offsetting gains and losses on the hedged items. Gains and losses representing hedge components excluded
from the assessment of effectiveness are recognized in other income (expense), net.

For derivative instruments designated as cash flow hedges, gains and losses are initially reported as a component of
other comprehensive income and subsequently recognized in other income (expense), net with the corresponding
hedged item. Gains and losses representing hedge components excluded from the assessment of effectiveness are
recognized in other income (expense), net.

For derivative instruments that are not designated as hedges, gains and losses from changes in fair values are primarily
recognized in other income (expense), net.


###### Fair Value Measurements

We account for certain assets and liabilities at fair value. The hierarchy below lists three levels of fair value based on
the extent to which inputs used in measuring fair value are observable in the market. We categorize each of our fair
value measurements in one of these three levels based on the lowest level input that is significant to the fair value
measurement in its entirety. These levels are:

· Level 1 - inputs are based upon unadjusted quoted prices for identical instruments in active markets. Our
Level 1 investments include U.S. government securities, common and preferred stock, and mutual
funds. Our Level 1 derivative assets and liabilities include those actively traded on exchanges.

· Level 2 - inputs are based upon quoted prices for similar instruments in active markets, quoted prices for
identical or similar instruments in markets that are not active, and model-based valuation techniques (e.g.
the Black-Scholes model) for which all significant inputs are observable in the market or can be corroborated
by observable market data for substantially the full term of the assets or liabilities. Where applicable, these
models project future cash flows and discount the future amounts to a present value using market-based
observable inputs including interest rate curves, credit spreads, foreign exchange rates, and forward and
spot prices for currencies. Our Level 2 investments include commercial paper, certificates of deposit, U.S.
agency securities, foreign government bonds, mortgage- and asset-backed securities, corporate notes and
bonds, and municipal securities. Our Level 2 derivative assets and liabilities include certain cleared swap
contracts and over-the-counter forward, option, and swap contracts.

<!-- PageNumber="65" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->

· Level 3 - inputs are generally unobservable and typically reflect management's estimates of assumptions
that market participants would use in pricing the asset or liability. The fair values are therefore determined
using model-based techniques, including option pricing models and discounted cash flow models. Our Level
3 assets and liabilities include investments in corporate notes and bonds, municipal securities, and goodwill
and intangible assets, when they are recorded at fair value due to an impairment charge. Unobservable
inputs used in the models are significant to the fair values of the assets and liabilities.

We measure equity investments without readily determinable fair values on a nonrecurring basis. The fair values of
these investments are determined based on valuation techniques using the best information available, and may include
quoted market prices, market comparables, and discounted cash flow projections.

Our other current financial assets and current financial liabilities have fair values that approximate their carrying values.


###### Inventories

Inventories are stated at average cost, subject to the lower of cost or net realizable value. Cost includes materials,
labor, and manufacturing overhead related to the purchase and production of inventories. Net realizable value is the
estimated selling price less estimated costs of completion, disposal, and transportation. We regularly review inventory
quantities on hand, future purchase commitments with our suppliers, and the estimated utility of our inventory. If our
review indicates a reduction in utility below carrying value, we reduce our inventory to a new cost basis through a
charge to cost of revenue.


###### Property and Equipment

Property and equipment is stated at cost less accumulated depreciation, and depreciated using the straight-line method
over the shorter of the estimated useful life of the asset or the lease term. The estimated useful lives of our property
and equipment are generally as follows: computer software developed or acquired for internal use, three years;
computer equipment, two to six years; buildings and improvements, five to 15 years; leasehold improvements, three
to 20 years; and furniture and equipment, one to 10 years. Land is not depreciated.


###### Leases

We determine if an arrangement is a lease at inception. Operating leases are included in operating lease right-of-use
("ROU") assets, other current liabilities, and operating lease liabilities in our consolidated balance sheets. Finance
leases are included in property and equipment, other current liabilities, and other long-term liabilities in our consolidated
balance sheets.

ROU assets represent our right to use an underlying asset for the lease term and lease liabilities represent our
obligation to make lease payments arising from the lease. Operating lease ROU assets and liabilities are recognized
at commencement date based on the present value of lease payments over the lease term. As most of our leases do
not provide an implicit rate, we generally use our incremental borrowing rate based on the estimated rate of interest
for collateralized borrowing over a similar term of the lease payments at commencement date. The operating lease
ROU asset also includes any lease payments made and excludes lease incentives. Our lease terms may include
options to extend or terminate the lease when it is reasonably certain that we will exercise that option. Lease expense
for lease payments is recognized on a straight-line basis over the lease term.

We have lease agreements with lease and non-lease components, which are generally accounted for separately. For
certain equipment leases, such as vehicles, we account for the lease and non-lease components as a single lease
component. Additionally, for certain equipment leases, we apply a portfolio approach to effectively account for the
operating lease ROU assets and liabilities.


###### Goodwill

Goodwill is tested for impairment at the reporting unit level (operating segment or one level below an operating
segment) on an annual basis (May 1) and between annual tests if an event occurs or circumstances change that would
more likely than not reduce the fair value of a reporting unit below its carrying value.

<!-- PageNumber="66" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->


###### Intangible Assets

Our intangible assets are subject to amortization and are amortized over the estimated useful life in proportion to the
economic benefits received. We evaluate the recoverability of intangible assets periodically by taking into account
events or circumstances that may warrant revised estimates of useful lives or that indicate the asset may be impaired.


###### Related Party Transactions

In March 2024, we entered into an agreement with Inflection Al, Inc. ("Inflection"), pursuant to which we obtained a
non-exclusive license to Inflection's intellectual property. Reid Hoffman, a member of our Board of Directors, is a co-
founder of and serves on the board of directors of Inflection. As of the date of the agreement with Inflection,
Reprogrammed Interchange LLC ("Reprogrammed") and entities affiliated with Greylock Ventures ("Greylock") each
held less than a 10% equity interest in Inflection. Mr. Hoffman may be deemed to beneficially own the shares held by
Reprogrammed and Greylock by virtue of his relationship with such entities. Mr. Hoffman did not participate in any
portions of the meetings of our Board of Directors or any committee thereof to review and approve the transaction with
Inflection.


###### Recent Accounting Guidance


####### Segment Reporting - Improvements to Reportable Segment Disclosures

In November 2023, the Financial Accounting Standards Board ("FASB") issued a new standard to improve reportable
segment disclosures. The guidance expands the disclosures required for reportable segments in our annual and interim
consolidated financial statements, primarily through enhanced disclosures about significant segment expenses. The
standard will be effective for us beginning with our annual reporting for fiscal year 2025 and interim periods thereafter,
with early adoption permitted. We are currently evaluating the impact of this standard on our segment disclosures.


####### Income Taxes - Improvements to Income Tax Disclosures

In December 2023, the FASB issued a new standard to improve income tax disclosures. The guidance requires
disclosure of disaggregated income taxes paid, prescribes standardized categories for the components of the effective
tax rate reconciliation, and modifies other income tax-related disclosures. The standard will be effective for us beginning
with our annual reporting for fiscal year 2026, with early adoption permitted. We are currently evaluating the impact of
this standard on our income tax disclosures.


######## NOTE 2 - EARNINGS PER SHARE

Basic earnings per share ("EPS") is computed based on the weighted average number of shares of common stock
outstanding during the period. Diluted EPS is computed based on the weighted average number of shares of common
stock plus the effect of dilutive potential common shares outstanding during the period using the treasury stock method.
Dilutive potential common shares include outstanding stock options and stock awards.

<!-- PageNumber="67" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->

The components of basic and diluted EPS were as follows:

(In millions, except per share amounts)


<table>
<tr>
<th>Year Ended June 30,</th>
<th>2024</th>
<th>2023</th>
<th>2022</th>
</tr>
<tr>
<td>Net income available for common shareholders (A)</td>
<td>$ 88,136</td>
<td>$ 72,361</td>
<td>$ 72,738</td>
</tr>
<tr>
<td>Weighted average outstanding shares of common stock (B)</td>
<td>7,431</td>
<td>7,446</td>
<td>7,496</td>
</tr>
<tr>
<td>Dilutive effect of stock-based awards</td>
<td>38</td>
<td>26</td>
<td>44</td>
</tr>
<tr>
<td>Common stock and common stock equivalents (C)</td>
<td>7,469</td>
<td>7,472</td>
<td>7,540</td>
</tr>
<tr>
<td>Earnings Per Share</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Basic (A/B)</td>
<td>$ 11.86</td>
<td>$ 9.72</td>
<td>$ 9.70</td>
</tr>
<tr>
<td>Diluted (A/C)</td>
<td>$ 11.80</td>
<td>$ 9.68</td>
<td>$ 9.65</td>
</tr>
</table>


Anti-dilutive stock-based awards excluded from the calculations of diluted EPS were immaterial during the periods
presented.


# NOTE 3 - OTHER INCOME (EXPENSE), NET

The components of other income (expense), net were as follows:

(In millions)


<table>
<tr>
<th>Year Ended June 30,</th>
<th>2024</th>
<th>2023</th>
<th>2022</th>
</tr>
<tr>
<td>Interest and dividends income</td>
<td>$ 157</td>
<td>$ 2,994</td>
<td>$ 2,09</td>
</tr>
<tr>
<td>Interest expense</td>
<td>(2,935)</td>
<td>(1,968)</td>
<td>(2,063)</td>
</tr>
<tr>
<td>Net recognized gains (losses) on investments</td>
<td>(118)</td>
<td>260</td>
<td>461</td>
</tr>
<tr>
<td>Net losses on derivatives</td>
<td>(187)</td>
<td>(456)</td>
<td>(52)</td>
</tr>
<tr>
<td>Net gains (losses) on foreign currency remeasurements</td>
<td>(244)</td>
<td>181</td>
<td>(75)</td>
</tr>
<tr>
<td>Other, net</td>
<td>(1,319)</td>
<td>(223)</td>
<td>(32)</td>
</tr>
<tr>
<td>Total</td>
<td>$ (1,646)</td>
<td>$ 788</td>
<td>$ 333</td>
</tr>
</table>


Other, net primarily reflects net recognized losses on equity method investments.


## Net Recognized Gains (Losses) on Investments

Net recognized gains (losses) on debt investments were as follows:

(In millions)


<table>
<tr>
<th>Year Ended June 30,</th>
<th>2024</th>
<th>2023</th>
<th>2022</th>
</tr>
<tr>
<td>Realized gains from sales of available-for-sale securities</td>
<td>$ 22</td>
<td>$ 36</td>
<td>$ 162</td>
</tr>
<tr>
<td>Realized losses from sales of available-for-sale securities</td>
<td>(98)</td>
<td>(124)</td>
<td>(138)</td>
</tr>
<tr>
<td>Impairments and allowance for credit losses</td>
<td>23</td>
<td>(10)</td>
<td>(81)</td>
</tr>
<tr>
<td>Total</td>
<td>$ (53)</td>
<td>$ (98)</td>
<td>$ (57)</td>
</tr>
</table>


<!-- PageNumber="68" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->

Net recognized gains (losses) on equity investments were as follows:


### (In millions)


<table>
<tr>
<th>Year Ended June 30,</th>
<th>2024</th>
<th>2023</th>
<th>2022</th>
</tr>
<tr>
<td>Net realized gains on investments sold</td>
<td>$ 18</td>
<td>$ 75</td>
<td>$ 29</td>
</tr>
<tr>
<td>Net unrealized gains on investments still held</td>
<td>146</td>
<td>303</td>
<td>509</td>
</tr>
<tr>
<td>Impairments of investments</td>
<td>(229)</td>
<td>(20)</td>
<td>(20)</td>
</tr>
<tr>
<td>Total</td>
<td>$ (65)</td>
<td>$ 358</td>
<td>$ 518</td>
</tr>
</table>


# NOTE 4 - INVESTMENTS


## Investment Components

The components of investments were as follows:


<table>
<tr>
<th>(In millions)</th>
<th>Fair Value Level</th>
<th>Adjusted Cost Basis</th>
<th>Unrealized Gains</th>
<th>Unrealized Losses</th>
<th>Recorded Basis</th>
<th>Cash and Cash Equivalents</th>
<th>Short-term Investments</th>
<th>Equity and Other Investments</th>
</tr>
<tr>
<td>June 30, 2024</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Changes in Fair Value Recorded in Other Comprehensive Income</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Commercial paper</td>
<td>Level 2</td>
<td>$ 4,666</td>
<td>$ 0</td>
<td>$ 0</td>
<td>$ 4,666</td>
<td>$ 4,666</td>
<td>$ 0</td>
<td>$ 0</td>
</tr>
<tr>
<td>Certificates of deposit</td>
<td>Level 2</td>
<td>1,547</td>
<td>0</td>
<td>0</td>
<td>1,547</td>
<td>1,503</td>
<td>44</td>
<td>0</td>
</tr>
<tr>
<td>U.S. government securities</td>
<td>Level 1</td>
<td>49,603</td>
<td>4</td>
<td>(2,948)</td>
<td>46,659</td>
<td>14</td>
<td>46,645</td>
<td>0</td>
</tr>
<tr>
<td>U.S. agency securities</td>
<td>Level 2</td>
<td>17</td>
<td>0</td>
<td>0</td>
<td>17</td>
<td>0</td>
<td>17</td>
<td>0</td>
</tr>
<tr>
<td>Foreign government bonds</td>
<td>Level 2</td>
<td>319</td>
<td>3</td>
<td>(16)</td>
<td>306</td>
<td>0</td>
<td>306</td>
<td>0</td>
</tr>
<tr>
<td>Mortgage- and asset-backed securities</td>
<td>Level 2</td>
<td>944</td>
<td>3</td>
<td>(35)</td>
<td>912</td>
<td>0</td>
<td>912</td>
<td>0</td>
</tr>
<tr>
<td>Corporate notes and bonds</td>
<td>Level 2</td>
<td>9,106</td>
<td>28</td>
<td>(318)</td>
<td>8,816</td>
<td>0</td>
<td>8,816</td>
<td>0</td>
</tr>
<tr>
<td>Corporate notes and bonds</td>
<td>Level 3</td>
<td>1,641</td>
<td>0</td>
<td>(1)</td>
<td>1,640</td>
<td>0</td>
<td>140</td>
<td>1,500</td>
</tr>
<tr>
<td>Municipal securities</td>
<td>Level 2</td>
<td>262</td>
<td>0</td>
<td>(13)</td>
<td>249</td>
<td>0</td>
<td>249</td>
<td>0</td>
</tr>
<tr>
<td>Municipal securities</td>
<td>Level 3</td>
<td>104</td>
<td>0</td>
<td>(17)</td>
<td>87</td>
<td>0</td>
<td>87</td>
<td>0</td>
</tr>
<tr>
<td>Total debt investments</td>
<td></td>
<td>$ 68,209</td>
<td>$ 38</td>
<td>$ (3,348)</td>
<td>$ 64,899</td>
<td>$ 6,183</td>
<td>$ 57,216</td>
<td>$ 1,500</td>
</tr>
<tr>
<td>Changes in Fair Value Recorded in Net Income</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Equity investments</td>
<td>Level 1</td>
<td></td>
<td></td>
<td></td>
<td>$ 3,547</td>
<td>$ 561</td>
<td>$ 0</td>
<td>$ 2,986</td>
</tr>
<tr>
<td>Equity investments</td>
<td>Other</td>
<td></td>
<td></td>
<td></td>
<td>10,114</td>
<td>0</td>
<td>0</td>
<td>10,114</td>
</tr>
<tr>
<td>Total equity investments</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td>$ 13,661</td>
<td>$ 561</td>
<td>$ 0</td>
<td>$ 13,100</td>
</tr>
<tr>
<td>Cash</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td>$ 11,571</td>
<td>$ 11,571</td>
<td>$ 0</td>
<td>$ 0</td>
</tr>
<tr>
<td>Derivatives, net (a)</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td>12</td>
<td>0</td>
<td>12</td>
<td>0</td>
</tr>
<tr>
<td>Total</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td>$ 90,143</td>
<td>$ 18,315</td>
<td>$ 57,228</td>
<td>$ 14,600</td>
</tr>
</table>


<!-- PageNumber="69" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->


<table>
<tr>
<th>(In millions)</th>
<th>Fair Value Level</th>
<th>Adjusted Cost Basis</th>
<th>Unrealized Gains</th>
<th>Unrealized Losses</th>
<th>Recorded Basis</th>
<th>Cash and Cash Equivalents</th>
<th>Short-term Investments</th>
<th>Equity and Other Investments</th>
</tr>
<tr>
<td>June 30, 2023</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Changes in Fair Value Recorded in Other Comprehensive Income</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Commercial paper</td>
<td>Level 2</td>
<td>$ 16,589</td>
<td>$ 0</td>
<td>$ 0</td>
<td>$ 16,589</td>
<td>$ 12,231</td>
<td>$ 4,358</td>
<td>$ 0</td>
</tr>
<tr>
<td>Certificates of deposit</td>
<td>Level 2</td>
<td>2,701</td>
<td>0</td>
<td>0</td>
<td>2,701</td>
<td>2,657</td>
<td>44</td>
<td>0</td>
</tr>
<tr>
<td>U.S. government securities</td>
<td>Level 1</td>
<td>65,237</td>
<td>2</td>
<td>(3,870)</td>
<td>61,369</td>
<td>2,991</td>
<td>58,378</td>
<td>0</td>
</tr>
<tr>
<td>U.S. agency securities</td>
<td>Level 2</td>
<td>2,703</td>
<td>0</td>
<td>0</td>
<td>2,703</td>
<td>894</td>
<td>1,809</td>
<td>0</td>
</tr>
<tr>
<td>Foreign government bonds</td>
<td>Level 2</td>
<td>498</td>
<td>1</td>
<td>(24)</td>
<td>475</td>
<td>0</td>
<td>475</td>
<td>0</td>
</tr>
<tr>
<td>Mortgage- and asset-backed securities</td>
<td>Level 2</td>
<td>824</td>
<td>1</td>
<td>(39)</td>
<td>786</td>
<td>0</td>
<td>786</td>
<td>0</td>
</tr>
<tr>
<td>Corporate notes and bonds</td>
<td>Level 2</td>
<td>10,809</td>
<td>8</td>
<td>(583)</td>
<td>10,234</td>
<td>0</td>
<td>10,234</td>
<td>0</td>
</tr>
<tr>
<td>Corporate notes and bonds</td>
<td>Level 3</td>
<td>120</td>
<td>0</td>
<td>0</td>
<td>120</td>
<td>0</td>
<td>120</td>
<td>0</td>
</tr>
<tr>
<td>Municipal securities</td>
<td>Level 2</td>
<td>285</td>
<td>1</td>
<td>(18)</td>
<td>268</td>
<td>7</td>
<td>261</td>
<td>0</td>
</tr>
<tr>
<td>Municipal securities</td>
<td>Level 3</td>
<td>103</td>
<td>0</td>
<td>(16)</td>
<td>87</td>
<td>0</td>
<td>87</td>
<td>0</td>
</tr>
<tr>
<td>Total debt investments</td>
<td></td>
<td>$ 99,869</td>
<td>$ 13</td>
<td>$ (4,550)</td>
<td>$ 95,332</td>
<td>$ 18,780</td>
<td>$ 76,552</td>
<td>$ 0</td>
</tr>
<tr>
<td>Changes in Fair Value Recorded in Net Income</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Equity investments</td>
<td>Level 1</td>
<td></td>
<td></td>
<td></td>
<td>$ 10,138</td>
<td>$ 7,446</td>
<td>$ 0</td>
<td>$ 2,692</td>
</tr>
<tr>
<td>Equity investments</td>
<td>Other</td>
<td></td>
<td></td>
<td></td>
<td>7,187</td>
<td>0</td>
<td>0</td>
<td>7,187</td>
</tr>
<tr>
<td>Total equity investments</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td>$ 17,325</td>
<td>$ 7,446</td>
<td>$ 0</td>
<td>$ 9,879</td>
</tr>
<tr>
<td>Cash</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td>$ 8,478</td>
<td>$ 8,478</td>
<td>$ 0</td>
<td>$ 0</td>
</tr>
<tr>
<td>Derivatives, net (a)</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td>6</td>
<td>0</td>
<td>6</td>
<td>0</td>
</tr>
<tr>
<td>Total</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td>$ 21,141</td>
<td>$ 34,704</td>
<td>$ 76,558</td>
<td>$ 9,879</td>
</tr>
</table>

(a) Refer to Note 5 - Derivatives for further information on the fair value of our derivative instruments.


Equity investments presented as "Other" in the tables above include investments without readily determinable fair
values measured using the equity method or measured at cost with adjustments for observable changes in price or
impairments, and investments measured at fair value using net asset value as a practical expedient which are not
categorized in the fair value hierarchy. As of June 30, 2024 and 2023, equity investments without readily determinable
fair values measured at cost with adjustments for observable changes in price or impairments were $3.9 billion and
$4.2 billion, respectively.


## Unrealized Losses on Debt Investments

Debt investments with continuous unrealized losses for less than 12 months and 12 months or greater and their related
fair values were as follows:


<table>
<tr>
<th rowspan="2">(In millions)</th>
<th colspan="2">Less than 12 Months</th>
<th colspan="2">12 Months or Greater</th>
<th rowspan="2">Total Fair Value</th>
<th rowspan="2">Total Unrealized Losses</th>
</tr>
<tr>
<th>Fair Value</th>
<th>Unrealized Losses</th>
<th>Fair Value</th>
<th>Unrealized Losses</th>
</tr>
<tr>
<td>June 30, 2024</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>U.S. government and agency securities</td>
<td>$ 529</td>
<td>$ (12)</td>
<td>$ 45,821</td>
<td>$ (2,936)</td>
<td>$ 46,350</td>
<td>$ (2,948)</td>
</tr>
<tr>
<td>Foreign government bonds</td>
<td>79</td>
<td>(2)</td>
<td>180</td>
<td>(14)</td>
<td>259</td>
<td>(16)</td>
</tr>
<tr>
<td>Mortgage- and asset-backed securities</td>
<td>201</td>
<td>(1)</td>
<td>409</td>
<td>(34)</td>
<td>610</td>
<td>(35)</td>
</tr>
<tr>
<td>Corporate notes and bonds</td>
<td>1,310</td>
<td>(9)</td>
<td>5,779</td>
<td>(310)</td>
<td>7,089</td>
<td>(319)</td>
</tr>
<tr>
<td>Municipal securities</td>
<td>38</td>
<td>(1)</td>
<td>243</td>
<td>(29)</td>
<td>281</td>
<td>(30)</td>
</tr>
<tr>
<td>Total</td>
<td>$ 2,157</td>
<td>$ (25)</td>
<td>$ 52,432</td>
<td>$ (3,323)</td>
<td>$ 54,589</td>
<td>$ (3,348)</td>
</tr>
</table>


<!-- PageNumber="70" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->


<table>
<tr>
<th rowspan="2">(In millions)</th>
<th colspan="2">Less than 12 Months</th>
<th colspan="2">12 Months or Greater</th>
<th rowspan="2">Total Fair Value</th>
<th rowspan="2">Total Unrealized Losses</th>
</tr>
<tr>
<th>Fair Value</th>
<th>Unrealized Losses</th>
<th>Fair Value</th>
<th>Unrealized Losses</th>
</tr>
<tr>
<td>June 30, 2023</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>U.S. government and agency securities</td>
<td>$ 7,950</td>
<td>$ (336)</td>
<td>$ 45,273</td>
<td>$ (3,534)</td>
<td>$ 53,223</td>
<td>$ (3,870)</td>
</tr>
<tr>
<td>Foreign government bonds</td>
<td>77</td>
<td>(5)</td>
<td>391</td>
<td>(19)</td>
<td>468</td>
<td>(24)</td>
</tr>
<tr>
<td>Mortgage- and asset-backed securities</td>
<td>257</td>
<td>(5)</td>
<td>412</td>
<td>(34)</td>
<td>669</td>
<td>(39)</td>
</tr>
<tr>
<td>Corporate notes and bonds</td>
<td>2,326</td>
<td>(49)</td>
<td>7,336</td>
<td>(534)</td>
<td>9,662</td>
<td>(583)</td>
</tr>
<tr>
<td>Municipal securities</td>
<td>111</td>
<td>(3)</td>
<td>186</td>
<td>(31)</td>
<td>297</td>
<td>(34)</td>
</tr>
<tr>
<td>Total</td>
<td>$ 10,721</td>
<td>$ (398)</td>
<td>$ 53,598</td>
<td>$ (4,152)</td>
<td>$ 64,319</td>
<td>$ (4,550)</td>
</tr>
</table>


Unrealized losses from fixed-income securities are primarily attributable to changes in interest rates. Management
does not believe any remaining unrealized losses represent impairments based on our evaluation of available
evidence.


## Debt Investment Maturities

The following table outlines maturities of our debt investments as of June 30, 2024:


<table>
<tr>
<th>(In millions)</th>
<th>Adjusted Cost Basis</th>
<th>Estimated Fair Value</th>
</tr>
<tr>
<td>June 30, 2024</td>
<td></td>
<td></td>
</tr>
<tr>
<td>Due in one year or less</td>
<td>$ 19,815</td>
<td>$ 19,596</td>
</tr>
<tr>
<td>Due after one year through five years</td>
<td>38,954</td>
<td>36,779</td>
</tr>
<tr>
<td>Due after five years through 10 years</td>
<td>8,028</td>
<td>7,242</td>
</tr>
<tr>
<td>Due after 10 years</td>
<td>1,412</td>
<td>1,282</td>
</tr>
<tr>
<td>Total</td>
<td>$ 68,209</td>
<td>$ 64,899</td>
</tr>
</table>


# NOTE 5 - DERIVATIVES

We use derivative instruments to manage risks related to foreign currencies, interest rates, equity prices, and credit;
to enhance investment returns; and to facilitate portfolio diversification. Our objectives for holding derivatives include
reducing, eliminating, and efficiently managing the economic impact of these exposures as effectively as possible. Our
derivative programs include strategies that both qualify and do not qualify for hedge accounting treatment.


## Foreign Currencies

Certain forecasted transactions, assets, and liabilities are exposed to foreign currency risk. We monitor our foreign
currency exposures daily to maximize the economic effectiveness of our foreign currency hedge positions.

Foreign currency risks related to certain non-U.S. dollar-denominated investments are hedged using foreign exchange
forward contracts that are designated as fair value hedging instruments. Foreign currency risks related to certain Euro-
denominated debt are hedged using foreign exchange forward contracts that are designated as cash flow hedging
instruments.

Certain options and forwards not designated as hedging instruments are also used to manage the variability in foreign
exchange rates on certain balance sheet amounts and to manage other foreign currency exposures.


## Interest Rate

Interest rate risks related to certain fixed-rate debt are hedged using interest rate swaps that are designated as fair
value hedging instruments to effectively convert the fixed interest rates to floating interest rates.

<!-- PageNumber="71" -->
<!-- PageBreak -->


# PART II Item 8

Securities held in our fixed-income portfolio are subject to different interest rate risks based on their maturities. We
manage the average maturity of our fixed-income portfolio to achieve economic returns that correlate to certain broad-
based fixed-income indices using option, futures, and swap contracts. These contracts are not designated as hedging
instruments and are included in "Other contracts" in the tables below.


# Equity

Securities held in our equity investments portfolio are subject to market price risk. At times, we may hold options,
futures, and swap contracts. These contracts are not designated as hedging instruments.


# Credit

Our fixed-income portfolio is diversified and consists primarily of investment-grade securities. We use credit default
swap contracts to manage credit exposures relative to broad-based indices and to facilitate portfolio diversification.
These contracts are not designated as hedging instruments and are included in "Other contracts" in the tables below.


# Credit-Risk-Related Contingent Features

Certain counterparty agreements for derivative instruments contain provisions that require our issued and outstanding
long-term unsecured debt to maintain an investment grade credit rating and require us to maintain minimum liquidity
of $1.0 billion. To the extent we fail to meet these requirements, we will be required to post collateral, similar to the
standard convention related to over-the-counter derivatives. As of June 30, 2024, our long-term unsecured debt rating
was AAA, and cash investments were in excess of $1.0 billion. As a result, no collateral was required to be posted.

The following table presents the notional amounts of our outstanding derivative instruments measured in U.S. dollar
equivalents:


<table>
<tr>
<th>(In millions)</th>
<th>June 30, 2024</th>
<th>June 30, 2023</th>
</tr>
<tr>
<td>Designated as Hedging Instruments</td>
<td></td>
<td></td>
</tr>
<tr>
<td>Foreign exchange contracts purchased</td>
<td>$ 1,492</td>
<td>$ 1,492</td>
</tr>
<tr>
<td>Interest rate contracts purchased</td>
<td>1,100</td>
<td>1,078</td>
</tr>
<tr>
<td>Not Designated as Hedging Instruments</td>
<td></td>
<td></td>
</tr>
<tr>
<td>Foreign exchange contracts purchased</td>
<td>7,167</td>
<td>7,874</td>
</tr>
<tr>
<td>Foreign exchange contracts sold</td>
<td>31,793</td>
<td>25,159</td>
</tr>
<tr>
<td>Equity contracts purchased</td>
<td>4,016</td>
<td>3,867</td>
</tr>
<tr>
<td>Equity contracts sold</td>
<td>2,165</td>
<td>2,154</td>
</tr>
<tr>
<td>Other contracts purchased</td>
<td>2,113</td>
<td>1,224</td>
</tr>
<tr>
<td>Other contracts sold</td>
<td>811</td>
<td>581</td>
</tr>
</table>


<!-- PageNumber="72" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->


# Fair Values of Derivative Instruments

The following table presents our derivative instruments:


<table>
<tr>
<th>(In millions)</th>
<th>Derivative Assets</th>
<th>Derivative Liabilities</th>
<th>Derivative Assets</th>
<th>Derivative Liabilities</th>
</tr>
<tr>
<th></th>
<th></th>
<th>June 30, 2024</th>
<th></th>
<th>June 30, 2023</th>
</tr>
<tr>
<td>Designated as Hedging Instruments</td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Foreign exchange contracts</td>
<td>$ 24</td>
<td>$ (76)</td>
<td>$ $\begin{array}{} 3 4 \\ 1 6 \end{array}$</td>
<td>$ $\left( 6 7 \right)$</td>
</tr>
<tr>
<td>Interest rate contracts</td>
<td>19</td>
<td>0</td>
<td></td>
<td>0</td>
</tr>
<tr>
<td>Not Designated as Hedging Instruments</td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Foreign exchange contracts</td>
<td>213</td>
<td>(230)</td>
<td>249</td>
<td>(332)</td>
</tr>
<tr>
<td>Equity contracts</td>
<td>63</td>
<td>(491)</td>
<td>165</td>
<td>(400)</td>
</tr>
<tr>
<td>Other contracts</td>
<td>12</td>
<td>(3)</td>
<td>5</td>
<td>(6)</td>
</tr>
<tr>
<td>Gross amounts of derivatives</td>
<td>331</td>
<td>(800)</td>
<td>469</td>
<td>(805)</td>
</tr>
<tr>
<td>Gross amounts of derivatives offset in the balance sheets</td>
<td>(151)</td>
<td>152</td>
<td>(202)</td>
<td>206</td>
</tr>
<tr>
<td>Cash collateral received</td>
<td>0</td>
<td>(104)</td>
<td>0</td>
<td>(125)</td>
</tr>
<tr>
<td>Net amounts of derivatives</td>
<td>$ 180</td>
<td>$ (752)</td>
<td>$ 267</td>
<td>$ (724)</td>
</tr>
<tr>
<td>Reported as</td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Short-term investments</td>
<td>$ 12</td>
<td>$ 0</td>
<td>$ 6</td>
<td>$ 0</td>
</tr>
<tr>
<td>Other current assets</td>
<td>149</td>
<td>0</td>
<td>245</td>
<td>0</td>
</tr>
<tr>
<td>Other long-term assets</td>
<td>19</td>
<td>0</td>
<td>16</td>
<td>0</td>
</tr>
<tr>
<td>Other current liabilities</td>
<td>0</td>
<td>(401)</td>
<td>0</td>
<td>(341)</td>
</tr>
<tr>
<td>Other long-term liabilities</td>
<td>0</td>
<td>(351)</td>
<td>0</td>
<td>(383)</td>
</tr>
<tr>
<td>Total</td>
<td>$ 180</td>
<td>$ (752)</td>
<td>$ 267</td>
<td>$ (724)</td>
</tr>
</table>


Gross derivative assets and liabilities subject to legally enforceable master netting agreements for which we have
elected to offset were $304 million and $800 million, respectively, as of June 30, 2024, and $442 million and $804
million, respectively, as of June 30, 2023.

The following table presents the fair value of our derivatives instruments on a gross basis:


<table>
<tr>
<th>(In millions)</th>
<th>Level 1</th>
<th>Level 2</th>
<th>Level 3</th>
<th>Total</th>
</tr>
<tr>
<td>June 30, 2024</td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Derivative assets</td>
<td>$ 0</td>
<td>$ 327</td>
<td>$ 4</td>
<td>$ 331</td>
</tr>
<tr>
<td>Derivative liabilities</td>
<td>(1)</td>
<td>(799)</td>
<td>0</td>
<td>(800)</td>
</tr>
<tr>
<td>June 30, 2023</td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Derivative assets</td>
<td>0</td>
<td>462</td>
<td>7</td>
<td>469</td>
</tr>
<tr>
<td>Derivative liabilities</td>
<td>0</td>
<td>(805)</td>
<td>0</td>
<td>(805)</td>
</tr>
</table>


<!-- PageNumber="73" -->
<!-- PageBreak -->


# PART II Item 8

Gains (losses) on derivative instruments recognized in other income (expense), net were as follows:


## (In millions)


<table>
<tr>
<th>Year Ended June 30,</th>
<th>2024</th>
<th>2023</th>
<th>2022</th>
</tr>
<tr>
<td>Designated as Fair Value Hedging Instruments</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Foreign exchange contracts</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Derivatives</td>
<td>$ 0</td>
<td>$ 0</td>
<td>$ 49</td>
</tr>
<tr>
<td>Hedged items</td>
<td>0</td>
<td>0</td>
<td>(50)</td>
</tr>
<tr>
<td>Excluded from effectiveness assessment</td>
<td>0</td>
<td>0</td>
<td>4</td>
</tr>
<tr>
<td>Interest rate contracts</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Derivatives</td>
<td>(23)</td>
<td>(65)</td>
<td>(92)</td>
</tr>
<tr>
<td>Hedged items</td>
<td>(25)</td>
<td>38</td>
<td>108</td>
</tr>
<tr>
<td>Designated as Cash Flow Hedging Instruments</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Foreign exchange contracts</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Amount reclassified from accumulated other comprehensive loss</td>
<td>(48)</td>
<td>61</td>
<td>(79)</td>
</tr>
<tr>
<td>Not Designated as Hedging Instruments</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Foreign exchange contracts</td>
<td>367</td>
<td>(73)</td>
<td>383</td>
</tr>
<tr>
<td>Equity contracts</td>
<td>(177)</td>
<td>(420)</td>
<td>13</td>
</tr>
<tr>
<td>Other contracts</td>
<td>(15)</td>
<td>(41)</td>
<td>(85)</td>
</tr>
</table>


Gains (losses), net of tax, on derivative instruments recognized in our consolidated comprehensive income statements
were as follows:

(In millions)


<table>
<tr>
<th>Year Ended June 30,</th>
<th>2024</th>
<th>2023</th>
<th>2022</th>
</tr>
<tr>
<td>Designated as Cash Flow Hedging Instruments</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td rowspan="2">Foreign exchange contracts Included in effectiveness assessment</td>
<td></td>
<td></td>
<td rowspan="2">$ (57)</td>
</tr>
<tr>
<td>$ (14)</td>
<td>$ 34</td>
</tr>
</table>


## NOTE 6 - INVENTORIES

The components of inventories were as follows:


<table>
<tr>
<td>(In millions)</td>
<td></td>
<td></td>
</tr>
<tr>
<td>June 30,</td>
<td>2024</td>
<td>2023</td>
</tr>
<tr>
<td>Raw materials</td>
<td>$ 394</td>
<td>$ 709</td>
</tr>
<tr>
<td>Work in process</td>
<td>7</td>
<td>23</td>
</tr>
<tr>
<td>Finished goods</td>
<td>845</td>
<td>1,768</td>
</tr>
<tr>
<td>Total</td>
<td>$ 1,246</td>
<td>$ 2,500</td>
</tr>
</table>


<!-- PageNumber="74" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->


## NOTE 7 - PROPERTY AND EQUIPMENT

The components of property and equipment were as follows:

(In millions)


<table>
<tr>
<th>June 30,</th>
<th>2024</th>
<th>2023</th>
</tr>
<tr>
<td>Land</td>
<td>$ 8,163</td>
<td>$ 5,683</td>
</tr>
<tr>
<td>Buildings and improvements</td>
<td>93,943</td>
<td>68,465</td>
</tr>
<tr>
<td>Leasehold improvements</td>
<td>9,594</td>
<td>8,537</td>
</tr>
<tr>
<td>Computer equipment and software</td>
<td>93,780</td>
<td>74,961</td>
</tr>
<tr>
<td>Furniture and equipment</td>
<td>6,532</td>
<td>6,246</td>
</tr>
<tr>
<td>Total, at cost</td>
<td>212,012</td>
<td>163,892</td>
</tr>
<tr>
<td>Accumulated depreciation</td>
<td>(76,421)</td>
<td>(68,251)</td>
</tr>
<tr>
<td>Total, net</td>
<td>$135,591</td>
<td>$ 95,641</td>
</tr>
</table>


During fiscal years 2024, 2023, and 2022, depreciation expense was $15.2 billion, $11.0 billion, and $12.6 billion,
respectively.

As of June 30, 2024, we have committed $35.4 billion for the construction of new buildings, building improvements,
and leasehold improvements, primarily related to datacenters.


### NOTE 8 - BUSINESS COMBINATIONS


#### Activision Blizzard, Inc.

On October 13, 2023, we completed our acquisition of Activision Blizzard, Inc. ("Activision Blizzard") for a total purchase
price of $75.4 billion, consisting primarily of cash. Activision Blizzard is a leader in game development and an interactive
entertainment content publisher. The acquisition will accelerate the growth in our gaming business across mobile, PC,
console, and cloud gaming. The financial results of Activision Blizzard have been included in our consolidated financial
statements since the date of the acquisition. Activision Blizzard is reported as part of our More Personal Computing
segment.

The purchase price allocation as of the date of acquisition was based on a preliminary valuation and is subject to
revision as more detailed analyses are completed and additional information about the fair value of assets acquired
and liabilities assumed becomes available. The primary areas that remain preliminary relate to the fair values of
goodwill and income taxes.

The major classes of assets and liabilities to which we have preliminarily allocated the purchase price were as follows:


<table>
<tr>
<th>(In millions)</th>
<th></th>
</tr>
<tr>
<td>Cash and cash equivalents</td>
<td>$ 12,976</td>
</tr>
<tr>
<td>Goodwill</td>
<td>50,969</td>
</tr>
<tr>
<td>Intangible assets</td>
<td>21,969</td>
</tr>
<tr>
<td>Other assets</td>
<td>2,501</td>
</tr>
<tr>
<td>Long-term debt</td>
<td>(2,799)</td>
</tr>
<tr>
<td>Long-term income taxes</td>
<td>(1,914)</td>
</tr>
<tr>
<td>Deferred income taxes</td>
<td>(4,677)</td>
</tr>
<tr>
<td>Other liabilities</td>
<td>(3,617)</td>
</tr>
<tr>
<td>Total purchase price</td>
<td>$ 75,408</td>
</tr>
</table>


<!-- PageNumber="75" -->
<!-- PageBreak -->


# PART II Item 8

Goodwill was assigned to our More Personal Computing segment. The goodwill was primarily attributed to increased
synergies that are expected to be achieved from the integration of Activision Blizzard. Substantially all of the goodwill
is expected to be non-deductible for income tax purposes.

Following are the details of the purchase price allocated to the intangible assets acquired:


<table>
<tr>
<th>(In millions, except average life)</th>
<th>Amount</th>
<th>Weighted Average Life</th>
</tr>
<tr>
<td>Marketing-related</td>
<td>$ 11,619</td>
<td>24 years</td>
</tr>
<tr>
<td>Technology-based</td>
<td>9,689</td>
<td>4 years</td>
</tr>
<tr>
<td>Customer-related</td>
<td>661</td>
<td>4 years</td>
</tr>
<tr>
<td>Fair value of intangible assets acquired</td>
<td>$ 21,969</td>
<td>15 years</td>
</tr>
</table>


Following is the net impact of the Activision Blizzard acquisition on our consolidated income statements since the date
of acquisition:


<table>
<tr>
<th>(In millions)</th>
<th></th>
</tr>
<tr>
<td>Year Ended June 30,</td>
<td>2024</td>
</tr>
<tr>
<td>Revenue</td>
<td>$ 5,729</td>
</tr>
<tr>
<td>Operating loss</td>
<td>(1,362)</td>
</tr>
</table>


The change of Activision Blizzard content from third-party to first-party is reflected in the net impact.

Following are the supplemental consolidated financial results of Microsoft Corporation on an unaudited pro forma basis,
as if the acquisition had been consummated on July 1, 2022:


## (In millions, except per share amounts)


<table>
<tr>
<th>Year Ended June 30,</th>
<th>2024 2023</th>
</tr>
<tr>
<td>Revenue</td>
<td>$ 247,442 $ 219,790</td>
</tr>
<tr>
<td>Net income</td>
<td>88,308 71,383</td>
</tr>
<tr>
<td>Diluted earnings per share</td>
<td>11.82 9.55</td>
</tr>
</table>


These pro forma results were based on estimates and assumptions, which we believe are reasonable. They are not
the results that would have been realized had we been a combined company during the periods presented and are not
necessarily indicative of our consolidated results of operations in future periods. The pro forma results include
adjustments related to purchase accounting, primarily amortization of intangible assets. Acquisition costs and other
nonrecurring charges were immaterial and are included in the earliest period presented.


### Nuance Communications, Inc.

On March 4, 2022, we completed our acquisition of Nuance Communications, Inc. ("Nuance") for a total purchase price
of $18.8 billion, consisting primarily of cash. Nuance is a cloud and artificial intelligence ("Al") software provider with
healthcare and enterprise Al experience, and the acquisition will build on our industry-specific cloud offerings. The
financial results of Nuance have been included in our consolidated financial statements since the date of the
acquisition. Nuance is reported as part of our Intelligent Cloud segment.

<!-- PageNumber="76" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->

The allocation of the purchase price to goodwill was completed as of December 31, 2022. The major classes of assets
and liabilities to which we have allocated the purchase price were as follows:


<table>
<tr>
<th>(In millions)</th>
<th></th>
</tr>
<tr>
<td>Goodwill (a)</td>
<td>$ 16,326</td>
</tr>
<tr>
<td>Intangible assets</td>
<td>4,365</td>
</tr>
<tr>
<td>Other assets</td>
<td>42</td>
</tr>
<tr>
<td>Other liabilities (b)</td>
<td>(1,972)</td>
</tr>
<tr>
<td>Total</td>
<td>$ 18,761</td>
</tr>
</table>

(a) Goodwill was assigned to our Intelligent Cloud segment and was primarily attributed to increased synergies that
are expected to be achieved from the integration of Nuance. None of the goodwill is expected to be deductible
for income tax purposes.

(b)
Includes $986 million of convertible senior notes issued by Nuance in 2015 and 2017, substantially all of which
have been redeemed.


Following are the details of the purchase price allocated to the intangible assets acquired:


<table>
<tr>
<th>(In millions, except average life)</th>
<th>Amount</th>
<th>Weighted Average Life</th>
</tr>
<tr>
<td>Customer-related</td>
<td>$ 2,610</td>
<td>9 years</td>
</tr>
<tr>
<td>Technology-based</td>
<td>1,540</td>
<td>5 years</td>
</tr>
<tr>
<td>Marketing-related</td>
<td>215</td>
<td>4 years</td>
</tr>
<tr>
<td>Total</td>
<td>$ 4,365</td>
<td>7 years</td>
</tr>
</table>


## NOTE 9 - GOODWILL

Changes in the carrying amount of goodwill were as follows:


<table>
<tr>
<th>(In millions)</th>
<th>June 30, 2022</th>
<th>Acquisitions</th>
<th>Other</th>
<th>June 30, 2023</th>
<th>Acquisitions</th>
<th>Other</th>
<th>June 30, 2024</th>
</tr>
<tr>
<td>Productivity and Business Processes</td>
<td>$ 24,811</td>
<td>$ 11</td>
<td>$ (47)</td>
<td>$ 24,775</td>
<td>$ 0</td>
<td>$ 2</td>
<td>$ 24,777</td>
</tr>
<tr>
<td>Intelligent Cloud</td>
<td>30,182</td>
<td>223</td>
<td>64</td>
<td>30,469</td>
<td>0</td>
<td>(28)</td>
<td>30,441</td>
</tr>
<tr>
<td>More Personal Computing</td>
<td>12,531</td>
<td>0</td>
<td>111</td>
<td>12,642</td>
<td>51,235 (a)</td>
<td>125(a)</td>
<td>64,002</td>
</tr>
<tr>
<td>Total</td>
<td>$ 67,524</td>
<td>$ 234</td>
<td>$ 128</td>
<td>$ 67,886</td>
<td>$ 51,235</td>
<td>$ 99</td>
<td>$ 119,220</td>
</tr>
</table>

(a) Includes goodwill of $51.0 billion related to Activision Blizzard. See Note 8 - Business Combinations for further
information.


The measurement periods for the valuation of assets acquired and liabilities assumed end as soon as information on
the facts and circumstances that existed as of the acquisition dates becomes available, but do not exceed 12 months.
Adjustments in purchase price allocations may require a change in the amounts allocated to goodwill during the periods
in which the adjustments are determined.

Any change in the goodwill amounts resulting from foreign currency translations and purchase accounting adjustments
are presented as "Other" in the table above. Also included in "Other" are business dispositions and transfers between
segments due to reorganizations, as applicable.

<!-- PageNumber="77" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->


### Goodwill Impairment

We test goodwill for impairment annually on May 1 at the reporting unit level, primarily using a discounted cash flow
methodology with a peer-based, risk-adjusted weighted average cost of capital. We believe use of a discounted cash
flow approach is the most reliable indicator of the fair values of the businesses.

No instances of impairment were identified in our May 1, 2024, May 1, 2023, or May 1, 2022 tests. As of June 30, 2024
and 2023, accumulated goodwill impairment was $11.3 billion.


# NOTE 10 - INTANGIBLE ASSETS

The components of intangible assets, all of which are finite-lived, were as follows:


<table>
<tr>
<th>(In millions)</th>
<th>Gross Carrying Amount</th>
<th>Accumulated Amortization</th>
<th>Net Carrying Amount</th>
<th>Gross Carrying Amount</th>
<th>Accumulated Amortization</th>
<th>Net Carrying Amount</th>
</tr>
<tr>
<td>June 30,</td>
<td></td>
<td></td>
<td>2024</td>
<td></td>
<td></td>
<td>2023</td>
</tr>
<tr>
<td>Marketing-related</td>
<td>$ 16,500</td>
<td>$ (3,101)</td>
<td>$ 13,399</td>
<td>$ 4,935</td>
<td>$ (2,473)</td>
<td>$ 2,462</td>
</tr>
<tr>
<td>Technology-based</td>
<td>21,913</td>
<td>(10,741)</td>
<td>11,172</td>
<td>11,245</td>
<td>(7,589)</td>
<td>3,656</td>
</tr>
<tr>
<td>Customer-related</td>
<td>6,038</td>
<td>(3,051)</td>
<td>2,987</td>
<td>7,281</td>
<td>(4,047)</td>
<td>3,234</td>
</tr>
<tr>
<td>Contract-based</td>
<td>58</td>
<td>(19)</td>
<td>39</td>
<td>29</td>
<td>(15)</td>
<td>14</td>
</tr>
<tr>
<td>Total</td>
<td>$ 44,509(a a)</td>
<td>$ (16,912)</td>
<td>$ 27,597</td>
<td>$ 23,490</td>
<td>$ (14,124)</td>
<td>$ 9,366</td>
</tr>
</table>

(a) Includes intangible assets of $22.0 billion related to Activision Blizzard. See Note 8 - Business Combinations for
further information.


No material impairments of intangible assets were identified during fiscal years 2024, 2023, or 2022. We estimate that
we have no significant residual value related to our intangible assets.

The components of intangible assets acquired during the periods presented were as follows:


<table>
<tr>
<th>(In millions)</th>
<th>Amount</th>
<th>Weighted Average Life</th>
<th>Amount</th>
<th>Weighted Average Life</th>
</tr>
<tr>
<td>Year Ended June 30,</td>
<td>2024</td>
<td></td>
<td>2023</td>
<td></td>
</tr>
<tr>
<td>Marketing-related</td>
<td>$11,619</td>
<td>24 years</td>
<td>$ 7</td>
<td>5 years</td>
</tr>
<tr>
<td>Technology-based</td>
<td>10,947</td>
<td>4 years</td>
<td>522</td>
<td>7 years</td>
</tr>
<tr>
<td>Customer-related</td>
<td>660</td>
<td>4 years</td>
<td>0</td>
<td>0 years</td>
</tr>
<tr>
<td>Contract-based</td>
<td>38</td>
<td>4 years</td>
<td>12</td>
<td>3 years</td>
</tr>
<tr>
<td>Total</td>
<td>$23,264</td>
<td>14 years</td>
<td>$ 541</td>
<td>6 years</td>
</tr>
</table>


Intangible assets amortization expense was $4.8 billion, $2.5 billion, and $2.0 billion for fiscal years 2024, 2023, and
2022, respectively.

<!-- PageNumber="78" -->
<!-- PageBreak -->


# PART II Item 8

The following table outlines the estimated future amortization expense related to intangible assets held as of June 30,
2024:

(In millions)


<table>
<tr>
<td>Year Ending June 30,</td>
<td></td>
</tr>
<tr>
<td>2025</td>
<td>$ 5,892</td>
</tr>
<tr>
<td>2026</td>
<td>4,471</td>
</tr>
<tr>
<td>2027</td>
<td>2,793</td>
</tr>
<tr>
<td>2028</td>
<td>1,909</td>
</tr>
<tr>
<td>2029</td>
<td>1,728</td>
</tr>
<tr>
<td>Thereafter</td>
<td>10,804</td>
</tr>
<tr>
<td>Total</td>
<td>$ 27,597</td>
</tr>
</table>


## NOTE 11 - DEBT


### Short-term Debt

As of June 30, 2024, we had $6.7 billion of commercial paper issued and outstanding, with a weighted average interest
rate of 5.4% and maturities ranging from 28 days to 152 days. The estimated fair value of this commercial paper
approximates its carrying value. As of June 30, 2023, we had no commercial paper issued or outstanding.


### Long-term Debt

The components of long-term debt were as follows:


<table>
<tr>
<th>(In millions, issuance by calendar year)</th>
<th>Maturities (calendar year)</th>
<th>Stated Interest Rate</th>
<th>Effective Interest Rate</th>
<th>June 30, 2024</th>
<th>June 30, 2023</th>
</tr>
<tr>
<td>2009 issuance of $3.8 billion</td>
<td>2039</td>
<td>5.20%</td>
<td>5.24%</td>
<td>$ 520</td>
<td>$ 520</td>
</tr>
<tr>
<td>2010 issuance of $4.8 billion</td>
<td>2040</td>
<td>4.50%</td>
<td>4.57%</td>
<td>486</td>
<td>486</td>
</tr>
<tr>
<td>2011 issuance of $2.3 billion</td>
<td>2041</td>
<td>5.30%</td>
<td>5.36%</td>
<td>718</td>
<td>718</td>
</tr>
<tr>
<td>2012 issuance of $2.3 billion</td>
<td>2042</td>
<td>3.50%</td>
<td>3.57%</td>
<td>454</td>
<td>454</td>
</tr>
<tr>
<td>2013 issuance of $5.2 billion</td>
<td>2043</td>
<td>3.75%-4.88%</td>
<td>3.83%-4.92%</td>
<td>314</td>
<td>1,814</td>
</tr>
<tr>
<td>2013 issuance of €4.1 billion</td>
<td>2028-2033</td>
<td>2.63%-3.13%</td>
<td>2.69%-3.22%</td>
<td>2,465</td>
<td>2,509</td>
</tr>
<tr>
<td>2015 issuance of $23.8 billion</td>
<td>2025-2055</td>
<td>2.70%-4.75%</td>
<td>2.77%-4.78%</td>
<td>9,805</td>
<td>9,805</td>
</tr>
<tr>
<td>2016 issuance of $19.8 billion</td>
<td>2026-2056</td>
<td>2.40%-3.95%</td>
<td>2.46%-4.03%</td>
<td>7,930</td>
<td>9,430</td>
</tr>
<tr>
<td>2017 issuance of $17.1 billion (a)</td>
<td>2026-2057</td>
<td>3.30%-4.50%</td>
<td>3.38%-5.49%</td>
<td>6,833</td>
<td>8,945</td>
</tr>
<tr>
<td>2020 issuance of $10.1 billion (a)</td>
<td>2030-2060</td>
<td>$1 . 3 5 - 2 . 6 8$</td>
<td>2.53%-5.43%</td>
<td>10,111</td>
<td>10,000</td>
</tr>
<tr>
<td>2021 issuance of $8.2 billion</td>
<td>2052-2062</td>
<td>2.92%-3.04%</td>
<td>2.92%-3.04%</td>
<td>8,185</td>
<td>8,185</td>
</tr>
<tr>
<td>2023 issuance of $0.1 billion (a)</td>
<td>2026-2050</td>
<td>$1 . 3 5 - 4 . 5 0$</td>
<td>5.16%-5.49%</td>
<td>56</td>
<td>0</td>
</tr>
<tr>
<td>2024 issuance of $3.3 billion (a)</td>
<td>2026-2050</td>
<td>$1 . 3 5 - 4 . 5 0$</td>
<td>5.16%-5.49%</td>
<td>3,344</td>
<td>0</td>
</tr>
<tr>
<td>Total face value</td>
<td></td>
<td></td>
<td></td>
<td>51,221</td>
<td>52,866</td>
</tr>
<tr>
<td>Unamortized discount and issuance costs</td>
<td></td>
<td></td>
<td></td>
<td>(1,227)</td>
<td>(438)</td>
</tr>
<tr>
<td>Hedge fair value adjustments (b)</td>
<td></td>
<td></td>
<td></td>
<td>(81)</td>
<td>(106)</td>
</tr>
<tr>
<td>Premium on debt exchange</td>
<td></td>
<td></td>
<td></td>
<td>(4,976)</td>
<td>(5,085)</td>
</tr>
<tr>
<td>Total debt</td>
<td></td>
<td></td>
<td></td>
<td>44,937</td>
<td>47,237</td>
</tr>
<tr>
<td>Current portion of long-term debt</td>
<td></td>
<td></td>
<td></td>
<td>(2,249)</td>
<td>(5,247)</td>
</tr>
<tr>
<td>Long-term debt</td>
<td></td>
<td></td>
<td></td>
<td>$ 42,688</td>
<td>$ 41,990</td>
</tr>
</table>

(a) Includes $3.6 billion of debt at face value related to the Activision Blizzard acquisition, the majority of which was
exchanged for Microsoft registered securities in June 2024. See Note 8 - Business Combinations for further
information.

(b) Refer to Note 5 - Derivatives for further information on the interest rate swaps related to fixed-rate debt.


<!-- PageNumber="79" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->

As of June 30, 2024 and 2023, the estimated fair value of long-term debt, including the current portion, was $42.3
billion and $46.2 billion, respectively. The estimated fair values are based on Level 2 inputs.

Debt in the table above is comprised of senior unsecured obligations and ranks equally with our other outstanding
obligations. Interest is paid semi-annually, except for the Euro-denominated debt, which is paid annually. Cash paid
for interest on our debt for fiscal years 2024, 2023, and 2022 was $1.7 billion, $1.7 billion, and $1.9 billion, respectively.

The following table outlines maturities of our long-term debt, including the current portion, as of June 30, 2024:


<table>
<tr>
<th>(In millions)</th>
<th></th>
</tr>
<tr>
<td>Year Ending June 30,</td>
<td></td>
</tr>
<tr>
<td>2025</td>
<td>$ 2,250</td>
</tr>
<tr>
<td>2026</td>
<td>3,000</td>
</tr>
<tr>
<td>2027</td>
<td>9,250</td>
</tr>
<tr>
<td>2028</td>
<td>0</td>
</tr>
<tr>
<td>2029</td>
<td>1,876</td>
</tr>
<tr>
<td>Thereafter</td>
<td>34,845</td>
</tr>
<tr>
<td>Total</td>
<td>$ 51,221</td>
</tr>
</table>


## NOTE 12 - INCOME TAXES


### Provision for Income Taxes

The components of the provision for income taxes were as follows:

(In millions)


<table>
<tr>
<th>Year Ended June 30,</th>
<th>2024</th>
<th>2023</th>
<th>2022</th>
</tr>
<tr>
<td>Current Taxes</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>U.S. federal</td>
<td>$ 12,165</td>
<td>$ 14,009</td>
<td>$ 8,329</td>
</tr>
<tr>
<td>U.S. state and local</td>
<td>2,366</td>
<td>2,322</td>
<td>1,679</td>
</tr>
<tr>
<td>Foreign</td>
<td>9,858</td>
<td>6,678</td>
<td>6,672</td>
</tr>
<tr>
<td>Current taxes</td>
<td>$ 24,389</td>
<td>$ 23,009</td>
<td>$ 16,680</td>
</tr>
<tr>
<td>Deferred Taxes</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>U.S. federal</td>
<td>$ (4,791)</td>
<td>$ (6,146)</td>
<td>$ (4,815)</td>
</tr>
<tr>
<td>U.S. state and local</td>
<td>(379)</td>
<td>(477)</td>
<td>(1,062)</td>
</tr>
<tr>
<td>Foreign</td>
<td>432</td>
<td>564</td>
<td>175</td>
</tr>
<tr>
<td>Deferred taxes</td>
<td>$ (4,738)</td>
<td>$ (6,059)</td>
<td>$ (5,702)</td>
</tr>
<tr>
<td>Provision for income taxes</td>
<td>19,651</td>
<td>$ 16,950</td>
<td>$ 10,978</td>
</tr>
</table>


<!-- PageNumber="80" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->


## U.S. and foreign components of income before income taxes were as follows:


### (In millions)


<table>
<tr>
<th>Year Ended June 30,</th>
<th>2024</th>
<th>2023</th>
<th>2022</th>
</tr>
<tr>
<td>U.S.</td>
<td>$ 62,886</td>
<td>$ 52,917</td>
<td>$ 47,837</td>
</tr>
<tr>
<td>Foreign</td>
<td>44,901</td>
<td>36,394</td>
<td>35,879</td>
</tr>
<tr>
<td>Income before income taxes</td>
<td>$107,787</td>
<td>$ 89,311</td>
<td>$ 83,716</td>
</tr>
</table>


### Effective Tax Rate

The items accounting for the difference between income taxes computed at the U.S. federal statutory rate and our
effective rate were as follows:


<table>
<tr>
<th>Year Ended June 30,</th>
<th>2024</th>
<th>2023</th>
<th>2022</th>
</tr>
<tr>
<td>Federal statutory rate</td>
<td>21.0%</td>
<td>21.0%</td>
<td>21.0%</td>
</tr>
<tr>
<td>Effect of:</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Foreign earnings taxed at lower rates</td>
<td>(1.4)%</td>
<td>(1.8)%</td>
<td>(1.3)%</td>
</tr>
<tr>
<td>Impact of intangible property transfers</td>
<td>0%</td>
<td>0%</td>
<td>(3.9)%</td>
</tr>
<tr>
<td>Foreign-derived intangible income deduction</td>
<td>(1.1)%</td>
<td>(1.3)%</td>
<td>(1.1)%</td>
</tr>
<tr>
<td>State income taxes, net of federal benefit</td>
<td>1.5%</td>
<td>1.6%</td>
<td>1.4%</td>
</tr>
<tr>
<td>Research and development credit</td>
<td>(1.1)%</td>
<td>(1.1)%</td>
<td>(0.9)%</td>
</tr>
<tr>
<td>Excess tax benefits relating to stock-based compensation</td>
<td>(1.1)%</td>
<td>(0.7)%</td>
<td>(1.9)%</td>
</tr>
<tr>
<td>Interest, net</td>
<td>1.1%</td>
<td>0.8%</td>
<td>0.5%</td>
</tr>
<tr>
<td>Other reconciling items, net</td>
<td>(0.7)%</td>
<td>0.5%</td>
<td>(0.7)%</td>
</tr>
<tr>
<td>Effective rate</td>
<td>18.2%</td>
<td>19.0%</td>
<td>13.1%</td>
</tr>
</table>


In the first quarter of fiscal year 2022, we transferred certain intangible properties from our Puerto Rico subsidiary to
the U.S. The transfer of intangible properties resulted in a $3.3 billion net income tax benefit in the first quarter of fiscal
year 2022, as the value of future U.S. tax deductions exceeded the current tax liability from the U.S. global intangible
low-taxed income ("GILTI") tax.

The decrease from the federal statutory rate in fiscal year 2024 and 2023 is primarily due to earnings taxed at lower
rates in foreign jurisdictions resulting from producing and distributing our products and services through our foreign
regional operations center in Ireland. The decrease from the federal statutory rate in fiscal year 2022 is primarily due
to the net income tax benefit related to the transfer of intangible properties, earnings taxed at lower rates in foreign
jurisdictions resulting from producing and distributing our products and services through our foreign regional operations
center in Ireland, and tax benefits relating to stock-based compensation. In fiscal years 2024 and 2023, our foreign
regional operating center in Ireland, which is taxed at a rate lower than the U.S. rate, generated 83% and 81% of our
foreign income before tax. In fiscal year 2022, our foreign regional operating centers in Ireland and Puerto Rico, which
are taxed at rates lower than the U.S. rate, generated 71% of our foreign income before tax. Other reconciling items,
net consists primarily of tax credits and GILTI tax, and in fiscal year 2024, includes tax benefits from tax law changes.
In fiscal year 2024, tax benefits from tax law changes primarily relates to the issuance of Notice 2023-55 and Notice
2023-80 by the Internal Revenue Service ("IRS") and U.S. Treasury Department. Notice 2023-55, issued in the first
quarter of fiscal year 2024, delayed the effective date of final foreign tax credit regulations to fiscal year 2024 for
Microsoft. Notice 2023-80, issued in the second quarter of fiscal year 2024, further delayed the effective date of final
foreign tax credit regulations indefinitely. In fiscal years 2024, 2023, and 2022, there were no individually significant
other reconciling items.

The decrease in our effective tax rate for fiscal year 2024 compared to fiscal year 2023 was primarily due to tax benefits
from tax law changes, including the delay of the effective date of final foreign tax credit regulations. The increase in
our effective tax rate for fiscal year 2023 compared to fiscal year 2022 was primarily due to a $3.3 billion net income

<!-- PageNumber="81" -->
<!-- PageBreak -->


# PART II Item 8

tax benefit in the first quarter of fiscal year 2022 related to the transfer of intangible properties and a decrease in tax
benefits relating to stock-based compensation.

The components of the deferred income tax assets and liabilities were as follows:

(In millions)


<table>
<tr>
<th>June 30,</th>
<th>2024</th>
<th>2023</th>
</tr>
<tr>
<td>Deferred Income Tax Assets</td>
<td></td>
<td></td>
</tr>
<tr>
<td>Stock-based compensation expense</td>
<td>$ 765</td>
<td>$ 681</td>
</tr>
<tr>
<td>Accruals, reserves, and other expenses</td>
<td>4,381</td>
<td>3,131</td>
</tr>
<tr>
<td>Loss and credit carryforwards</td>
<td>1,741</td>
<td>1,441</td>
</tr>
<tr>
<td>Amortization</td>
<td>4,159</td>
<td>9,440</td>
</tr>
<tr>
<td>Leasing liabilities</td>
<td>6,504</td>
<td>5,041</td>
</tr>
<tr>
<td>Unearned revenue</td>
<td>3,717</td>
<td>3,296</td>
</tr>
<tr>
<td>Book/tax basis differences in investments and debt</td>
<td>9</td>
<td>373</td>
</tr>
<tr>
<td>Capitalized research and development</td>
<td>11,442</td>
<td>6,958</td>
</tr>
<tr>
<td>Other</td>
<td>426</td>
<td>489</td>
</tr>
<tr>
<td>Deferred income tax assets</td>
<td>33,144</td>
<td>30,850</td>
</tr>
<tr>
<td>Less valuation allowance</td>
<td>(1,045)</td>
<td>(939)</td>
</tr>
<tr>
<td>Deferred income tax assets, net of valuation allowance</td>
<td>$ 32,099</td>
<td>$ 29,911</td>
</tr>
<tr>
<td>Deferred Income Tax Liabilities</td>
<td></td>
<td></td>
</tr>
<tr>
<td>Leasing assets</td>
<td>$ (6,503)</td>
<td>$ (4,680)</td>
</tr>
<tr>
<td>Depreciation</td>
<td>(3,940)</td>
<td>(2,674)</td>
</tr>
<tr>
<td>Deferred tax on foreign earnings</td>
<td>(1,837)</td>
<td>(2,738)</td>
</tr>
<tr>
<td>Other</td>
<td>(167)</td>
<td>(89)</td>
</tr>
<tr>
<td>Deferred income tax liabilities</td>
<td>$ (12,447)</td>
<td>$ (10,181)</td>
</tr>
<tr>
<td>Net deferred income tax assets</td>
<td>$ 19,652</td>
<td>$ 19,730</td>
</tr>
<tr>
<td>Reported As</td>
<td></td>
<td></td>
</tr>
<tr>
<td>Other long-term assets</td>
<td>$ 22,270</td>
<td>$ 20,163</td>
</tr>
<tr>
<td>Long-term deferred income tax liabilities</td>
<td>(2,618)</td>
<td>(433)</td>
</tr>
<tr>
<td>Net deferred income tax assets</td>
<td>$ 19,652</td>
<td>$ 19,730</td>
</tr>
</table>


Deferred income tax balances reflect the effects of temporary differences between the carrying amounts of assets and
liabilities and their tax bases and are stated at enacted tax rates expected to be in effect when the taxes are paid or
recovered.

As of June 30, 2024, we had federal, state, and foreign net operating loss carryforwards of $476 million, $899 million,
and $2.6 billion, respectively. The federal and state net operating loss carryforwards have varying expiration dates
ranging from fiscal year 2025 to 2044 or indefinite carryforward periods, if not utilized. The majority of our foreign net
operating loss carryforwards do not expire. Certain acquired net operating loss carryforwards are subject to an annual
limitation but are expected to be realized with the exception of those which have a valuation allowance. As of June 30,
2024, we had $456 million federal capital loss carryforwards for U.S. tax purposes from our acquisition of Nuance. The
federal capital loss carryforwards are subject to an annual limitation and will expire in fiscal year 2025.

The valuation allowance disclosed in the table above relates to the foreign net operating loss carryforwards, federal
capital loss carryforwards, and other net deferred tax assets that may not be realized.

Income taxes paid, net of refunds, were $23.4 billion, $23.1 billion, and $16.0 billion in fiscal years 2024, 2023, and
2022, respectively.

<!-- PageNumber="82" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->


# Uncertain Tax Positions

Gross unrecognized tax benefits related to uncertain tax positions as of June 30, 2024, 2023, and 2022, were $22.8
billion, $17.1 billion, and $15.6 billion, respectively, which were primarily included in long-term income taxes in our
consolidated balance sheets. If recognized, the resulting tax benefit would affect our effective tax rates for fiscal years
2024, 2023, and 2022 by $19.6 billion, $14.4 billion, and $13.3 billion, respectively.

As of June 30, 2024, 2023, and 2022, we had accrued interest expense related to uncertain tax positions of $6.8 billion,
$5.2 billion, and $4.3 billion, respectively, net of income tax benefits. The provision for income taxes for fiscal years
2024, 2023, and 2022 included interest expense related to uncertain tax positions of $1.5 billion, $918 million, and $36
million, respectively, net of income tax benefits.

The aggregate changes in the gross unrecognized tax benefits related to uncertain tax positions were as follows:


<table>
<tr>
<th>(In millions)</th>
<th colspan="3"></th>
</tr>
<tr>
<th>Year Ended June 30,</th>
<th>2024</th>
<th>2023</th>
<th>2022</th>
</tr>
<tr>
<td>Beginning unrecognized tax benefits</td>
<td>$ 17,120</td>
<td>$ 15,593</td>
<td>$ 14,550</td>
</tr>
<tr>
<td>Decreases related to settlements</td>
<td>(76)</td>
<td>(329)</td>
<td>(317)</td>
</tr>
<tr>
<td>Increases for tax positions related to the current year</td>
<td>1,903</td>
<td>1,051</td>
<td>1,145</td>
</tr>
<tr>
<td>Increases for tax positions related to prior years (a)</td>
<td>4,289</td>
<td>870</td>
<td>461</td>
</tr>
<tr>
<td>Decreases for tax positions related to prior years</td>
<td>(464)</td>
<td>(60)</td>
<td>(246)</td>
</tr>
<tr>
<td>Decreases due to lapsed statutes of limitations</td>
<td>(12)</td>
<td>(5)</td>
<td>0</td>
</tr>
<tr>
<td>Ending unrecognized tax benefits</td>
<td>$ 22,760</td>
<td>$ 17,120</td>
<td>$ 15,593</td>
</tr>
</table>

(a) Fiscal year 2024 includes unrecognized tax benefits of $3.4 billion related to the acquisition of Activision Blizzard.
See Note 8 - Business Combinations for further information.


We remain under audit by the IRS for tax years 2014 to 2017. With respect to the audit for tax years 2004 to 2013, on
September 26, 2023, we received Notices of Proposed Adjustment ("NOPAs") from the IRS. The primary issues in the
NOPAs relate to intercompany transfer pricing. In the NOPAs, the IRS is seeking an additional tax payment of $28.9
billion plus penalties and interest. As of June 30, 2024, we believe our allowances for income tax contingencies are
adequate. We disagree with the proposed adjustments and will vigorously contest the NOPAs through the IRS's
administrative appeals office and, if necessary, judicial proceedings. We do not expect a final resolution of these issues
in the next 12 months. Based on the information currently available, we do not anticipate a significant increase or
decrease to our income tax contingencies for these issues within the next 12 months.

We are subject to income tax in many jurisdictions outside the U.S. Our operations in certain jurisdictions remain
subject to examination for tax years 1996 to 2023, some of which are currently under audit by local tax authorities. The
resolution of each of these audits is not expected to be material to our consolidated financial statements.

NOTE 13 - UNEARNED REVENUE

Unearned revenue by segment was as follows:

(In millions)


<table>
<tr>
<th>June 30,</th>
<th>2024</th>
<th>2023</th>
</tr>
<tr>
<td>Productivity and Business Processes</td>
<td>$ 30,879</td>
<td>$ 27,572</td>
</tr>
<tr>
<td>Intelligent Cloud</td>
<td>23,117</td>
<td>21,563</td>
</tr>
<tr>
<td>More Personal Computing</td>
<td>6,188</td>
<td>4,678</td>
</tr>
<tr>
<td>Total</td>
<td>$ 60,184</td>
<td>$ 53,813</td>
</tr>
</table>


<!-- PageNumber="83" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->

Changes in unearned revenue were as follows:


<table>
<tr>
<th>(In millions)</th>
<th></th>
</tr>
<tr>
<td>Year Ended June 30, 2024</td>
<td></td>
</tr>
<tr>
<td>Balance, beginning of period</td>
<td>$ 53,813</td>
</tr>
<tr>
<td>Deferral of revenue</td>
<td>148,701</td>
</tr>
<tr>
<td>Recognition of unearned revenue</td>
<td>(142,330)</td>
</tr>
<tr>
<td>Balance, end of period</td>
<td>$ 60,184</td>
</tr>
</table>


Revenue allocated to remaining performance obligations, which includes unearned revenue and amounts that will be
invoiced and recognized as revenue in future periods, was $275 billion as of June 30, 2024, of which $269 billion is
related to the commercial portion of revenue. We expect to recognize approximately 45% of our total company
remaining performance obligation revenue over the next 12 months and the remainder thereafter.


## NOTE 14 - LEASES

We have operating and finance leases for datacenters, corporate offices, research and development facilities, Microsoft
Experience Centers, and certain equipment. Our leases have remaining lease terms of less than 1 year to 17 years,
some of which include options to extend the leases for up to 5 years, and some of which include options to terminate
the leases within 1 year.

The components of lease expense were as follows:


<table>
<tr>
<th>(In millions)</th>
<th colspan="3"></th>
</tr>
<tr>
<th>Year Ended June 30,</th>
<th>2024</th>
<th>2023</th>
<th>2022</th>
</tr>
<tr>
<td>Operating lease cost</td>
<td>$ 3,555</td>
<td>$ 2,875</td>
<td>$ 2,461</td>
</tr>
<tr>
<td>Finance lease cost:</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Amortization of right-of-use assets</td>
<td>$ 1,800</td>
<td>$ 1,352</td>
<td>$ 980</td>
</tr>
<tr>
<td>Interest on lease liabilities</td>
<td>734</td>
<td>501</td>
<td>429</td>
</tr>
<tr>
<td>Total finance lease cost</td>
<td>$ 2,534</td>
<td>$ 1,853</td>
<td>$ 1,409</td>
</tr>
</table>


Supplemental cash flow information related to leases was as follows:

(In millions)


<table>
<tr>
<th>Year Ended June 30,</th>
<th>2024</th>
<th>2023</th>
<th>2022</th>
</tr>
<tr>
<td>Cash paid for amounts included in the measurement of lease liabilities:</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Operating cash flows from operating leases</td>
<td>$ 3,550</td>
<td>$ 2,706</td>
<td>$ 2,368</td>
</tr>
<tr>
<td>Operating cash flows from finance leases</td>
<td>734</td>
<td>501</td>
<td>429</td>
</tr>
<tr>
<td>Financing cash flows from finance leases</td>
<td>1,286</td>
<td>1,056</td>
<td>896</td>
</tr>
<tr>
<td>Right-of-use assets obtained in exchange for lease obligations:</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Operating leases</td>
<td>6,703</td>
<td>3,514</td>
<td>5,268</td>
</tr>
<tr>
<td>Finance leases</td>
<td>11,633</td>
<td>3,128</td>
<td>4,234</td>
</tr>
</table>


Supplemental balance sheet information related to leases was as follows:

(In millions, except lease term and discount rate)


<table>
<tr>
<th>June 30,</th>
<th>2024</th>
<th>2023</th>
</tr>
<tr>
<td>Operating Leases</td>
<td></td>
<td></td>
</tr>
<tr>
<td>Operating lease right-of-use assets</td>
<td>$ 18,961</td>
<td>$ 14,346</td>
</tr>
<tr>
<td>Other current liabilities</td>
<td>$ 3,580</td>
<td>$ 2,409</td>
</tr>
</table>


<!-- PageNumber="84" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->


### (In millions, except lease term and discount rate)


<table>
<tr>
<th>Operating lease liabilities</th>
<th>15,497</th>
<th>12,728</th>
</tr>
<tr>
<td>Total operating lease liabilities</td>
<td>$ 19,077</td>
<td>$ 15,137</td>
</tr>
<tr>
<td>Finance Leases</td>
<td></td>
<td></td>
</tr>
<tr>
<td>Property and equipment, at cost</td>
<td>$ 32,248</td>
<td>$ 20,538</td>
</tr>
<tr>
<td>Accumulated depreciation</td>
<td>(6,386)</td>
<td>(4,647)</td>
</tr>
<tr>
<td>Property and equipment, net</td>
<td>$ 25,862</td>
<td>$ 15,891</td>
</tr>
<tr>
<td>Other current liabilities</td>
<td>$ 2,349</td>
<td>$ 1,197</td>
</tr>
<tr>
<td>Other long-term liabilities</td>
<td>24,796</td>
<td>15,870</td>
</tr>
<tr>
<td>Total finance lease liabilities</td>
<td>$ 27,145</td>
<td>$ 17,067</td>
</tr>
<tr>
<td>Weighted Average Remaining Lease Term</td>
<td></td>
<td></td>
</tr>
<tr>
<td>Operating leases</td>
<td>7 years</td>
<td>8 years</td>
</tr>
<tr>
<td>Finance leases</td>
<td>12 years</td>
<td>11 years</td>
</tr>
<tr>
<td>Weighted Average Discount Rate</td>
<td></td>
<td></td>
</tr>
<tr>
<td>Operating leases</td>
<td>3.3%</td>
<td>2.9%</td>
</tr>
<tr>
<td>Finance leases</td>
<td>3.9%</td>
<td>3.4%</td>
</tr>
</table>


The following table outlines maturities of our lease liabilities as of June 30, 2024:

(In millions)


<table>
<tr>
<th>Year Ending June 30,</th>
<th>Operating Leases</th>
<th>Finance Leases</th>
</tr>
<tr>
<td>2025</td>
<td>$ 4,124</td>
<td>$ 3,311</td>
</tr>
<tr>
<td>2026</td>
<td>3,549</td>
<td>3,021</td>
</tr>
<tr>
<td>2027</td>
<td>2,981</td>
<td>3,037</td>
</tr>
<tr>
<td>2028</td>
<td>2,405</td>
<td>3,026</td>
</tr>
<tr>
<td>2029</td>
<td>1,924</td>
<td>2,638</td>
</tr>
<tr>
<td>Thereafter</td>
<td>6,587</td>
<td>19,116</td>
</tr>
<tr>
<td>Total lease payments</td>
<td>21,570</td>
<td>34,149</td>
</tr>
<tr>
<td>Less imputed interest</td>
<td>(2,493)</td>
<td>(7,004)</td>
</tr>
<tr>
<td>Total</td>
<td>$ 19,077</td>
<td>$ 27,145</td>
</tr>
</table>


As of June 30, 2024, we had additional operating and finance leases, primarily for datacenters, that had not yet
commenced of $8.6 billion and $108.4 billion, respectively. These operating and finance leases will commence
between fiscal year 2025 and fiscal year 2030 with lease terms of 1 year to 20 years.

<!-- PageNumber="85" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->


#### NOTE 15 - CONTINGENCIES


##### U.S. Cell Phone Litigation

Microsoft Mobile Oy, a subsidiary of Microsoft, along with other handset manufacturers and network operators, is a
defendant in 45 lawsuits filed in the Superior Court for the District of Columbia by individual plaintiffs who allege that
radio emissions from cellular handsets caused their brain tumors and other adverse health effects. We assumed
responsibility for these claims in our agreement to acquire Nokia's Devices and Services business and have been
substituted for the Nokia defendants. Twelve of these cases were consolidated for certain pre-trial proceedings; the
remaining cases are stayed. In a separate 2009 decision, the Court of Appeals for the District of Columbia held that
adverse health effect claims arising from the use of cellular handsets that operate within the U.S. Federal
Communications Commission radio frequency emission guidelines ("FCC Guidelines") are pre-empted by federal law.
The plaintiffs allege that their handsets either operated outside the FCC Guidelines or were manufactured before the
FCC Guidelines went into effect. The lawsuits also allege an industry-wide conspiracy to manipulate the science and
testing around emission guidelines.

In 2013, the defendants in the consolidated cases moved to exclude the plaintiffs' expert evidence of general causation
on the basis of flawed scientific methodologies. In 2014, the trial court granted in part and denied in part the defendants'
motion to exclude the plaintiffs' general causation experts. The defendants filed an interlocutory appeal to the District
of Columbia Court of Appeals challenging the standard for evaluating expert scientific evidence. In October 2016, the
Court of Appeals issued its decision adopting the standard advocated by the defendants and remanding the cases to
the trial court for further proceedings under that standard. The plaintiffs have filed supplemental expert evidence,
portions of which were stricken by the court. A hearing on general causation took place in September of 2022. In April
of 2023, the court granted defendants' motion to strike the testimony of plaintiffs' experts that cell phones cause brain
cancer and entered an order excluding all of plaintiffs' experts from testifying. The parties agreed to a stipulated
dismissal of the consolidated cases to allow plaintiffs to appeal the expert testimony order. Plaintiffs appealed the
court's order in August of 2023, and the parties have filed their briefs on the appeal. A hearing on the status of the
stayed cases occurred in December of 2023. In July 2024, the court entered summary judgment in nine of the stayed
cases on the grounds that plaintiffs had agreed to be bound by the general causation outcome in the consolidated
cases.


##### Irish Data Protection Commission Matter

In 2018, the Irish Data Protection Commission ("IDPC") began investigating a complaint against Linkedln as to whether
LinkedIn's targeted advertising practices violated the recently implemented European Union General Data Protection
Regulation ("GDPR"). Microsoft cooperated throughout the period of inquiry. In April 2023, the IDPC provided Linkedln
with a non-public preliminary draft decision alleging GDPR violations and proposing a fine. In July 2024, the IDPC
provided LinkedIn with a revised non-public draft decision. There is no set timeline for the IDPC to issue a final decision,
at which time Microsoft will consider its options to appeal.


##### Other Contingencies

We also are subject to a variety of other claims and suits that arise from time to time in the ordinary course of our
business. Although management currently believes that resolving claims against us, individually or in aggregate, will
not have a material adverse impact in our consolidated financial statements, these matters are subject to inherent
uncertainties and management's view of these matters may change in the future.

As of June 30, 2024, we accrued aggregate legal liabilities of $641 million. While we intend to defend these matters
vigorously, adverse outcomes that we estimate could reach approximately $600 million in aggregate beyond recorded
amounts are reasonably possible. Were unfavorable final outcomes to occur, there exists the possibility of a material
adverse impact in our consolidated financial statements for the period in which the effects become reasonably
estimable.


#### NOTE 16 - STOCKHOLDERS' EQUITY


##### Shares Outstanding

<!-- PageNumber="86" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->

Shares of common stock outstanding were as follows:
(In millions)


<table>
<tr>
<th>Year Ended June 30,</th>
<th>2024</th>
<th>2023</th>
<th>2022</th>
</tr>
<tr>
<td>Balance, beginning of year</td>
<td>7,432</td>
<td>7,464</td>
<td>7,519</td>
</tr>
<tr>
<td>Issued</td>
<td>34</td>
<td>37</td>
<td>40</td>
</tr>
<tr>
<td>Repurchased</td>
<td>(32)</td>
<td>(69)</td>
<td>(95)</td>
</tr>
<tr>
<td>Balance, end of year</td>
<td>7,434</td>
<td>7,432</td>
<td>7,464</td>
</tr>
</table>


##### Share Repurchases

On September 18, 2019, our Board of Directors approved a share repurchase program authorizing up to $40.0 billion
in share repurchases. This share repurchase program commenced in February 2020 and was completed in November
2021.

On September 14, 2021, our Board of Directors approved a share repurchase program authorizing up to $60.0 billion
in share repurchases. This share repurchase program commenced in November 2021, following completion of the
program approved on September 18, 2019, has no expiration date, and may be terminated at any time. As of June 30,
2024, $10.3 billion remained of this $60.0 billion share repurchase program.

We repurchased the following shares of common stock under the share repurchase programs:


<table>
<tr>
<th>(In millions)</th>
<th>Shares</th>
<th>Amount</th>
<th>Shares</th>
<th>Amount</th>
<th>Shares</th>
<th>Amount</th>
</tr>
<tr>
<td>Year Ended June 30,</td>
<td></td>
<td>2024</td>
<td></td>
<td>2023</td>
<td></td>
<td>2022</td>
</tr>
<tr>
<td>First Quarter</td>
<td>11</td>
<td>$ 3,560</td>
<td>17</td>
<td>$ 4,600</td>
<td>21</td>
<td>$ 6,200</td>
</tr>
<tr>
<td>Second Quarter</td>
<td>7</td>
<td>2,800</td>
<td>20</td>
<td>4,600</td>
<td>20</td>
<td>6,233</td>
</tr>
<tr>
<td>Third Quarter</td>
<td>7</td>
<td>2,800</td>
<td>18</td>
<td>4,600</td>
<td>26</td>
<td>7,800</td>
</tr>
<tr>
<td>Fourth Quarter</td>
<td>7</td>
<td>2,800</td>
<td>14</td>
<td>4,600</td>
<td>28</td>
<td>7,800</td>
</tr>
<tr>
<td>Total</td>
<td>32</td>
<td>$ 11,960</td>
<td>69</td>
<td>$ 18,400</td>
<td>95</td>
<td>$ 28,033</td>
</tr>
</table>


All repurchases were made using cash resources. Shares repurchased during the first quarter of fiscal year 2022 were
under the share repurchase program approved on September 18, 2019. Shares repurchased during the second quarter
of fiscal year 2022 were under the share repurchase programs approved on September 18, 2019 and September 14,
2021. All other shares repurchased were under the share repurchase program approved on September 14, 2021. The
above table excludes shares repurchased to settle employee tax withholding related to the vesting of stock awards of
$5.3 billion, $3.8 billion, and $4.7 billion for fiscal years 2024, 2023, and 2022, respectively.

<!-- PageNumber="87" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->


##### Dividends

Our Board of Directors declared the following dividends:


<table>
<tr>
<th>Declaration Date</th>
<th>Record Date</th>
<th>Payment Date</th>
<th>Dividend Per Share</th>
<th>Amount</th>
</tr>
<tr>
<td>Fiscal Year 2024</td>
<td></td>
<td></td>
<td></td>
<td>(In millions)</td>
</tr>
<tr>
<td>September 19, 2023</td>
<td>November 16, 2023</td>
<td>December 14, 2023</td>
<td>$ 0.75</td>
<td>$ 5,574</td>
</tr>
<tr>
<td>November 28, 2023</td>
<td>February 15, 2024</td>
<td>March 14, 2024</td>
<td>0.75</td>
<td>5,573</td>
</tr>
<tr>
<td>March 12, 2024</td>
<td>May 16, 2024</td>
<td>June 13, 2024</td>
<td>0.75</td>
<td>5,574</td>
</tr>
<tr>
<td>June 12, 2024</td>
<td>August 15, 2024</td>
<td>September 12, 2024</td>
<td>0.75</td>
<td>5,575</td>
</tr>
<tr>
<td>Total</td>
<td></td>
<td></td>
<td>$ 3.00</td>
<td>$ 22,296</td>
</tr>
<tr>
<td>Fiscal Year 2023</td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>September 20, 2022</td>
<td>November 17, 2022</td>
<td>December 8, 2022</td>
<td>$ 0.68</td>
<td>$ 5,066</td>
</tr>
<tr>
<td>November 29, 2022</td>
<td>February 16, 2023</td>
<td>March 9, 2023</td>
<td>0.68</td>
<td>5,059</td>
</tr>
<tr>
<td>March 14, 2023</td>
<td>May 18, 2023</td>
<td>June 8, 2023</td>
<td>0.68</td>
<td>5,054</td>
</tr>
<tr>
<td>June 13, 2023</td>
<td>August 17, 2023</td>
<td>September 14, 2023</td>
<td>0.68</td>
<td>5,051</td>
</tr>
<tr>
<td>Total</td>
<td></td>
<td></td>
<td>$ 2.72</td>
<td>$ 20,230</td>
</tr>
</table>


The dividend declared on June 12, 2024 was included in other current liabilities as of June 30, 2024.

<!-- PageNumber="88" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->


###### NOTE 17 - ACCUMULATED OTHER COMPREHENSIVE INCOME (LOSS)

The following table summarizes the changes in accumulated other comprehensive income (loss) by component:


<table>
<tr>
<th colspan="4">(In millions)</th>
</tr>
<tr>
<th>Year Ended June 30,</th>
<th>2024</th>
<th>2023</th>
<th>2022</th>
</tr>
<tr>
<td>Derivatives</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Balance, beginning of period</td>
<td>$ (27)</td>
<td>$ (13)</td>
<td>$ (19)</td>
</tr>
<tr>
<td>Unrealized gains (losses), net of tax of $(4), $9, and $(15)</td>
<td>(14)</td>
<td>34</td>
<td>(57)</td>
</tr>
<tr>
<td>Reclassification adjustments for (gains) losses included in other income (expense), net</td>
<td>48</td>
<td>(61)</td>
<td>79</td>
</tr>
<tr>
<td>Tax expense (benefit) included in provision for income taxes</td>
<td>(10)</td>
<td>13</td>
<td>(16)</td>
</tr>
<tr>
<td>Amounts reclassified from accumulated other comprehensive loss</td>
<td>38</td>
<td>(48)</td>
<td>63</td>
</tr>
<tr>
<td>Net change related to derivatives, net of tax of $6, $(4), and $1</td>
<td>24</td>
<td>(14)</td>
<td>6</td>
</tr>
<tr>
<td>Balance, end of period</td>
<td>$ (3)</td>
<td>$ (27)</td>
<td>$ (13)</td>
</tr>
<tr>
<td>Investments</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Balance, beginning of period</td>
<td>$ (3,582)</td>
<td>$ (2,138)</td>
<td>$ 3,222</td>
</tr>
<tr>
<td>Unrealized gains (losses), net of tax of $247, $(393), and $(1,440)</td>
<td>915</td>
<td>(1,523)</td>
<td>(5,405)</td>
</tr>
<tr>
<td>Reclassification adjustments for losses included in other income (expense), net</td>
<td>53</td>
<td>99</td>
<td>57</td>
</tr>
<tr>
<td>Tax benefit included in provision for income taxes</td>
<td>(11)</td>
<td>(20)</td>
<td>(12)</td>
</tr>
<tr>
<td>Amounts reclassified from accumulated other comprehensive loss</td>
<td>42</td>
<td>79</td>
<td>45</td>
</tr>
<tr>
<td>Net change related to investments, net of tax of $258, $(373), and $(1,428)</td>
<td>957</td>
<td>(1,444)</td>
<td>(5,360)</td>
</tr>
<tr>
<td>Balance, end of period</td>
<td>$ (2,625)</td>
<td>$ (3,582)</td>
<td>$ (2,138)</td>
</tr>
<tr>
<td>Translation Adjustments and Other</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Balance, beginning of period</td>
<td>$ (2,734)</td>
<td>$ (2,527)</td>
<td>$ (1,381)</td>
</tr>
<tr>
<td>Translation adjustments and other, net of tax of $0, $0, and $0</td>
<td>(228)</td>
<td>(207)</td>
<td>(1,146)</td>
</tr>
<tr>
<td>Balance, end of period</td>
<td>$ (2,962)</td>
<td>$ (2,734)</td>
<td>$ (2,527)</td>
</tr>
<tr>
<td>Accumulated other comprehensive loss, end of period</td>
<td>$ (5,590)</td>
<td>$ (6,343)</td>
<td>: $ (4,678)</td>
</tr>
</table>


####### NOTE 18 - EMPLOYEE STOCK AND SAVINGS PLANS

We grant stock-based compensation to employees and directors. Awards that expire or are canceled without delivery
of shares generally become available for issuance under the plans. We issue new shares of Microsoft common stock
to satisfy vesting of awards granted under our stock plans. We also have an ESPP for all eligible employees.

Stock-based compensation expense and related income tax benefits were as follows:

(In millions)


<table>
<tr>
<th>Year Ended June 30,</th>
<th>2024</th>
<th>2023</th>
<th>2022</th>
</tr>
<tr>
<td>Stock-based compensation expense</td>
<td>$ 10,734</td>
<td>$ 9,611</td>
<td>$ 7,502</td>
</tr>
<tr>
<td>Income tax benefits related to stock-based compensation</td>
<td>1,826</td>
<td>1,651</td>
<td>1,293</td>
</tr>
</table>


<!-- PageNumber="89" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->


##### Stock Plans

Stock awards entitle the holder to receive shares of Microsoft common stock as the award vests. Stock awards
generally vest over a service period of four years or five years.


##### Executive Incentive Plan

Under the Executive Incentive Plan, the Compensation Committee approves stock awards to executive officers and
certain senior executives. RSUs generally vest ratably over a service period of four years. PSUs generally vest over a
performance period of three years. The number of shares the PSU holder receives is based on the extent to which the
corresponding performance goals have been achieved.


##### Activity for All Stock Plans

The fair value of stock awards was estimated on the date of grant using the following assumptions:


<table>
<tr>
<th>Year ended June 30,</th>
<th>2024</th>
<th>2023</th>
<th>2022</th>
</tr>
<tr>
<td>Dividends per share (quarterly amounts)</td>
<td>$ 0.68- 0.75</td>
<td>$ 0.62- 0.68</td>
<td>$ 0.56- 0.62</td>
</tr>
<tr>
<td>Interest rates</td>
<td>3.8%- 5.6%</td>
<td>2.0%-5.4%</td>
<td>0.03%- 3.6%</td>
</tr>
</table>


During fiscal year 2024, the following activity occurred under our stock plans:


<table>
<tr>
<th></th>
<th>Shares</th>
<th>Weighted Average Grant-Date Fair Value</th>
</tr>
<tr>
<td></td>
<td colspan="2">(In millions)</td>
</tr>
<tr>
<td>Stock Awards</td>
<td></td>
<td></td>
</tr>
<tr>
<td>Nonvested balance, beginning of year</td>
<td>96</td>
<td>$ 250.37</td>
</tr>
<tr>
<td>Granted (a)</td>
<td>41</td>
<td>339.46</td>
</tr>
<tr>
<td>Vested</td>
<td>(42)</td>
<td>246.71</td>
</tr>
<tr>
<td>Forfeited</td>
<td>(7)</td>
<td>270.59</td>
</tr>
<tr>
<td>Nonvested balance, end of year</td>
<td>88</td>
<td>$ 292.28</td>
</tr>
</table>

(a) Includes 1 million of PSUs granted at target and performance adjustments above target levels for each of the
fiscal years 2024, 2023, and 2022.


As of June 30, 2024, total unrecognized compensation costs related to stock awards were $20.3 billion. These costs
are expected to be recognized over a weighted average period of three years. The weighted average grant-date fair
value of stock awards granted was $339.46, $252.59, and $291.22 for fiscal years 2024, 2023, and 2022, respectively.
The fair value of stock awards vested was $16.0 billion, $11.9 billion, and $14.1 billion, for fiscal years 2024, 2023, and
2022, respectively. As of June 30, 2024, an aggregate of 129 million shares were authorized for future grant under our
stock plans.


##### Employee Stock Purchase Plan

We have an ESPP for all eligible employees. Shares of our common stock may be purchased by employees at three-
month intervals at 90% of the fair market value on the last trading day of each three-month period. Employees may
purchase shares having a value not exceeding 15% of their gross compensation during an offering period.

<!-- PageNumber="90" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->

Employees purchased the following shares during the periods presented:


#### (Shares in millions)


<table>
<tr>
<td>Year Ended June 30,</td>
<td>2024</td>
<td>2023</td>
<td>2022</td>
</tr>
<tr>
<td>Shares purchased</td>
<td>6</td>
<td>7</td>
<td>7</td>
</tr>
<tr>
<td>Average price per share</td>
<td>$ 339.46</td>
<td>$ 245.59</td>
<td>$ 259.55</td>
</tr>
</table>


As of June 30, 2024, 68 million shares of our common stock were reserved for future issuance through the ESPP.


##### Savings Plans

We have savings plans in the U.S. that qualify under Section 401(k) of the Internal Revenue Code, and a number of
savings plans in international locations. Eligible U.S. employees may contribute a portion of their salary into the savings
plans, subject to certain limitations. We match a portion of each dollar a participant contributes into the plans. Employer-
funded retirement benefits for all plans were $1.7 billion, $1.6 billion, and $1.4 billion in fiscal years 2024, 2023, and
2022, respectively, and were expensed as contributed.


### NOTE 19 - SEGMENT INFORMATION AND GEOGRAPHIC DATA

In its operation of the business, management, including our chief operating decision maker, who is also our Chief
Executive Officer, reviews certain financial information, including segmented internal profit and loss statements
prepared on a basis not consistent with GAAP. During the periods presented, we reported our financial performance
based on the following segments: Productivity and Business Processes, Intelligent Cloud, and More Personal
Computing.

Our reportable segments are described below.


#### Productivity and Business Processes

Our Productivity and Business Processes segment consists of products and services in our portfolio of productivity,
communication, and information services, spanning a variety of devices and platforms. This segment primarily
comprises:

· Office Commercial (Office 365 subscriptions, the Office 365 portion of Microsoft 365 Commercial
subscriptions, and Office licensed on-premises), comprising Office, Exchange, SharePoint, Microsoft
Teams, Office 365 Security and Compliance, Microsoft Viva, and Copilot for Microsoft 365.

· Office Consumer, including Microsoft 365 Consumer and Copilot Pro subscriptions, Office licensed on-
premises, and other Office services.

· Linkedln, including Talent Solutions, Marketing Solutions, Premium Subscriptions, and Sales Solutions.

· Dynamics business solutions, including Dynamics 365, comprising a set of intelligent, cloud-based
applications across ERP, CRM, Power Apps, and Power Automate; and on-premises ERP and CRM
applications.


#### Intelligent Cloud

Our Intelligent Cloud segment consists of our public, private, and hybrid server products and cloud services that can
power modern business and developers. This segment primarily comprises:

. Server products and cloud services, including Azure and other cloud services; SQL Server, Windows
Server, Visual Studio, System Center, and related Client Access Licenses ("CALs"); and Nuance and
GitHub.

· Enterprise and partner services, including Enterprise Support Services, Industry Solutions, Nuance
professional services, Microsoft Partner Network, and Learning Experience.

<!-- PageNumber="91" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->


#### More Personal Computing

Our More Personal Computing segment consists of products and services that put customers at the center of the
experience with our technology. This segment primarily comprises:

· Windows, including Windows OEM licensing and other non-volume licensing of the Windows operating
system; Windows Commercial, comprising volume licensing of the Windows operating system, Windows
cloud services, and other Windows commercial offerings; patent licensing; and Windows Internet of Things.

· Devices, including Surface, HoloLens, and PC accessories.

· Gaming, including Xbox hardware and Xbox content and services, comprising first-party content (such as
Activision Blizzard) and third-party content, including games and in-game content; Xbox Game Pass and
other subscriptions; Xbox Cloud Gaming; advertising; third-party disc royalties; and other cloud services.

. Search and news advertising, comprising Bing (including Copilot), Microsoft News, Microsoft Edge, and
third-party affiliates.

Revenue and costs are generally directly attributed to our segments. However, due to the integrated structure of our
business, certain revenue recognized and costs incurred by one segment may benefit other segments. Revenue from
certain contracts is allocated among the segments based on the relative value of the underlying products and services,
which can include allocation based on actual prices charged, prices when sold separately, or estimated costs plus a
profit margin. Cost of revenue is allocated in certain cases based on a relative revenue methodology. Operating
expenses that are allocated primarily include those relating to marketing of products and services from which multiple
segments benefit and are generally allocated based on relative gross margin.

In addition, certain costs are incurred at a corporate level and allocated to our segments. These allocated costs
generally include legal, including settlements and fines, information technology, human resources, finance, excise
taxes, field selling, shared facilities services, customer service and support, and severance incurred as part of a
corporate program. Each allocation is measured differently based on the specific facts and circumstances of the costs
being allocated and is generally based on relative gross margin or relative headcount.

Segment revenue and operating income were as follows during the periods presented:


<table>
<tr>
<th>(In millions)</th>
<th colspan="3"></th>
</tr>
<tr>
<th>Year Ended June 30,</th>
<th>2024</th>
<th>2023</th>
<th>2022</th>
</tr>
<tr>
<td>Revenue</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Productivity and Business Processes</td>
<td>$ 77,728</td>
<td>$ 69,274</td>
<td>$ 63,364</td>
</tr>
<tr>
<td>Intelligent Cloud</td>
<td>105,362</td>
<td>87,907</td>
<td>74,965</td>
</tr>
<tr>
<td>More Personal Computing</td>
<td>62,032</td>
<td>54,734</td>
<td>59,941</td>
</tr>
<tr>
<td>Total</td>
<td>$ 245,122</td>
<td>$211,915</td>
<td>$ 198,270</td>
</tr>
<tr>
<td>Operating Income</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>Productivity and Business Processes</td>
<td>$ 40,540</td>
<td>$ 34,189</td>
<td>$ 29,690</td>
</tr>
<tr>
<td>Intelligent Cloud</td>
<td>49,584</td>
<td>37,884</td>
<td>33,203</td>
</tr>
<tr>
<td>More Personal Computing</td>
<td>19,309</td>
<td>16,450</td>
<td>20,490</td>
</tr>
<tr>
<td>Total</td>
<td>$ 109,433</td>
<td>$ 88,523</td>
<td>$ 83,383</td>
</tr>
</table>


<!-- PageNumber="92" -->
<!-- PageBreak -->


# PART II Item 8

No sales to an individual customer or country other than the United States accounted for more than 10% of revenue
for fiscal years 2024, 2023, or 2022. Revenue, classified by the major geographic areas in which our customers were
located, was as follows:

(In millions)


<table>
<tr>
<th>Year Ended June 30,</th>
<th>2024</th>
<th>2023</th>
<th>2022</th>
</tr>
<tr>
<td>United States (a)</td>
<td>$124,704</td>
<td>$ 106,744</td>
<td>$ 100,218</td>
</tr>
<tr>
<td>Other countries</td>
<td>120,418</td>
<td>105,171</td>
<td>98,052</td>
</tr>
<tr>
<td>Total</td>
<td>$245,122</td>
<td>$ 211,915</td>
<td>$ 198,270</td>
</tr>
</table>

(a) Includes billings to OEMs and certain multinational organizations because of the nature of these businesses and
the impracticability of determining the geographic source of the revenue.


Revenue, classified by significant product and service offerings, was as follows:

(In millions)


<table>
<tr>
<th>Year Ended June 30,</th>
<th>2024</th>
<th>2023</th>
<th>2022</th>
</tr>
<tr>
<td>Server products and cloud services</td>
<td>$ 97,726</td>
<td>$ 79,970</td>
<td>$ 67,350</td>
</tr>
<tr>
<td>Office products and cloud services</td>
<td>54,875</td>
<td>48,848</td>
<td>44,970</td>
</tr>
<tr>
<td>Windows</td>
<td>23,244</td>
<td>21,507</td>
<td>24,732</td>
</tr>
<tr>
<td>Gaming</td>
<td>21,503</td>
<td>15,466</td>
<td>16,230</td>
</tr>
<tr>
<td>LinkedIn</td>
<td>16,372</td>
<td>14,989</td>
<td>13,631</td>
</tr>
<tr>
<td>Search and news advertising</td>
<td>12,576</td>
<td>12,158</td>
<td>11,526</td>
</tr>
<tr>
<td>Enterprise and partner services</td>
<td>7,594</td>
<td>7,900</td>
<td>7,605</td>
</tr>
<tr>
<td>Dynamics products and cloud services</td>
<td>6,481</td>
<td>5,437</td>
<td>4,687</td>
</tr>
<tr>
<td>Devices</td>
<td>4,706</td>
<td>5,521</td>
<td>7,306</td>
</tr>
<tr>
<td>Other</td>
<td>45</td>
<td>119</td>
<td>233</td>
</tr>
<tr>
<td>Total</td>
<td>$ 245,122</td>
<td>$211,915</td>
<td>$198,270</td>
</tr>
</table>


We have recast certain prior period amounts to conform to the way we internally manage and monitor our business.

Our Microsoft Cloud revenue, which includes Azure and other cloud services, Office 365 Commercial, the commercial
portion of LinkedIn, Dynamics 365, and other commercial cloud properties, was $137.4 billion, $111.6 billion, and $91.4
billion in fiscal years 2024, 2023, and 2022, respectively. These amounts are primarily included in Server products and
cloud services, Office products and cloud services, LinkedIn, and Dynamics products and cloud services in the table
above.

Assets are not allocated to segments for internal reporting presentations. A portion of amortization and depreciation is
included with various other costs in an overhead allocation to each segment. It is impracticable for us to separately
identify the amount of amortization and depreciation by segment that is included in the measure of segment profit or
loss.

Long-lived assets, excluding financial instruments and tax assets, classified by the location of the controlling statutory
company and with countries over 10% of the total shown separately, were as follows:

(In millions)


<table>
<tr>
<th>June 30,</th>
<th>2024</th>
<th>2023</th>
<th>2022</th>
</tr>
<tr>
<td>United States</td>
<td>$ 186,106</td>
<td>$ 114,380</td>
<td>$ 106,430</td>
</tr>
<tr>
<td>Other countries</td>
<td>115,263</td>
<td>72,859</td>
<td>59,938</td>
</tr>
<tr>
<td>Total</td>
<td>$ 301,369</td>
<td>$ 187,239</td>
<td>$ 166,368</td>
</tr>
</table>


<!-- PageNumber="93" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->


# REPORT OF INDEPENDENT REGISTERED PUBLIC ACCOUNTING FIRM


## To the Stockholders and the Board of Directors of Microsoft Corporation


### Opinion on the Financial Statements

We have audited the accompanying consolidated balance sheets of Microsoft Corporation and subsidiaries (the
"Company") as of June 30, 2024 and 2023, the related consolidated statements of income, comprehensive
income, cash flows, and stockholders' equity, for each of the three years in the period ended June 30, 2024, and
the related notes (collectively referred to as the "financial statements"). In our opinion, the financial statements
present fairly, in all material respects, the financial position of the Company as of June 30, 2024 and 2023, and
the results of its operations and its cash flows for each of the three years in the period ended June 30, 2024, in
conformity with accounting principles generally accepted in the United States of America.

We have also audited, in accordance with the standards of the Public Company Accounting Oversight Board
(United States) (PCAOB), the Company's internal control over financial reporting as of June 30, 2024, based on
criteria established in Internal Control - Integrated Framework (2013) issued by the Committee of Sponsoring
Organizations of the Treadway Commission and our report dated July 30, 2024, expressed an unqualified opinion
on the Company's internal control over financial reporting.


### Basis for Opinion

These financial statements are the responsibility of the Company's management. Our responsibility is to express
an opinion on the Company's financial statements based on our audits. We are a public accounting firm registered
with the PCAOB and are required to be independent with respect to the Company in accordance with the U.S.
federal securities laws and the applicable rules and regulations of the Securities and Exchange Commission and
the PCAOB.

We conducted our audits in accordance with the standards of the PCAOB. Those standards require that we plan
and perform the audit to obtain reasonable assurance about whether the financial statements are free of material
misstatement, whether due to error or fraud. Our audits included performing procedures to assess the risks of
material misstatement of the financial statements, whether due to error or fraud, and performing procedures that
respond to those risks. Such procedures included examining, on a test basis, evidence regarding the amounts
and disclosures in the financial statements. Our audits also included evaluating the accounting principles used
and significant estimates made by management, as well as evaluating the overall presentation of the financial
statements. We believe that our audits provide a reasonable basis for our opinion.


### Critical Audit Matters

The critical audit matters communicated below are matters arising from the current-period audit of the financial
statements that were communicated or required to be communicated to the audit committee and that (1) relate
to accounts or disclosures that are material to the financial statements and (2) involved our especially challenging,
subjective, or complex judgments. The communication of critical audit matters does not alter in any way our
opinion on the financial statements, taken as a whole, and we are not, by communicating the critical audit matters
below, providing separate opinions on the critical audit matters or on the accounts or disclosures to which they
relate.

<!-- PageNumber="94" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->


## Revenue Recognition - Refer to Note 1 to the financial statements


### Critical Audit Matter Description

The Company recognizes revenue upon transfer of control of promised products or services to customers in an
amount that reflects the consideration the Company expects to receive in exchange for those products or
services. The Company offers customers the ability to acquire multiple licenses of software products and
services, including cloud-based services, in its customer agreements through its volume licensing programs.

Significant judgment is exercised by the Company in determining revenue recognition for certain customer
agreements, and includes the following:

· Determination of whether products and services are considered distinct performance obligations that should
be accounted for separately versus together, such as software licenses and related services that are sold
with cloud-based services.

· The pattern of delivery (i.e., timing of when revenue is recognized) for each distinct performance obligation.

· Identification and treatment of contract terms that may impact the timing and amount of revenue recognized
(e.g., variable consideration, optional purchases, and free services).

· Determination of stand-alone selling prices for each distinct performance obligation and for products and
services that are not sold separately.

Given these factors and due to the volume of transactions, the related audit effort in evaluating management's
judgments in determining revenue recognition for certain customer agreements was extensive and required a
high degree of auditor judgment.

How the Critical Audit Matter Was Addressed in the Audit

Our principal audit procedures related to the Company's revenue recognition for certain customer agreements
included the following:

. We tested the effectiveness of controls related to the identification of distinct performance obligations, the
determination of the timing of revenue recognition, and the estimation of variable consideration.

· We evaluated management's significant accounting policies related to certain customer agreements for
reasonableness.

· We selected a sample of customer agreements and performed the following procedures:

\- Obtained and read contract source documents for each selection, including master agreements, and other
documents that were part of the agreement.

\- Tested management's identification and treatment of contract terms.

\- Assessed the terms in the customer agreement and evaluated the appropriateness of management's
application of their accounting policies, along with their use of estimates, in the determination of revenue
recognition conclusions.

. We evaluated the reasonableness of management's estimate of stand-alone selling prices for products and
services that are not sold separately.

. We tested the mathematical accuracy of management's calculations of revenue and the associated timing of
revenue recognized in the financial statements.

<!-- PageNumber="95" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 8" -->


## Income Taxes - Uncertain Tax Positions - Refer to Note 12 to the financial statements


### Critical Audit Matter Description

The Company's long-term income taxes liability includes uncertain tax positions related to transfer pricing issues
that remain unresolved with the Internal Revenue Service ("IRS"). The Company remains under IRS audit, or
subject to IRS audit, for tax years subsequent to 2003. In the current fiscal year, the Company received Notices
of Proposed Adjustments ("NOPAs") for the tax years 2004 to 2013, primarily related to intercompany transfer
pricing. While the Company has settled a portion of the IRS audits, resolution of the remaining matters could
have a material impact on the Company's financial statements.

Conclusions on recognizing and measuring uncertain tax positions involve significant estimates and management
judgment and include complex considerations of the Internal Revenue Code, related regulations, tax case laws,
and prior-year audit settlements. Given the complexity and the subjective nature of certain transfer pricing issues
that remain unresolved with the IRS, evaluating management's estimates relating to their determination of
uncertain tax positions required extensive audit effort and a high degree of auditor judgment, including
involvement of our tax specialists.


### How the Critical Audit Matter Was Addressed in the Audit

Our principal audit procedures to evaluate management's estimates of uncertain tax positions related to
unresolved transfer pricing issues included the following:

. We evaluated the appropriateness and consistency of management's methods and assumptions used in the
identification, recognition, measurement, and disclosure of uncertain tax positions, which included testing the
effectiveness of the related internal controls.

. We read and evaluated management's documentation, including relevant accounting policies and information
obtained by management from outside tax specialists, that detailed the basis of the uncertain tax positions.

. We tested the reasonableness of management's judgments regarding the future resolution of the uncertain
tax positions, including an evaluation of the technical merits of the uncertain tax positions.

. For those uncertain tax positions that had not been effectively settled, we evaluated whether management
had appropriately considered new information, including the NOPAs received in the current fiscal year, that
could significantly change the recognition, measurement, or disclosure of the uncertain tax positions.

. We evaluated the reasonableness of management's estimates by considering how tax law, including statutes,
regulations, and case law, impacted management's judgments.


## Business Combinations - Estimate for Valuation of Acquired Intangible Assets - Refer to Note 8 to the financial statements


### Critical Audit Matter Description

On October 13, 2023, the Company completed the acquisition of Activision Blizzard, Inc. The Company
accounted for the Activision Blizzard, Inc., acquisition as a business combination and, accordingly, allocated the
purchase price to the assets acquired and liabilities assumed based on their respective estimated fair values as
of the date of acquisition. Identifiable intangible assets acquired included marketing-related intangible assets,
technology-based intangible assets, and customer-related intangible assets. The excess of the purchase
consideration over the fair value of identifiable assets acquired and liabilities assumed was recorded as goodwill.

We identified the fair value determination of certain marketing-related and technology-based intangible assets
for the business combination as a critical audit matter due to the significant judgment required in determining
their estimated fair values. Management's estimates of fair value included assumptions for revenue and expense
forecasts and the selection of appropriate discount rates. There was a high degree of auditor judgment and
subjectivity in applying audit procedures and evaluating the significant assumptions relating to the estimates,
including involvement of our fair value specialists.

How the Critical Audit Matter Was Addressed in the Audit

<!-- PageNumber="96" -->
<!-- PageBreak -->


# PART II Item 8

Our audit procedures related to management's estimates of the fair value of certain marketing-related and
technology-based intangible assets acquired included the following, among others:

· We tested the operating effectiveness of internal controls over the business combination, including internal
controls over the revenue and expense forecasts and the selection of appropriate discount rates.

· We assessed the knowledge, skills, abilities, and objectivity of management's valuation specialist and
evaluated the work performed.

· When assessing the reasonableness of assumptions related to forecasted revenue and expenses, we
evaluated whether the assumptions used were reasonable considering historical financial information of
Activision Blizzard, Inc., and the Company's forecasted financial information.

· With the assistance of our fair value specialists, we evaluated the reasonableness of the discount rates by:

\- Testing the source information underlying the discount rates and testing the mathematical accuracy of the
calculations.

\- Developing a range of independent estimates and comparing those to the discount rates selected by
management.

/s/ DELOITTE & TOUCHE LLP

Seattle, Washington
July 30, 2024

We have served as the Company's auditor since 1983.

<!-- PageNumber="97" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 9, 9A" -->


# ITEM 9. CHANGES IN AND DISAGREEMENTS WITH ACCOUNTANTS ON ACCOUNTING AND FINANCIAL DISCLOSURE

Not applicable.


## ITEM 9A. CONTROLS AND PROCEDURES

Under the supervision and with the participation of our management, including the Chief Executive Officer and Chief
Financial Officer, we have evaluated the effectiveness of our disclosure controls and procedures as required by
Exchange Act Rule 13a-15(b) as of the end of the period covered by this report. Based on that evaluation, the Chief
Executive Officer and Chief Financial Officer have concluded that these disclosure controls and procedures are
effective.


### REPORT OF MANAGEMENT ON INTERNAL CONTROL OVER FINANCIAL REPORTING

Our management is responsible for establishing and maintaining adequate internal control over financial reporting for
the Company. Internal control over financial reporting is a process to provide reasonable assurance regarding the
reliability of our financial reporting for external purposes in accordance with accounting principles generally accepted
in the United States of America. Internal control over financial reporting includes maintaining records that in reasonable
detail accurately and fairly reflect our transactions; providing reasonable assurance that transactions are recorded as
necessary for preparation of our consolidated financial statements; providing reasonable assurance that receipts and
expenditures of company assets are made in accordance with management authorization; and providing reasonable
assurance that unauthorized acquisition, use, or disposition of company assets that could have a material effect on our
consolidated financial statements would be prevented or detected on a timely basis. Because of its inherent limitations,
internal control over financial reporting is not intended to provide absolute assurance that a misstatement of our
consolidated financial statements would be prevented or detected.

Management conducted an evaluation of the effectiveness of our internal control over financial reporting based on the
framework in Internal Control - Integrated Framework (2013) issued by the Committee of Sponsoring Organizations of
the Treadway Commission. Our assessment of, and conclusion on, the effectiveness of internal control over financial
reporting did not include the internal controls of Activision Blizzard, Inc., acquired on October 13, 2023, which is
included in our consolidated financial statements since the date of acquisition and represented less than 1% of our
total assets as of June 30, 2024 after excluding goodwill and intangible assets acquired, and 2% of our total revenues
for the year ended June 30, 2024. Based on this evaluation, management concluded that the Company's internal
control over financial reporting was effective as of June 30, 2024. There were no changes in our internal control over
financial reporting during the quarter ended June 30, 2024 that have materially affected, or are reasonably likely to
materially affect, our internal control over financial reporting. Deloitte & Touche LLP has audited our internal control
over financial reporting as of June 30, 2024; their report is included in Item 9A.

<!-- PageNumber="98" -->
<!-- PageBreak -->

<!-- PageHeader="PART II Item 9A" -->


### REPORT OF INDEPENDENT REGISTERED PUBLIC ACCOUNTING FIRM


#### To the Stockholders and the Board of Directors of Microsoft Corporation


##### Opinion on Internal Control over Financial Reporting

We have audited the internal control over financial reporting of Microsoft Corporation and subsidiaries (the
"Company") as of June 30, 2024, based on criteria established in Internal Control - Integrated Framework (2013)
issued by the Committee of Sponsoring Organizations of the Treadway Commission (COSO). In our opinion, the
Company maintained, in all material respects, effective internal control over financial reporting as of June 30,
2024, based on criteria established in Internal Control - Integrated Framework (2013) issued by COSO.

We have also audited, in accordance with the standards of the Public Company Accounting Oversight Board
(United States) (PCAOB), the consolidated financial statements as of and for the year ended June 30, 2024, of
the Company and our report dated July 30, 2024, expressed an unqualified opinion on those financial statements.

As described in the Report of Management on Internal Control over Financial Reporting, management excluded
from its assessment the internal control over financial reporting at Activision Blizzard, Inc., which was acquired
on October 13, 2023, and whose financial statements constitute less than 1 percent of total assets as of June
30, 2024 after excluding goodwill and intangible assets acquired, and 2 percent of total revenues for the year
ended June 30, 2024. Accordingly, our audit did not include the internal control over financial reporting at
Activision Blizzard, Inc.


##### Basis for Opinion

The Company's management is responsible for maintaining effective internal control over financial reporting and
for its assessment of the effectiveness of internal control over financial reporting, included in the accompanying
Report of Management on Internal Control over Financial Reporting. Our responsibility is to express an opinion
on the Company's internal control over financial reporting based on our audit. We are a public accounting firm
registered with the PCAOB and are required to be independent with respect to the Company in accordance with
the U.S. federal securities laws and the applicable rules and regulations of the Securities and Exchange
Commission and the PCAOB.

We conducted our audit in accordance with the standards of the PCAOB. Those standards require that we plan
and perform the audit to obtain reasonable assurance about whether effective internal control over financial
reporting was maintained in all material respects. Our audit included obtaining an understanding of internal
control over financial reporting, assessing the risk that a material weakness exists, testing and evaluating the
design and operating effectiveness of internal control based on the assessed risk, and performing such other
procedures as we considered necessary in the circumstances. We believe that our audit provides a reasonable
basis for our opinion.


##### Definition and Limitations of Internal Control over Financial Reporting

A company's internal control over financial reporting is a process designed to provide reasonable assurance
regarding the reliability of financial reporting and the preparation of financial statements for external purposes in
accordance with generally accepted accounting principles. A company's internal control over financial reporting
includes those policies and procedures that (1) pertain to the maintenance of records that, in reasonable detail,
accurately and fairly reflect the transactions and dispositions of the assets of the company; (2) provide reasonable
assurance that transactions are recorded as necessary to permit preparation of financial statements in
accordance with generally accepted accounting principles, and that receipts and expenditures of the company
are being made only in accordance with authorizations of management and directors of the company; and (3)
provide reasonable assurance regarding prevention or timely detection of unauthorized acquisition, use, or
disposition of the company's assets that could have a material effect on the financial statements.

Because of its inherent limitations, internal control over financial reporting may not prevent or detect
misstatements. Also, projections of any evaluation of effectiveness to future periods are subject to the risk that
controls may become inadequate because of changes in conditions, or that the degree of compliance with the
policies or procedures may deteriorate.

/s/ DELOITTE & TOUCHE LLP
Seattle, Washington
July 30, 2024

<!-- PageNumber="99" -->
<!-- PageBreak -->

<!-- PageHeader="PART II, III Item 9B, 9C, 10" -->


## ITEM 9B. OTHER INFORMATION


### Insider Trading Arrangements

None of our officers or directors, as defined in Rule 16a-1(f) of the Securities Exchange Act of 1934, adopted, modified,
or terminated a "Rule 10b5-1 trading arrangement" or a "non-Rule 10b5-1 trading arrangement," as defined in Item 408
of Regulation S-K, during the three months ended June 30, 2024.


## ITEM 9C. DISCLOSURE REGARDING FOREIGN JURISDICTIONS THAT PREVENT INSPECTIONS

Not applicable.


## PART III


### ITEM 10. DIRECTORS, EXECUTIVE OFFICERS, AND CORPORATE GOVERNANCE

A list of our executive officers and biographical information appears in Part I, Item 1 of this Form 10-K. Information
about our directors may be found under the caption "Our Director Nominees" in our Proxy Statement for the Annual
Meeting of Shareholders to be held December 10, 2024 (the "Proxy Statement"). Information about our Audit
Committee may be found under the caption "Board Committees" in the Proxy Statement. That information is
incorporated herein by reference.

We have adopted the Microsoft Finance Code of Professional Conduct (the "finance code of ethics"), a code of ethics
that applies to our Chief Executive Officer, Chief Financial Officer, Chief Accounting Officer, and other finance
organization employees. The finance code of ethics is publicly available on our website at
https://aka.ms/FinanceCodeProfessionalConduct. If we make any substantive amendments to the finance code of
ethics or grant any waiver, including any implicit waiver, from a provision of the code to our Chief Executive Officer,
Chief Financial Officer, or Chief Accounting Officer, we will disclose the nature of the amendment or waiver on that
website or in a report on Form 8-K.

We have adopted insider trading policies and procedures applicable to our directors, officers, and employees, and
have implemented processes for the company, that we believe are reasonably designed to promote compliance with
insider trading laws, rules, and regulations, and the Nasdaq Stock Market LLC listing standards.

Our General Insider Trading Policy prohibits our employees and related persons and entities from trading in securities
of Microsoft and other companies while in possession of material, nonpublic information. Our General Insider Trading
Policy also prohibits our employees from disclosing material, nonpublic information Microsoft, or another publicly traded
company, to others who may trade on the basis of that information. A copy of our General Insider Trading Policy is
filed as Exhibit 19.1 to this Form 10-K.

Our Restricted Trading Window Policy requires that certain officers of the company (corporate vice presidents and
above) and other designated employees only transact in Microsoft securities during an open window period, subject to
limited exceptions. In addition, certain officers of the company are required to obtain approval in advance of
transactions in Microsoft securities. A copy of our Restricted Trading Window Policy is filed as Exhibit 19.2 to this Form
10-K.

Our executive officers and directors must also comply with additional trading restrictions. A copy of our Insider Trading
Compliance and Preclearance Policies for Section 16 Officers and Directors of Microsoft is filed as Exhibit 19.3 to this
Form 10-K.

<!-- PageNumber="100" -->
<!-- PageBreak -->

<!-- PageHeader="PART III Item 11, 12, 13, 14" -->


### ITEM 11. EXECUTIVE COMPENSATION

The information in the Proxy Statement set forth under the captions "Director Compensation," "Named Executive Officer
Compensation," "Compensation Committee Report," and, if required, "Compensation Committee Interlocks and Insider
Participation," is incorporated herein by reference.


### ITEM 12. SECURITY OWNERSHIP OF CERTAIN BENEFICIAL OWNERS AND MANAGEMENT AND RELATED STOCKHOLDER MATTERS

The information in the Proxy Statement set forth under the captions "Stock Ownership Information," "Principal
Shareholders," and "Equity Compensation Plan Information" is incorporated herein by reference.


### ITEM 13. CERTAIN RELATIONSHIPS AND RELATED TRANSACTIONS, AND DIRECTOR INDEPENDENCE

The information set forth in the Proxy Statement under the captions "Director Independence Guidelines" and "Certain
Relationships and Related Transactions" is incorporated herein by reference.


### ITEM 14. PRINCIPAL ACCOUNTANT FEES AND SERVICES

Information concerning fees and services provided by our principal accountant, Deloitte & Touche LLP (PCAOB ID No.
34), appears in the Proxy Statement under the headings "Fees Billed by Deloitte & Touche" and "Policy on Audit
Committee Pre-Approval of Audit and Permissible Non-Audit Services of Independent Auditor" and is incorporated
herein by reference.

<!-- PageNumber="101" -->
<!-- PageBreak -->

<!-- PageHeader="PART IV Item 15" -->


# PART IV


## ITEM 15. EXHIBIT AND FINANCIAL STATEMENT SCHEDULES


### (a) Financial Statements and Schedules

The financial statements are set forth under Part II, Item 8 of this Form 10-K, as indexed below. Financial statement
schedules have been omitted since they either are not required, not applicable, or the information is otherwise included.


<table>
<tr>
<th>Index to Financial Statements</th>
<th>Page</th>
</tr>
<tr>
<td>Income Statements</td>
<td>55</td>
</tr>
<tr>
<td>Comprehensive Income Statements</td>
<td>56</td>
</tr>
<tr>
<td>Balance Sheets</td>
<td>57</td>
</tr>
<tr>
<td>Cash Flows Statements</td>
<td>58</td>
</tr>
<tr>
<td>Stockholders' Equity Statements</td>
<td>59</td>
</tr>
<tr>
<td>Notes to Financial Statements</td>
<td>60</td>
</tr>
<tr>
<td>Report of Independent Registered Public Accounting Firm</td>
<td>94</td>
</tr>
</table>


### (b) Exhibit Listing


<table>
<tr>
<th rowspan="2">Exhibit Number</th>
<th rowspan="2">Exhibit Description</th>
<th rowspan="2">Filed Herewith</th>
<th colspan="4">Incorporated by Reference</th>
</tr>
<tr>
<th>Form</th>
<th>Period Ending</th>
<th>Exhibit</th>
<th>Filing Date</th>
</tr>
<tr>
<td>3.1</td>
<td>Amended and Restated Articles of Incorporation of Microsoft Corporation</td>
<td></td>
<td>8-K</td>
<td></td>
<td>3.1</td>
<td>12/1/2016</td>
</tr>
<tr>
<td>3.2</td>
<td>Bylaws of Microsoft Corporation</td>
<td></td>
<td>8-K</td>
<td></td>
<td>3.2</td>
<td>7/3/2023</td>
</tr>
<tr>
<td>4.1</td>
<td>Indenture, dated as of May 18, 2009, between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as Trustee ("Base Indenture")</td>
<td></td>
<td>S-3ASR</td>
<td></td>
<td>4.1</td>
<td>10/29/2015</td>
</tr>
<tr>
<td>4.2</td>
<td>Form of First Supplemental Indenture for 2.95% Notes due 2014, 4.20% Notes due 2019, and 5.20% Notes due 2039, dated as of May 18, 2009, between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as Trustee, to the Base Indenture</td>
<td></td>
<td>8-K</td>
<td></td>
<td>4.2</td>
<td>5/15/2009</td>
</tr>
</table>


<!-- PageNumber="102" -->
<!-- PageBreak -->


# PART IV Item 15


<table>
<tr>
<th rowspan="2">Exhibit Number</th>
<th rowspan="2">Exhibit Description</th>
<th rowspan="2">Filed Herewith</th>
<th colspan="4">Incorporated by Reference</th>
</tr>
<tr>
<th>Form</th>
<th>Period Ending</th>
<th>Exhibit</th>
<th>Filing Date</th>
</tr>
<tr>
<td>4.5</td>
<td rowspan="2">Form of Second Supplemental Indenture for 0.875% Notes due 2013, 1.625% Notes due 2015, 3.00% Notes due 2020, and 4.50% Notes due 2040, dated as of September 27, 2010, between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as Trustee, to the Indenture, dated as of May 18, 2009, between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as Trustee Third Supplemental Indenture for 2.500% Notes due 2016, 4.000% Notes due 2021, and 5.300% Notes due 2041, dated as of February 8, 2011, between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as Trustee, to the Indenture, dated as of May 18, 2009, between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as Trustee</td>
<td rowspan="2"></td>
<td rowspan="2">8-K $8 - K$</td>
<td rowspan="2"></td>
<td rowspan="2">4.2 4.2</td>
<td>9/27/2010</td>
</tr>
<tr>
<td>4.6</td>
<td>2/8/2011</td>
</tr>
</table>


<!-- PageNumber="103" -->
<!-- PageBreak -->


## PART IV Item 15


<table>
<tr>
<th rowspan="2">Exhibit Number</th>
<th rowspan="2">Exhibit Description</th>
<th rowspan="2">Filed Herewith</th>
<th colspan="4">Incorporated by Reference</th>
</tr>
<tr>
<th>Form</th>
<th>Period Ending</th>
<th>Exhibit</th>
<th>Filing Date</th>
</tr>
<tr>
<td>4.7</td>
<td>Fourth Supplemental Indenture for 0.875% Notes due 2017, 2.125% Notes due 2022, and 3.500% Notes due 2042, dated as of November 7, 2012, between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as Trustee, to the Indenture, dated as of May 18, 2009, between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as Trustee</td>
<td rowspan="2"></td>
<td>8-K</td>
<td></td>
<td>4.1</td>
<td>11/7/2012</td>
</tr>
<tr>
<td>4.8</td>
<td>Fifth Supplemental Indenture for 2.625% Notes due 2033, dated as of May 2, 2013, between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as Trustee, to the Indenture, dated as of May 18, 2009, between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as Trustee</td>
<td>8-K</td>
<td></td>
<td>4.1</td>
<td>5/1/2013</td>
</tr>
<tr>
<td>4.9</td>
<td>Sixth Supplemental Indenture for 1.000% Notes due 2018, 2.375% Notes due 2023, and 3.750% Notes due 2043, dated as of May 2, 2013, between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as Trustee, to the Indenture, dated as of May 18, 2009, between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as Trustee</td>
<td></td>
<td>8-K</td>
<td></td>
<td>4.2</td>
<td>5/1/2013</td>
</tr>
</table>


<!-- PageNumber="104" -->
<!-- PageBreak -->


## PART IV Item 15


<table>
<tr>
<th rowspan="2">Exhibit Number</th>
<th rowspan="2">Exhibit Description</th>
<th rowspan="2">Filed Herewith</th>
<th colspan="4">Incorporated by Reference</th>
</tr>
<tr>
<th>Form</th>
<th>Period Ending</th>
<th>Exhibit</th>
<th>Filing Date</th>
</tr>
<tr>
<td>4.10 4.11</td>
<td>Seventh Supplemental Indenture for 2.125% Notes due 2021 and 3.125% Notes due 2028, dated as of December 6, 2013, between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as Trustee, to the Indenture, dated as of May 18, 2009, between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as Trustee Eighth Supplemental Indenture for 1.625% Notes due 2018, 3.625% Notes due 2023, and 4.875% Notes due 2043, dated as of December 6, 2013, between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as Trustee, to the Indenture, dated as of May 18, 2009, between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as Trustee</td>
<td></td>
<td>8-K 8-K</td>
<td></td>
<td>4.1 4.2</td>
<td>12/6/2013 12/6/2013</td>
</tr>
</table>


<!-- PageNumber="105" -->
<!-- PageBreak -->


### PART IV Item 15


<table>
<tr>
<th rowspan="2">Exhibit Number</th>
<th rowspan="2">Exhibit Description</th>
<th rowspan="2">Filed Herewith</th>
<th colspan="4">Incorporated by Reference</th>
</tr>
<tr>
<th>Form</th>
<th>Period Ending</th>
<th>Exhibit</th>
<th>Filing Date</th>
</tr>
<tr>
<td>4.12 4.13</td>
<td>Ninth Supplemental Indenture for 1.850% Notes due 2020, 2.375% Notes due 2022, 2.700% Notes due 2025, 3.500% Notes due 2035, 3.750% Notes due 2045, and 4.000% Notes due 2055, dated as of February 12, 2015, between Microsoft Corporation and U.S. Bank National Association, as Trustee, to the Indenture, dated as of May 18, 2009, between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as trustee Tenth Supplemental Indenture for 1.300% Notes due 2018, 2.000% Notes due 2020, 2.650% Notes due 2022, 3.125% Notes due 2025, 4.200% Notes due 2035, 4.450% Notes due 2045, and 4.750% Notes due 2055, dated as of November 3, 2015, between Microsoft Corporation and U.S. Bank National Association, as Trustee, to the Indenture, dated as of May 18, 2009, between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as trustee</td>
<td></td>
<td>8-K $8 - K$</td>
<td></td>
<td>4.1 4.1</td>
<td>$2 / 1 2 / 2 0 1 5$ 11/3/2015</td>
</tr>
</table>


<!-- PageNumber="106" -->
<!-- PageBreak -->


### PART IV Item 15


<table>
<tr>
<th rowspan="2">Exhibit Number</th>
<th rowspan="2">Exhibit Description</th>
<th rowspan="2">Filed Herewith</th>
<th colspan="4">Incorporated by Reference</th>
</tr>
<tr>
<th>Form</th>
<th>Period Ending</th>
<th>Exhibit</th>
<th>Filing Date</th>
</tr>
<tr>
<td>4.14 4.15</td>
<td>Eleventh Supplemental Indenture for 1.100% Notes due 2019, 1.550% Notes due 2021, 2.000% Notes due 2023, 2.400% Notes due 2026, 3.450% Notes due 2036, 3.700% Notes due 2046, and 3.950% Notes due 2056, dated as of August 8, 2016, between Microsoft Corporation and U.S. Bank, National Association, as Trustee, to the Indenture, dated as of May 18, 2009, between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as trustee Twelfth Supplemental Indenture for 1.850% Notes due 2020, 2.400% Notes due 2022, 2.875% Notes due 2024, 3.300% Notes due 2027, 4.100% Notes due 2037, 4.250% Notes due 2047, and 4.500% Notes due 2057, dated as of February 6, 2017, between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as Trustee, to the Indenture, dated as of May 18, 2009, between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as trustee</td>
<td></td>
<td>8-K 8-K</td>
<td></td>
<td>4.1 4.1</td>
<td>8/5/2016 2/3/2017</td>
</tr>
</table>


<!-- PageNumber="107" -->
<!-- PageBreak -->


# PART IV Item 15


<table>
<tr>
<th rowspan="2">Exhibit Number</th>
<th rowspan="2">Exhibit Description</th>
<th rowspan="2">Filed Herewith</th>
<th colspan="4">Incorporated by Reference</th>
</tr>
<tr>
<th>Form</th>
<th>Period Ending</th>
<th>Exhibit</th>
<th>Filing Date</th>
</tr>
<tr>
<td>4.16</td>
<td>Thirteenth Supplemental Indenture for 2.525% Notes due 2050 and 2.675% Notes due 2060, dated as of June 1, 2020, between Microsoft Corporation and U.S. Bank National Association, as Trustee, to the Indenture, dated as of May 18, 2009, between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as trustee</td>
<td></td>
<td>8-K</td>
<td></td>
<td>4.1</td>
<td>6/1/2020</td>
</tr>
<tr>
<td>4.17</td>
<td>Fourteenth Supplemental Indenture for 2.921% Notes due 2052 and 3.041% Notes due 2062, dated as of March 17, 2021, between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as Trustee, to the Indenture, dated as of May 18, 2009, between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as trustee</td>
<td></td>
<td>8-K</td>
<td></td>
<td>4.1</td>
<td>3/17/2021</td>
</tr>
<tr>
<td>4.18</td>
<td>Fifteenth Supplemental Indenture, dated as of November 6, 2023, by and between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as Trustee</td>
<td></td>
<td>$8 - K$</td>
<td></td>
<td>4.2</td>
<td>11/6/2023</td>
</tr>
<tr>
<td>4.19</td>
<td>Indenture, dated as of September 19, 2016, by and between Activision Blizzard, Inc. and Wells Fargo Bank, National Association, as Trustee, with respect to Activision Blizzard, Inc.'s 3.400% Senior Notes due 2026</td>
<td></td>
<td>$8 - K$</td>
<td></td>
<td>4.9</td>
<td>11/6/2023</td>
</tr>
</table>


<!-- PageNumber="108" -->
<!-- PageBreak -->


# PART IV Item 15


<table>
<tr>
<th rowspan="2">Exhibit Number</th>
<th rowspan="2">Exhibit Description</th>
<th rowspan="2">Filed Herewith</th>
<th colspan="4">Incorporated by Reference</th>
</tr>
<tr>
<th>Form</th>
<th>Period Ending</th>
<th>Exhibit</th>
<th>Filing Date</th>
</tr>
<tr>
<td>4.20</td>
<td>Base Indenture, dated as of May 26, 2017, by and between Activision Blizzard, Inc. and Wells Fargo Bank, National Association, as Trustee, with respect to Activision Blizzard, Inc.'s 3.400% Senior Notes due 2027, 1.350% Senior Notes due 2030, 4.500% Senior Notes due 2047 and 2.500% Senior Notes due 2050</td>
<td></td>
<td>8-K</td>
<td></td>
<td>4.10</td>
<td>11/6/2023</td>
</tr>
<tr>
<td>4.21</td>
<td>First Supplemental Indenture, dated as of May 26, 2017, by and between Activision Blizzard, Inc. and Wells Fargo Bank, National Association, as Trustee, with respect to Activision Blizzard, Inc.'s 3.400% Senior Notes due 2027 and 4.500% Senior Notes due 2047</td>
<td></td>
<td>8-K</td>
<td></td>
<td>4.11</td>
<td>11/6/2023</td>
</tr>
<tr>
<td>4.22</td>
<td>Second Supplemental Indenture, dated as of August 10, 2020, by and between Activision Blizzard, Inc. and Wells Fargo Bank, National Association, as Trustee, with respect to Activision Blizzard, Inc.'s 1.350% Senior Notes due 2030 and 2.500% Senior Notes due 2050</td>
<td></td>
<td>$8 - K$</td>
<td></td>
<td>4.12</td>
<td>11/6/2023</td>
</tr>
<tr>
<td>4.23</td>
<td>First Supplemental Indenture, dated as of October 27, 2023, by and between Activision Blizzard, Inc. and Computershare Trust Company, N.A., with respect to Activision Blizzard, Inc.'s 3.400% Senior Notes due 2026</td>
<td></td>
<td>8-K</td>
<td></td>
<td>4.13</td>
<td>11/6/2023</td>
</tr>
</table>


<!-- PageNumber="109" -->
<!-- PageBreak -->


## PART IV Item 15


<table>
<tr>
<th rowspan="2">Exhibit Number</th>
<th rowspan="2">Exhibit Description</th>
<th rowspan="2">Filed Herewith</th>
<th colspan="4">Incorporated by Reference</th>
</tr>
<tr>
<th>Form</th>
<th>Period Ending</th>
<th>Exhibit</th>
<th>Filing Date</th>
</tr>
<tr>
<td>4.24</td>
<td>Third Supplemental Indenture, dated as of October 27, 2023, by and between Activision Blizzard, Inc. and Computershare Trust Company, N.A., with respect to Activision Blizzard, Inc.'s 3.400% Senior Notes due 2027 and 4.500% Senior Notes due 2047</td>
<td></td>
<td>8-K</td>
<td></td>
<td>4.14</td>
<td>11/6/2023</td>
</tr>
<tr>
<td>4.25</td>
<td>Fourth Supplemental Indenture, dated as of October 27, 2023, by and between Activision Blizzard, Inc. and Computershare Trust Company, N.A., with respect to Activision Blizzard, Inc.'s 1.350% Senior Notes due 2030 and 2.500% Senior Notes due 2050</td>
<td></td>
<td>8-K</td>
<td></td>
<td>4.15</td>
<td>11/6/2023</td>
</tr>
<tr>
<td>4.26</td>
<td>Description of Securities</td>
<td>X</td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>10.1*</td>
<td>Microsoft Corporation 2001 Stock Plan</td>
<td></td>
<td>10-Q</td>
<td>9/30/2016</td>
<td>10.1</td>
<td>10/20/2016</td>
</tr>
<tr>
<td>10.4*</td>
<td>Microsoft Corporation Employee Stock Purchase Plan</td>
<td></td>
<td>10-K</td>
<td>6/30/2012</td>
<td>10.4</td>
<td>7/26/2012</td>
</tr>
<tr>
<td>$1 0 . 5 ^ { * }$</td>
<td>Microsoft Corporation Deferred Compensation Plan</td>
<td>$X$</td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>$1 0 . 6 ^ { * }$</td>
<td>Microsoft Corporation 2017 Stock Plan</td>
<td></td>
<td>DEF14A</td>
<td></td>
<td>Annex C</td>
<td>10/16/2017</td>
</tr>
<tr>
<td>10.7*</td>
<td>Form of Stock Award Agreement Under the Microsoft Corporation 2017 Stock Plan</td>
<td></td>
<td>10-Q</td>
<td>3/31/2018</td>
<td>10.26</td>
<td>4/26/2018</td>
</tr>
<tr>
<td>10.8*</td>
<td>Form of Performance Stock Award Agreement Under the Microsoft Corporation 2017 Stock Plan</td>
<td></td>
<td>10-Q</td>
<td>3/31/2018</td>
<td>10.27</td>
<td>4/26/2018</td>
</tr>
</table>


<!-- PageNumber="110" -->
<!-- PageBreak -->


## PART IV Item 15


<table>
<tr>
<th rowspan="2">Exhibit Number</th>
<th rowspan="2">Exhibit Description</th>
<th rowspan="2">Filed Herewith</th>
<th colspan="4">Incorporated by Reference</th>
</tr>
<tr>
<th>Form</th>
<th>Period Ending</th>
<th>Exhibit</th>
<th>Filing Date</th>
</tr>
<tr>
<td>10.9</td>
<td>Amended and Restated Officers' Indemnification Trust Agreement between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as trustee</td>
<td></td>
<td>10-Q</td>
<td>9/30/2016</td>
<td>10.12</td>
<td>10/20/2016</td>
</tr>
<tr>
<td>10.10</td>
<td>Assumption of Beneficiaries' Representative Obligations Under Amended and Restated Officers' Indemnification Trust Agreement</td>
<td></td>
<td>10-K</td>
<td>6/30/2020</td>
<td>10.25</td>
<td>7/30/2020</td>
</tr>
<tr>
<td>10.11</td>
<td>Form of Indemnification Agreement and Amended and Restated Directors' Indemnification Trust Agreement between Microsoft Corporation and The Bank of New York Mellon Trust Company, N.A., as trustee</td>
<td></td>
<td>10-K</td>
<td>6/30/2019</td>
<td>10.13</td>
<td>8/1/2019</td>
</tr>
<tr>
<td>10.12</td>
<td>Assumption of Beneficiaries' Representative Obligations Under Amended and Restated Directors' Indemnification Trust Agreement</td>
<td></td>
<td>10-K</td>
<td>6/30/2020</td>
<td>10.26</td>
<td>7/30/2020</td>
</tr>
<tr>
<td>10.14*</td>
<td>Microsoft Corporation Deferred Compensation Plan for Non-Employee Directors</td>
<td></td>
<td>10-Q</td>
<td>12/31/2017</td>
<td>10.14</td>
<td>1/31/2018</td>
</tr>
<tr>
<td>10.15*</td>
<td>Microsoft Corporation Executive Incentive Plan</td>
<td></td>
<td>$8 - K$</td>
<td></td>
<td>10.1</td>
<td>9/19/2018</td>
</tr>
<tr>
<td>10.19*</td>
<td>Microsoft Corporation Executive Incentive Plan</td>
<td></td>
<td>10-Q</td>
<td>9/30/2016</td>
<td>10.17</td>
<td>10/20/2016</td>
</tr>
<tr>
<td>10.20*</td>
<td>Form of Executive Incentive Plan (Executive Officer SAs) Stock Award Agreement under the Microsoft Corporation 2001 Stock Plan</td>
<td></td>
<td>10-Q</td>
<td>9/30/2016</td>
<td>10.18</td>
<td>10/20/2016</td>
</tr>
</table>


<!-- PageNumber="111" -->
<!-- PageBreak -->


# PART IV Item 15


<table>
<tr>
<th rowspan="2">Exhibit Number</th>
<th rowspan="2">Exhibit Description</th>
<th rowspan="2">Filed Herewith</th>
<th colspan="4">Incorporated by Reference</th>
</tr>
<tr>
<th>Form</th>
<th>Period Ending</th>
<th>Exhibit</th>
<th>Filing Date</th>
</tr>
<tr>
<td>10.21*</td>
<td>Form of Executive Incentive Plan Performance Stock Award Agreement under the Microsoft Corporation 2001 Stock Plan</td>
<td></td>
<td>10-Q</td>
<td>9/30/2016</td>
<td>10.25</td>
<td>10/20/2016</td>
</tr>
<tr>
<td>10.22*</td>
<td>Senior Executive Severance Benefit Plan</td>
<td></td>
<td>10-Q</td>
<td>9/30/2016</td>
<td>10.22</td>
<td>10/20/2016</td>
</tr>
<tr>
<td>10.23*</td>
<td>Offer Letter, dated February 3, 2014, between Microsoft Corporation and Satya Nadella</td>
<td></td>
<td>$8 - K$</td>
<td></td>
<td>10.1</td>
<td>2/4/2014</td>
</tr>
<tr>
<td>10.24*</td>
<td>Long-Term Performance Stock Award Agreement between Microsoft Corporation and Satya Nadella</td>
<td></td>
<td>10-Q</td>
<td>12/31/2014</td>
<td>10.24</td>
<td>1/26/2015</td>
</tr>
<tr>
<td>10.25*</td>
<td>Offer Letter, dated October 25, 2020, between Microsoft Corporation and Christopher Young</td>
<td></td>
<td>10-Q</td>
<td>9/30/2021</td>
<td>10.27</td>
<td>10/26/2021</td>
</tr>
<tr>
<td>19.1</td>
<td>General Insider Trading Policy</td>
<td>☒ X</td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>19.2</td>
<td>Restricted Trading Window Policy</td>
<td>$x$ ☒</td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>19.3</td>
<td>Insider Trading Compliance and Preclearance Policies for Section 16 Officers and Directors of Microsoft</td>
<td>$x$ ☒</td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>21</td>
<td>Subsidiaries of Registrant</td>
<td>☒ X</td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>23.1</td>
<td>Consent of Independent Registered Public Accounting Firm</td>
<td>$X$ ☒</td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>31.1</td>
<td>Certification of Chief Executive Officer Pursuant to Section 302 of the Sarbanes-Oxley Act of 2002</td>
<td>$x$ ☒</td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>31.2</td>
<td>Certification of Chief Financial Officer Pursuant to Section 302 of the Sarbanes-Oxley Act of 2002</td>
<td>☒ X</td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
</table>


<!-- PageNumber="112" -->
<!-- PageBreak -->


## PART IV Item 15


<table>
<tr>
<th rowspan="2">Exhibit Number</th>
<th rowspan="2">Exhibit Description</th>
<th rowspan="2">Filed Herewith</th>
<th colspan="4">Incorporated by Reference</th>
</tr>
<tr>
<th>Form</th>
<th>Period Ending</th>
<th>Exhibit</th>
<th>Filing Date</th>
</tr>
<tr>
<td>32.1 **</td>
<td>Certification of Chief Executive Officer Pursuant to Section 906 of the Sarbanes-Oxley Act of 2002</td>
<td>$x$ ☒</td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>32.2 **</td>
<td>Certification of Chief Financial Officer Pursuant to Section 906 of the Sarbanes-Oxley Act of 2002</td>
<td>$x$ ☒</td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>97.1*</td>
<td>Microsoft Corporation Executive Compensation Recovery Policy</td>
<td>$x$ ☒</td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>101.INS</td>
<td>Inline XBRL Instance Document-the instance document does not appear in the Interactive Data File as its XBRL tags are embedded within the Inline XBRL document</td>
<td>$x$ ☒</td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>101.SCH</td>
<td>Inline XBRL Taxonomy Extension Schema With Embedded Linkbase Documents</td>
<td>X ☒</td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td>104</td>
<td>Cover page formatted as Inline XBRL and contained in Exhibit 101</td>
<td>X ☒</td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
</table>

\* Indicates a management contract or compensatory plan or arrangement.
Furnished, not filed.


<!-- PageNumber="113" -->
<!-- PageBreak -->


# PART IV Item 16


## ITEM 16. FORM 10-K SUMMARY

None.

<!-- PageNumber="114" -->
<!-- PageBreak -->


### SIGNATURES

Pursuant to the requirements of Section 13 or 15(d) of the Securities Exchange Act of 1934, the Registrant
has duly caused this report to be signed on its behalf by the undersigned; thereunto duly authorized, in the
City of Redmond, State of Washington, on July 30, 2024.

MICROSOFT CORPORATION

/s/ ALICE L. JOLLA

Alice L. Jolla
Corporate Vice President and Chief Accounting
Officer (Principal Accounting Officer)

<!-- PageNumber="115" -->
<!-- PageBreak -->

Pursuant to the requirements of the Securities Exchange Act of 1934, this report has been signed below by the following
persons on behalf of Registrant and in the capacities indicated on July 30, 2024.

Signature

Title

/s/ SATYA NADELLA

Chairman and Chief Executive Officer (Principal
Executive Officer)

Satya Nadella

/s/ REID HOFFMAN

Director

Reid Hoffman

/s/ HUGH F. JOHNSTON
Director

Hugh F. Johnston

/s/ TERI L. LIST

Director

Teri L. List

/s/ CATHERINE MACGREGOR
Director

Catherine MacGregor

/s/ MARK A. L. MASON
Director

Mark A. L. Mason

/s/ SANDRA E. PETERSON
Lead Independent Director

Sandra E. Peterson

/s/ PENNY S. PRITZKER
Director

Penny S. Pritzker

/s/ CARLOS A. RODRIGUEZ
Director

Carlos A. Rodriguez

/s/ CHARLES W. SCHARF
Director

Charles W. Scharf

/s/ JOHN W. STANTON
Director

John W. Stanton

/s/ EMMA N. WALMSLEY
Director

Emma N. Walmsley

/s/ AMY E. HOOD

Executive Vice President and Chief Financial Officer
(Principal Financial Officer)

Amy E. Hood

/s/ ALICE L. JOLLA

Alice L. Jolla

Corporate Vice President and Chief Accounting Officer
(Principal Accounting Officer)

<!-- PageNumber="116" -->
<!-- PageBreak -->

<!-- PageHeader="Exhibit 4.26" -->


# DESCRIPTION OF THE REGISTRANT'S SECURITIES REGISTERED PURSUANT TO SECTION 12 OF THE SECURITIES EXCHANGE ACT OF 1934

As of July 30, 2024, Microsoft Corporation has three classes of securities registered under Section
12 of the Securities Exchange Act of 1934, as amended (the "Exchange Act"): (1) our Common
Stock; (2) our 3.125% Notes due 2028; and (3) our 2.625% Notes due 2033.


## Description of Common Stock

The following description of our Common Stock is a summary and does not purport to be complete. It
is subject to and qualified in its entirety by reference to our Amended and Restated Articles of
Incorporation (the "Articles of Incorporation") and our Amended and Restated Bylaws (the "Bylaws"),
each of which are incorporated by reference as an exhibit to the Annual Report on Form 10-K of
which this Exhibit 4.1 is a part. We encourage you to read our Articles of Incorporation, our Bylaws
and the applicable provisions of Washington Business Corporation Act, Title 23B of the Revised
Code of Washington, for additional information.


### Authorized Capital Shares

Our authorized capital shares consist of 24,000,000,000 shares of common stock, $0.00000625 par
value per share ("Common Stock"), and 100,000,000 shares of series preferred stock, $0.01 par
value per share ("Preferred Stock"). The outstanding shares of our Common Stock are fully paid and
nonassessable.


### Voting Rights

Holders of Common Stock are entitled to one vote per share on all matters voted on by the
stockholders, including the election of directors. Our Common Stock does not have cumulative voting
rights.


### Dividend Rights

Subject to the rights of holders of outstanding shares of Preferred Stock, if any, the holders of
Common Stock are entitled to receive dividends, if any, as may be declared from time to time by the
Board of Directors in its discretion out of funds legally available for the payment of dividends.


### Liquidation Rights

Subject to any preferential rights of outstanding shares of Preferred Stock, holders of Common Stock
will share ratably in all assets legally available for distribution to our stockholders in the event of
dissolution.


### Other Rights and Preferences

Our Common Stock has no sinking fund or redemption provisions or preemptive, conversion or
exchange rights. Holders of Common Stock may act by unanimous written consent.


### Listing

The Common Stock is traded on The Nasdaq Stock Market LLC under the trading symbol "MSFT."


### Description of the Notes

The following description of our 3.125% Notes due 2028 (the "2028 Notes") and our 2.625% Notes
due 2033 (the "2033 Notes" and, together with the 2028 Notes, the "notes"), is a summary and does
not purport to be complete. It is subject to and qualified in its entirety by reference to the indenture,

<!-- PageBreak -->

dated as of May 18, 2009 (the "Base Indenture"), between Microsoft Corporation and The Bank of
New York Mellon Trust Company, N.A., as trustee, as supplemented in the case of the 2028 Notes,
by the seventh supplemental indenture, dated as of December 6, 2013, and, as supplemented in the
case of the 2033 Notes, by the fifth supplemental indenture dated as of May 1, 2013 (the Base
Indenture, as supplemented by the fifth and seventh supplemental indentures, the "indenture"),
which are incorporated by reference as exhibits to the Annual Report on Form 10-K of which this
Exhibit 4.1 is a part. The 2028 Notes and the 2033 Notes are each traded on The New York Stock
Exchange under the bond trading symbols of "MSFT28" and "MSFT33," respectively.

We encourage you to read the above referenced indenture, as supplemented, for additional
information.


### General

The following is a description of certain of the specific terms and conditions of the indenture
supplements with respect to each of the notes.

The 2033 Notes were initially issued in an €550,000,000 aggregate principal amount. The 2028
Notes were initially issued in an €1,750,000,000 aggregate principal amount. We are permitted to
issue additional notes of each series of notes without the consent of the holders of that series of
notes, but we will not issue such additional notes unless they are fungible for U.S. federal income tax
purposes with the relevant series of notes offered. As of July 30, 2024 no such additional notes have
been issued.

The notes are senior unsecured obligations and rank equally with our other unsecured and
unsubordinated debt from time to time outstanding.

The maturity date of the 2028 Notes is December 6, 2028. The maturity date of the 2033 Notes is
May 2, 2033.

The notes will be subject to legal defeasance and covenant defeasance as provided under the
"Discharge, Defeasance and Covenant Defeasance" section set forth below.

The notes were issued in a form of one or more fully registered global securities, without coupons, in
denominations of €100,000 in principal amount and integral multiples of €1,000 in excess thereof.

The notes are not redeemable at the option of the holder prior to maturity and will not benefit from
any sinking fund.


### Interest and Principal

The 2028 Notes bear interest from December 6, 2013 at a fixed interest rate of 3.125% per annum.
The 2033 Notes bear interest from May 2, 2013 at a fixed interest rate of 2.625% per annum. Interest
is paid annually on December 6 for the 2028 Notes and on May 2 for the 2033 Notes, and on the
maturity date of each series of notes (the "interest payment date"). We will pay interest on the notes
to the persons in whose names the notes are registered at the close of business on the May 1 or
December 5 or May 1, as applicable (in each case, whether or not a business day) immediately
preceding the related interest payment date. Interest on the notes will be computed on the basis of
the actual number of days in the period for which interest is being calculated and the actual number
of days from and including the last date on which interest was paid on the notes (or the original issue
date if no interest has been paid or duly provided for on the notes), to but excluding the next date on
which interest is paid or duly provided for. This payment convention is referred to as Actual/Actual
(ICMA) as defined in the rulebook of the International Capital Market Association.

We will pay the principal of and interest on each note to the registered holder in euro in immediately
available funds; provided that for the 2028 Notes, if the euro is unavailable to us due to the
imposition of exchange controls or other circumstances beyond our control or if the euro is no longer
being used by the then member states of the European Monetary Union that have adopted the euro
as their currency or for the settlement of transactions by public institutions of or within the

<!-- PageBreak -->

international banking community, then all payments in respect of the notes will be made in U.S.
dollars until the euro is again available to us or so used. The amount payable on any date in euro will
be converted into U.S. dollars at the rate mandated by the U.S. Federal Reserve Board as of the
close of business on the second business day prior to the relevant payment date or, in the event the
U.S. Federal Reserve Board has not mandated a rate of conversion, on the basis of the most recent
U.S. dollar/euro exchange rate published in The Wall Street Journal on or prior to the second
business day prior to the relevant payment date. Any payment in respect of the notes so made in
U.S. dollars will not constitute an event of default under the notes or the indenture governing the
notes. So long as the notes are in book-entry form, we will make payments of principal and interest
through the London paying agent described below.

Interest payable on any interest payment date for a series of notes or the maturity date for that series
of notes will be the amount of interest accrued from, and including, the next preceding interest
payment date for such notes in respect of which interest has been paid or duly provided for (or from
and including the original issue date, if no interest has been paid or duly provided for with respect to
the notes of that series) to, but excluding, such interest payment date or maturity date, as the case
may be. If any interest payment date falls on a day that is not a business day, the interest payment
will be made on the next succeeding business day, and we will not be liable for any additional
interest as a result of the delay in payment. If a maturity date falls on a day that is not a business
day, the related payment of principal and interest will be made on the next succeeding business day,
and no interest will accrue on the amounts so payable for the period from and after such date to the
next succeeding business day. The term "business day" means any day, other than a Saturday or a
Sunday, (1) which is not a day on which banking institutions are authorized or obligated by law or
executive order to close in New York City or London and (2) on which the Trans-European
Automated Real-time Gross Settlement Express Transfer system (the TARGET2 system), or any
successor thereto, is open.


### Optional Redemption

At any time prior to September 6, 2028, we will have the right at our option to redeem the 2028
Notes, in whole or in part, at any time or from time to time, on at least 30 days' but not more than 60
days' prior notice mailed to the registered address of each holder of the 2028 Notes, at a redemption
price, calculated by us, equal to the greater of (1) 100% of the principal amount of the 2028 Notes to
be redeemed and (2) the sum of the present values of each remaining scheduled payment of
principal and interest on the 2028 Notes to be redeemed (exclusive of interest accrued to the date of
redemption) discounted to the redemption date on an annual basis (Actual/Actual (ICMA)) at the
applicable Bond Rate plus 20 basis points.

At any time prior to February 2, 2033, we will have the right at our option to redeem the notes, in
whole or in part, at any time or from time to time, on at least 30 days' but not more than 60 days'
prior notice mailed to the registered address of each holder of the notes, at a redemption price equal
to the greater of (1) 100% of the principal amount of the notes to be redeemed and (2) the sum of the
present values of each remaining scheduled payment of principal and interest on the notes to be
redeemed (exclusive of interest accrued to the date of redemption) discounted to the redemption
date on an annual basis (Actual/Actual (ICMA)) at the applicable Bond Rate plus 12.5 basis points.

At any time on or after September 6, 2028, we will have the right at our option to redeem the 2028
Notes, in whole or in part, on at least 30 days' but not more than 60 days' notice, at any time at a
redemption price equal to 100% of the principal amount of the 2028 Notes to be redeemed.

At any time on or after February 2, 2033, we will have the right at our option to redeem the 2033
Notes, in whole or in part, on at least 30 days' but not more than 60 days' notice, at any time at a
redemption price equal to 100% of the principal amount of the notes to be redeemed.

The redemption price for the notes will include, in each case, accrued and unpaid interest on the
principal amount of the notes to be redeemed to the redemption date. The redemption price paid for
the notes upon any such redemption will be paid in euro.

<!-- PageBreak -->

"Bond Rate" means, with respect to any redemption date, the rate per annum equal to the annual
equivalent yield to maturity or interpolated maturity (on a day count basis) of the applicable
Comparable Government Issue (computed as of the third business day immediately preceding the
redemption date), assuming a price for such Comparable Government Issue (expressed as a
percentage of its principal amount) equal to the applicable Comparable Price for such redemption
date.

"Comparable Government Issue" means the euro-denominated security issued by the German
government selected by an Independent Investment Banker as having an actual or interpolated
maturity comparable to the remaining term of the series of notes to be redeemed that would be
utilized, at the time of selection and in accordance with customary financial practice, in pricing new
issues of corporate debt securities of a comparable maturity to the remaining term of the notes of
such series.

"Comparable Price" means, with respect to any redemption date (1) the arithmetic average of the
Reference Dealer Quotations for such redemption date, after excluding the highest and lowest such
Reference Dealer Quotations or (2) if we obtain fewer than four such Reference Dealer Quotations,
the arithmetic average of all such quotations for such redemption date.

"Independent Investment Banker" means an investment bank of international standing appointed by
us.

"Reference Dealer" means a broker of, or a market maker in, the Comparable Government Issue
selected by the Independent Investment Banker.

"Reference Dealer Quotation" means, with respect to each Reference Dealer and any redemption
date, the arithmetic average, as determined by us, of the bid and asked prices for the applicable
Comparable Government Issue (expressed in each case as a percentage of its principal amount)
quoted in writing to us by such Reference Dealer at 11:00 a.m. (London time) on the third business
day preceding such redemption date.

On and after a redemption date, interest will cease to accrue on the notes called for redemption or
any portion of any series of the notes called for redemption (unless we default in the payment of the
redemption price and accrued and unpaid interest). On or before the redemption date, we will
deposit with the London paying agent money sufficient to pay the redemption price of and (unless
the redemption date shall be an interest payment date) accrued and unpaid interest to the
redemption date on the notes to be redeemed on such date. If less than all of the notes of a series
are to be redeemed, the notes of such series to be redeemed will be selected by the trustee by such
method as the trustee will deem fair and appropriate; provided, however, that no notes of a principal
amount of €100,000 or less shall be redeemed in part.


### Redemption Upon Tax Event

We may redeem the notes of any series at our option in whole, but not in part, on at least 15 days'
but not more than 60 days' notice, at a redemption price equal to 100% of their principal amount
(plus any accrued interest and additional amounts then payable with respect to the notes of that
series), if we determine that (A) as a result of any change or amendment to the laws, treaties,
regulations or rulings of the United States or any political subdivision or taxing authority thereof,
which change or amendment is announced and becomes effective after the date of the applicable
prospectus supplement, we have become obligated to pay additional amounts as described under "-
Payment of Additional Amounts" on any notes of such series or (B) after the date of the applicable
prospectus supplement, any change in the official application, enforcement or interpretation of those
laws, treaties, regulations or rulings, including a holding by a court of competent jurisdiction in the
United States or any other action, taken by any taxing authority or a court of competent jurisdiction in
the United States, whether or not such action was taken or made with respect to us, results in a
material probability that we have or will become obligated to pay additional amounts as described
under "-Payment of Additional Amounts" on any notes of such series; provided that we determine, in
our business judgment, that the obligation to pay such additional amounts cannot be avoided by use

<!-- PageBreak -->

of reasonable measures available to us, not including substitution of the obligor under the notes of
such series. Prior to the mailing of any notice of such a redemption, we will deliver to the trustee (1)
an officer's certificate stating that we are entitled to effect such a redemption and setting forth a
statement of facts showing that the conditions precedent to the right of our company to so redeem
have occurred and (2) an opinion of counsel to that effect based on that statement of facts.


### Payment of Additional Amounts

All payments of principal and interest in respect of the notes will be made free and clear of, and
without deduction or withholding for or on account of any present or future taxes, duties,
assessments or other governmental charges of whatsoever nature imposed, levied, collected,
withheld or assessed by the United States or any political subdivision or taxing authority of or in the
United States, unless such withholding or deduction is required by law.

We will pay to the beneficial owner of notes who is a Non-U.S. Person (as defined below) additional
amounts as may be necessary so that every net payment of the principal of and premium, if any, and
interest on such holder's note, after deduction or withholding for or on account of any present or
future tax, assessment or other governmental charge imposed upon that beneficial owner by the
United States or any taxing authority thereof or therein, will not be less than the amount provided in
such holder's notes to be then due and payable. We will not be required, however, to make any
payment of additional amounts for or on account of:

(a) any tax, assessment or other governmental charge that would not have been imposed but for
(1) the existence of any present or former connection (other than a connection arising solely
from the ownership of those notes or the receipt of payments in respect of those notes)
between that beneficial owner, or between a fiduciary, settlor, beneficiary of, member or
shareholder of, or possessor of a power over, that beneficial owner, if that beneficial owner is
an estate, trust, partnership or corporation, and the United States, including that beneficial
owner, or that fiduciary, settlor, beneficiary, member, shareholder or possessor, being or
having been a citizen or resident or treated as a resident of the United States or being or
having been engaged in trade or business or present in the United States or having had a
permanent establishment in the United States or (2) the presentation of a debt security for
payment on a date more than 30 days after the later of the date on which that payment
becomes due and payable and the date on which payment is duly provided for;

(b) any estate, inheritance, gift, sales, transfer, excise, personal property, wealth, interest
equalization or similar tax, assessment or other governmental charge;

(c) any tax, assessment or other governmental charge imposed on foreign personal holding
company income or by reason of that beneficial owner's past or present status as a passive
foreign investment company, a controlled foreign corporation, a foreign tax exempt
organization or a personal holding company with respect to the United States or as a
corporation that accumulates earnings to avoid U.S. federal income tax;

(d) any tax, assessment or other governmental charge which is payable otherwise than by
withholding from payment of principal of or premium, if any, or interest on such holder's notes;

(e) any tax, assessment or other governmental charge required to be withheld by any paying
agent from any payment of principal of and premium, if any, or interest on any note if that
payment can be made without withholding by any other paying agent;

(f) any tax, assessment or other governmental charge which would not have been imposed but
for the failure of a beneficial owner or any holder of notes to comply with our request to
comply with certification, information, documentation or other reporting requirements
concerning the nationality, residence, identity or connections with the United States of the
beneficial owner or any holder of the notes (including, but not limited to, the requirement to
provide Internal Revenue Service Forms W-8BEN, Forms W-8ECI, or any subsequent
versions thereof or successor thereto, and including, without limitation, any documentation
requirement under an applicable income tax treaty);

<!-- PageBreak -->

(g) any tax, assessment or other governmental charge imposed on interest received by (1) a
10% shareholder (as defined in Section 871(h)(3)(B) of the U.S. Internal Revenue Code of
1986, as amended (the "Code"), and the regulations that may be promulgated thereunder) of
our company or (2) a controlled foreign corporation that is related to us within the meaning of
Section 864(d)(4) of the Code, or (3) a bank receiving interest described in Section
881(c)(3)(A) of the Code;

(h) any withholding or deduction that is imposed on a payment to an individual and is required to

(i) be made pursuant to European Council Directive 2003/48/EC relating to the taxation of
savings, or any law implementing or complying with, or introduced in order to conform to,
such Directive;
with respect to the 2028 Notes, any taxes payable under Sections 1471 through 1474 of the
Code (or any amended or successor version of such Sections), any regulations or other
guidance thereunder, or any agreement (including any intergovernmental agreement) entered
into in connection therewith; or

(j) any combination of items (a), (b), (c), (d), (e), (f), (g), (h) and (i);

nor will we pay any additional amounts to any beneficial owner or holder of notes who is a fiduciary
or partnership to the extent that a beneficiary or settlor with respect to that fiduciary or a member of
that partnership or a beneficial owner thereof would not have been entitled to the payment of those
additional amounts had that beneficiary, settlor, member or beneficial owner been the beneficial
owner of those notes.

As used in the preceding paragraph, "Non-U.S. Person" means any corporation, partnership,
individual or fiduciary that is, for United States federal income tax purposes, a foreign corporation, a
non-resident alien individual who has not made a valid election to be treated as a United States
resident, a non-resident fiduciary of a foreign estate or trust or a foreign partnership, one or more of
the members of which is, as to the United States, a foreign corporation, a non-resident alien
individual or a non-resident fiduciary of a foreign estate or trust.


### Book-Entry and Settlement

Each series of notes was issued in the form of one or more global securities, in definitive, fully
registered form without interest coupons, each of which we refer to as a "global security." Each such
global security was deposited with The Bank of New York Mellon, as common depositary (the
"Common Depositary") and registered in the name of the Common Depositary or its nominee.

Beneficial interests in the global securities are represented, and transfers of such beneficial interest
was effected, through accounts of financial institutions acting on behalf of beneficial owners as direct
or indirect participants in Clearstream Banking, société anonyme, which we refer to as
"Clearstream," or Euroclear Bank SA/ NV, as operator of the Euroclear System, which we refer to as
"Euroclear." Investors may hold notes directly through Clearstream or Euroclear, if they are
participants in such systems, or indirectly through organizations that are participants in such
systems.

Beneficial interests in the global securities will be shown on, and transfers of beneficial interests in
the global securities are made only through, records maintained by Clearstream or Euroclear and
their participants. The London paying agent will wire payments on the notes to the Common
Depositary as the holder of the global securities. The trustee, the London paying agent and we will
treat the Common Depositary or any successor nominee to the Common Depositary as the owner of
the global securities for all purposes. Accordingly, the trustee, the London paying agent and we will
have no direct responsibility or liability to pay amounts due with respect to the global securities to
you or any other beneficial owners in the global securities. Any redemption or other notices with
respect to the notes will be sent by us directly to Clearstream or Euroclear, which will, in turn, inform
the direct participants (or the indirect participants), which will then contact you as a beneficial holder,
all in accordance with the rules of Clearstream or Euroclear, as the case may be, and the internal
procedures of the direct participant (or the indirect participant) through which beneficial interest in the
notes are held.

<!-- PageBreak -->


### Certificated Notes

Subject to certain conditions, the notes represented by the global securities are exchangeable for
certificated notes in definitive form of like tenor in minimum denominations of €100,000 principal
amount and integral multiples of €1,000 in excess thereof if:

(1) the Common Depositary notifies us that it is unwilling or unable to continue as depositary or if
the Common Depositary ceases to be eligible under the indenture and we do not appoint a
successor depository within 90 days;

(2) we determine that the notes will no longer be represented by global securities and execute
and deliver to the trustee an order to that effect; or

(3) an event of default with respect to the notes will have occurred and be continuing.

Any note that is exchangeable as above is exchangeable for certificated notes issuable in authorized
denominations and registered in such names as the Common Depositary shall direct. Subject to the
foregoing, a global security is not exchangeable, except for a global security of the same aggregate
denomination to be registered in the name of the Common Depositary or its nominee.


### Trustee, Paying Agents and Security Registrar

The Bank of New York Mellon Trust Company, N.A. is the trustee under the indenture governing the
notes. The Bank of New York Mellon Trust Company, N.A. is a national banking association
organized under and governed by the laws of the United States of America, and provides trust
services and acts as indenture trustee for numerous corporate securities issuances, including for
other series of debt securities of which we are the issuer. The Bank of New York Mellon, London
Branch, is the paying agent for the notes in London.


### Base Indenture Provisions:


#### Governing Law

The indenture and the notes are governed by, and construed in accordance with, the laws of the
State of New York.


#### Consolidation, Merger and Sale of Assets

The indenture provides that we may consolidate with or merge with or into any other person, and
may sell, transfer, or lease or convey all or substantially all of our properties and assets to another
person; provided that the following conditions are satisfied:

· we are the continuing entity, or the resulting, surviving or transferee person (the
"Successor") is a person organized and existing under the laws of the United States of
America, any state thereof or the District of Columbia and the Successor (if not us) will
expressly assume, by supplemental indenture, all of our obligations under the debt
securities and the indenture and, for each security that by its terms provides for
conversion, provide for the right to convert such security in accordance with its terms;

· immediately after giving effect to such transaction, no default or event of default under
the indenture has occurred and is continuing; and

. if requested, the trustee receives from us, an officers' certificate and an opinion of
counsel that the merger, consolidation or transfer and such supplemental indenture, as
the case may be, complies with the applicable provisions of the indenture.

If we consolidate or merge with or into any other person or sell, transfer, lease or convey all or
substantially all of our properties and assets in accordance with the indenture, the Successor will be
substituted for us in the indenture, with the same effect as if it had been an original party to the
indenture. As a result, the Successor may exercise our rights and powers under the indenture, and
we will be released from all our liabilities and obligations under the indenture and under the debt
securities.

<!-- PageBreak -->

Any substitution of the Successor for us might be deemed for federal income tax purposes to be an
exchange of the debt securities for "new" debt securities, resulting in recognition of gain or loss for
such purposes and possibly certain other adverse tax consequences to beneficial owners of the debt
securities. Holders should consult their own tax advisors regarding the tax consequences of any
such substitution.

For purposes of this covenant, "person" means any individual, corporation, partnership, limited
liability company, joint venture, association, joint-stock company, trust, unincorporated organization
or government or any agency or political subdivision thereof or any other entity.


### Events of Default

Each of the following events are defined in the indenture as an "event of default" (whatever the
reason for such event of default and whether or not it will be voluntary or involuntary or be effected
by operation of law or pursuant to any judgment, decree or order of any court or any order, rule or
regulation of any administrative or governmental body) with respect to the debt securities of any
series:

(1) default in the payment of any installment of interest on any debt securities of that series for 30
days after becoming due;

(2) default in the payment of principal of or premium, if any, on any debt securities of that series
when it becomes due and payable at its stated maturity, upon optional redemption, upon declaration
or otherwise;

(3) default in the deposit of any sinking fund payment, when and as due by the terms of any debt
securities of that series;

(4) default in the performance, or breach, of any covenant or agreement of ours in the indenture with
respect to the debt securities of that series (other than as referred to in clause (1), (2) or (3) above),
which continues for a period of 90 days after written notice to us by the trustee or to us and the
trustee by the holders of at least 25% in aggregate principal amount of the outstanding debt
securities of that series;

(5) we pursuant to or within the meaning of the Bankruptcy Law:

· commence a voluntary case or proceeding;

· consent to the entry of an order for relief against us in an involuntary case or
proceeding;

· consent to the appointment of a Custodian of us or for all or substantially all of our
property;

· make a general assignment for the benefit of our creditors;

· file a petition in bankruptcy or answer or consent seeking reorganization or relief;

· consent to the filing of such petition or the appointment of or taking possession by a
Custodian; or

· take any comparable action under any foreign laws relating to insolvency;

(6) a court of competent jurisdiction enters an order or decree under any Bankruptcy Law that:

· is for relief against us in an involuntary case, or adjudicates us insolvent or
bankrupt;

· appoints a Custodian of us or for all or substantially all of our property; or

· orders the winding-up or liquidation of us (or any similar relief is granted under any
foreign laws);

and the order or decree remains unstayed and in effect for 90 days; or

(7) any other event of default provided with respect to debt securities of that series occurs.

<!-- PageBreak -->

"Bankruptcy Law" means Title 11, United States Code or any similar federal or state or foreign law
for the relief of debtors.

"Custodian" means any custodian, receiver, trustee, assignee, liquidator or other similar official under
any Bankruptcy Law.

If an event of default with respect to debt securities of any series (other than an event of default
relating to certain events of bankruptcy, insolvency, or reorganization of us) occurs and is continuing,
the trustee by notice to us, or the holders of at least 25% in aggregate principal amount of the
outstanding debt securities of that series by notice to us and the trustee, may, and the trustee at the
request of these holders will, declare the principal of and premium, if any, and accrued and unpaid
interest on all the debt securities of that series to be due and payable. Upon such a declaration, such
principal, premium and accrued and unpaid interest will be due and payable immediately. If an event
of default relating to certain events of bankruptcy, insolvency, or reorganization of us occurs and is
continuing, the principal of and premium, if any, and accrued and unpaid interest on the debt
securities of that series will become and be immediately due and payable without any declaration or
other act on the part of the trustee or any holders.

The holders of not less than a majority in aggregate principal amount of the outstanding debt
securities of any series may rescind a declaration of acceleration and its consequences, if we have
deposited certain sums with the trustee and all events of default with respect to the debt securities of
that series, other than the non-payment of the principal or interest which have become due solely by
such acceleration, have been cured or waived, as provided in the indenture.

An event of default for a particular series of debt securities does not necessarily constitute an event
of default for any other series of debt securities issued under the indenture.

We are required to furnish the trustee annually a statement by certain of our officers to the effect
that, to the best of their knowledge, we are not in default in the fulfillment of any of our obligations
under the indenture or, if there has been a default in the fulfillment of any such obligation, specifying
each such default.

No holder of any debt securities of any series will have any right to institute any judicial or other
proceeding with respect to the indenture, or for the appointment of a receiver or trustee, or for any
other remedy unless:

(1) an event of default has occurred and is continuing and such holder has given the trustee prior
written notice of such continuing event of default with respect to the debt securities of that series;

(2) the holders of not less than 25% of the aggregate principal amount of the outstanding debt
securities of that series have requested the trustee to institute proceedings in respect of such event
of default;

(3) the trustee has been offered indemnity reasonably satisfactory to it against its costs, expenses
and liabilities in complying with such request;

(4) the trustee has failed to institute proceedings 60 days after the receipt of such notice, request and
offer of indemnity; and

(5) no direction inconsistent with such written request has been given for 60 days by the holders of a
majority in aggregate principal amount of the outstanding debt securities of that series.

The holders of a majority in aggregate principal amount of outstanding debt securities of a series will
have the right, subject to certain limitations, to direct the time, method and place of conducting any
proceeding for any remedy available to the trustee with respect to the debt securities of that series or
exercising any trust or power conferred to the trustee, and to waive certain defaults. The indenture
provides that if an event of default occurs and is continuing, the trustee will exercise such of its rights
and powers under the indenture, and use the same degree of care and skill in their exercise, as a
prudent person would exercise or use under the circumstances in the conduct of such person's own

<!-- PageBreak -->

affairs. Subject to such provisions, the trustee will be under no obligation to exercise any of its rights
or powers under the indenture at the request of any of the holders of the debt securities of a series
unless they will have offered to the trustee security or indemnity satisfactory to the trustee against
the costs, expenses and liabilities which might be incurred by it in compliance with such request.

Notwithstanding the foregoing, the holder of any debt security will have an absolute and
unconditional right to receive payment of the principal of and premium, if any, and interest on that
debt security on or after the due dates expressed in that debt security and to institute suit for the
enforcement of payment.


### Discharge, Defeasance and Covenant Defeasance

We may discharge certain obligations to holders of the debt securities of a series that have not
already been delivered to the trustee for cancellation and that either have become due and payable
or will become due and payable within one year (or scheduled for redemption within one year) by
depositing with the trustee, in trust, money in an amount sufficient to pay the entire indebtedness
including the principal and premium, if any, and interest to the date of such deposit (if the debt
securities have become due and payable) or to the maturity thereof or the redemption date of the
debt securities of that series, as the case may be. We may direct the trustee to invest such funds in
U.S. Treasury securities with a maturity of one year or less or in a money market fund that invests
solely in short-term U.S. Treasury securities.

The indenture provides that we may elect either (1) to defease and be discharged from any and all
obligations with respect to the debt securities of a series (except for, among other things, obligations
to register the transfer or exchange of the debt securities, to replace temporary or mutilated,
destroyed, lost or stolen debt securities, to maintain an office or agency with respect to the debt
securities and to hold moneys for payment in trust) ("legal defeasance") or (2) to be released from
our obligations to comply with the restrictive covenants under the indenture, and any omission to
comply with such obligations will not constitute a default or an event of default with respect to the
debt securities of a series and clauses (4) and (7) under "-Events of Default" will no longer be applied
("covenant defeasance"). Legal defeasance or covenant defeasance, as the case may be, will be
conditioned upon, among other things, the irrevocable deposit by us with the trustee, in trust, of an
amount, or U.S. government obligations, or both, applicable to the debt securities of that series
which through the scheduled payment of principal and interest in accordance with their terms will
provide money in an amount sufficient to pay the principal or premium, if any, and interest on the
debt securities on the scheduled due dates therefor.

If we effect covenant defeasance with respect to the debt securities of any series, the amount, or
U.S. government obligations, or both, on deposit with the trustee will be sufficient, in the opinion of a
nationally recognized firm of independent accountants, to pay amounts due on the debt securities of
that series at the time of the stated maturity but may not be sufficient to pay amounts due on the debt
securities of that series at the time of the acceleration resulting from such event of default. However,
we would remain liable to make payment of such amounts due at the time of acceleration.

We have elected pursuant to Section 1301 of the indenture, to have both Section 1302 (legal
defeasance) and Section 1303 (covenant defeasance) of the indenture apply to the all of the notes.

With respect to the 2028 Notes, "U.S. government obligation" means $\left( I \right) \left( x \right)$ any security which is (i) a
direct obligation of the United States of America for the payment of which the full faith and credit of
the United States of America is pledged or (ii) an obligation of a Person (as defined in the indenture)
controlled or supervised by and acting as an agency or instrumentality of the United States of
America the payment of which is unconditionally guaranteed as a full faith and credit obligation by
the United States of America, which, in either case (I)(x)(i) or (ii), is not callable or redeemable at the
option of the issuer thereof, and (y) any depositary receipt issued by a bank (as defined in Section
3(a)(2) of the Securities Act) as custodian with respect to any U.S. government obligation which is
specified in clause (x) above and held by such bank for the account of the holder of such depositary
receipt, or with respect to any specific payment of principal of or interest on any U.S. government
obligation which is so specified and held; provided that (except as required by law) such custodian is

<!-- PageBreak -->

not authorized to make any deduction from the amount payable to the holder of such depositary
receipt from any amount received by the custodian in respect of the U.S. government obligation or
the specific payment of principal or interest evidenced by such depositary receipt or (II)(x) any
security which is (i) a direct obligation of the German Government (as defined in the indenture) or (ii)
an obligation of a Person (as defined in the indenture) controlled or supervised by and acting as an
agency or instrumentality of the German Government (as defined in the indenture) the payment of
which is fully and unconditionally guaranteed by the German Government (as defined in the
indenture), the central bank of the German Government (as defined in the indenture) or a
governmental agency of the German Government (as defined in the indenture), which, in either case
(II)(x)(i) or (ii), is not callable or redeemable at the option of the issuer thereof, and (y) certificates,
depositary receipts or other instruments which evidence a direct ownership interest in obligations
described in clause (II)(x)(i) or (ii) above or in any specific principal or interest payments due in
respect thereof.

With respect to the 2033 Notes, "U.S. government obligation" means (x) any security which is (i) a
direct obligation of the United States of America for the payment of which the full faith and credit of
the United States of America is pledged or (ii) an obligation of a Person (as defined in the indenture)
controlled or supervised by and acting as an agency or instrumentality of the United States of
America the payment of which is unconditionally guaranteed as a full faith and credit obligation by
the United States of America, which, in either case (i) or (ii), is not callable or redeemable at the
option of the issuer thereof, and (y) any depositary receipt issued by a bank (as defined in Section
3(a)(2) of the Securities Act of 1933, as amended (the "Securities Act")) as custodian with respect to
any U.S. government obligation which is specified in clause (x) above and held by such bank for the
account of the holder of such depositary receipt, or with respect to any specific payment of principal
of or interest on any U.S. government obligation which is so specified and held; provided that (except
as required by law) such custodian is not authorized to make any deduction from the amount
payable to the holder of such depositary receipt from any amount received by the custodian in
respect of the U.S. government obligation or the specific payment of principal or interest evidenced
by such depositary receipt.

We will be required to deliver to the trustee an opinion of counsel that the deposit and related
defeasance will not cause the holders and beneficial owners of the debt securities of that series to
recognize income, gain or loss for federal income tax purposes. If we elect legal defeasance, that
opinion of counsel must be based upon a ruling from the U.S. Internal Revenue Service or a change
in law to that effect.

We may exercise our legal defeasance option notwithstanding our prior exercise of our covenant
defeasance option.

Exhibit 10.5


# MICROSOFT CORPORATION DEFERRED COMPENSATION PLAN (Amended and Restated Effective as of April 1, 2024)


## 1. Purpose.

The purpose of the Microsoft Corporation Deferred Compensation Plan (the "Plan")
is to further the long-term growth of Microsoft Corporation (the "Company") by allowing
selected Company executives and other senior management or highly compensated
employees to defer receipt of certain compensation in order to keep such employees'
financial interests aligned with the Company and provide them with a long-term incentive to
continue employment with the Company.

<!-- PageBreak -->

The Plan was formerly known as the 1998 Microsoft Corporation Stock Option Gain
and Bonus Deferral Program. The name of the Plan was changed pursuant to a
restatement effective January 1, 2006.

This Plan is intended (1) to comply with section 409A of the Internal Revenue Code,
as amended (the "Code") and official guidance issued thereunder (except with respect to
amounts covered by Appendix B), and (2) to be "a plan which is unfunded and is maintained
by an employer primarily for the purpose of providing deferred compensation for a select
group of management or highly compensated employees" within the meaning of sections
201(2), 301(a)(3) and 401(a)(1) of the Employee Retirement Income Security Act of 1974.
Notwithstanding any other provision of this Plan, this Plan shall be interpreted, operated and
administered in a manner consistent with these intentions.


# 2. Effective Date.

The Plan was originally effective November 18, 1998. Except as specifically set
forth below, this restatement of the Plan is effective as of April 1, 2024.


# 3. Definitions.

Account - means a bookkeeping account established by the Company for each
Participant electing to defer Eligible Income under the Plan, which may include sub-
accounts for different types of Eligible Income deferred and for amounts payable at
different times and/or payable in different forms.

Acquisition Retention Bonus - means a bonus provided to a Newly Hired Eligible
Employee who continues employment with the Company or a Designated Subsidiary after
the acquisition of a business by the Company or a Designated Subsidiary or who begins
employment with the Company or a Designated Subsidiary as part of a strategic alliance.

Affiliate - means any corporation or other entity that is treated as a single employer
with the Company under Code section 414.

Annual Base Salary - means the regular annual base salary paid to an Eligible
Employee.

Annual Bonus - means the amount payable to an Eligible Employee as an annual
bonus that is awarded in connection with the Company's annual process under the Annual
Bonus Plan or the cash portion of awards under the Executive Incentive Plan.

Board - means the Board of Directors of Microsoft Corporation.

Code - means the Internal Revenue Code of 1986, as amended.

Company - means Microsoft Corporation.

Date of Hire - means the date of a Participant's first day of active employment with
the Company and its Affiliates.

Designated Subsidiary - means a subsidiary of the Company that has been
approved for participation in the Plan by the Senior HR Officer. A listing of the Designated
Subsidiaries is in Appendix A.

Disabled - means:

<!-- PageBreak -->

(a)
A Participant (1) is unable to engage in any substantial gainful activity by
reason of any medically determinable physical or mental impairment which can be expected
to result in death or can be expected to last for a continuous period of not less than 12
months, or (2) is, by reason of any medically determinable physical or mental impairment
which can be expected to result in death or can be expected to last for a continuous period
of not less than 12 months, receiving income replacement benefits for a period of not less
than 3 months under an accident and health plan covering employees of the participant's
employer.

(b)
The Plan Administrator, in its complete and sole discretion, shall determine
whether a Participant is Disabled. The Plan Administrator may require that the Participant
submit to an examination on an annual basis, at the expense of the Company, by a
competent physician or medical clinic selected by the Plan Administrator to assist in
determining whether the Participant is Disabled. On the basis of such medical evidence, the
determination of the Plan Administrator as to whether or not the Participant is Disabled (or
whether such Participant continues to be Disabled) shall be conclusive.

Eligible Employee - means:

(a)
An Employee of the Company or a Designated Subsidiary working in the
U.S. at the Company's stock level 68 or above and, effective with respect to Eligible Income
earned for periods beginning on or after January 1, 2012, an Employee of the Company or a
Designated Subsidiary working in the U.S. at the Company's stock level 67 or above.

(b)
An Employee meeting the criteria of subsection (a) will not fail to be
considered an Eligible Employee solely as a result of being on paid or unpaid leave.

Eligible Income - means compensation which may be deferred under the Plan, as
from time to time determined by the Plan Administrator, including without limitation (1)
Regular Enrollment Compensation and (2) New Hire Enrollment Compensation.
Amounts will qualify as "Eligible Income" only if the Participant is on the U.S. payroll of the
Company or its Affiliates at the time the amount is payable to the Participant absent deferral.
For the avoidance of doubt, "Eligible Income" does not include severance pay, revenue-
based incentives, commitment-based incentives, SCA bonuses, or any other pay that the
Plan Administrator does not classify as Regular Enrollment Compensation or New Hire
Enrollment Compensation.

Employee - means an individual who is a regular employee on the U.S. payroll of
the Company or its Affiliates. The term "Employee" shall not include a person hired as an
independent contractor, leased employee, consultant, or a person otherwise designated by
the Company or an Affiliate as not eligible to participate in the Plan, even if such person is
determined to be a common law employee of the Company or an Affiliate, retroactively or
prospectively, by any governmental or judicial authority.

ERISA - means the Employee Retirement Income Security Act of 1974, as
amended.

Fiscal Year Compensation - means "fiscal year compensation" as defined under
Treas. Reg. § 1.409A-2(a)(6) or any successor thereto.

Hire Date - means the date an Employee becomes employed by the Company or a
Designated Subsidiary. In the case of an individual who becomes an Employee upon the

<!-- PageBreak -->

acquisition of a business by the Company or a Designated Subsidiary, the Employee's
"Hire Date" shall be such Employee's transfer date.

Investment Options - means a set of investment options, which may include
investment options offered under the 401(k) Plan, and which are from time to time
determined by the Plan Administrator and used to credit earnings, gains, and losses on
Account balances.

Key Employee - means an employee treated as a "specified employee" under Code
section 409A(a)(2)(B)(i) as of such employee's Separation from Service (i.e., a key
employee (as defined under Code section 416(i) without regard to paragraph (5) thereof) of
a corporation any stock of which is publicly traded on an established securities market or
otherwise). Key Employees shall be determined in accordance with Code section 409A,
using a December 31 identification date. A listing of Key Employees as of an identification
date shall be effective for the 12-month period beginning on the April 1 following the
identification date.

New Hire Enrollment Compensation - means compensation for a Newly Hired
Eligible Employee which is from time to time determined by the Plan Administrator,
including without limitation a (1) Signing Bonus and (2) Acquisition Retention Bonus.

Newly Hired Eligible Employee - means an individual hired by the Company or a
Designated Subsidiary who meets the criteria for an Eligible Employee on such individual's
Hire Date, provided that an individual who has previously worked for the Company or an
Affiliate will only qualify as a "Newly Hired Eligible Employee" if such individual meets the
requirements of Treas. Reg. § 1.409A-2(a)(7) or any successor thereto. Generally, a re-
hired individual will meet these requirements if (1) such individual has been paid any and
all amounts due under the Plan (and any plans required to be aggregated with the Plan
under Code section 409A) prior to re-hire, or (2) such individual has not been eligible to
participate, other than the accrual of earnings, in the Plan (or any other plan required to be
aggregated with the Plan under Code section 409A) for at least 24 months.

Open Enrollment - means the period or periods during each Plan Year when
Eligible Employees may elect to defer amounts under the Plan. Open Enrollment shall be
held at the time or times designated by the Plan Administrator.

Participant - means an Eligible Employee who elects to defer Eligible Income under
the Plan.

Performance-Based Compensation - means "performance-based compensation"
as defined under Code section 409A.

Plan - means the Microsoft Corporation Deferred Compensation Plan, as amended
from time to time.

Plan Administrator - means the Senior HR Officer or, with respect to the eligibility of
executive officers of the Company to participate in the Plan, the Compensation Committee
of the Board.

Plan Year - means the 12-month period from January 1 to December 31.

Pre-2014 Election Amount - means an amount deferred under the Plan (and
earnings thereon) pursuant to a deferral election that (1) becomes irrevocable during an
Open Enrollment period occurring before July 1, 2013, or (2) is made by a Newly Hired

<!-- PageBreak -->

Eligible Employee under Section 5.1(b)(ii) and becomes irrevocable on or before December
31, 2013. Thus, Annual Base Salary amounts deferred after 2013 and Annual Bonuses
deferred based on elections made after 2013 will not be considered Pre-2014 Election
Amounts.

Regular Enrollment Compensation - means compensation which is from time to
time determined by the Plan Administrator, including without limitation (1) Annual Base
Salary and (2) Annual Bonus.

Retirement - means a Separation from Service after attaining Retirement Age.

Retirement Age - means one specified date for each Participant occurring on the
earlier of: (1) Participant's attainment of age sixty-five (65), or (2) the later of Participant's
attainment of age fifty-five (55) or the tenth (10th) anniversary of such Participant's Date of
Hire. When an Employee becomes eligible to participate in the Plan, the Plan Administrator
shall determine the Retirement Age for the Employee as one specified date in accordance
with the foregoing.

Senior HR Officer - means the senior officer in charge of the Human Resources
department.

Separation from Service or Separates from Service - means a "separation from
service" with the Company and its Affiliates within the meaning of Code section 409A.

Signing Bonus - means a bonus provided to a Newly Hired Eligible Employee upon
acceptance of an offer of employment with the Company or a Designated Subsidiary,
including bonuses provided to a Newly Hired Eligible Employee who accepted an offer to
continue employment with the Company or a Designated Subsidiary after the acquisition of
a business by the Company or a Designated Subsidiary or to begin employment with the
Company or Designated Subsidiary as part of a strategic alliance.

401(k) Plan - means the Microsoft Corporation Savings Plus 401(k) Plan.

4\.
Participation.

4.1
An Eligible Employee becomes an active Participant in the Plan on the date
the Eligible Employee first enrolls in the Plan by electing to defer all or any portion of such
Eligible Employee's Eligible Income. An Eligible Employee may enroll in the Plan during
Open Enrollment in accordance with Section 5.1(b)(i) or pursuant to Section 5.1(c). A Newly
Hired Eligible Employee may enroll before such Newly Hired Eligible Employee's Hire Date
in accordance with 5.1(b)(ii).

4.2
An Eligible Employee who has been an active Participant under the Plan will
cease to be a Participant on the date such Eligible Employee's Account is fully distributed.

5\.
Participant Accounts.

5.1
Elections to Defer Eligible Income.

(a)
Initial Deferral Election. An Eligible Employee may make an
irrevocable election to defer the following types of Eligible Income in one (1) percent
increments up to the specified maximum percentages:

(i)
An Eligible Employee may elect to defer up to 75% of such

<!-- PageBreak -->

Eligible Employee's Annual Base Salary.

(ii)
An Eligible Employee may elect to defer up to 100% of an
Annual Bonus.

(iii)
An Eligible Employee may elect to defer up to 90% of New

Hire Enrollment Compensation.

Eligible Employees are not permitted to defer gains on the exercise of a stock option
under the Plan after December 31, 2004.

(b)
Time and Manner of Making an Initial Election.

(i)
An Eligible Employee may make an election to defer one or
more types of Regular Enrollment Compensation during an Open Enrollment period that
occurs in the Plan Year preceding the Plan Year in which the Regular Enrollment
Compensation begins to be earned. A deferral election shall be made in accordance with
procedures established by the Plan Administrator. An Employee's election during such an
Open Enrollment period will not be given effect if the Employee ceases to be an Eligible
Employee by the last day of the month in which the Open Enrollment period occurs.

(ii)
A Newly Hired Eligible Employee may make an election to
defer one or more types of New Hire Enrollment Compensation in accordance with
procedures established by the Plan Administrator, provided such election occurs before
such Newly Hired Eligible Employee's Hire Date and such election shall only apply to
amounts earned after the election is filed. A Newly Hired Eligible Employee may make an
election to defer Regular Enrollment Compensation during an Open Enrollment period that
follows or coincides with such Newly Hired Eligible Employee's Hire Date.

(c)
Alternative Election Deadlines. Notwithstanding the rules in
subsection (b), if the Plan Administrator, in its sole discretion, determines that:

(i)
Eligible Income constitutes Performance-Based
Compensation that is based on services performed over a performance period of at least
twelve (12) months, the Plan Administrator may establish procedures, including an Open
Enrollment period, under which an Eligible Employee may elect to defer such
Performance-Based Compensation, but such election must be made no later than six (6)
months before the end of the performance period; or

(ii)
Eligible Income constitutes Fiscal Year Compensation, the
Plan Administrator may establish procedures, including an Open Enrollment period, under
which an Eligible Employee may elect to defer such Fiscal Year Compensation, but such
election must be made no later than the last day of the Company's fiscal year
immediately preceding the first fiscal year in which services are performed related to such
Eligible Income.

An Employee's election under this Section will not be given effect if the
Employee ceases to be an Eligible Employee by the deadline stated above for making
such an election.

(d)
Cancellation of Election. If a Participant becomes Disabled,
receives a hardship withdrawal under the 401(k) Plan, or the Plan Administrator
determines the Participant meets the requirements to obtain a distribution under Section
6.6 on account of an unforeseeable emergency (regardless of whether such distribution

<!-- PageBreak -->

is actually made) during a Plan Year, such Participant's deferrals for such Plan Year shall
be cancelled. For the avoidance of doubt, this Section 5.1(d) shall not affect such
Participant's deferral election made during Open Enrollment in such Plan Year with
respect to amounts earned or payable in a subsequent Plan Year.

5.2 Crediting of Deferrals. Eligible Income deferred by a Participant under the Plan
shall be credited to the Participant's Account as soon as practicable after the amounts would
have otherwise been paid to the Participant.

5.3
3 Vesting. A Participant shall at all times be one-hundred (100) percent vested in
any amounts credited to such Participant's Account.

5.4
Investments and Earnings. The Company shall periodically credit gains, losses
and earnings to a Participant's Account, until the full balance of the Account has been
distributed. Amounts shall be credited to a Participant's Account under this Section based on
the results that would have been achieved had amounts credited to the Account been
invested as soon as practicable after crediting into the Investment Options selected by the
Participant. The Plan Administrator shall specify procedures to allow Participants to make
elections as to the deemed investment of amounts newly credited to such Participants'
Accounts, as well as the deemed investment of amounts previously credited to such
Participants' Accounts. Nothing in this Section or otherwise in the Plan, however, will require
the Company to actually invest any amounts in such Investment Options or otherwise.

5.5
Employment, State and Local Taxes. The Participant's share of FICA and
FUTA taxes, or any state or local taxes, owed on Eligible Income the Participant elects to
defer shall be deducted from the amount deferred or from other compensation payable to the
Participant, at the election of the Company in accordance with Code section 409A.

6\.
Distribution of Account Balances.

6.1
Distribution Form.

(a)
A Participant may elect to have amounts deferred under the Plan
(and earnings thereon) distributed in a lump sum payment or in annual installments over a
period ranging from three (3) to fifteen (15) years.

(b)
A Participant must specify the form in which a deferred amount
(and earnings thereon) will be distributed at the time of making the initial deferral election
under Section 5.1.

(c)
Notwithstanding the distribution form elected under subsection (a),
if at the time a portion of a Participant's Account is to be distributed, the portion of the
balance to be distributed is less than $50,000, that portion shall be distributed in a lump
sum payment at such time, provided that this subsection (c) shall not apply to any
amounts deferred under the Plan pursuant to a deferral election that becomes irrevocable
on or after June 30, 2011 (and earnings thereon).

(d)
Distribution of a Participant's Account balance shall be made in
cash.

6.2
Distribution Time of Post-2013 Election Amounts. The rules in this Section 6.2
shall apply to amounts deferred under the Plan, excluding any portion of a Participant's
Account attributable to Pre-2014 Election Amounts.

<!-- PageBreak -->

(a)
A Participant may elect to have distribution of a deferred amount
(and earnings thereon) commence as of the following dates:

(i)
A specified time (a particular month and year); or

(ii)
Upon the Participant's Separation from Service (in which
case distributions will commence in the month following Separation from Service).

(b)
A Participant must specify such Participant's distribution
commencement election at the time of making the initial deferral election under Section
5.1.

(c)
If a Participant elects to have a deferred amount distributed as of a
specified time, the specified time must be at least twelve (12) months after the date on
which the final payment of the deferred amount would have been made to the Participant
absent deferral.

6.3
Distribution of Pre-2014 Election Amounts Upon Separation from Service. The
rules in this Section 6.3 shall only apply to Pre-2014 Election Amounts.

(a) If a Participant has reached Retirement Age at the time of
Separation from Service, the distribution of a Pre-2014 Election Amount will commence
as follows:

(i)
If the Participant elected commencement upon
Retirement, the distribution will commence in the month following Retirement.

(ii)
If the Participant elected commencement upon a
specified time, the distribution will commence in the specified month and year.

(b) Notwithstanding a Participant's elections with respect to Pre-2014
Election Amounts, (i) if a Participant Separates from Service prior to reaching Retirement
Age, the portion of such Participant's Account balance attributable to Pre-2014 Election
Amounts shall be distributed in an immediate lump sum payment in the month following
the Separation from Service, and (ii) if a Participant becomes Disabled prior to attaining
Retirement Age while employed with the Company or an Affiliate, the portion of such
Participant's Account balance attributable to Pre-2014 Election Amounts shall be
distributed in an immediate lump sum payment in the month following the date the
Participant becomes Disabled.

6.4
Key Employee Delay. Except as otherwise permitted under IRS guidance, if a
distribution is to be made upon the Separation from Service of a Key Employee, distribution
may not be made before the date which is six months after the date of the Key Employee's
Separation from Service (or, if earlier, before the date of death of the Key Employee). Any
payments that would otherwise be made during this period of delay shall be paid in
accordance with the elected distribution method and the terms of the Plan in the seventh
month following Separation from Service (or, if earlier, following the Key Employee's death in
accordance with Sections 6.5(a) and (b) below).

6.5
Distributions Upon Death.

(a)
Notwithstanding a Participant's elections with respect to Pre-2014
Election Amounts, if a Participant dies prior to attaining Retirement Age while employed
with the Company or an Affiliate, the portion of such Participant's Account balance

<!-- PageBreak -->

attributable to the Pre-2014 Election Amounts shall be distributed to the Participant's
beneficiary in an immediate single lump sum payment as soon as administratively feasible
following the Participant's death. For the avoidance of doubt, if a Participant attains
Retirement Age while employed with the Company or an Affiliate, and dies thereafter
(whether employed by the Company or an Affiliate at the time of death or not), the portion
of such Participant's Account balance attributable to the Pre-2014 Election Amounts shall
be distributed to the Participant's beneficiary in the time and form of payment elected by
the Participant under Section 6.1 and 6.3.

(b)
Notwithstanding a Participant's elections under Sections 6.1 and
6.2, if a Participant dies, such Participant's Account balance, excluding any portion
attributable to Pre-2014 Election Amounts, shall be distributed to the Participant's
beneficiary in a single lump sum payment as soon as administratively feasible following
the Participant's death.

(c)
A Participant shall designate a beneficiary prior to death in
accordance with procedures established by the Plan Administrator. If a Participant has not
properly designated a beneficiary or if no designated beneficiary is living on the date of
distribution, the Participant's Account shall be distributed to the Participant's beneficiary
designated under the 401(k) Plan, or if no designated beneficiary under the 401(k) Plan is
living, in accordance with the default provisions under the 401(k) Plan.

(d)
For purposes of determining the proper death beneficiary under this
Plan, this Plan shall not be interpreted as preempting applicable state law regarding the
ownership rights of Accounts upon a Participant's death. For example, although this Plan
states that upon a Participant's death, Account balances will be paid to such Participant's
beneficiary, the personal representative may be obligated to pay any benefits owed to a
spouse or otherwise as a result of any applicable community property laws.

6.6 Withdrawals for Unforeseeable Emergency. A Participant may withdraw all or
any portion of such Participant's Account balance for an Unforeseeable Emergency. The
amounts distributed with respect to an Unforeseeable Emergency may not exceed the
amounts necessary to satisfy such Unforeseeable Emergency plus amounts necessary to
pay taxes reasonably anticipated as a result of the distribution, after taking into account the
extent to which such hardship is or may be relieved through reimbursement or compensation
by insurance or otherwise or by liquidation of the Participant's assets (to the extent the
liquidation of such assets would not itself cause severe financial hardship) or by cessation of
deferrals under the Plan. "Unforeseeable Emergency" means for this purpose a severe
financial hardship to a Participant resulting from an illness or accident of the Participant, the
Participant's spouse, or a dependent of the Participant, loss of the Participant's property due
to casualty, or other similar extraordinary and unforeseeable circumstances arising as a
result of events beyond the control of the Participant.

Unless otherwise required by IRS guidance, a Participant shall not be required to
take any available hardship withdrawals from the 401(k) Plan before being eligible to
receive a withdrawal under this section.

6.7
Changes in Time or Form of Distribution. To the extent permitted in
accordance with procedures established by the Plan Administrator, including (without
limitation) restrictions with respect to the frequency with which participants can make such
elections, a Participant may make one or more subsequent elections to change the time or
form of a distribution to be made as of a specified time or upon the occurrence of a
distributable event for a deferred amount, but such an election will be effective only if the
following conditions are satisfied:

<!-- PageBreak -->

(a)
The election may not take effect until at least twelve (12)
months after the date on which the election is made;

(b)
A distribution may not be made earlier than at least five (5)
years from the date the distribution would have otherwise been made;

(c)
In the case of an election to change the time or form of a
distribution payable as of a specified time, the election must be made at least
twelve (12) months before the date of the first scheduled distribution; and

(d)
The election may not result in an impermissible acceleration
of payment prohibited under Code section 409A.

6.8 Effect of Taxation. If a portion of the Participant's Account balance is includible
in income under Code section 409A, such portion shall be distributed immediately to the
Participant.

6.9 Payment of Taxes. If state, local, or foreign tax obligations arise from
participation in the Plan that apply to an amount deferred under the Plan before such amount
is paid or made available to the Participant (the "Taxes"), the Company shall pay a portion of
such deferred amount by distribution (a) to the Participant in the form of withholding pursuant
to provisions of applicable state, local, or foreign law; or (b) directly to the Participant. In no
event shall the total payment under this Section 6.9 exceed the aggregate amount of the
Taxes, and the income tax withholding related to such Taxes.

6.10 Settlement of Bona Fide Dispute. Subject to certain presumptions under Code
section 409A, if an arm's length, bona fide dispute between a Participant and the Company
arises as to the Participant's right to an amount deferred under the Plan, the payment of the
deferred amount as part of a settlement of such dispute shall be distributed immediately to
the Participant.

6.11 Offset for Obligations to Company. If the Participant has any debt, obligation
or other liability representing an amount owing to the Company (the "Debt"), incurred in the
ordinary course of such Participant's employment relationship, the Company shall offset the
Debt against the Participant's Account balance. The Company shall reduce the Participant's
Account balance in satisfaction of the Debt at the same time and in the same amount as the
Debt otherwise would have been due and collected from the Participant; provided however,
in no event shall the amount of such offset in any of the Company's taxable years exceed
$5,000.

6.12 2005 Deferred Compensation. Except as provided in Appendix C, Sections 6.1
\- 6.11 shall govern the distribution of compensation earned and deferred under the Plan
during the 2005 Plan Year.

6.13 Pre-2005 Deferrals. Notwithstanding the foregoing, Appendix B governs the
distribution of amounts that were earned and vested (within the meaning of Code section
409A and regulations thereunder) under the Plan prior to 2005 (and earnings thereon) and
are exempt from the requirements of Code section 409A.

7\.
Administration.

7.1
General Administration. The Plan Administrator shall be responsible for the
operation and administration of the Plan and for carrying out the provisions hereof. The Plan

<!-- PageBreak -->

Administrator shall have the full authority and discretion to make, amend, interpret, and
enforce all appropriate rules and regulations for the administration of this Plan and decide or
resolve any and all questions, including interpretations of this Plan, as may arise in
connection with this Plan. Except as otherwise provided in Section 7.2, any such action
taken by the Plan Administrator shall be final and conclusive on any party. To the extent the
Plan Administrator has been granted discretionary authority under the Plan, the Plan
Administrator's prior exercise of such authority shall not obligate it to exercise its authority in
a like fashion thereafter. The Plan Administrator shall be entitled to rely conclusively upon
all tables, valuations, certificates, opinions and reports furnished by any actuary,
accountant, controller, counsel or other person employed or engaged by the Company with
respect to the Plan. The Plan Administrator may, from time to time, employ agents and
delegate to such agents, including other employees of the Company, such administrative
duties as it sees fit.


## 7.2 Claims for Benefits.

(a)
Filing a Claim. A Participant or the Participant's authorized
representative may file a claim for benefits under the Plan. Any claim must be in writing
and submitted to the Senior HR Officer at such address as may be specified from time to
time. Claimants will be notified in writing of approved claims. A claim is considered
approved only if its approval is communicated in writing to a claimant.

(b)
Denial of Claim. If a Participant's claim is denied, in whole or in
part, a written notice will be furnished to the claimant within 90 days of the date on which
the claim is received by the Senior HR Officer. If special circumstances (such as for a
hearing) require a longer period, the claimant will be notified in writing, prior to the
expiration of the 90-day period, of the reasons for an extension of time; provided, however,
that no extensions will be permitted beyond 90 days after the expiration of the initial 90-
day period.

(c)
Reasons for Denial. A denial or partial denial of a claim will be
dated and signed by the Senior HR Officer and will clearly set forth:

(i)
the specific reason or reasons for the denial;

(ii)
specific reference to pertinent Plan provisions on which the
denial is based;

(iii)
a description of any additional material or information
necessary for the claimant to perfect the claim and an explanation of why such material or
information is necessary; and

(iv)
)an explanation of the procedure for review of the denied or
partially denied claim set forth below, including the claimant's right to bring a civil action
under ERISA section 502(a) following an adverse benefit determination on review.

(d)
Review of Denial. Upon denial of a claim, in whole or in part, a
claimant or such claimant's duly authorized representative will have the right to submit a
written request to the Senior HR Officer for a full and fair review of the denied claim by
filing a written notice of appeal with the Senior HR Officer within 60 days of the receipt by
the claimant of written notice of the denial of the claim. A claimant or the claimant's
authorized representative will have, upon request and free of charge, reasonable access
to, and copies of, all documents, records, and other information relevant to the claimant's
claim for benefits and may submit issues and comments in writing. The review will take

<!-- PageBreak -->

into account all comments, documents, records, and other information submitted by the
claimant relating to the claim, without regard to whether such information was submitted or
considered in the initial benefit determination.

If the claimant fails to file a request for review within 60 days of the claim denial
notification, the claim will be deemed abandoned and the claimant precluded from
reasserting it. If the claimant does file a request for review, the claimant's request must
include a description of the issues and evidence the claimant deems relevant. Failure to
raise issues or present evidence on review will preclude those issues or evidence from
being presented in any subsequent proceeding or judicial review of the claim.

(e)
Decision Upon Review. The Senior HR Officer will provide a
prompt written decision on review. If the claim is denied on review, the decision shall set
forth:

(i)
the specific reason or reasons for the adverse determination;

(ii)
specific reference to pertinent Plan provisions on which the
adverse determination is based;

(iii)
a statement that the claimant is entitled to receive, upon
request and free of charge, reasonable access to, and copies of, all documents, records, and
other information relevant to the claimant's claim for benefits; and

(iv)
a statement describing any voluntary appeal procedures
offered by the Plan and the claimant's right to obtain the information about such procedures,
as well as a statement of the claimant's right to bring an action under ERISA section 502(a).

A decision will be rendered no more than 60 days after the Senior HR Officer's
receipt of the request for review, except that such period may be extended for an
additional 60 days if the Senior HR Officer determines that special circumstances (such as
the need for a hearing) require such extension. If an extension of time is required, written
notice of the extension will be furnished to the claimant before the end of the initial 60-day
period.

(f)
Finality of Determinations; Exhaustion of Remedies. To the extent
permitted by law, decisions reached under the claims procedures set forth in this Section
shall be final and binding on all parties. No legal action for benefits under the Plan shall be
brought unless and until the claimant has exhausted such claimant's remedies under this
Section. In any such legal action, the claimant may only present evidence and theories
which the claimant presented during the internal claims and appeals procedure. Any
claims which the claimant does not in good faith pursue through the review stage of the
procedure shall be treated as having been irrevocably waived. Judicial review of a
claimant's denied claim shall be limited to a determination of whether the denial was an
abuse of discretion based on the evidence and theories the claimant presented during the
internal claims and appeals procedure. Any suit or legal action initiated by a claimant
under the Plan must be brought by the claimant no later than one year following a final
decision on appeal by the Senior HR Officer. The one-year limitation on suits for benefits
will apply in any forum where a claimant initiates such suit or legal action.

(g)
Disability Claims. Claims for disability benefits shall be determined
under the DOL Regulation section 2560.503-1 which is hereby incorporated by reference.

8\.
Amendment and Termination.

<!-- PageBreak -->

8.1
Amendment or Termination. The Company reserves the right to amend or
terminate the Plan when, in the sole discretion of the Company, such amendment or
termination is advisable, pursuant to a resolution or other action taken by the Plan
Administrator.

Notwithstanding the foregoing, no amendment of the Plan shall apply to amounts
that were earned and vested (within the meaning of Code section 409A and regulations
thereunder) under the Plan prior to 2005, unless the amendment specifically provides that
it applies to such amounts. The purpose of this restriction is to prevent a Plan amendment
from resulting in an inadvertent "material modification" to amounts that are
"grandfathered" and exempt from the requirements of Code section 409A.

8.2
Effect of Amendment or Termination. No amendment or termination of the
Plan shall decrease the amounts credited to a Participant's Account as of such
amendment or termination. Upon termination of the Plan, Participants' Account balances
shall be distributed in accordance with the terms of Section 6, unless the Company
determines in its sole discretion that all such amounts shall be distributed upon termination
in accordance with the requirements under Code section 409A.

9\.
General Provisions.

9.1 Rights Unsecured. The right of a Participant or the Participant's beneficiary
to receive a distribution hereunder shall be an unsecured claim against the general assets
of the Company, and neither the Participant nor the Participant's beneficiary shall have
any rights in or against any amount credited to any Account or any other assets of the
Company. The Plan at all times shall be considered entirely unfunded for tax purposes.
Any funds set aside by the Company for the purpose of meeting its obligations under the
Plan, including any amounts held by a trustee, shall continue for all purposes to be part of
the general assets of the Company and shall be available to its general creditors in the
event of the Company's bankruptcy or insolvency. The Company's obligation under this
Plan shall be that of an unfunded and unsecured promise to pay money in the future.

9.2 No Right to Eligible Income. Nothing in this Plan shall be construed to give
any Eligible Employee any right to be granted Eligible Income or any other type of
compensation.

9.3 No Enlargement of Rights. No Participant or beneficiary shall have any right
to receive a distribution under the Plan except in accordance with the terms of the Plan.
Establishment of the Plan shall not be construed to give any Participant the right to
continue to be employed by or provide services to the Company or its affiliates or to
employment that is not terminable at will.

9.4 No Guarantee of Benefits. Nothing contained in the Plan shall constitute a
guarantee by the Company or any other person or entity that the assets of the Company
will be sufficient to pay any benefits hereunder.

9.5 Nonalienation of Benefits. This Plan inures to the benefit of and is binding
upon the parties hereto and their successors, heirs and assigns; provided, however, that
the amounts credited to a Participant's Account are not, except as provided in Sections
9.6 and 6.11, subject in any manner to anticipation, alienation, sale, transfer, assignment,
pledge, encumbrance, charge, garnishment, execution or levy of any kind, either voluntary
or involuntary, and any attempt to anticipate, alienate, sell, transfer, assign, pledge,
encumber, charge or otherwise dispose of any right to any benefits payable hereunder,

<!-- PageBreak -->

will be null and void and not binding on the Plan or the Company.

9.6 Taxes. In addition to its rights under Section 5.5, the Company or other
payor may withhold from a benefit payment under the Plan or a Participant's wages any
federal, state, or local taxes required by law to be withheld with respect to a payment or
accrual under the Plan, and shall report such payments and other Plan-related information
to the appropriate governmental agencies as required under applicable law.

9.7 Participant's Cooperation. The Participant shall cooperate with the
Company by furnishing any and all information requested by the Plan Administrator in
order to facilitate the payment of benefits hereunder, taking such physical examinations as
the Plan Administrator may deem necessary and taking such other actions as may be
requested by the Plan Administrator. If the Participant refuses to cooperate, the Company
shall have no further obligation to the Participant under the Plan.

9.8 Incapacity of Recipient. If any person entitled to a distribution under the Plan
is deemed in the Plan Administrator's sole discretion to be incapacitated or otherwise
incapable of personally receiving such payment, then, unless and until a claim for such
payment is made by a duly appointed guardian or other legal representative of such
person, the Plan Administrator may provide for such payment or any part thereof to be
made to any other person or institution then contributing toward or providing for the care
and maintenance of such person. Any such payment shall be a payment for the account of
such person and a complete discharge of any liability of the Company and the Plan with
respect to the payment.

9.9 Legally Binding. In the event of any consolidation, merger, acquisition or
reorganization, the obligations of the Company under this Plan shall continue and be
binding on the Company and its successors or assigns. The rights, privileges, benefits and
obligations under the Plan are intended to be legal obligations of the Company and
binding upon the Company, its successors and assigns.

9.10 Unclaimed Benefits. Each Participant shall keep the Plan Administrator
informed of the Participant's current address and the current address of the Participant's
designated beneficiary. The Plan Administrator shall not be obligated to search for the
whereabouts of any person if the location of a person is not made known to the Plan
Administrator.

9.11 Severability. In the event any provision of the Plan shall be held invalid or
illegal for any reason, any illegality or invalidity shall not affect the remaining parts of the
Plan, but the Plan shall be construed and enforced as if the illegal or invalid provision had
never been inserted.

9.12 Words and Headings. Words in any gender shall include all genders and
the singular shall include the plural, and vice versa, unless qualified by the context. Any
headings used herein are included for ease of reference only, and are not to be construed
so as to alter the terms hereof.

9.13 Applicable law and Venue. To the extent not preempted by federal law, the
Plan shall be governed by the laws of the State of Washington. In the event the Company
or any Participant (or beneficiary) initiates litigation related to this Plan, the venue for such
action will be in King County, Washington.

9.14 Waiver of Breach. The waiver by the Company of any breach of any
provision of the Plan by the Participant shall not operate or be construed as a waiver of

<!-- PageBreak -->

any subsequent breach by the Participant.

9.15 Notice. Any notice or filing required or permitted to be given to the Plan
Administrator under the Plan shall be sufficient if in writing and hand-delivered, or sent by
first class mail to the principal office of the Company, directed to the attention of the Plan
Administrator. Such notice shall be deemed given as of the date of delivery, or, if delivery
is made by mail, as of the date shown on the postmark.

9.16 Attorneys' Fees and Costs. In the event that a dispute regarding benefits
arises between the Company or Plan Administrator and a Participant (or beneficiary) and
such dispute is resolved through arbitration or litigation in court, the prevailing party(ies)
shall be entitled to their reasonable attorneys' fees and costs incurred in such action.

The Company has caused this restated Plan to be duly adopted and executed on

[Kathleen Hogan signature block]

.

<!-- PageBreak -->


# APPENDIX A DESIGNATED SUBSIDIARIES (As of April 1, 2024)

1654: MOL Corporation
1693: Vexcel Corporation
1548: Microsoft Online, Inc.

<!-- PageBreak -->


# APPENDIX B GRANDFATHERED AMOUNTS

Distribution of amounts that were earned and vested (within the meaning of Code
section 409A and regulations thereunder) under the Plan prior to 2005 (and earnings
thereon) and are exempt from the requirements of Code section 409A shall be made in
accordance with the Plan terms as in effect on December 31, 2004 and as summarized in
this Appendix B.

B.1
Timing. As soon as practicable following the final day of the Deferral Period
for a specific deferral, the Company will distribute to the Participant (or in the case of the
Participant's death, the Participant's estate), all proceeds in the Participant's Deferred
Bonus Account and will issue to the Participant (or in the event of the Participant's death,
the personal representative or beneficiaries of the Participant's estate) shares of Stock
credited to the Participant's Deferred Stock Option Gain Account, that are attributed to that
deferral. With respect to a specific deferral, the final day of the Deferral Period shall be the
earliest of the last day of the Deferral Period selected by the Participant or the date such
Participant has a Termination of Employment. Upon Termination of Employment, a
Participant will have the same rights with respect to an unexercised Option that such
Participant would have if the Participant had not elected to defer the Stock Option Gain
relating to that Option. The portion of a Participant's Accounts that can be attributed to a
specific deferral shall be determined in the sole discretion of the Plan Administrator.

B.2
Extension of Deferral Period. On a one time basis with respect to each
deferral, a Participant may elect in accordance with procedures established by the Plan
Administrator to extend the Deferral Period for a Bonus or Stock Option Gain for an
additional five (5), seven (7), or ten (10) years, provided that such extension is elected in
the calendar year prior, and at least six (6) months prior, to the expiration of the initial
Deferral Period and the Participant is an Eligible Executive at the time such Participant
makes the election to extend the Deferral Period.

B.3
Disability. In the event of a Participant's Disability and upon application by
such Participant, the Plan Administrator may determine that payment of all, or part, of such
Participant's Accounts shall be made in a different manner, or on an earlier date than the
time or times specified in Section 8.1 above, but only to the extent determined by the Plan
Administrator to be reasonably required to satisfy the Participant's need.

B.4
Investment of Accounts. Notwithstanding Section 5.4, a Participant shall not
have the right to select among Investment Options for amounts credited to the Participant's
Deferred Stock Option Gain Account. Such amounts shall be treated as if invested in Stock
at all times.

B.5
Definitions. For purposes of this Appendix B, the following terms shall have
the meanings indicated below:

Bonus means the amount payable by the Company to an Eligible Employee
as an individual performance bonus, executive bonus or any other bonus/incentive award
that is approved by the Plan Administrator for deferral under the Plan.

Deferral Period means with respect to a specific deferral of a Bonus or Stock
Option Gain, the period of five (5), seven (7), or ten (10) years from the date on which the
corresponding Bonus would otherwise have been paid or the date the Option was scheduled
to expire had it not been exercised; provided that, in the event of the Participant's

<!-- PageBreak -->

Termination of Employment, the Deferral Period shall end on the date of Termination of
Employment.

Deferred Bonus Account means a bookkeeping account established for
Bonuses deferred under the Plan.

Deferred Stock Option Gain Account means a bookkeeping account
established for Stock Option Gains deferred under the Plan.

Disability means any long-term disability as defined under the Company's
long-term disability plan. The Plan Administrator, in its complete and sole discretion, shall
determine a Participant's Disability. The Plan Administrator may require that the Participant
submit to an examination on an annual basis, at the expense of the Company, by a
competent physician or medical clinic selected by the Plan Administrator to assist in the
determination of Disability. On the basis of such medical evidence, the determination of the
Plan Administrator as to whether or not a condition of Disability exists or continues shall be
conclusive.

Eligible Executive means a full-time employee of the Company who is (i) an
elected officer of the Company, (ii) at the level of Vice President or above, (iii) at Level 16
or above on the Company's salary range, and (iv) working within the United States of
America. In addition, the Plan Administrator may, in the Plan Administrator's discretion,
extend coverage to persons who are selected by the Plan Administrator and who either (y)
☒
meet all of the foregoing requirements except that such person works outside of the United
States of America, or (z) is an officer of a subsidiary of the Company.

Mature Shares means shares of the Company's Stock delivered by a
Participant in payment of the exercise price of an Option; provided that Mature Shares shall
not include any shares of the Company's Stock that may be received upon exercise of such
Option, nor Stock that the Participant purchased pursuant to a prior stock option exercise
which occurred less than six months prior to the exercise of such Option.

Option shall mean one or more non-qualified stock options, issued to a
Participant under any stock option plan of the Company, with respect to which the
Participant has elected to defer the Stock Option Gain. Option shall not include any rights
under the Company's Employee Stock Purchase Plan.

Stock means Microsoft Corporation common stock.

Stock Option Gain means the number of shares underlying an Option minus
the number of Mature Shares required to pay the exercise price for those shares. For
example, if a Participant elects to defer the gain on 100 shares and is required to deliver 10
shares of Stock as payment for the exercise price on the 100 shares, the Stock Option Gain
will be 90 shares.

Termination of Employment means the termination of the Participant's
employment relationship with the Company for any reason including, without limitation,
involuntary termination with or without cause, voluntary termination, disability, death, or
retirement.

<!-- PageBreak -->


## APPENDIX C 2005 DEFERRED COMPENSATION

This Appendix C sets forth the special rules applicable to compensation eligible for deferral under
the Plan from January 1, 2005 through December 31, 2005. Unless otherwise defined herein, capitalized
terms used but not otherwise defined herein shall have the meanings given to them in the Plan and
Appendix B.

C-1. 2005 Initial Deferral Elections. Notwithstanding anything in Section 5.1 of the Plan to the
contrary and only with respect to compensation earned during the 2005 Plan Year ("2005 Income"), an
Eligible Employee may make an irrevocable election to defer up to 100% of a Bonus in ten (10) percent
increments. Eligible Employees are not permitted to defer gains on the exercise of a stock option under the
Plan after December 31, 2004.

C-2. Time of Distribution. The Company will distribute to the Participant (or in the case of the
Participant's death, the Participant's estate) all proceeds in the Participant's Deferred Bonus Account that
are attributed to a specific deferral upon the earlier of: (1) the last day of the Deferral Period elected by the
Participant; or (2) the date of the Participant's Separation from Service; provided that, if a distribution is to
be made upon the Separation from Service of a Key Employee, such distribution is subject to the six month
delay set forth in Section 6.4 of the Plan.

For purposes of this Appendix C, "Deferral Period" means with respect to a specific deferral of a
Bonus, the period, as elected by the Participant at the time of the deferral election, of five (5), seven (7), or
ten (10) years from the date on which the corresponding Bonus would otherwise have been paid.

C-3. Changes in Time or Form of Distribution. To the extent the Company allows a
Participant to make a subsequent election to change the time or form of distribution of 2005 Income
deferred under the Plan, such election will be effective only if the conditions set forth in Section 6.7 of
the Plan are satisfied.

C-4. General Application of the Plan. Other than as set forth above, the terms of the Plan in all
other respects and in compliance with Code section 409A shall govern the distribution of 2005 Income
deferred under the Plan from January 1, 2005 through December 31, 2005.

<!-- PageBreak -->

<!-- PageHeader="Exhibit 19.1" -->


# General Insider Trading Policy


## 1. PURPOSE

To prevent the misuse of material, nonpublic information about Microsoft or other publicly traded companies
obtained by virtue of your position at Microsoft.


## 2. SUMMARY

This policy and Federal and State securities laws prohibit a person from trading in securities of a company when
that person has material, nonpublic information relating to that company.

As an employee of Microsoft, you may have access to financial, business, or other information about Microsoft, or
other companies, that is both material and not available to the public. If you are in possession of material,
nonpublic information about any company, including Microsoft, the securities laws and Microsoft prohibit you from
trading in (or gifting) the securities of that company, and you may not disclose the information to anyone else,
except as specifically authorized in the performance of your job responsibilities.

This policy also applies to:

· immediate family members who live with you or use your address as their regular address (e.g., spouse,
domestic partner, minor children, college students, parents, grandparents, grandchildren, siblings and in-
laws);

· trusts, if you are: (1) a trustee who has or shares investment control of the trust securities and you or a
member of your immediate family (whether or not they live with you) has an economic interest in the trust
securities; (2) a beneficiary who has or shares investment control; or (3) a person who has the power to
revoke the trust; and

· partnerships or other entities over which you have a controlling influence.

You are responsible for assuring that individuals or entities listed above comply with this policy.


## 3. REQUIREMENTS


### A. Do Not Trade While in Possession of Material, Nonpublic Information

As an employee of Microsoft, you may have access to financial, business and other information about Microsoft or
other companies. When you possess information about Microsoft that is both material and nonpublic, the
securities laws and Microsoft prohibit you from buying or selling (or gifting) Microsoft securities.

Similarly, when you possess material, nonpublic information about any publicly traded company that you acquired
through your role at Microsoft, including but not limited to a customer or partner of Microsoft or an economically-
linked company such as a competitor of Microsoft, the securities laws and Microsoft prohibit you from buying or
selling securities in that company.

<!-- PageBreak -->

· Information is 'material' if there is a substantial likelihood that a reasonable investor would consider it
important in deciding whether to buy or sell that company's securities.

· Information is 'nonpublic' if it has not been broadly communicated to the marketplace in a manner making
it generally available to the investing public. Rumors, even if true and widely reported in the media, do not
constitute public disclosure unless publicly confirmed by the company to which it relates.

Common examples of material information may include:

· Financial results

· Internal projections that significantly differ from external expectations

· Significant proposed or pending acquisitions, dispositions, mergers or joint ventures

· Changes in key executives

· Significant plans, developments or problems related to important products and services

· Changes in the terms of a significant contract with a major customer or partner, or the acquisition or loss of
such a contract

· Cybersecurity risks and incidents (including vulnerabilities and breaches)

· Significant litigation or regulatory actions

· Other notable transactions, events or developments outside the ordinary course of business

Material information can be good news or bad news, and can relate to any aspect of a company's business. Either
way, the law prohibits making profits and avoiding losses by using material, nonpublic information. It makes no
difference whether you receive the information during the course of your job responsibilities or overhear it in the
hallway or lunchroom, or anywhere at or away from your normal workplace.


### B. Do Not Disclose Material, Nonpublic Information to Others

Do not disclose material, nonpublic information about Microsoft, or another publicly traded company, to anyone
else. This is considered illegal 'tipping' and you could be held criminally liable under federal law even if you yourself
did not make a trade.

Do not recommend or suggest that anyone else trade the securities of any company, including Microsoft, even if
you do not disclose the material, nonpublic information.


### C. Limited Exceptions to These Restrictions

There are a few situations where transactions in securities are not prohibited even if you have material, nonpublic
information.

· Purchases of Microsoft stock through an existing purchase election under the Employee Stock Purchase
Plan (ESPP). However, you may sell your ESPP shares only at a time when you do not have material,
nonpublic information.

· Acquiring Microsoft stock through the vesting of employee Stock Awards. You may sell your Stock Award
shares only at a time when you do not have material, nonpublic information.

· Exercising options, if you pay the exercise price and withholding taxes in cash. Again, you may sell your
shares only at a time when you do not have material, nonpublic information.

· Trades made pursuant to a predetermined, written Rule 10b5-1 Trading Plan. For information on Trading
Plans and the specific policies and procedures associated with them, please visit the Rule 10b5-1 Trading
plans page.

<!-- PageBreak -->


### D. Restrictions on Microsoft Officers and Other Designated Employees

Microsoft has a policy requiring certain officers of the company (corporate vice presidents and above) and other
designated employees to only transact in Microsoft securities during specific times throughout the year, commonly
referred to as 'trading windows'. Those officers and employees will be notified if they are subject to the Restricted
Trading Window Policy.


#### E. Ask Questions

If you have any questions about this policy or how it applies under certain circumstances, please visit the Insider
trading compliance site or contact ( ** )@microsoft.com.


### 4. PROCEDURE

Any person who knows of, or believes there is, a violation of this policy must report the violation immediately to
** )@microsoft.com.
☒


### 5. EXCEPTIONS

There are no exceptions to this policy.


### 6. ENFORCEMENT

Violation of this policy may result in disciplinary action, up to and including immediate termination of employment.
Employees who engage in insider trading or tipping may also be subject to civil and criminal fines, as well as
imprisonment.


### 7. APPLICATION

This policy applies to all employees.


### 8. COUNTRY OR BUSINESS UNIT SUPPLEMENT


### 9. RELATED DOCUMENTS

Restricted Trading Window Policy


### 10. TRANSLATIONS

11\. DOCUMENT HISTORY

<!-- PageBreak -->

<!-- PageHeader="Exhibit 19.2" -->


# Restricted Trading Window Policy


## 1. PURPOSE

This policy describes the restrictions on trading Microsoft securities that apply to Microsoft officers and other
designated persons due to their regular access to material, nonpublic information about Microsoft.

Members of the Senior Leadership Team ("SLT") who are direct reports to the CEO ("SLT CEO Direct Reports") are
subject to an additional notification requirement described below.


## 2. SUMMARY

Microsoft's officers and other designated employees are subject to additional trading restrictions for Microsoft
securities to those set forth in the General Insider Trading Policy. In general, if you fall in this group you may only
trade (or gift) Microsoft securities during an open window period when your trading status is designated as Trading
Allowed in the Insider Trading Compliance Tool.


## 3. REQUIREMENTS


### A. Application of the Policy

This policy applies to all transactions in Microsoft securities by Microsoft's officers and other employees with
regular access to material, nonpublic information (collectively 'Insiders'). In general, you will know you are an
Insider and subject to this policy because you will be notified. The requirements in this policy are in addition to the
General Insider Trading Policy, which applies to all employees.

This policy applies to transactions involving Microsoft securities that are beneficially owned by Insiders. 'Beneficial
ownership' is based on the Insider's direct or economic interest in the securities. Some common examples of
relationships that may give rise to beneficial ownership include, but are not limited to:

· Microsoft shares owned by immediate family members who live with you or use your address as their
regular address (e.g., spouse, domestic partner, minor children, college students, parents, grandparents,
grandchildren, siblings and in-laws).

· Microsoft shares held in a trust, if the Insider is: (1) a trustee who has or shares investment control of the
trust securities and the Insider or a member of his or her immediate family (whether or not they live with
you) has an economic interest in the trust securities; (2) a beneficiary who has or shares investment
control; or (3) a person who has the power to revoke the trust.

· Microsoft shares held in the name of a partnership or other entity over which the Insider has a controlling
influence.

Insiders are responsible for assuring that individuals or entities who may have an interest in securities that are
beneficially owned by the Insider ('Covered Persons') comply with this policy. Accordingly, any reference below to
'Insiders' should be read to include 'Covered Persons.'

<!-- PageBreak -->


### B. Open Trading Window

If you are an Insider, you may only trade (or gift) Microsoft securities during an Open Window Period and when
your status is Trading Allowed in the Insider Trading Compliance Tool. However, even during an Open Window
Period, you may not trade Microsoft securities if you possess material, nonpublic information about Microsoft. For
more information about material, nonpublic information, see the General Insider Trading Policy.

An Open Window Period means a defined period of time following Microsoft's quarterly earnings release when the
public has been apprised of all relevant material information about Microsoft's recent business and financial
performance.

· Open Window Periods typically begin with the opening of trading on the second full trading day following
Microsoft's public release of quarterly earnings, and end at the close of trading 15 business days before the
end of the fiscal quarter in which the earnings were released.

. For example, if Microsoft made an earnings announcement on a Thursday, the Open Window Period would
open the following Monday, and would end at the close of the trading 15 business days before the end of
that fiscal quarter.

. After you become subject to this Restricted Trading Window Policy, and as long as you have an active
restriction reason, you will receive three emails each quarter from the insider trading compliance tool. (1)
You will receive an email on the first business day after the public release of quarterly earnings to confirm
the date the trading window will open and that your trading status will be Trading Allowed. (2) You will
receive an email 14 calendar days before the trading window closes to let you know the date when your
trading status will change to No Trading Allowed. (3) You will receive an email on the date the window
closes to confirm that your trading status is No Trading Allowed.

. Occasionally, a quarterly window may be delayed, closed early, or skipped. You will be notified if this
happens.

· Microsoft may also institute additional 'blackout periods' during which no trades may be made, at which
time you should receive notification of such restriction.

Do not make a trade in Microsoft securities until you receive notice that the Open Window Period has begun and
you have confirmed that your status in the Insider Trading Compliance Tool is Trading Allowed.

You may not disclose to any outside third party, including brokers or financial advisers, the terms of Microsoft's
Open Window Period, the date on which an Open Window Period has begun or ended, or that a blackout period
has been designated. You may disclose your status in the Insider Trading Compliance Tool as Trading Allowed or No
Trading Allowed.

If your employment with Microsoft ends during a blackout period, you will be subject to the remainder of that
blackout period. If your employment with Microsoft ends during an Open Window Period, you will not be subject to
the next blackout period. However, even if you are not subject to our blackout period after you leave Microsoft,
you should not trade in Microsoft securities if you are aware of material, nonpublic information. That restriction
stays with you as long as the information you possess is material and not publicly disseminated within the meaning
of the General Insider Trading Policy.


### C. Application of Trading Restrictions to Particular Situations


#### 1. Stock Awards

Transactions involving Microsoft stock received by an Insider pursuant to a Stock Award may be made only during
an Open Window Period and your status in the Insider Trading Compliance Tool is Trading Allowed.

<!-- PageBreak -->


#### 2. Gifts

Gifts of Microsoft stock may be made only during an Open Window Period and your status in the Insider Trading
Compliance Tool is Trading Allowed.


## 3. Employee Stock Purchase Plan

You may purchase Microsoft stock under Microsoft's Employee Stock Purchase Plan ('ESPP') at any time. However,
you may only sell Microsoft stock purchased pursuant to the ESPP during an Open Window Period and your status
in the Insider Trading Compliance Tool is Trading Allowed.


## 4. Stock Options

You may exercise stock options outside an Open Window Period only if you pay the exercise price and withholding
taxes in cash, and then hold the shares (known as an 'exercise and hold'). Any sale of stock as part of a broker-
assisted cashless exercise of an option, any other market sale for the purpose of generating the cash needed to pay
the exercise price of an option, or any delivery of shares to pay the exercise price (if permissible under the
applicable stock plan) may only be done during an Open Window Period.


## 5. Transactions Pursuant to Rule 10b5-1 Plans

Sales of Microsoft stock outside of the Open Window Period or during a blackout period can only be made pursuant
to an approved written trading plan (known as a '10b5-1 Plan') that you entered into at a time that sales could be
made in accordance with the restrictions set forth in Section B above. To learn more, please visit the Rule 10b5-1
Trading plans page.


## D. Senior Leadership Team Advance Notification Requirement

In addition to the requirements above, SLT CEO Direct Reports are also required to notify CELA in advance of
transactions in Microsoft securities, including, but not limited to, option exercises, gifts, purchases, sales, and
pledges of Microsoft stock or debt. It does not apply to purchases made pursuant to the ESPP or sales made
pursuant to a 10b5-1 Plan, but does apply to the other scenarios described in Section C above.

The notification must include a brief description of the nature of the transaction (e.g., exercise of stock option,
purchase, sale, gift, pledge of stock for loan, etc.), the type of Microsoft securities involved, and the proposed date
on which the Insider will enter into the transaction. Notifications should be given by sending email to
$\left( * * \right.$ )@microsoft.com, or by phoning any of the following individuals ("Contacts"):
☒

( ** )
☒

Advance notification can be provided by the Insider or his or her broker.

As soon as possible following notification, one of the Contacts will review the transaction and advise the Insider
whether he or she may proceed with the transaction. During this process, the Insider or his or her broker will be
asked to answer some preclearance questions from a Preclearance Checklist. A copy of the Preclearance Checklist
may be requested from ( ** )@microsoft.com. If advance notification of a transaction is provided via e-mail, the
Insider may wish to attach a completed Preclearance Checklist for the proposed transaction.

E. Questions

<!-- PageBreak -->

If you have a question about this policy, its application in relation to a proposed transaction, or reporting
responsibilities, you should email ( ** )@microsoft.com.


## 4. PROCEDURE


## 5. EXCEPTIONS

There are no exceptions to this policy.


## 6. ENFORCEMENT

Violation of this policy may result in disciplinary action, up to and including immediate termination of employment.

Insiders who engage in insider trading or tipping may also be subject to civil and criminal fines, as well as
imprisonment.


## 7. APPLICATION

This policy applies to Microsoft officers and other designated persons, and their Covered Persons.


## 8. COUNTRY OR BUSINESS UNIT SUPPLEMENT


## 9. RELATED DOCUMENTS

Corporate Governance Guidelines

General Insider Trading Policy


## 10. TRANSLATIONS


### 11. DOCUMENT HISTORY

<!-- PageBreak -->

<!-- PageHeader="Exhibit 19.3" -->


# Insider Trading Compliance and Preclearance Policies for Section 16 Officers and Directors of Microsoft

This document sets forth the insider trading compliance policies that apply to transactions in Microsoft securities by
Microsoft officers and directors who are subject to Section 16 of the Securities Exchange Act of 1934 ("Insiders").

The policies contained in this document apply to all securities "beneficially owned" by Insiders. Insiders are
responsible for assuring that other individuals or entities ("Covered Persons") who may have an interest in
securities that are "beneficially owned" by an Insider comply with these policies. For more information regarding
beneficial ownership of securities and Covered Persons, please see Section 6 below.

Covered Persons must comply with the policies below. Accordingly, any reference below to "Insiders" should be
read to include Covered Persons. In addition to the policies below, all Directors must comply with the Director
stock ownership requirements located in the Corporate Governance Guidelines and Executive Officers must comply
with the Stock Ownership Requirements for Microsoft Corporation Executives.


## 1. Restrictions on Trading

A. Prohibition on Trading While in Possession of Material, Nonpublic Information. Under Federal
securities law and regulations, no Insider may trade in Microsoft securities while possessing material, nonpublic
information concerning the Company, even if a quarterly window period described in Section 2 is open. The
window period is not a safe harbor that eliminates the need for caution about whether an Insider should refrain
from trading because he or she possesses material, nonpublic information about Microsoft. For more information,
see the General Insider Trading Policy.

B. Trading Restricted to Open Window Periods. No Insider may trade in Microsoft securities outside of an
applicable Open Window Period described below in Section 2, or during any trading blackout periods designated by
Microsoft, except pursuant to the terms of a Rule 10b5-1 Plan as outlined in Section 4.E below. For more
information, see the Restricted Trading Window Policy.

C.
Prohibition on Trading in Derivatives. Trading in derivative securities (other than derivative
securities issued by Microsoft as compensation) related to Microsoft stock or debt such as options, puts, calls,
warrants or similar financial instruments may create the appearance among regulators and Microsoft shareholders
that an Insider is engaging in short-term, speculative transactions or inappropriately hedging the risk of declines in
Microsoft stock. For this reason, Insiders may not trade in options, puts, calls or other derivative instruments
related to Microsoft stock or debt.

D.
Prohibition on Pledging. No Insider may purchase Microsoft stock on margin, borrow against
Microsoft stock held in a margin account, or pledge Microsoft stock as collateral for a loan.


## 2. Trading Window and Blackout Periods

A. An Insider may trade in Microsoft securities only during quarterly window periods that follow each
quarterly earnings announcement. This policy is intended to help prevent accusations of trading on material,
nonpublic information. The theory underlying the policy is that the period of time following an earnings release is a
time when the public has been apprised of all relevant material information about the Company's recent business
and financial performance. The window period generally begins on the commencement of trading on the second
full trading day following Microsoft's public release of quarterly earnings, and ends at the close of trading 15
business days prior to the end of the third month of the fiscal quarter in which the earnings are released (the "Open

<!-- PageBreak -->

Window Period"). If, for example, Microsoft made an earnings announcement on Thursday, the trading window
would be expected to open the following Monday. There are rare situations when a quarterly window period is
delayed, closed early or skipped. Microsoft employees and directors who are subject to the trading window policy
are sent email each quarter advising them of the first and last day of that quarter's Open Window Period. If an
Open Window Period is delayed, closed early or skipped, Insiders will be sent email alerting them to this
development. An Insider should not trade unless he or she has received notice about the opening of the trading
window.

B. Insiders who have material, nonpublic information concerning Microsoft may not trade in Microsoft
securities even during Open Window Periods.

C. Insiders may not trade in Microsoft securities outside of an Open Window Period or during any
blackout period that Microsoft may designate. Insiders may not disclose to any outside third party, including
brokers or financial advisers, the terms of the trading window policy, the date on which an Open Window Period
has begun or ended, or that a blackout period has been designated.


## 3. Pre-Clearance Policies and Procedures

A. Potential Section 16(b) Liability. Insiders are subject to the short-swing profit recapture rules o
of
Section 16 of the Securities Exchange Act of 1934. Compliance with the law includes prompt reporting of an
Insider's transactions involving Microsoft securities. The Section 16 reporting requirements also apply to
transactions in Microsoft securities owned by certain other individuals and entities related to an Insider, as
explained in greater detail in Section 5 below. Any late or delinquent filings are required to be reported in
Microsoft's annual proxy statement. In order to assist Insiders in complying with these reporting requirements, the
procedures described below have been adopted so that CELA, Corporate Legal Group may prepare and file with the
SEC on an Insider's behalf the appropriate form reporting an Insider's transactions.

The purpose of these filings with the SEC is to give any interested party the ability to examine the filings to
determine whether an Insider has violated the so-called "short-swing profit" rules, which prohibit a specified officer
or director of a public company from profiting on sales and purchases of stock that occur within 6 months of each
other. Neither lack of familiarity with the rules nor inadvertent non-compliance will excuse a violation of the short-
swing profit rules. Generally, this law should not present Insiders at Microsoft with a problem because Microsoft
stock purchases that an Insider makes from the Company (e.g. stock acquired on exercise of stock options and stock
purchased under the Employee Stock Purchase Plan) are all exempt from short swing profit liability under SEC rules,
and therefore any sales of stock an Insider undertakes typically would not have any non-exempt purchases of stock
with which they could be matched. However, an Insider may potentially have a problem if the Insider purchases
Microsoft shares in the open market or changes an investment election under the Microsoft 401(k) plan with
respect to investing in Microsoft stock. The advance notification procedures described below are intended to help
Insiders avoid transactions that would violate Section 16(b), as well as meet the 2-day filing deadline for reporting
most Insider transactions in Microsoft securities. Insiders are expected to cooperate with the Company fully and
promptly in disclosing their transactions involving Company securities.

B.
Preclearance Requirement. All Insiders are required to obtain approval from the CELA Corporate
Legal Group before entering into any transaction involving Microsoft securities, including, but not limited to, option
exercises, gifts, purchases, and sales, of Microsoft stock or debt. Some very limited exceptions apply as described in
Section 4 below. The Insider or his or her broker will be asked to answer some preclearance questions in a
Preclearance Checklist, which includes a brief description of the nature of the transaction (e.g., exercise of stock
option, purchase, sale, gift, etc.), the type and number of Microsoft securities involved, and the proposed date on
which the Insider will enter into the transaction. A copy of the Preclearance Checklist may be requested from
** )@microsoft.com.

<!-- PageBreak -->

Preclearance requests should be sent by email to ( ** )@microsoft.com. For questions, please call any of the
following individuals ("Contacts"):

( ** )
☒

As soon as possible following the preclearance request, one of the Contacts will review the transaction and advise
the Insider whether he or she may proceed with the transaction.

C.
Confirmation of Transaction. Immediately after the transaction is executed, the Insider or his or
her broker must provide the details of the transaction to one or more of the Contacts, either by phone or e-mail to
** )@microsoft.com. This information must include the following:

i. type of Microsoft security involved;

ii. a brief description of the nature of the transaction (e.g., purchase, sale, exercise, etc.);

iii. number of shares involved;

iv. price per share;

v. the total aggregate price/value; and

vi. the transaction date.

Please note that this information must be provided on the date the transaction is executed (commonly referred to
as the "trade date"), as opposed to on the date the transaction is settled. In nearly all cases, the trade date is the
date that gives rise to the Section 16 reporting obligation.


# 4. Application of the Trading Restrictions and Preclearance Policies to Particular Situations

A.
Stock Awards.

i. Trading Restrictions. Transactions involving Microsoft stock received by an Insider pursuant to a
Stock Award may be made only in accordance with, and at times permitted by, the trading restrictions
contained in Sections 1 and 2 above.

ii. Preclearance and Confirmation. Insiders do not need to request preclearance or provide
confirmation of receipt of a Stock Award. Insiders must request preclearance and provide confirmation
of a transaction as outlined in Section 3 above for any sales of Microsoft stock received under a Stock
Award.

B. Gifts.

i. Trading Restrictions. Gifts of Microsoft stock may be made only in accordance with, and at times
permitted by, the trading restrictions contained in Sections 1 and 2 above.

ii. Preclearance and Confirmation. Any gift is subject to the preclearance requirement and
confirmation of the transaction as outlined in Section 3 above.

C.
Employee Stock Purchase Plan.

<!-- PageBreak -->

i. Trading Restrictions. There is no restriction on purchases of Microsoft stock under Microsoft's
Employee Stock Purchase Plan ("ESPP"). However, an Insider's sale of Microsoft stock purchased
pursuant to the ESPP may be made only in accordance with, and at times permitted by, the trading
restrictions contained in Sections 1 and 2 above.

ii. Preclearance and Confirmation. Insiders do not need to request preclearance or provide
confirmation of purchases of Microsoft stock pursuant to an election under the ESPP. However,
Insiders must request preclearance and provide confirmation of the transaction as outlined in Section 3
above for any sales of Microsoft stock purchased pursuant to the ESPP.


## D. Stock Options.

i. Trading Restrictions. Stock options may be exercised outside an Open Window Period only if the
Insider pays the exercise price and withholding taxes in cash, and then holds the shares (also known as
an "exercise and hold"). Any sale of stock as part of a broker-assisted cashless exercise of an option,
any other market sale for the purpose of generating the cash needed to pay the exercise price of an
option, or any delivery of shares to pay the exercise price (if permissible under the applicable stock
plan) may only be done in accordance with, and at times permitted by, the trading restrictions
described in Section 1 and 2 above.

ii. Preclearance and Confirmation. Any exercise of an employee stock option is subject to the
preclearance requirement and confirmation of transaction process as outlined in Section 3 above.


## E. Transactions in Microsoft Stock Pursuant to Rule 10b5-1 Plans.

i. Trading Restrictions. SEC Rule 10b5-1 allows trading under certain circumstances by Insiders during
periods when the Insider would otherwise be prohibited from trading (for example, during periods
when the trading window is closed or the insider possesses material, nonpublic information). In order
to qualify for the affirmative defense offered by this Rule, the plan must be contained in an agreement
that meets certain requirements, and the Insider must not exercise any discretion regarding the
transactions under the plan during a period when trading would otherwise be prohibited (typically
referred to as a "10b5-1 Plan"). An Insider may make sales of Microsoft stock outside of the Open
Window Period or during a blackout period only if such sales are made pursuant to a 10b5-1 Plan that
consists of an agreement with the selling broker that has been prepared by a broker-dealer or a lawyer
representing the Insider, reviewed and approved by the CELA Corporate Legal Group, and entered into
by the Insider at a time that sales could be made in accordance with the restrictions set forth in
Sections 1 and 2 above. For additional information regarding 10b5-1 Plans and the policies associated
with them, please send email to ( ** )@microsoft.com.

ii. Preclearance and Confirmation. Insiders engaging in transactions under an approved 10b5-1 Plan
generally will not have an obligation to request preclearance of such transactions. However, Insiders
(or their brokers) must immediately confirm a transaction as provided in Section 3 above upon
execution of a transaction under a 10b5-1 Plan.


## 5. Tax Services from Microsoft Independent Auditors

Microsoft's independent auditor (currently Deloitte & Touche LLP) is prohibited from providing tax services to
individuals in a financial reporting oversight role at Microsoft, or immediate family members of such persons, who
do not meet the exceptions provided by PCAOB Rule 3523.

<!-- PageBreak -->

Employee Insiders not in a financial reporting oversight role and directors may use Microsoft's independent auditor
for individual tax services, after providing advance notice to and receiving approval from CELA (via the audsvc alias).

If you are uncertain whether you a person in a financial reporting oversight role covered by this policy or are
otherwise prohibited from using Microsoft's independent auditor for tax services, you may contact
the( ** )@microsoft.com alias.


## 6. Covered Persons

Insiders are subject to the Section 16 reporting requirements with respect to all Microsoft securities that they
beneficially own. The determination of "beneficial ownership" is based on the Insider's direct or economic interest
in the securities. Although not exhaustive, we have listed below the primary types of relationships that may give
rise to beneficial ownership, thus triggering the Section 16 reporting requirement.

· Microsoft shares owned by immediate family members who live with the Insider or use the Insider's
address as their regular address (e.g., spouse, domestic partner, minor children, college students, parents,
grandparents, grandchildren, siblings and in-laws).

· Microsoft shares held in a trust, if the Insider is: (1) a trustee who has or shares investment control of the
trust securities and the Insider or a member of his or her immediate family (whether or not they share the
same household) has an economic interest in the trust securities; (2) a beneficiary and has or shares
investment control; or (3) a person who has the power to revoke the trust.

· Microsoft shares held in the name of a partnership or other entity over which the Insider has a controlling
influence.

Insiders may wish to consult with a Contact or send e-mail to ( ** )@microsoft.com to clarify reporting
responsibilities with respect to any Microsoft securities over which the Insider has voting control or in which the
Insider has an economic interest.


## 7. Sanctions

Persons who are found to have traded on material, nonpublic information can face serious criminal and civil
penalties. Insiders that violate the prohibition on short-swing profit transactions will be required to disgorge the
deemed profit from the transaction to the Company. Any late or delinquent filings are required to be reported in
Microsoft's annual proxy statement. In addition, the SEC has broad authority to seek sanctions from late filers;
these sanctions may include significant fines and other penalties. Violation of these policies may result in
disciplinary action up to and including immediate termination of employment. Violation of these policies also may
be reported in Company filings with the SEC.


## 8. Questions

Anyone with questions about these policies and procedures, or the application of them to a proposed transaction,
may obtain additional guidance by sending e-mail to ( ** )@microsoft.com. The ultimate responsibility for adhering
to these policies and avoiding unlawful transactions rests with the Insider.

<!-- PageBreak -->

<!-- PageHeader="Exhibit 21" -->


# SUBSIDIARIES OF REGISTRANT

The following is a list of subsidiaries of Microsoft Corporation as of June 30, 2024, omitting subsidiaries which,
considered in the aggregate, would not constitute a significant subsidiary.


<table>
<tr>
<th>Name</th>
<th>Where Incorporated</th>
</tr>
<tr>
<td>Microsoft Ireland Research</td>
<td>Ireland</td>
</tr>
<tr>
<td>Microsoft Global Finance</td>
<td>Ireland</td>
</tr>
<tr>
<td>Microsoft Ireland Operations Limited</td>
<td>Ireland</td>
</tr>
<tr>
<td>Microsoft Online, Inc.</td>
<td>United States</td>
</tr>
<tr>
<td>LinkedIn Corporation</td>
<td>United States</td>
</tr>
<tr>
<td>LinkedIn Ireland Unlimited Company</td>
<td>Ireland</td>
</tr>
<tr>
<td>Nuance Communications, Inc.</td>
<td>United States</td>
</tr>
<tr>
<td>Activision Blizzard, Inc.</td>
<td>United States</td>
</tr>
<tr>
<td>Activision Publishing, Inc.</td>
<td>United States</td>
</tr>
</table>


<!-- PageBreak -->

<!-- PageHeader="Exhibit 23.1" -->


# CONSENT OF INDEPENDENT REGISTERED PUBLIC ACCOUNTING FIRM

We consent to the incorporation by reference in Registration Statement Nos. 333-109185, 333-118764, 333-
52852, 333-132100, 333-161516, 333-75243, 333-185757, and 333-221833 on Form S-8 and Registration
Statement No. 333-261590 on Form S-3 of our reports dated July 30, 2024, relating to the financial statements
of Microsoft Corporation and the effectiveness of Microsoft Corporation's internal control over financial reporting
appearing in this Annual Report on Form 10-K of Microsoft Corporation for the year ended June 30, 2024.

/s/ DELOITTE & TOUCHE LLP
Seattle, Washington
July 30, 2024

<!-- PageBreak -->

<!-- PageHeader="Exhibit 31.1" -->


# CERTIFICATION

I, Satya Nadella, certify that:

1\. I have reviewed this annual report on Form 10-K of Microsoft Corporation;

2\. Based on my knowledge, this report does not contain any untrue statement of a material fact or omit to state a
material fact necessary to make the statements made, in light of the circumstances under which such statements were
made, not misleading with respect to the period covered by this report;

3\. Based on my knowledge, the financial statements, and other financial information included in this report, fairly
present in all material respects the financial condition, results of operations and cash flows of the registrant as of, and
for, the periods presented in this report;

4\. The registrant's other certifying officer and I are responsible for establishing and maintaining disclosure controls and
procedures (as defined in Exchange Act Rules 13a-15(e) and 15d-15(e)) and internal control over financial reporting
(as defined in Exchange Act Rules 13a-15(f) and 15d-15(f)) for the registrant and have:

a) Designed such disclosure controls and procedures, or caused such disclosure controls and procedures to be
designed under our supervision, to ensure that material information relating to the registrant, including its consolidated
subsidiaries, is made known to us by others within those entities, particularly during the period in which this report is
being prepared;

b) Designed such internal control over financial reporting, or caused such internal control over financial reporting to be
designed under our supervision, to provide reasonable assurance regarding the reliability of financial reporting and the
preparation of financial statements for external purposes in accordance with generally accepted accounting principles;

c) Evaluated the effectiveness of the registrant's disclosure controls and procedures and presented in this report our
conclusions about the effectiveness of the disclosure controls and procedures, as of the end of the period covered by
this report based on such evaluation; and

d) Disclosed in this report any change in the registrant's internal control over financial reporting that occurred during
the registrant's most recent fiscal quarter (the registrant's fourth fiscal quarter in the case of an annual report) that has
materially affected, or is reasonably likely to materially affect, the registrant's internal control over financial reporting;
and

5\. The registrant's other certifying officer and I have disclosed, based on our most recent evaluation of internal control
over financial reporting, to the registrant's auditors and the audit committee of registrant's Board of Directors (or
persons performing the equivalent functions):

a) All significant deficiencies and material weaknesses in the design or operation of internal control over financial
reporting which are reasonably likely to adversely affect the registrant's ability to record, process, summarize and report
financial information; and

b) Any fraud, whether or not material, that involves management or other employees who have a significant role in the
registrant's internal control over financial reporting.

/s/ SATYA NADELLA
Satya Nadella
Chief Executive Officer
July 30, 2024

<!-- PageBreak -->

<!-- PageHeader="Exhibit 31.2" -->


# CERTIFICATION

I, Amy E. Hood, certify that:

1\. I have reviewed this annual report on Form 10-K of Microsoft Corporation;

2\. Based on my knowledge, this report does not contain any untrue statement of a material fact or omit to state a
material fact necessary to make the statements made, in light of the circumstances under which such statements were
made, not misleading with respect to the period covered by this report;

3\. Based on my knowledge, the financial statements, and other financial information included in this report, fairly
present in all material respects the financial condition, results of operations and cash flows of the registrant as of, and
for, the periods presented in this report;

4\. The registrant's other certifying officer and I are responsible for establishing and maintaining disclosure controls and
procedures (as defined in Exchange Act Rules 13a-15(e) and 15d-15(e)) and internal control over financial reporting
(as defined in Exchange Act Rules 13a-15(f) and 15d-15(f)) for the registrant and have:

a) Designed such disclosure controls and procedures, or caused such disclosure controls and procedures to be
designed under our supervision, to ensure that material information relating to the registrant, including its consolidated
subsidiaries, is made known to us by others within those entities, particularly during the period in which this report is
being prepared;

b) Designed such internal control over financial reporting, or caused such internal control over financial reporting to be
designed under our supervision, to provide reasonable assurance regarding the reliability of financial reporting and the
preparation of financial statements for external purposes in accordance with generally accepted accounting principles;

c) Evaluated the effectiveness of the registrant's disclosure controls and procedures and presented in this report our
conclusions about the effectiveness of the disclosure controls and procedures, as of the end of the period covered by
this report based on such evaluation; and

d) Disclosed in this report any change in the registrant's internal control over financial reporting that occurred during
the registrant's most recent fiscal quarter (the registrant's fourth fiscal quarter in the case of an annual report) that has
materially affected, or is reasonably likely to materially affect, the registrant's internal control over financial reporting;
and

5\. The registrant's other certifying officer and I have disclosed, based on our most recent evaluation of internal control
over financial reporting, to the registrant's auditors and the audit committee of registrant's Board of Directors (or
persons performing the equivalent functions):

a) All significant deficiencies and material weaknesses in the design or operation of internal control over financial
reporting which are reasonably likely to adversely affect the registrant's ability to record, process, summarize and report
financial information; and

b) Any fraud, whether or not material, that involves management or other employees who have a significant role in the
registrant's internal control over financial reporting.

/s/ AMY E. HOOD
Amy E. Hood
Executive Vice President and
Chief Financial Officer

July 30, 2024

<!-- PageBreak -->

<!-- PageHeader="Exhibit 32.1" -->


# CERTIFICATION PURSUANT TO SECTION 906 OF THE SARBANES-OXLEY ACT OF 2002 (18 U.S.C. SECTION 1350)

In connection with the Annual Report of Microsoft Corporation, a Washington corporation (the "Company"), on Form 10-
$\mathrm { K }$ for the year ended June 30, 2024, as filed with the Securities and Exchange Commission (the "Report"), Satya
Nadella, Chief Executive Officer of the Company, does hereby certify, pursuant to § 906 of the Sarbanes-Oxley Act of
2002 (18 U.S.C. § 1350), that to his knowledge:

(1) The Report fully complies with the requirements of section 13(a) or 15(d) of the Securities Exchange Act of 1934;
and

(2) The information contained in the Report fairly presents, in all material respects, the financial condition and results
of operations of the Company.

/s/ SATYA NADELLA
Satya Nadella
Chief Executive Officer
July 30, 2024

[A signed original of this written statement required by Section 906 has been provided to Microsoft Corporation and will
be retained by Microsoft Corporation and furnished to the Securities and Exchange Commission or its staff upon
request.]

<!-- PageBreak -->

<!-- PageHeader="Exhibit 32.2" -->


# CERTIFICATION PURSUANT TO SECTION 906 OF THE SARBANES-OXLEY ACT OF 2002 (18 U.S.C. SECTION 1350)

In connection with the Annual Report of Microsoft Corporation, a Washington corporation (the "Company"), on Form 10-
$\mathrm { K }$ for the year ended June 30, 2024, as filed with the Securities and Exchange Commission (the "Report"), Amy E.
Hood, Chief Financial Officer of the Company, does hereby certify, pursuant to § 906 of the Sarbanes-Oxley Act of
2002 (18 U.S.C. § 1350), that to her knowledge:

(1) The Report fully complies with the requirements of section 13(a) or 15(d) of the Securities Exchange Act of 1934;
and

(2) The information contained in the Report fairly presents, in all material respects, the financial condition and results
of operations of the Company.

/s/ AMY E. HOOD
Amy E. Hood
Executive Vice President and
Chief Financial Officer

July 30, 2024

[A signed original of this written statement required by Section 906 has been provided to Microsoft Corporation and will
be retained by Microsoft Corporation and furnished to the Securities and Exchange Commission or its staff upon
request.]

<!-- PageBreak -->

<!-- PageHeader="Exhibit 97.1" -->


# Microsoft Corporation Executive Compensation Recovery Policy

This policy covers Microsoft's Covered Officers and explains when Microsoft will be required or authorized, as
applicable, to seek recovery of Incentive Compensation awarded or paid to Covered Officers. Please refer to Exhibit
A attached hereto (the "Definitions Exhibit") for the definitions of capitalized terms used throughout this Policy.

1\. Miscalculation of Financial Performance Measure Results. In the event of a Restatement, Microsoft will
seek to recover, reasonably promptly, all Recoverable Incentive Compensation from a Covered Officer
during the Applicable Period. Such recovery, in the case of a Restatement, will be made without regard to
any individual knowledge or responsibility related to the Restatement or the Recoverable Incentive
Compensation. Notwithstanding the foregoing, if Microsoft is required to undertake a Restatement,
Microsoft will not be required to recover the Recoverable Incentive Compensation if the Compensation
Committee determines it Impracticable to do so, after exercising a normal due process review of all the
relevant facts and circumstances.

Microsoft will seek to recover all Recoverable Incentive Compensation that was awarded or paid in
accordance with the definition of "Recoverable Incentive Compensation" set forth on the Definitions
Exhibit. If such Recoverable Incentive Compensation was not awarded or paid on a formulaic basis,
Microsoft will seek to recover the amount that the Compensation Committee determines in good faith
should be recouped.

2\. Legal and Compliance Violations. Compliance with the law and Microsoft's Standards of Business Conduct
and other corporate policies is a pre-condition to earning Incentive Compensation. If Microsoft in its sole
discretion concludes that a Covered Officer (1) committed a significant legal or compliance violation in
connection with the Covered Officer's employment, including a violation of Microsoft's corporate policies
or Microsoft's Standards of Business Conduct (each, "Misconduct"), or (2) was aware of or willfully blind to
Misconduct that occurred in an area over which the Covered Officer had supervisory authority, Microsoft
may, at the direction of the Compensation Committee, seek recovery of all or a portion of the Recoverable
Incentive Compensation awarded or paid to the Covered Officer for the Applicable Period in which the
violation occurred. In addition, Microsoft may, at the direction of the Compensation Committee, conclude
that any unpaid or unvested Incentive Compensation has not been earned and must be forfeited.

In the event of Misconduct, Microsoft may seek recovery of Recoverable Incentive Compensation even if
the Misconduct did not result in an award or payment greater than would have been awarded or paid
absent the Misconduct.

In the event of Misconduct, in determining whether to seek recovery and the amount, if any, by which the
payment or award should be reduced, the Compensation Committee may consider-among other things-
the seriousness of the Misconduct, whether the Covered Officer was unjustly enriched, whether seeking
the recovery would prejudice Microsoft's interests in any way, including in a proceeding or investigation,
and any other factors it deems relevant to the determination.

3\.
Other Actions. The Compensation Committee may, subject to applicable law, seek recovery in the manner
it chooses, including by seeking reimbursement from the Covered Officer of all or part of the compensation
awarded or paid, by electing to withhold unpaid compensation, by set-off, or by rescinding or canceling
unvested stock.

<!-- PageBreak -->

In the reasonable exercise of its business judgment under this Policy, the Compensation Committee may in
its sole discretion determine whether and to what extent additional action is appropriate to address the
circumstances surrounding a Restatement or Misconduct to minimize the likelihood of any recurrence and
to impose such other discipline as it deems appropriate.

4\.
No Indemnification or Reimbursement. Notwithstanding the terms of any other policy, program,
agreement or arrangement, in no event will Microsoft or any of its affiliates indemnify or reimburse a
Covered Officer for any loss under this Policy and in no event will Microsoft or any of its affiliates pay
premiums on any insurance policy that would cover a Covered Officer's potential obligations with respect
to Recoverable Incentive Compensation under this Policy.

5\.
Administration of Policy. The Compensation Committee will have full authority to administer this Policy.
Actions of the Compensation Committee pursuant to this Policy will be taken by the vote of a majority of its
members. The Compensation Committee will, subject to the provisions of this Policy and Rule 10D-1 of the
Securities Exchange Act of 1934, as amended (the "Exchange Act"), and Microsoft's applicable exchange
listing standards, make such determinations and interpretations and take such actions in connection with
this Policy as it deems necessary, appropriate or advisable. All determinations and interpretations made by
the Compensation Committee will be final, binding and conclusive.

6\.
Other Claims and Rights. The remedies under this Policy are in addition to, and not in lieu of, any legal and
equitable claims Microsoft or any of its affiliates may have or any actions that may be imposed by law
enforcement agencies, regulators, administrative bodies, or other authorities. Further, the exercise by the
Compensation Committee of any rights pursuant to this Policy will not impact any other rights that
Microsoft or any of its affiliates may have with respect to any Covered Officer subject to this Policy.

7\.
Condition to Eligibility for Incentive Compensation. All Incentive Compensation subject to this Policy will
not be earned, even if already paid, until the Policy ceases to apply to such Incentive Compensation and any
other vesting conditions applicable to such Incentive Compensation are satisfied.

8\.
Amendment; Termination. The Board or the Compensation Committee may amend or terminate this
Policy at any time.

9\.
Effectiveness. Except as otherwise determined in writing by the Compensation Committee, this Policy will
apply to any Incentive Compensation that (a) in the case of any Restatement, is Received by Covered
Officers prior to, on or following the Effective Date, and (b) in the case of Misconduct, is awarded or paid to
a Covered Officer on or after the Effective Date. This Policy will survive and continue notwithstanding any
termination of a Covered Officer's employment with Microsoft and its affiliates.

10\.
Successors. This Policy shall be binding and enforceable against all Covered Officers and their successors,
beneficiaries, heirs, executors, administrators, or other legal representatives.

11\.
Governing Law. To the extent not preempted by U.S. federal law, this Policy will be governed by and
construed in accordance with the laws of the State of Washington, without reference to principles of
conflict of laws.

<!-- PageBreak -->


# EXHIBIT A


## DEFINITIONS

"Applicable Period" means (a) in the case of any Restatement, the three completed fiscal years of Microsoft
immediately preceding the earlier of (i) the date the Board, a committee of the Board, or the officer or officers of
Microsoft authorized to take such action if Board action is not required, concludes (or reasonably should have
concluded) that a Restatement is required or (ii) the date a regulator, court or other legally authorized entity directs
Microsoft to undertake a Restatement, and (b) in the case of any Misconduct, such period as the Compensation
Committee or Board determines to be appropriate in light of the scope and nature of the Misconduct. The
"Applicable Period" also includes any transition period (that results from a change in Microsoft's fiscal year) within
or immediately following the three completed fiscal years identified in the preceding sentence.

"Board" means the Board of Directors of Microsoft.

"Compensation Committee" means Microsoft's committee of independent directors responsible for executive
compensation decisions, or in the absence of such a committee, a majority of the independent directors serving on
the Board.

"Covered Officer" means (a) in the case of any Restatement, any person who is, or was at any time, during the
Applicable Period, an Executive Officer of Microsoft or an SLT Member, and (b) in the case of any Misconduct, any
person who was an Executive Officer or an SLT Member at the time of the Misconduct. For the avoidance of doubt,
a Covered Officer may include a former Executive Officer or SLT Member that left Microsoft, retired, or transitioned
to an employee role (including after serving as an Executive Officer in an interim capacity) during the Applicable
Period.

"Effective Date" means July 1, 2023.

"Executive Officer" means Microsoft's president, principal financial officer, principal accounting officer (or if there is
no such accounting officer, the controller), any vice-president in charge of a principal business unit, division, or
function (such as sales, administration, or finance), any other officer who performs a policy-making function, or any
other person (including an officer of Microsoft's parent(s) or subsidiaries) who performs similar policy-making
functions for Microsoft.

"Financial Performance Measure" means a measure that is determined and presented in accordance with the
accounting principles used in preparing Microsoft's financial statements (including "non-GAAP" financial measures,
such as those appearing in Microsoft's earnings releases or Management Discussion and Analysis), and any measure
that is derived wholly or in part from such measure. Stock price and total shareholder return (and any measures
derived wholly or in part therefrom) shall be considered Financial Performance Measures.

"Impracticable." The Compensation Committee may determine in good faith that recovery of Recoverable Incentive
Compensation is "Impracticable" (a) in the case of any Restatement, if: (i) pursuing such recovery would violate
home country law of the jurisdiction of incorporation of the Company where that law was adopted prior to
November 28, 2022 and Microsoft provides an opinion of counsel to that effect acceptable to Microsoft's listing
exchange; (ii) the direct expense paid to a third party to assist in enforcing this Policy would exceed the Recoverable
Incentive Compensation and Microsoft has (A) made a reasonable attempt to recover such amounts and (B)
provided documentation of such attempts to recover to Microsoft's applicable listing exchange; or (iii) recovery
would likely cause an otherwise tax-qualified retirement plan, under which benefits are broadly available to
employees of Microsoft, to fail to meet the requirements of the Internal Revenue Code of 1986, as amended, and
(b) in the case of any Misconduct, in its sole discretion, in light of the scope and nature of the Misconduct.

<!-- PageBreak -->

"Incentive Compensation" means any compensation that is granted, earned, or vested based wholly or in part upon
the attainment of a Financial Performance Measure. Incentive Compensation does not include any base salaries
(except with respect to any salary increases earned wholly or in part based on the attainment of a Financial
Performance Measure performance goal); bonuses paid solely at the discretion of the Compensation Committee or
Board that are not paid from a "bonus pool" that is determined by satisfying a Financial Performance Measure
performance goal; bonuses paid solely upon satisfying one or more subjective standards and/or completion of a
specified employment period; non-equity incentive plan awards earned solely upon satisfying one or more strategic
measures or operational measures; and equity awards that vest solely based on the passage of time and/or
attaining one or more non-Financial Performance Measures. Notwithstanding the foregoing, in the case of any
Misconduct, Incentive Compensation will include all forms of cash and equity incentive compensation, including,
without limitation, cash bonuses and equity awards that are received or vest solely based on the passage of time
and/or attaining one or more non-Financial Performance Measures.

"Received." Incentive Compensation is deemed "Received" in Microsoft's fiscal period during which the Financial
Performance Measure specified in the Incentive Compensation award is attained, even if the payment or grant of
the Incentive Compensation occurs after the end of that period.

"Recoverable Incentive Compensation" means (a) in the case of any Restatement, the amount of any Incentive
Compensation (calculated on a pre-tax basis) Received by a Covered Officer during the Applicable Period that is in
excess of the amount that otherwise would have been Received if the calculation were based on the Restatement,
and (b) in the case of any Misconduct, the amount of any Incentive Compensation (calculated on a pre-tax basis)
awarded or paid to a Covered Officer during the Applicable Period that the Compensation Committee determines,
in its sole discretion, to be appropriate in light of the scope and nature of the Misconduct. For the avoidance of
doubt, in the case of any Restatement, Recoverable Incentive Compensation does not include any Incentive
Compensation Received by a person (i) before such person began service as a Covered Officer and (ii) who did not
serve as a Covered Officer at any time during the performance period for that Incentive Compensation. For the
avoidance of doubt, in the case of any Restatement, Recoverable Incentive Compensation may include Incentive
Compensation Received by a person while serving as an employee if such person previously served as a Covered
Officer and then transitioned to an employee role. For Incentive Compensation based on (or derived from) stock
price or total shareholder return where the amount of Recoverable Incentive Compensation is not subject to
mathematical recalculation directly from the information in the applicable Restatement, the amount will be
determined by the Compensation Committee based on a reasonable estimate of the effect of the Restatement on
the stock price or total shareholder return upon which the Incentive Compensation was Received (in which case,
Microsoft will maintain documentation of such determination of that reasonable estimate and provide such
documentation to Microsoft's applicable listing exchange).

"Restatement" means an accounting restatement of any of Microsoft's financial statements filed with the Securities
and Exchange Commission under the Exchange Act, or the Securities Act of 1933, as amended, due to Microsoft's
material noncompliance with any financial reporting requirement under U.S. securities laws, regardless of whether
Microsoft or Covered Officer misconduct was the cause for such restatement. "Restatement" includes any required
accounting restatement to correct an error in previously issued financial statements that is material to the
previously issued financial statements (commonly referred to as "Big $R ^ { \prime \prime }$ restatements), or that would result in a
material misstatement if the error were corrected in the current period or left uncorrected in the current period
(commonly referred to as "little $r ^ { \prime \prime }$ restatements).

"SLT Member" means any person who has been designated as a member of Microsoft's Senior Leadership Team,
other than an Executive Officer.

<!-- PageFooter="Updated July 1, 2023" -->


## Process rest of the documents and save their results

In [8]:
DI_result = process_document_and_save_results(INPUT_FILE_PATHS[3], PICKLE_DIR)
DI_result = process_document_and_save_results(INPUT_FILE_PATHS[4], PICKLE_DIR)
DI_result = process_document_and_save_results(INPUT_FILE_PATHS[5], PICKLE_DIR)

starting up...
entered...
	Caption: 
page numer: 1
Opening PDF file: %s /content/Agentic_RAG_Workshop/data/Annual Reports/Apple_2023.pdf
image cropped
image saved as/content/Agentic_RAG_Workshop/DI_output/imagesApple_2023_cropped_image_0.png
entered...
	Caption: 
page numer: 1
Opening PDF file: %s /content/Agentic_RAG_Workshop/data/Annual Reports/Apple_2023.pdf
image cropped
image saved as/content/Agentic_RAG_Workshop/DI_output/imagesApple_2023_cropped_image_1.png
entered...
	Caption: 
page numer: 2
Opening PDF file: %s /content/Agentic_RAG_Workshop/data/Annual Reports/Apple_2023.pdf
image cropped
image saved as/content/Agentic_RAG_Workshop/DI_output/imagesApple_2023_cropped_image_2.png
entered...
	Caption: 
page numer: 23
Opening PDF file: %s /content/Agentic_RAG_Workshop/data/Annual Reports/Apple_2023.pdf
image cropped
image saved as/content/Agentic_RAG_Workshop/DI_output/imagesApple_2023_cropped_image_3.png
starting up...
entered...
	Caption: 
page numer: 1
Opening PDF file: %s /con